In [0]:
!pip install -r Req_ristretto.txt

In [0]:
dbutils.library.restartPython()

In [0]:
%python 
%run my_import.py

In [0]:
%python  %run prepro.py 

Su VSCode PARTO DIRETTAMENTE DA QUI

In [ ]:
%run my_import.py

In [ ]:
%run prepro.py

ELENCO DATASET CHE HO QUI IN MEMORIA PER I MODELLI:\
series\
oil_prices\
missdata\
out\
df_solar\
f1\
df_elec\
favorita_train\
favorita_test

# MODELS IMPLEMENTATION

### **WHITE NOISE**

In [0]:
#preparazione dataset
df = series.reset_index()
df = df.reset_index().rename(columns={'index': 'ds','level_0':'indice'})
df['unique_id'] = 'serie_1'
df = df.rename(columns={0: 'y'})

In [0]:
df.head(2)

##### ARIMA + ETS

In [0]:
# divisione in train e test
train_size = int(len(df) * 0.8)
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]
# queste due ci serviranno dopo
season_length = 7 # for monthly data = 12 
horizon = len(test_df) # number of predictions

In [0]:
# train e test plot
fig = go.Figure()
#train dat
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Train'))
#test data 
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Test'))
#layout
fig.update_layout(template='plotly_dark', title='Train and Test Data Plot')
fig.show()

In [0]:
# definisco i modelli
models = [
    AutoARIMA(season_length=season_length),
    AutoETS(season_length=season_length)
]
# creo il forecaster
sf = StatsForecast(models=models, freq='D')
# fit
sf.fit(train_df)

In [0]:
#summary of the model ARIMA
result=sf.fitted_[0,0].model_
print(result.keys())
print("Parametri ARMA:",result['arma'])

Il modello AutoARIMA ha rilevato come miglior modello un ARIMA (0,0,0). Quindi previsioni costanti. Normale per un White Noise

In [0]:
#summary of the model ETS
result2 = sf.fitted_[0,1].model_
print(result2.keys())
print("Modello:",result2['method'])

AutoETS ha rilevato come miglior modello un ANN cioè errore additivo (A), no trend, no stagionalità.

In [0]:
# residuals analysis for both models
residual=pd.DataFrame(result2.get("residuals"), columns=["residual Model"])
fig, axs = plt.subplots(nrows=2, ncols=2)
print("---------Residui per ETS---------")
residual.plot(ax=axs[0,0])
axs[0,0].set_title("Residuals")
sns.distplot(residual, ax=axs[0,1])
axs[0,1].set_title("Density plot - Residual")
stats.probplot(residual["residual Model"], dist="norm", plot=axs[1,0])
axs[1,0].set_title('Plot Q-Q')
plot_acf(residual,  lags=35, ax=axs[1,1],color="fuchsia")
axs[1,1].set_title("Autocorrelation")
plt.show()

residual=pd.DataFrame(result.get("residuals"), columns=["residual Model"])
fig, axs = plt.subplots(nrows=2, ncols=2)
print("---------Residui per ARIMA---------")
residual.plot(ax=axs[0,0])
axs[0,0].set_title("Residuals")
sns.distplot(residual, ax=axs[0,1])
axs[0,1].set_title("Density plot - Residual")
stats.probplot(residual["residual Model"], dist="norm", plot=axs[1,0])
axs[1,0].set_title('Plot Q-Q')
plot_acf(residual,  lags=35, ax=axs[1,1],color="fuchsia")
axs[1,1].set_title("Autocorrelation")
plt.show()

In [0]:
# let's forecast on the test lenght + 100
Y_hat_df = sf.forecast(df=train_df.drop(columns=['indice']), h=(horizon+100), fitted=True, level=[95])
#see fitted values vs true values
values=sf.forecast_fitted_values() #qui viene aggiunta anche la vera y
values.head()

In [0]:
# metrics
metrics_etsA = evaluate(
    test_df.drop(columns=['indice']).merge(Y_hat_df),
    metrics=[ufl.mae, ufl.mape, partial(ufl.mase, seasonality=season_length), ufl.rmse, ufl.smape],
    train_df=train_df,
).drop(columns=['unique_id'])

In statsforecast mape a smape vengono restituiti in forma decimale: in realtà sarebbero 122% e 92%. Li sistemo.

In [0]:
metrics_etsA.loc[[1,4], ['AutoARIMA','AutoETS']] *= 100

In [0]:
metrics_etsA

In [0]:
# Forecast plot
# Creiamo un dataframe completo che include training, test e previsioni future
full_df = pd.concat([train_df, test_df], axis=0)
# Aggiungiamo le previsioni al dataframe completo
forecast_df_etsA = full_df.merge(Y_hat_df, how='outer', on=['unique_id', 'ds'])

fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=values['AutoARIMA'], mode='lines', 
    name='AutoARIMA training', line=dict(color='green'),showlegend=False))
fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoARIMA'], mode='lines', 
    name='AutoARIMA Forecast', line=dict(color='green')))

# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=values['AutoETS'], mode='lines', 
    name='AutoETS training', line=dict(color='orange', dash='dot'),showlegend=False))
fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoETS'], mode='lines', 
    name='AutoETS Forecast', line=dict(color='orange', dash='dot')))

# Intervallo di confidenza AUTOARIMA 
if 'AutoARIMA-lo-95' in forecast_df_etsA.columns and 'AutoARIMA-hi-95' in forecast_df_etsA.columns:
    fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoARIMA-lo-95'], 
        mode='lines', line_color='green', showlegend=False))
    
    fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoARIMA-hi-95'],
        mode='lines', fill='tonexty', fillcolor='rgba(200,200,200,0.2)',
        line_color='green', name='95% CI AutoARIMA'
    ))

# Intervallo di confidenza AUTOETS 
if 'AutoETS-lo-95' in forecast_df_etsA.columns and 'AutoETS-hi-95' in forecast_df_etsA.columns:
    fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoETS-lo-95'], 
        mode='lines', line_color='orange', showlegend=False))
    
    fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoETS-hi-95'],
        mode='lines', fill='tonexty', fillcolor='rgba(255,165,0,0.15)',
        line_color='orange', name='95% CI AutoETS'))

# Layout del grafico
fig.update_layout(title="Forecast Comparison: AutoARIMA vs AutoETS", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

Residual analysis on test set

In [0]:
# testing residuals analysis
ultimo_test = test_df['ds'].max()
primo_test = test_df['ds'].min()
merged_df = forecast_df_etsA.loc[(forecast_df_etsA['ds']<=ultimo_test) & (forecast_df_etsA['ds']>=primo_test)]

residuals_test_arima = pd.Series(merged_df['y'] - merged_df['AutoARIMA'])
residuals_test_ets = pd.Series(merged_df['y'] - merged_df['AutoETS'])
plot_residuals(residuals_test_arima, model_name="ARIMA - test")
plot_residuals(residuals_test_ets, model_name="ETS - test")

In [0]:
# cross validation
crossvalidation_df = sf.cross_validation(df=train_df,
                                         h=horizon, 
                                         step_size=100,
                                         n_windows=5) 

##### XGBOOST

In [0]:
# funzione per creare feature
def create_time_series_features(df, target_col):
    df = df.copy()

    # Lag features (1-7)
    for lag in range(1, 8):
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    
    # Rolling features, 7, 30 e std 7
    df['rolling_mean_7'] = df[target_col].shift(1).rolling(window=7).mean()
    df['rolling_mean_30'] = df[target_col].shift(1).rolling(window=30).mean()
    df['rolling_std_7'] = df[target_col].shift(1).rolling(window=7).std()
    
    # Differencing features, 1 e 7
    df['diff_1'] = df[target_col] - df[target_col].shift(1)
    df['diff_7'] = df[target_col] - df[target_col].shift(7)
    
    # Time-based features per future regressors
    df['day_of_week'] = df['ds'].dt.dayofweek
    df['month'] = df['ds'].dt.month
    return df

In [0]:
# df_xgb
df_xgb = create_time_series_features(df, target_col='y') 
df_xgb.drop(columns=['unique_id'],inplace=True)
df_xgb = df_xgb.dropna().reset_index(drop=True) #droppo i NaN creati dai lag, quindi le prime 30 obs.
print(df_xgb.columns)
df_xgb.head()

In [0]:
# Split train/test
train_size = int(len(df_xgb) * 0.8)
train_df_xgb = df_xgb.iloc[:train_size]
test_df_xgb = df_xgb.iloc[train_size:]

# Separazione features e target
feature_cols = [col for col in df_xgb.columns if col not in ['y', 'ds']]
X_train_xgb, y_train_xgb = train_df_xgb[feature_cols], train_df_xgb['y']
X_test_xgb, y_test_xgb = test_df_xgb[feature_cols], test_df_xgb['y']
X_train_xgb.shape, y_train_xgb.shape, X_test_xgb.shape, y_test_xgb.shape

Ottimizzazione Optuna (Bayesiana)

In [0]:
# Optuna con TimeSeriesSplit
cv_result_global = []
def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'verbosity': 0,
        'seed': 42
    }
    
    # Applicazione di TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)
    y = df_xgb['y']
    df_xgb_feature = df_xgb.drop(columns=['y', 'ds'])
    all_rmses = []

    for train_index, test_index in tscv.split(df_xgb_feature):
        X_train_cv, X_test_cv = df_xgb_feature.iloc[train_index], df_xgb_feature.iloc[test_index]
        y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]

        dtrain = xgb.DMatrix(X_train_cv.values, label=y_train_cv.values)
        dtest = xgb.DMatrix(X_test_cv.values, label=y_test_cv.values)

        model = xgb.train(params, dtrain, num_boost_round=500, evals=[(dtest, 'eval')],
                          early_stopping_rounds=30, verbose_eval=False)

        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test_cv.values, preds))
        all_rmses.append(rmse)
    cv_result_global.append(all_rmses)
    # Restituisci la media degli RMSE di tutte le fold
    mean_rmse = np.mean(all_rmses)
    print(f"Mean RMSE across time series folds: {mean_rmse}")
    return mean_rmse

In [0]:
# Optuna optimization
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)

In [0]:
# Best parameters
print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)

In [0]:
# Plot about CV-Optuna
# hyperparameter importance
fig = vis.plot_param_importances(study)
fig.show()

# RMSE per ogni fold
cv_df = pd.DataFrame(cv_result_global)
cv_df.columns = [f"Fold {i+1}" for i in range(cv_df.shape[1])]

# Boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(data=cv_df)
plt.ylabel("RMSE")
plt.title("Distribuzione RMSE su ogni fold (Optuna CV)")
plt.grid(True)
plt.show()

Training coi migliori parametri trovati da Optuna

In [0]:
# Addestriamo il modello con i best params
best_params = study.best_params
best_params.update({'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 42})

dtrain_xgb = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
model = xgb.train(best_params, dtrain_xgb, num_boost_round=500)

In [0]:
# Plot feature importance
xgb.plot_importance(model, importance_type='gain', max_num_features=20)
plt.title('Feature Importance (Gain)')
plt.tight_layout()
plt.show()

Analisi dei residui (training)

In [0]:
# Training residuals analysis
pred_train = model.predict(xgb.DMatrix(X_train_xgb,label=y_train_xgb))
residuals_train_xgb = y_train_xgb - pred_train
plot_residuals(residuals_train_xgb, model_name="XGBoost - training")

MULTI-STEP FORECAST

In [0]:
def direct_multistep_forecast(train_data, feature_cols, target_col, horizon=None):
    """
    Addestra modelli separati per ogni orizzonte temporale futuro
    """
    forecasts = []
    models = []
    
    # Crea dataframe per date future
    last_date = train_data['ds'].max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=horizon, freq='D') #dopo modificare freq = M
    
    # Per ogni step futuro, addestra un modello dedicato
    for h in range(1, horizon+1):
        print(f"Training model for horizon {h}")
        
        # Prepara target con shift inverso per prevedere h passi avanti
        df_horizon = train_data.copy()
        df_horizon[f'y_horizon_{h}'] = df_horizon[target_col].shift(-h)
        df_horizon = df_horizon.dropna()
        
        # Prendi features e target per questo orizzonte
        X = df_horizon[feature_cols]
        y = df_horizon[f'y_horizon_{h}']
        
        # Split train/validation
        train_size = int(len(X) * 0.8)
        X_train, X_val = X.iloc[:train_size], X.iloc[train_size:]
        y_train, y_val = y.iloc[:train_size], y.iloc[train_size:]
        
        # Addestra modello
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        params = best_params
        model = xgb.train(
            params,
            dtrain,
            evals=[(dtrain, 'train'), (dval, 'val')],
            num_boost_round=100,
            early_stopping_rounds=20,
            verbose_eval=False
        )
        models.append(model)
        
    # Crea input per la previsione (l'ultimo punto noto)
    last_point = xgb.DMatrix(train_data.iloc[[-1]][feature_cols])
    
    # Prevedi ciascun orizzonte con il modello dedicato
    for i, model in enumerate(models):
        pred = model.predict(last_point)[0]
        forecasts.append(pred)
    return pd.DataFrame({'ds': future_dates, 'forecast': forecasts})

In [0]:
# Forecasting
forecast_xgb = direct_multistep_forecast(train_df_xgb, feature_cols, target_col='y',horizon=len(test_df_xgb)+100)
print(f"Previsioni per i prossimi {(len(test_df_xgb)+100)} step:", forecast_xgb.head()) #ho scelto +100 liberamente

In [0]:
# Plot forecast
fig = go.Figure()
# Training data
fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=train_df_xgb['y'], mode='lines', name='Training'))
# pred_train = model.predict(xgb.DMatrix(X_train_xgb,label=y_train_xgb)) #just to plot, shouldn't be used
# fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=pred_train, mode='lines', name='Fit'))

# Test data
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=test_df_xgb['y'], mode='lines', name='Test'))
# Forecast
fig.add_trace(go.Scatter(x=forecast_xgb['ds'], y=forecast_xgb['forecast'], mode='lines', 
                         name='Forecast', line=dict(color='deepskyblue')))
fig.update_layout(title='Forecast vs Real Data', template='plotly_dark')
fig.show()

Calcolo metriche

In [0]:
# Calcola metriche
merged_df = pd.merge(test_df_xgb[['ds', 'y']], forecast_xgb, on='ds', how='left')
metriche_xgb = calcola_metriche(merged_df['y'],merged_df['forecast'],train_df_xgb['y'],modelname="XGBoost")
metriche_xgb

Analisi dei residui (test)

In [0]:
# testing residuals analysis
residuals_test_xgb = pd.Series(merged_df['y'] - merged_df['forecast'])
plot_residuals(residuals_test_xgb, model_name="XGBoost - test")

##### XGBOOST 2 (easier but worse)

In [0]:
# df_xgb
df_xgb = create_time_series_features(df, target_col='y') 
df_xgb.drop(columns=['unique_id'],inplace=True)
df_xgb = df_xgb.dropna().reset_index(drop=True) #droppo i NaN creati dai lag, quindi le prime 30 obs.
# Split train/test
train_size = int(len(df_xgb) * 0.8)
train_df_xgb = df_xgb.iloc[:train_size]
test_df_xgb = df_xgb.iloc[train_size:]

# Separazione features e target
feature_cols = [col for col in df_xgb.columns if col not in ['y', 'ds']]
X_train_xgb, y_train_xgb = train_df_xgb[feature_cols], train_df_xgb['y']
X_test_xgb, y_test_xgb = test_df_xgb[feature_cols], test_df_xgb['y']
X_train_xgb.shape, y_train_xgb.shape, X_test_xgb.shape, y_test_xgb.shape

In [0]:
# Optuna con TimeSeriesSplit
def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'verbosity': 10,
        'seed': 42
    }

    # Applicazione di TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)
    y = df_xgb['y']
    df_xgb_feature = df_xgb.drop(columns=['y', 'ds'])
    all_rmses = []

    for train_index, test_index in tscv.split(df_xgb_feature):
        X_train_cv, X_test_cv = df_xgb_feature.iloc[train_index], df_xgb_feature.iloc[test_index]
        y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]

        dtrain = xgb.DMatrix(X_train_cv.values, label=y_train_cv.values)
        dtest = xgb.DMatrix(X_test_cv.values, label=y_test_cv.values)

        model = xgb.train(params, dtrain, num_boost_round=500, evals=[(dtest, 'eval')],
                          early_stopping_rounds=30, verbose_eval=False)

        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test_cv.values, preds))
        all_rmses.append(rmse)

    # Restituisci la media degli RMSE di tutte le fold
    mean_rmse = np.mean(all_rmses)
    print(f"Mean RMSE across time series folds: {mean_rmse}")
    return mean_rmse

In [0]:
best_params = study.best_params
best_params.update({
    'booster': 'gbtree',
    'objective': 'reg:squarederror',
    'n_estimators': 1000,
    'early_stopping_rounds': 50,
    'eval_metric': 'rmse'
})

reg_final = xgb.XGBRegressor(**best_params)
reg_final.fit(X_train_xgb, y_train_xgb,
              eval_set=[(X_train_xgb, y_train_xgb), (X_test_xgb, y_test_xgb)],
              verbose=10)

In [0]:
#Predictions
pred = reg_final.predict(X_test_xgb)

In [0]:
# Plot forecast
fig = go.Figure()
# Training data
fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=train_df_xgb['y'], mode='lines', name='Training'))
# Test data
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=test_df_xgb['y'], mode='lines', name='Test'))
# Forecast
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=pred, mode='lines', 
                         name='Forecast', line=dict(color='deepskyblue')))
fig.update_layout(title='Forecast vs Real Data', template='plotly_dark')
fig.show()

In [0]:
# Calcola metriche
metriche_xgb2 = calcola_metriche(test_df_xgb['y'], pred, train_df_xgb['y'], modelname="XGBoost2")
metriche_xgb2

##### LSTM (neuralforecast)

In [0]:
# divisione in train e test
df_lstm = df.copy()
df_lstm.drop(columns=['indice'],inplace = True)
train_size = int(len(df_lstm) * 0.8)
train_df_lstm = df_lstm.iloc[:train_size]
test_df_lstm = df_lstm.iloc[train_size:]
print(train_df_lstm.columns)
train_df_lstm.head()

In [0]:
# definiamo il modello
nf = NeuralForecast(
    models=[LSTM(h=len(test_df_lstm)+100,
                 input_size=len(test_df_lstm)+100,
                 loss=DistributionLoss(distribution="Normal", level=[80, 95]),
                 scaler_type='robust',
                 encoder_n_layers=2,
                 encoder_hidden_size=128,
                 decoder_hidden_size=128,
                 decoder_layers=2,
                 max_steps=10,
                 early_stop_patience_steps=5,
                 recurrent=False,
                 start_padding_enabled=True
                 ),
    ],
    freq='D')

In [0]:
# time series cross validation
cv_df = nf.cross_validation(
    val_size = len(test_df_lstm)+100,
    df=train_df_lstm,       # Deve essere un df con colonne: unique_id, ds, y
    #h=len(test_df_lstm),
    n_windows=5,
    step_size=100,
    refit=True          # True = rifitta il modello da zero per ogni finestra
)

In [0]:
mae_lstm = mean_absolute_error(cv_df['y'], cv_df['LSTM'])
print(f"MAE LSTM: {mae_lstm:.3f}")

In [0]:
#fit
nf.fit(df=train_df_lstm, val_size=len(test_df_lstm)+100)

In [0]:
# creazione future dataframe
start_date = test_df_lstm['ds'].min()
all_dates = pd.date_range(start=start_date, periods=len(test_df_lstm)+100, freq='D')

# Crea il DataFrame completo
data = pd.DataFrame({
    'ds': all_dates.strftime('%Y-%m-%d'),
    'unique_id': 'serie_1'
})

# Aggiungi i valori y dai dati originali
data['y'] = pd.Series(
    list(test_df_lstm['y'].values) + [np.nan] * 100)
data['ds'] = pd.to_datetime(data['ds'])

In [0]:
# Predict on test + 100
Y_hat_df_lstm = nf.predict(futr_df=data)

In [0]:
# predict insample, si blocca VSC
# Y_hat_insample = nf.predict_insample()
# print(Y_hat_insample.columns)
# print(Y_hat_insample.head())

Calcolo metriche

In [0]:
# Calcola metriche
merged_df_lstm = pd.merge(test_df[['ds', 'y']], Y_hat_df_lstm, on='ds', how='left')
metriche_lstm = calcola_metriche(merged_df_lstm['y'],merged_df_lstm['LSTM'],train_df['y'],modelname="LSTM")
metriche_lstm

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM'], mode='lines', 
    name='MEAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIANA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='yellow')))

# Intervallo di confidenza
if 'LSTM-lo-95' in Y_hat_df_lstm.columns and 'LSTM-hi-95' in Y_hat_df_lstm.columns:
    fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM-lo-95'], 
        mode='lines', line_color='green', showlegend=False))
    
    fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM-hi-95'],
        mode='lines', fill='tonexty', fillcolor='rgba(200,200,200,0.2)',
        line_color='green', name='95% CI LSTM'
    ))

# Layout del grafico
fig.update_layout(title="LSTM forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")

fig.show()

In [0]:
# testing residuals analysis
ultimo_test = test_df['ds'].max()
primo_test = test_df['ds'].min()
merged_df = Y_hat_df_lstm['LSTM'].loc[(Y_hat_df_lstm['ds']<=ultimo_test) & (Y_hat_df_lstm['ds']>=primo_test)]
residuals_test_lstm = pd.Series(merged_df - test_df['y'].values)
plot_residuals(residuals_test_lstm, model_name="LSTM - test")

##### NEURAL PROPHET

In [0]:
# divisione in train e test
df_np = df.drop(columns=['indice'])
train_size = int(len(df_np) * 0.8)
train_df_np = df_np.iloc[:train_size]
test_df_np = df_np.iloc[train_size:]

In [0]:
# definizione del modello
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.01, 
    batch_size=32,
    # n_forecasts=1,                            # Specifica l'orizzonte di previsione
    # n_lags=14,                                # componente AR, 14 gg, due settimane di dati; aggiunge 14 NaN
    # daily_seasonality=Auto (True/False),      # Disattiva la stagionalità giornaliera se non necessaria
    # weekly_seasonality=Auto,                  # Attiva stagionalità settimanale
    # yearly_seasonality=Auto,                  # Attiva stagionalità annuale
    loss_func='Huber'                           # Più robusta agli outlier rispetto a MSE
)

In [0]:
# training & predict                        
init = time()

#fit
metrics_df = neuralprophet.fit(train_df_np[['ds','y']], validation_df=test_df_np[['ds','y']].head(30))
# print("\nTraining metrics:")
# print(metrics_df)

#Prediction on test set
forecast_test = neuralprophet.predict(test_df_np[['ds','y']])
#Predictions out of sample
future = neuralprophet.make_future_dataframe(df=df_np[['ds','y']], periods=100)
forecast_future = neuralprophet.predict(future)

end = time()
time_np = (end - init) / 60
print(f'Prophet Time: {time_np:.2f} minutes')

In [0]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
# Plot Loss
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['Loss_val'], label='Validation Loss', color='lightblue')

# Plot MAE
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['MAE_val'], label='Validation MAE', color='lightgreen')

# Plot RMSE
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.plot(metrics_df['epoch'], metrics_df['RMSE_val'], label='Validation RMSE', color='orange')

plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
# residual analysis (training)
forecast_train = neuralprophet.predict(train_df_np[['ds','y']])
residuals_train_np = pd.Series(forecast_train['y'] - forecast_train['yhat1'])
plot_residuals(residuals_train_np, model_name="NeuralProphet - train")

In [0]:
# selecting columns & changing names
forecast_test = forecast_test[['ds', 'yhat1', 'yhat1 2.5%', 'yhat1 97.5%','trend','season_yearly','season_weekly']]
forecast_test.columns = ['ds', 'NeuralProphet', 'NeuralProphet-lo-95', 'NeuralProphet-hi-95', 
                       'trend','season_yearly','season_weekly']
forecast_future = forecast_future[['ds', 'yhat1', 'yhat1 2.5%', 'yhat1 97.5%','trend','season_yearly','season_weekly']]
forecast_future.columns = ['ds', 'NeuralProphet', 'NeuralProphet-lo-95', 'NeuralProphet-hi-95', 
                       'trend','season_yearly','season_weekly']

In [0]:
params = neuralprophet.plot_parameters()
params.show() #Mostra la forza di ogni componente del modello e come contribuisce alla previsione

In [0]:
fig_components = neuralprophet.plot_components(forecast_test)
fig_components.show() # Plot components like trends and different seasonalities

In [0]:
forecast_future.tail() #last forecasts available

In [0]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_np['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_np['y'], mode='lines', name='Testing Data'))

# PREVISIONI NEURAL PROPHET
# training period
fig.add_trace(go.Scatter(
    x=forecast_train['ds'], 
    y=forecast_train['yhat1'], 
    mode='lines', 
    line=dict(color='green'),
    showlegend=False,
    name = "NeuralProphet-train set"
))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['NeuralProphet'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))

# Out-of-sample forecast (future)
fig.add_trace(go.Scatter(
    x=forecast_future['ds'], 
    y=forecast_future['NeuralProphet'], 
    mode='lines', 
    line=dict(color='green'),
    showlegend=False,  # Evita duplicati nella legenda
    name="Out-of-sample"
))

# Intervallo di confidenza - test period
if 'NeuralProphet-lo-95' in forecast_test.columns and 'NeuralProphet-hi-95' in forecast_test.columns:
    fig.add_trace(go.Scatter(
        x=forecast_test['ds'], 
        y=forecast_test['NeuralProphet-lo-95'], 
        mode='lines', 
        line=dict(color='green', width=0), 
        name = "95% CI",
        showlegend=False
    ))
    
    fig.add_trace(go.Scatter(
        x=forecast_test['ds'], 
        y=forecast_test['NeuralProphet-hi-95'],
        mode='lines', 
        fill='tonexty', 
        fillcolor='rgba(200,200,200,0.2)',
        line=dict(color='green', width=0), 
        name='95% CI'
    ))

# Intervallo di confidenza - future period
if 'NeuralProphet-lo-95' in forecast_future.columns and 'NeuralProphet-hi-95' in forecast_future.columns:
    fig.add_trace(go.Scatter(
        x=forecast_future['ds'], 
        y=forecast_future['NeuralProphet-lo-95'], 
        mode='lines', 
        line=dict(color='green', width=0), 
        name = "95% CI",
        showlegend=False
    ))
    
    fig.add_trace(go.Scatter(
        x=forecast_future['ds'], 
        y=forecast_future['NeuralProphet-hi-95'],
        mode='lines', 
        fill='tonexty', 
        fillcolor='rgba(200,200,200,0.2)',
        line=dict(color='green', width=0), 
        name = "95% CI",
        showlegend=False 
    ))
    
# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast_test['ds'], 'forecast': forecast_test['NeuralProphet']})
merged_df = pd.merge(test_df_np[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
mae_val = mean_absolute_error(merged_df['y'], merged_df['forecast'])
metriche_np = calcola_metriche(merged_df['y'], merged_df['forecast'],train_df['y'],modelname="NeuralProphet")
metriche_np

In [0]:
# testing residuals analysis
residuals_test_np = pd.Series(merged_df['y'] - merged_df['forecast'])
plot_residuals(residuals_test_np, model_name="NeuralProphet - test")

In [0]:
# save and re-import a model
# import pickle
# # Save the model
# with open("neuralprophet_model.pkl", "wb") as file:
#     pickle.dump(neuralprophet, file) #pickle.dump
# print("Model saved as 'neuralprophet_model.pkl'.")
# # Load the saved model
# with open("neuralprophet_model.pkl", "rb") as file:
#     loaded_model = pickle.load(file) #pickle.load
# print("Model loaded successfully!")

##### final evaluations

In [0]:
metrics_etsA = metrics_etsA.set_index('metric')
final_metrics = metrics_etsA.join(metriche_xgb).join(metriche_xgb2).join(metriche_lstm).join(metriche_np).round(4)

In [0]:
final_metrics

In [0]:
# FINAL PLOT FOR WHITE NOISE
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_np['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_np['y'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoARIMA'], 
    mode='lines', 
    name='ARIMA', 
    line=dict(color='yellow')
))

# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoETS'], 
    mode='lines', 
    name='ETS', 
    line=dict(color='green')
))

# PREVISIONI XGBOOST
fig.add_trace(go.Scatter(
    x=forecast_xgb['ds'], 
    y=forecast_xgb['forecast'], 
    mode='lines', 
    name='XGB', 
    line=dict(color='purple')
))

# PREVISIONI LSTM
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=Y_hat_df_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='lightpink')
))

# PREVISIONI NEURAL PROPHET
# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['NeuralProphet'], 
    mode='lines', 
    name='NP in-sample', 
    line=dict(color='aquamarine')
))

# Out-of-sample forecast (future)
fig.add_trace(go.Scatter(
    x=forecast_future['ds'], 
    y=forecast_future['NeuralProphet'], 
    mode='lines', 
    line=dict(color='aquamarine'),
    name="NP Out-of-sample"
))
    
# Layout del grafico
fig.update_layout(
    title="Forecasts for White Noise", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

### **SOLAR ENERGY**

In [ ]:
#preparazione dataset
df = df_solar.drop(columns=['AC_POWER'])
df = df.reset_index().rename(columns={'DATE_TIME': 'ds','DC_POWER':'y'})
df = df.dropna()
df['unique_id'] = 'serie_1'

In [ ]:
# mancano due righe di notte, le fillo easy
row_to_copy = df[df['ds'] == '2020-06-17 06:00:00']
# Crea copie modificate per le 06:15 e 06:30
row_0615 = row_to_copy.copy()
row_0615['ds'] = pd.to_datetime('2020-06-17 06:15:00')

row_0630 = row_to_copy.copy()
row_0630['ds'] = pd.to_datetime('2020-06-17 06:30:00')

df = pd.concat([df, row_0615, row_0630], ignore_index=True)
df['ds'] = pd.to_datetime(df['ds'])
df = df.sort_values(by='ds').reset_index(drop=True)

In [ ]:
# funzione per creare feature
def create_time_series_features(df, target_col):
    df = df.copy()
    lags = [1, 2, 4, 24, 36, 48, 96]
    # Lag features (1-4)
    for lag in lags:
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    
    # Differencing features, 1, 4, 96
    df['diff_1'] = (df[target_col] - df[target_col].shift(1)).shift(1) #diff tra t-1 e t-2
    df['diff_4'] = (df[target_col] - df[target_col].shift(4)).shift(1) #diff tra t-1 e t-5
    df['diff_96'] = (df[target_col] - df[target_col].shift(96)).shift(1) #diff tra t-1 e t-97
    
    # Caratteristiche cicliche per rappresentare meglio la stagionalità
    df['day_sin'] = np.sin(2 * np.pi * df['giorno_settimana']/7)
    df['day_cos'] = np.cos(2 * np.pi * df['giorno_settimana']/7)
    df['hour_sin'] = np.sin(2 * np.pi * df['ora_del_giorno']/24)
    df['hour_cos'] = np.cos(2 * np.pi * df['ora_del_giorno']/24)
    
    # rolling means 
    df['rolling_mean_4'] = df[target_col].rolling(window=4).mean().shift(4) #ultima ora
    df['rolling_mean_48'] = df[target_col].rolling(window=6).mean().shift(1) #ultime 12 ore
    df['rolling_mean_96'] = df[target_col].rolling(window=12).mean().shift(1) #ultimo giorno
    
    return df

##### ARIMA + ETS

In [0]:
# divisione in train e test
df_arima = df[['y','ds','unique_id']].iloc[97:]
train_size = int(len(df_arima) * 0.9)
train_df = df_arima.iloc[:train_size]
test_df = df_arima.iloc[train_size:]
# queste due ci serviranno dopo
season_length = 96
horizon = len(test_df) # number of predictions
train_df.shape, test_df.shape

In [0]:
# train e test plot
fig = go.Figure()
#train dat
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Train'))
#test data 
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Test'))
#layout
fig.update_layout(template='plotly_dark', title='Train and Test Data Plot')
fig.show()

In [0]:
# definisco i modelli
models = [
    AutoARIMA(), 
    AutoETS() 
]
# creo il forecaster
sf = StatsForecast(models=models, freq='15min')

start_time = time.time()
# fit
sf.fit(train_df)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
#summary of the model ARIMA
result=sf.fitted_[0,0].model_
print(result.keys())
print("Parametri ARMA:",result['arma'])

In [0]:
arima_string(sf.fitted_[0,0].model_)

In [0]:
#summary of the model ETS
result2 = sf.fitted_[0,1].model_
print(result2.keys())
print("Modello:",result2['method'])

In [0]:
# let's forecast
Y_hat_df = sf.forecast(df=train_df, h=len(test_df), fitted=True, level=[95])
#see fitted values vs true values
values=sf.forecast_fitted_values() #qui viene aggiunta anche la vera y
values.head(2)

In [0]:
values.tail(2)

In [0]:
# TIME SERIES CROSS VALIDATION
crossvalidation_df = sf.cross_validation(
    df=train_df,
    h=horizon, 
    step_size=len(test_df),
    n_windows=5
)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = crossvalidation_df['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = crossvalidation_df[crossvalidation_df['cutoff'] == fold]
    
    for model in ['AutoARIMA', 'AutoETS']:
        # Filtra i dati per il modello corrente
        model_data = fold_data[['y', f'{model}']]
        model_data = model_data.dropna()
        
        # Calcola MAE e RMSE
        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data[f'{model}'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data[f'{model}'])
        
        # Aggiungi i risultati
        cv_metrics.append({
            'Fold': i+1,
            'Model': model,
            f'MAE_{model}': mae,
            f'RMSE_{model}': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_arima_ets = pd.DataFrame(cv_metrics)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = crossvalidation_df[crossvalidation_df['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df['unique_id'].unique():
    series_data = train_df[train_df['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

# Plot MAE sul primo asse
for model in cv_metrics_df_arima_ets['Model'].unique():
    model_data = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == model]
    ax1.plot(model_data['Fold'], model_data[f'MAE_{model}'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_arima_ets['Model'].unique():
    model_data = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == model]
    ax2.plot(model_data['Fold'], model_data[f'RMSE_{model}'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()

In [0]:
fold_intervals #EXPANDING WINDOW CROSS VALIDATION

In [0]:
# prediction on test set
test_pred = sf.forecast(df=train_df, h=len(test_df), level=[95])
Y_hat_df = test_df.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])

In [0]:
# Calcola metriche
metriche_arima = calcola_metriche(Y_hat_df['y'],Y_hat_df['AutoARIMA'],train_df['y'], 
                                modelname="ARIMA").round(10)
metriche_ets = calcola_metriche(Y_hat_df['y'],Y_hat_df['AutoETS'],train_df['y'], 
                                modelname="ETS").round(10)
metriche_arima, metriche_ets

In [0]:
# Forecast plot
# Creiamo un dataframe completo che include training, test e previsioni future
full_df = pd.concat([train_df, test_df], axis=0)
# Aggiungiamo le previsioni al dataframe completo
forecast_df_etsA = full_df.merge(Y_hat_df, how='outer', on=['unique_id', 'ds'])

fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Training Data'))

# Dati di test
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(x=test_pred['ds'], y=test_pred['AutoARIMA'], mode='lines', 
    name='AutoARIMA Forecast', line=dict(color='green')))

# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(x=test_pred['ds'], y=test_pred['AutoETS'], mode='lines', 
    name='AutoETS Forecast', line=dict(color='orange', dash='dot')))

# # Intervallo di confidenza AUTOARIMA 
# if 'AutoARIMA-lo-95' in forecast_df_etsA.columns and 'AutoARIMA-hi-95' in forecast_df_etsA.columns:
#     fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoARIMA-lo-95'], 
#         mode='lines', line_color='green', showlegend=False))
    
#     fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoARIMA-hi-95'],
#         mode='lines', fill='tonexty', fillcolor='rgba(200,200,200,0.2)',
#         line_color='green', name='95% CI AutoARIMA'
#     ))

# # Intervallo di confidenza AUTOETS 
# if 'AutoETS-lo-95' in forecast_df_etsA.columns and 'AutoETS-hi-95' in forecast_df_etsA.columns:
#     fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoETS-lo-95'], 
#         mode='lines', line_color='orange', showlegend=False))
    
#     fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoETS-hi-95'],
#         mode='lines', fill='tonexty', fillcolor='rgba(255,165,0,0.15)',
#         line_color='orange', name='95% CI AutoETS'))

# Layout del grafico
fig.update_layout(title="Forecast Comparison: AutoARIMA vs AutoETS", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

##### XGBOOST

In [0]:
# df_xgb
df_xgb = create_time_series_features(df, target_col='y') 
df_xgb.drop(columns=['unique_id'],inplace=True)
df_xgb = df_xgb.dropna().reset_index(drop=True) #droppo i NaN creati dai lag
print(df_xgb.columns)
df_xgb.head()

In [0]:
# Split train/test
train_size = int(len(df_xgb) * 0.9)
train_df_xgb = df_xgb.iloc[:train_size]
test_df_xgb = df_xgb.iloc[train_size:]

# Separazione features e target
feature_cols = [col for col in df_xgb.columns if col not in ['y', 'ds']]
X_train_xgb, y_train_xgb = train_df_xgb[feature_cols], train_df_xgb['y']
X_test_xgb, y_test_xgb = test_df_xgb[feature_cols], test_df_xgb['y']
X_train_xgb.shape, y_train_xgb.shape, X_test_xgb.shape, y_test_xgb.shape

In [0]:
# Scaling dei dati

#features
scaler_x = MinMaxScaler()
X_train_scaled = scaler_x.fit_transform(X_train_xgb)
X_test_scaled = scaler_x.transform(X_test_xgb)

# target, non serve che scalo y_test
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train_xgb.values.reshape(-1, 1))

# Ricreo df con stessi indici e nomi
X_train_xgb = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train_xgb.index)
X_test_xgb = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test_xgb.index)
y_train_xgb = pd.Series(y_train_scaled.flatten(), name='y', index=y_train_xgb.index)

#aggiorno train_df_xgb
train_df_xgb[feature_cols] = X_train_xgb
train_df_xgb['y'] = y_train_xgb

#controllo le shape
X_train_xgb.shape, X_test_xgb.shape, y_train_xgb.shape, y_test_xgb.shape

Ottimizzazione Optuna (Bayesiana)

In [0]:
# Optuna function
cv_metrics_log = []  # List globale per salvare metriche fold per fold

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'verbosity': 0,
        'seed': 42
    }

    tscv = TimeSeriesSplit(n_splits=5,test_size=len(test_df_xgb))
    y = train_df_xgb['y']
    df_xgb_feature = train_df_xgb.drop(columns=['y','ds'])
    all_rmse = []

    for fold, (train_idx, test_idx) in enumerate(tscv.split(df_xgb_feature)):
        X_train, X_test = df_xgb_feature.iloc[train_idx], df_xgb_feature.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        dtest = xgb.DMatrix(X_test.values, label=y_test.values)

        model = xgb.train(params, dtrain, num_boost_round=500,
                          evals=[(dtest, 'eval')],
                          early_stopping_rounds=30, verbose_eval=False)

        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mae = calcola_mae(y_test, preds)  # tua funzione custom

        all_rmse.append(rmse)

        # Recupera le date (se esiste colonna 'ds')
        start_date = df_xgb.iloc[test_idx]['ds'].min() if 'ds' in df_xgb.columns else None
        end_date = df_xgb.iloc[test_idx]['ds'].max() if 'ds' in df_xgb.columns else None

        # Logga le metriche della fold
        cv_metrics_log.append({
            'Trial': trial.number,
            'Fold': fold + 1,
            'MAE_XGB': mae,
            'RMSE_XGB': rmse,
            'start_date': start_date,
            'end_date': end_date,
            'Model': 'XGBoost'  # Puoi modificarlo se usi più modelli
        })
    return np.mean(all_rmse)

In [0]:
# Optuna optimization
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50) #aumentare su databricks

In [0]:
# Best parameters
print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)
print("Best trial:",study.best_trial.number)
besttrial = study.best_trial.number

In [0]:
cv_metrics_log = pd.DataFrame(cv_metrics_log)
cv_metrics_log = cv_metrics_log[cv_metrics_log['Trial']==besttrial]
cv_metrics_log

Vediamo come ha performato nel miglior trial (best parameters)

In [0]:
#PLOT CV
cv_metrics_df_xgboost = cv_metrics_log.copy()

# Stampa le metriche per modello
print("Metriche per modello:")
print(cv_metrics_df_xgboost)

# Calcola e stampa le metriche medie per modello
mean_metrics = cv_metrics_df_xgboost.mean(numeric_only=True).reset_index()

# Crea il DataFrame con gli intervalli delle fold
fold_intervals_df = cv_metrics_df_xgboost[['Fold', 'start_date', 'end_date']].drop_duplicates()

# Ora creiamo il grafico combinato
plt.figure(figsize=(16, 12))

# === PLOT 1: Serie temporale originale ===
plt.subplot(2, 1, 1)
#for unique_id in train_df_xgb['unique_id'].unique():
series_data = train_df_xgb.copy()
plt.plot(series_data['ds'], series_data['y'], label=f'Solar energy')

# Evidenzia le fold con colori
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)],
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10,
             bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# === PLOT 2: Metriche MAE e RMSE ===
plt.subplot(2, 1, 2)
ax1 = plt.gca()
ax2 = ax1.twinx()

# MAE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_XGB'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# RMSE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_XGB'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Legenda combinata
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


Training coi migliori parametri trovati da Optuna

In [0]:
# Aggiorniamo il modello con i best params
best_params = study.best_params
best_params.update({'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 42})

dtrain_xgb = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
start_time = time.time()
#fit 
model = xgb.train(best_params, dtrain_xgb, num_boost_round=500)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Plot feature importance
ax = xgb.plot_importance(model, importance_type='gain', max_num_features=10)
for text in ax.texts:
    text.set_visible(False)
plt.title('Feature Importance (Gain)')
plt.tight_layout()
plt.show()

MULTI-STEP FORECAST

In [0]:
def direct_multistep_forecast(train_data, feature_cols, target_col, horizon=None):
    """
    Addestra modelli separati per ogni orizzonte temporale futuro
    """
    forecasts = []
    models = []
    
    # Crea dataframe per date future
    last_date = train_data['ds'].max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta(minutes=15), periods=horizon, freq='15min') #dopo modificare
    
    # Per ogni step futuro, addestra un modello dedicato
    for h in range(1, horizon+1):
        print(f"Training model for horizon {h}")
        
        # Prepara target con shift inverso per prevedere h passi avanti
        df_horizon = train_data.copy()
        df_horizon[f'y_horizon_{h}'] = df_horizon[target_col].shift(-h)
        df_horizon = df_horizon.dropna()
        
        # Prendi features e target per questo orizzonte
        X = df_horizon[feature_cols]
        y = df_horizon[f'y_horizon_{h}']
        
        # Split train/validation
        train_size = int(len(X) * 0.8)
        X_train, X_val = X.iloc[:train_size], X.iloc[train_size:]
        y_train, y_val = y.iloc[:train_size], y.iloc[train_size:]
        
        # Addestra modello
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        params = best_params
        model = xgb.train(
            params,
            dtrain,
            evals=[(dtrain, 'train'), (dval, 'val')],
            num_boost_round=100,
            early_stopping_rounds=20,
            verbose_eval=False
        )
        models.append(model)
        
    # Crea input per la previsione (l'ultimo punto noto)
    last_point = xgb.DMatrix(train_data.iloc[[-1]][feature_cols])
    
    # Prevedi ciascun orizzonte con il modello dedicato
    for i, model in enumerate(models):
        pred = model.predict(last_point)[0]
        forecasts.append(pred)
    return pd.DataFrame({'ds': future_dates, 'forecast': forecasts})

In [0]:
# Forecasting
#forecast_xgb = direct_multistep_forecast(train_df_xgb, feature_cols, target_col='y',horizon=len(test_df_xgb)) #306

RECURSIVE FORECAST

In [0]:
dtest_xgb = xgb.DMatrix(X_test_xgb)
forecast_xgb = pd.DataFrame(model.predict(dtest_xgb),columns=['forecast'])
forecast_xgb['ds'] = test_df_xgb['ds'].values

Rescaling dei dati

In [0]:
# RE-Scaling dei dati
forecast_xgb['forecast'] = scaler_y.inverse_transform(forecast_xgb[['forecast']])
train_df_xgb['y'] = scaler_y.inverse_transform(train_df_xgb[['y']])

In [0]:
# Plot forecast
fig = go.Figure()
# Training data
fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=train_df_xgb['y'], mode='lines', name='Training'))
# Test data
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=test_df_xgb['y'], mode='lines', name='Test'))
# Forecast
fig.add_trace(go.Scatter(x=forecast_xgb['ds'], y=forecast_xgb['forecast'], mode='lines', 
                         name='Forecast', line=dict(color='deepskyblue')))
fig.update_layout(title='Forecast vs Real Data', template='plotly_dark')
fig.show()

Calcolo metriche

In [0]:
# Calcola metriche
merged_df = pd.merge(test_df_xgb[['ds', 'y']], forecast_xgb, on='ds', how='left')
merged_df.dropna(inplace=True)
metriche_xgb = calcola_metriche(merged_df['y'],merged_df['forecast'],train_df_xgb['y'], 
                                modelname="XGBoost").round(10)
metriche_xgb

##### LSTM

In [0]:
# divisione in train e test
df_lstm = create_time_series_features(df, target_col='y') 
df_lstm = df_lstm.dropna().reset_index(drop=True)

train_size = int(len(df_lstm) * 0.9)
train_df_lstm = df_lstm.iloc[:train_size]
test_df_lstm = df_lstm.iloc[train_size:]
print(train_df_lstm.columns)
train_df_lstm.head()

In [0]:
# Optuna function
futr_exog_list = train_df_lstm.drop(columns=['unique_id', 'ds', 'y']).columns.tolist()
folds = pd.DataFrame()
h = len(test_df_lstm)
cv_metrics_df_lstm = pd.DataFrame()
def objective(trial):
    cv_metrics = []
    # Hyperparametri da ottimizzare
    input_size = trial.suggest_categorical("input_size", [h, int(1.75*h), int(3*h)] )
    encoder_n_layers = trial.suggest_int("encoder_n_layers", 1, 4)
    encoder_hidden_size = trial.suggest_categorical("encoder_hidden_size", [32, 64, 128, 256])
    decoder_hidden_size = trial.suggest_categorical("decoder_hidden_size", [32, 64, 128, 256])
    decoder_layers = trial.suggest_int("decoder_layers", 1, 4)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 96, 128])

    # Fisso il seed per riproducibilità
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

    # Definizione modello LSTM
    lstm = LSTM(
        h=h,  # nel tuning
        input_size=input_size,
        loss=DistributionLoss(distribution="Normal", level=[95]),
        scaler_type='robust',
        encoder_n_layers=encoder_n_layers,
        encoder_hidden_size=encoder_hidden_size,
        decoder_hidden_size=decoder_hidden_size,
        decoder_layers=decoder_layers,
        futr_exog_list=futr_exog_list,
        max_steps=100, #aumentare a 150
        batch_size=batch_size,
        learning_rate=learning_rate,
        recurrent=False
    )

    nf = NeuralForecast(models=[lstm], freq='15min')

    # cross-validation per tuning
    df_cv = nf.cross_validation(
        df=train_df_lstm,
        step_size=h,
        n_windows=5
    )

    # Calcola MAE per ogni fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'LSTM']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'LSTM',
            'MAE_lstm': mae,
            'RMSE_lstm': rmse
        })

    # Media MAE su tutte le fold
    cv_metrics_df_lstm = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_lstm['MAE_lstm'].mean()

    return mean_mae 

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50) 

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
# salvo i best params trovati da Optuna
# with open('pickles/LSTM_bestPar_SolarEnergy.pkl', 'wb') as file:
#     pickle.dump(best_params, file)

Carico i best params

In [0]:
with open('pickles/LSTM_bestPar_SolarEnergy.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [0]:
# final model
best_lstm = LSTM(
    h=len(test_df_lstm),
    input_size=3*len(test_df_lstm),
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=100,
    #early_stop_patience_steps=20,
    batch_size=best_params['batch_size'],
    learning_rate=0.01,
    recurrent=False
)
nf = NeuralForecast(models=[best_lstm], freq='15min')

In [0]:
# final CV
# df_cv = nf.cross_validation(
#     df=train_df_lstm,
#     step_size=len(test_df_lstm),
#     n_windows=5
# )
# with open('pickles/LSTM_CV_SolarEnergy.pkl', 'wb') as file:
#      pickle.dump(df_cv, file)

with open('pickles/LSTM_CV_SolarEnergy.pkl', 'rb') as file:
    df_cv = pickle.load(file)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'LSTM']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'LSTM',
        'MAE_lstm': mae,
        'RMSE_lstm': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_lstm = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_lstm)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_lstm.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_lstm['unique_id'].unique():
    series_data = train_df_lstm[train_df_lstm['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_lstm['Model'] = 'lstm'
# Plot MAE sul primo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_lstm'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_lstm'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


In [0]:
# Final training

# Fisso il seed per riproducibilità
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

final_model = LSTM(
    h=len(test_df_lstm),
    input_size=best_params['input_size'],
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=50, #aumentare a 150
    early_stop_patience_steps=15,
    batch_size=64,
    learning_rate=0.01, 
    recurrent=False
)

nf = NeuralForecast(models=[final_model], freq='15min')
start_time = time.time()
#fit 
nf.fit(df=train_df_lstm, val_size=len(test_df_lstm))
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

Previsioni

In [0]:
Y_hat_df_lstm = nf.predict(futr_df=test_df_lstm)
# salvo previsioni in pickle
Y_hat_df_lstm.to_pickle('pickles/LSTM_pred_solarEnergy.pkl')

In [0]:
#carico le previsioni
Y_hat_df_lstm = pd.read_pickle('pickles/LSTM_pred_solarEnergy.pkl')

In [0]:
# Plot forecast
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_lstm['ds'], y=train_df_lstm['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_lstm['ds'], y=test_df_lstm['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM'], mode='lines', 
    name='MEAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIANA LSTM
# fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM-median'], mode='lines', 
#     name='MEDIAN Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="LSTM forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_lstm['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")

fig.show()

In [0]:
# Calcola metriche
merged_df_lstm = pd.merge(test_df_lstm[['ds', 'y']], Y_hat_df_lstm, on='ds', how='left')
merged_df_lstm.dropna(inplace=True)
metriche_lstm = calcola_metriche(merged_df_lstm['y'],merged_df_lstm['LSTM'],
                                 train_df_lstm['y'],modelname="LSTM").round(10)
metriche_lstm

##### NEURAL PROPHET

In [0]:
df_np = create_time_series_features(df, target_col='y') 
df_np = df_np.dropna().reset_index(drop=True)
df_np = df_np.drop(columns=['unique_id'])
df_np.columns

In [0]:
# Controllo gli zeri nel dataframe
df = df_np.copy()
df['ds'] = pd.to_datetime(df['ds'])

# 2. Definisci gli orari di interesse
start_time = pd.to_datetime('18:45').time()
end_time = pd.to_datetime('05:45').time()

# 3. Crea una maschera booleana per selezionare le righe nell'intervallo orario
mask = (df['ds'].dt.time >= start_time) | (df['ds'].dt.time <= end_time)

# 4. Applica la maschera al DataFrame per ottenere le righe nell'intervallo orario
df_notturno = df[mask]

# 5. Verifica se ci sono valori diversi da zero nella colonna 'y' nel subset notturno
valori_non_zero_notte = df_notturno[df_notturno['y'] != 0]

# 6. Stampa il risultato
if not valori_non_zero_notte.empty:
    print("Sono stati trovati valori diversi da zero nella colonna 'y' tra le 18:45 e le 05:45 (estremi inclusi):")
    print(valori_non_zero_notte)
else:
    print("Non sono stati trovati valori diversi da zero nella colonna 'y' tra le 18:45 e le 05:45 (estremi inclusi).")

In [0]:
train_size = int(len(df_np) * 0.9)
train_df_np = df_np.iloc[:train_size]
test_df_np = df_np.iloc[train_size:]

In [0]:
# Scaling dei dati
train_df_original = train_df_np.copy()
test_df_original = test_df_np.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_np['ds']
test_meta = test_df_np['ds']

feature_cols = [col for col in train_df_np.columns if col not in ['ds']]
train_idx = train_df_np.index
test_idx = test_df_np.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_np[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_np[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_np = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_np = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_np.shape, test_df_np.shape

In [0]:
# preparazione dataset
np_df_train = train_df_np[['ds', 'y']].copy()
np_df_test = test_df_np[['ds', 'y']].copy()

# Aggiungi le colonne dei regressori
regressors = ['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION',
       'giorno_settimana', 'ora_del_giorno', 'lag_1', 'lag_2',
       'lag_4', 'lag_24', 'lag_36', 'lag_48', 'lag_96', 'diff_1', 'diff_4',
       'diff_96', 'day_sin', 'day_cos', 'hour_sin', 'hour_cos']

for reg in regressors:
    np_df_train[reg] = train_df_np[reg]
    np_df_test[reg] = test_df_np[reg]
# Definisci il modello
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.001, 
    batch_size=32,
    daily_seasonality=True,
    weekly_seasonality=True,
    loss_func='Huber'
)

# Aggiungi i regressori
for reg in regressors:
    neuralprophet.add_future_regressor(reg)

TimeSeriesCV

In [0]:
# TIME SERIES CROSS VALIDATION
initial_train_size = 1223
h = 306  # dimensione test
n_windows = 5

# Verifica
required_length = initial_train_size + h * n_windows
if len(np_df_train) < required_length:
    raise ValueError("Dataset troppo corto per questo schema di cross-validation.")

# Reset risultati
results = []

for i in range(n_windows):
    print(f"\n--- Fold {i+1}/{n_windows} ---")

    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    train_window = np_df_train.iloc[:train_end]
    test_window = np_df_train.iloc[test_start:test_end]

    print(f"Train: {train_window.shape}, Test: {test_window.shape}")

    # Modello
    model = NeuralProphet(
        quantiles=[0.025, 0.975],
        learning_rate=0.001,
        batch_size=64,
        daily_seasonality=True,
        weekly_seasonality=True,
        loss_func='Huber',
        global_normalization=True,
        unknown_data_normalization=True
    )

    for reg in regressors:
        model.add_future_regressor(reg)

    # Fit
    _ = model.fit(train_window, freq="15min", epochs=100)

    # Previsione
    forecast = model.predict(test_window)

    # Metriche
    y_true = test_window['y'].values
    y_pred = forecast['yhat1'].values

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    results.append({'Fold': i+1, 'MAE_nprophet': mae, 'RMSE_nprophet': rmse})
    print(f"Split {i+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# Risultati
results_df_np = pd.DataFrame(results)
print("\n=== Backtest Results ===")
print(results_df_np)

# Plot
results_df_np.plot(x='Fold', y=['MAE_nprophet', 'RMSE_nprophet'], marker='o', title='Backtesting Errors')

In [0]:
# Estraggo date
# Crea lista per i dati delle fold
fold_data = []

for i in range(n_windows):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    # Ricava date da colonna 'ds' o dall'indice
    if 'ds' in np_df_train.columns:
        start_date = np_df_train['ds'].iloc[test_start]
        end_date = np_df_train['ds'].iloc[test_end - 1]
    else:
        start_date = np_df_train.index[test_start]
        end_date = np_df_train.index[test_end - 1]

    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]

    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)

# Converto le date in datetime se necessario
if all(isinstance(date, str) for date in fold_intervals_df['start_date']):
    fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
    fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])

print("Fold Intervals DataFrame:")
print(fold_intervals_df)

In [0]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_np['ds'], train_df_np['y'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

Now let's training on the whole dataset

In [0]:
# preparazione dataset
np_df_train = train_df_np[['ds', 'y']].copy()
np_df_test = test_df_np[['ds', 'y']].copy()

# Aggiungi le colonne dei regressori
regressors = ['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION',
       'giorno_settimana', 'ora_del_giorno', 'lag_1', 'lag_2',
       'lag_4', 'lag_24', 'lag_36', 'lag_48', 'lag_96', 'diff_1', 'diff_4',
       'diff_96', 'day_sin', 'day_cos', 'hour_sin', 'hour_cos']

for reg in regressors:
    np_df_train[reg] = train_df_np[reg]
    np_df_test[reg] = test_df_np[reg]
    

# Fisso il seed per riproducibilità
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


# Definisci il modello
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],
    learning_rate=0.005,
    batch_size=64,
    daily_seasonality=True,
    weekly_seasonality=True,
    loss_func='Huber',
)

# Aggiungi i regressori
for reg in regressors:
    neuralprophet.add_future_regressor(reg)

In [0]:
# FIT
start_time = time.time()
#fit 
metrics_df = neuralprophet.fit(np_df_train, 
                               freq="15min",
                               epochs=150,
                               )
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
forecast_test = neuralprophet.predict(np_df_test)
#salvo in pickle
#forecast_test.to_pickle('pickles/NP_solarEnergy.pkl')
forecast_test.tail() #last forecasts available

CARICO LE PREVISIONI DEL TEST SET

In [0]:
# carico le previsioni
#forecast_test = pd.read_pickle('pickles/NP_solarEnergy.pkl')

In [0]:
#Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
forecast_test['yhat1'] = forecast_test['yhat1'] * (max_val - min_val) + min_val
forecast_test['yhat1 2.5%'] = forecast_test['yhat1 2.5%'] * (max_val - min_val) + min_val
forecast_test['yhat1 97.5%'] = forecast_test['yhat1 97.5%'] * (max_val - min_val) + min_val

In [0]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))


# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast_test['ds'], 'forecast': forecast_test['yhat1']})
merged_df = pd.merge(test_df_original[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
mae_val = mean_absolute_error(merged_df['y'], merged_df['forecast'])
metriche_np = calcola_metriche(merged_df['y'], merged_df['forecast'],train_df_original['y'],
                               modelname="NeuralProphet").round(10)
metriche_np

#### TFT (H=1, ricorsivo)

In [0]:
# Preparing dataset
df_tft = create_time_series_features(df, target_col='y') 
df_tft = df_tft.dropna().reset_index(drop=True)
# 2. Suddivisione in train/test
train_size = int(len(df_tft) * 0.9)
train_df_tft = df_tft.iloc[:train_size]
test_df_tft = df_tft.iloc[train_size:]
train_df_tft.shape, test_df_tft.shape

In [0]:
# Scaling dei dati
#Salvo per dopo
train_df_original = train_df_tft.copy() 
test_df_original = test_df_tft.copy()

# Salvo le colonne da riattaccare
train_df_tft['unique_id'] = 'Serie unica'
test_df_tft['unique_id'] = 'Serie unica'
train_meta = train_df_tft[['ds', 'unique_id']]
test_meta = test_df_tft[['ds', 'unique_id']]

feature_cols = [col for col in train_df_tft.columns if col not in ['ds', 'unique_id']]
train_idx = train_df_tft.index
test_idx = test_df_tft.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_tft[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_tft[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_tft = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_tft = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_tft.shape, test_df_tft.shape

##### TimeSeriesCV - fine tuning Optuna (LENTO)

In [0]:
# Optuna function
futr_exog_list = train_df_tft.drop(columns=['unique_id', 'ds', 'y']).columns.tolist()
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_tft = pd.DataFrame()
h = len(test_df_tft)

def objective(trial):
    # Hyperparameters to optimize for TFT
    input_size = trial.suggest_categorical("input_size", [h, int(1.5*h), int(2*h)])
    hidden_size = trial.suggest_categorical("hidden_size", [32, 64, 128, 256])
    n_rnn_layers = trial.suggest_int("n_rnn_layers", 1, 4)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    grn_activation = trial.suggest_categorical("grn_activation", ["ReLU", "ELU", "Sigmoid"])
    
    # Define TFT model
    tft = TFT(
        h=h,  # forecast horizon
        input_size=input_size,
        hidden_size=hidden_size,
        n_rnn_layers=n_rnn_layers,
        rnn_type='lstm',
        grn_activation=grn_activation,
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=learning_rate,
        futr_exog_list=futr_exog_list,
        max_steps=50, #aumentare a 100
        val_check_steps=10,
        batch_size=batch_size,
        early_stop_patience_steps=-1,
        scaler_type="robust",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=64
)

    nf = NeuralForecast(models=[tft], freq='15min')

    # Cross-validation for tuning
    df_cv = nf.cross_validation(
        df=train_df_tft,
        step_size=h,
        n_windows=5
    )

    # Calculate MAE for each fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'TFT']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'TFT',
            'MAE_tft': mae,
            'RMSE_tft': rmse
        })

    # Average MAE across all folds
    cv_metrics_df_tft = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_tft['MAE_tft'].mean()

    return mean_mae

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)  #aumentare a 50 su databricks

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params
#salvo i best params in pickle
# with open('pickles/TFT_bestPars_SolarEnergy.pkl', 'wb') as file:
#     pickle.dump(best_params, file)

In [0]:
# #carico i best params
with open('pickles/TFT_bestPars_SolarEnergy.pkl', 'rb') as file:
    best_params = pickle.load(file)

La CV è troppo lenta, si blocca!

In [0]:
# final CV

# final model
best_tft = TFT(
        h=h,  
        input_size=int(best_params['input_size']/2),
        hidden_size=best_params['hidden_size'],
        n_rnn_layers=best_params['n_rnn_layers'],
        rnn_type='lstm',
        grn_activation=best_params['grn_activation'],
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=best_params['learning_rate'],
        futr_exog_list=futr_exog_list,
        max_steps=50, #aumentare a 100
        val_check_steps=10,
        batch_size=best_params['batch_size'],
        early_stop_patience_steps=-1,
        scaler_type="robust",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=64
)
nf = NeuralForecast(models=[best_tft],freq="15min")
# final CV
df_cv = nf.cross_validation(
    df=train_df_tft[1100:],
    step_size=int(len(test_df_tft)/3),
    n_windows=5
)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'TFT']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'TFT',
        'MAE_TFT': mae,
        'RMSE_TFT': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_tft = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_tft)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_tft.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_cv_metrics_df_SolarEnergy.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(cv_metrics_df_tft, f)

In [0]:
# Carico la cv
with open('pickles/TFT_cv_metrics_df_SolarEnergy.pkl', 'rb') as file:
    cv_metrics_df_tft = pickle.load(file)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_tft['ds'], train_df_tft['y'], label='Serie unica')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_tft['Model'] = 'TFT'
# Plot MAE sul primo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_TFT'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_TFT'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


##### Final training

In [0]:
# model fit, LENTO
h= len(test_df_tft)
nf = NeuralForecast(
    models=[
        TFT(
            h=h,
            input_size=int(h/1.5),
            hidden_size=4,  # Deve essere divisibile per 4
            grn_activation="ReLU",  # Prima era ELU
            rnn_type="lstm",
            n_rnn_layers=1,  # Consigliato fra 2 e 5
            one_rnn_initial_state=False,
            loss=DistributionLoss(distribution="StudentT", level=[90]),
            learning_rate=0.01,
            hist_exog_list=train_df_tft.drop(columns=['unique_id', 'ds', 'y']).columns.tolist(),
            #futr_exog_list=future_features,
            max_steps=100,
            val_check_steps=10,
            batch_size=4,
            early_stop_patience_steps=15,
            scaler_type="standard",
            enable_progress_bar=True,
            accelerator="auto",
        ),
    ],
    freq='15min',
)
import time
start_time = time.time()
#fit 
nf.fit(df=train_df_tft[1000:], val_size=h)

end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Predict
Y_hat_df_tft = nf.predict(futr_df=test_df_tft)

In [0]:
# #salvo le prediction
# with open('pickles/TFT_SolarEnergy_multistep.pkl', 'wb') as file:
#     pickle.dump(Y_hat_df_tft, file)

In [0]:
# # #carico le prediction
with open('pickles/TFT_SolarEnergy.pkl', 'rb') as file:
    Y_hat_df_tft = pickle.load(file)

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
Y_hat_df_tft['TFT'] = Y_hat_df_tft['TFT'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-median'] = Y_hat_df_tft['TFT-median'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-lo-90'] = Y_hat_df_tft['TFT-lo-90'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-hi-90'] = Y_hat_df_tft['TFT-hi-90'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_tft['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_tft['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='lightgreen')))

# QUANTILI TFT
# Fascia tra P10 e P90
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_tft['ds']) + 
    list(Y_hat_df_tft['ds'][::-1]),
    y=list(Y_hat_df_tft['TFT-hi-90']) + list(Y_hat_df_tft['TFT-lo-90'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="TFT forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_tft['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# Calcola metriche
merged_df_tft = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_tft, on='ds', how='left')
merged_df_tft.rename(columns={'TFT-lo-90': '0.1','TFT-median':'0.5', 'TFT-hi-90':'0.9'},inplace=True)
merged_df_tft.dropna(inplace=True)
metriche_tft = calcola_metriche(merged_df_tft['y'],merged_df_tft['TFT'],train_df_original['y'],
                                y_pred_quantiles=merged_df_tft[['0.1','0.5','0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="TFT").round(10)
metriche_tft

##### TFT Explainability

In [0]:
attention = nf.models[0].attention_weights()
def plot_attention(
    self, plot: str = "time", output: str = "plot", width: int = 800, height: int = 400
):
    """
    Plot the attention weights.

    Args:
        plot (str, optional): The type of plot to generate. Can be one of the following:
            - 'time': Display the mean attention weights over time.
            - 'all': Display the attention weights for each horizon.
            - 'heatmap': Display the attention weights as a heatmap.
            - An integer in the range [1, model.h) to display the attention weights for a specific horizon.
        output (str, optional): The type of output to generate. Can be one of the following:
            - 'plot': Display the plot directly.
            - 'figure': Return the plot as a figure object.
        width (int, optional): Width of the plot in pixels. Default is 800.
        height (int, optional): Height of the plot in pixels. Default is 400.

    Returns:
        matplotlib.figure.Figure: If `output` is 'figure', the function returns the plot as a figure object.
    """

    attention = (
        self.mean_on_batch(self.interpretability_params["attn_wts"])
        .mean(dim=0)
        .cpu()
        .numpy()
    )

    fig, ax = plt.subplots(figsize=(width / 100, height / 100))

    if plot == "time":
        attention = attention[self.input_size :, :].mean(axis=0)
        ax.plot(np.arange(-self.input_size, self.h), attention)
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Mean Attention")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "all":
        for i in range(self.input_size, attention.shape[0]):
            ax.plot(
                np.arange(-self.input_size, self.h),
                attention[i, :],
                label=f"horizon {i-self.input_size+1}",
            )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Attention per horizon")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "heatmap":
        cax = ax.imshow(
            attention,
            aspect="auto",
            cmap="viridis",
            extent=[-self.input_size, self.h, -self.input_size, self.h],
        )
        fig.colorbar(cax)
        ax.set_title("Attention Heatmap")
        ax.set_xlabel("Attention (current time step)")
        ax.set_ylabel("Attention (previous time step)")

    elif isinstance(plot, int) and (plot in np.arange(1, self.h + 1)):
        i = self.input_size + plot - 1
        ax.plot(
            np.arange(-self.input_size, self.h),
            attention[i, :],
            label=f"horizon {plot}",
        )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title(f"Attention weight for horizon {plot}")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    else:
        raise ValueError(
            'plot has to be in ["time","all","heatmap"] or integer in range(1,model.h)'
        )

    plt.tight_layout()

    if output == "plot":
        plt.show()
    elif output == "figure":
        return fig
    else:
        raise ValueError(f"Invalid output: {output}. Expected 'plot' or 'figure'.")

In [0]:
# MEAN ATTENTION
plot_attention(nf.models[0], plot="time")
# pesi di attenzione medi nel tempo su tutti gli orizzonti di previsione.
# La curva blu rappresenta quanto il modello ha "guardato" (attention) 
# quel punto temporale in input durante la previsione.
# Valori più alti significano che il modello ritiene quel punto temporale
# particolarmente rilevante per fare le previsioni

In [0]:
# ATTENTION OF ALL FUTURE TIME STEPS
plot_attention(nf.models[0], plot="all")

In [0]:
# ATTENTION OF A SPECIFIC TIME STEPS
plot_attention(nf.models[0], plot=44)
# pesi di attenzione per ogni orizzonte di previsione separatamente.

In [0]:
plot_attention(nf.models[0], plot="heatmap")

In [0]:
# PAST VARIABLE IMPORTANCES
feature_importances = nf.models[0].feature_importances()
print(feature_importances.keys())
print("\n ==========PAST FEATURE IMPORTANCES==========")
feature_importances["Past variable importance over time"].mean().sort_values().plot(kind="barh")

Questo grafico mostra quanto il modello ha fatto affidamento sulle feature passate \
(quelle che il modello ha potuto osservare prima del punto di previsione). Serve a costruire il contesto dell'input

In [0]:
# FUTURE VARIABLE IMPORTANCES
print("\n ==========FUTURE FEATURE IMPORTANCES==========")
feature_importances["Future variable importance over time"].mean().sort_values().plot(kind="barh")

Mostra quanto il modello ha sfruttato le feature che sono servite a guidare le previsioni passo-passo nel decoder

DIFFERENZA TRA PAST E FUTURE: \
La diversa importanza riflette quanto ciascuna variabile aiuta nell’analisi del contesto passato (encoder -->> PAST) \
o nella generazione delle previsioni (decoder -->> FUTURE).

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME
df_importances = feature_importances["Past variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances.index))

for col in df_importances.columns:
    p = ax.bar(np.arange(-len(df_importances), 0), df_importances[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances[col]
ax.set_title("Past variable importance over time")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)

plt.show()

Questo grafico mostra QUANDO (e QUANTO) nel passato ogni feature è stata più utile per costruire il contesto predittivo.

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME
df_importances2 = feature_importances["Future variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances2.index))
for col in df_importances2.columns:
    p = ax.bar(np.arange(-len(df_importances2), 0), df_importances2[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances2[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.grid(True)
ax.legend()
plt.show()

Per ogni feature viene mostrato quanto è stata importante per il modello a ciascun orizzonte futuro

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances3 = feature_importances["Past variable importance over time"]
mean_attention = (
    nf.models[0]
    .attention_weights()[nf.models[0].input_size :, :]
    .mean(axis=0)[: nf.models[0].input_size]
)
df_importances3 = df_importances3.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances3.index))

for col in df_importances3.columns:
    p = ax.bar(np.arange(-len(df_importances3), 0), df_importances3[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances3[col]
ax.set_title("Past variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances3), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances4 = feature_importances["Future variable importance over time"]
mean_attention = (
    nf.models[0]
    .attention_weights()[nf.models[0].input_size :, :]
    .mean(axis=0)[: nf.models[0].input_size]
)
df_importances4 = df_importances4.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances4.index))

for col in df_importances4.columns:
    p = ax.bar(np.arange(-len(df_importances4), 0), df_importances4[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances4[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances4), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
nf.models[0].feature_importance_correlations()

#### CHRONOS

In [0]:
# Preparing dataset
df_chronos = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos = df_chronos.dropna().reset_index(drop=True) 
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target','unique_id':'item_id'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)
df_chronos.columns

In [0]:
# Split train e test
known_covariates_names = [
    'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION',
       'giorno_settimana', 'ora_del_giorno', 'lag_1', 'lag_2', 'lag_4',
       'lag_24', 'lag_36', 'lag_48', 'lag_96', 'diff_1', 'diff_4', 'diff_96',
       'day_sin', 'day_cos', 'hour_sin', 'hour_cos'
]

# Split train e test
train_size = int(len(df_chronos) * 0.9)
train_df_chronos = df_chronos.iloc[:train_size]
test_df_chronos = df_chronos.iloc[train_size:]

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [0]:
# Scaling dei dati
train_df_original = train_df_chronos.copy() 
test_df_original = test_df_chronos.copy()

train_idx = train_df_chronos.index
test_idx = test_df_chronos.index

feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

#scaler
scaler_x = MinMaxScaler()
train_df_chronos = pd.DataFrame(scaler_x.fit_transform(train_df_chronos[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_chronos = pd.DataFrame(scaler_x.transform(test_df_chronos[feature_cols]),
                            columns=feature_cols, index=test_idx)
#controllo le shape
train_df_chronos.shape, test_df_chronos.shape

TimeSeriesCV

In [0]:
#TimeSeriesCV
h = len(test_df_chronos)  # prediction length
n_splits = 5
initial_train_size = len(train_df_chronos) - 5*h
results = []

for fold in range(n_splits):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")
    
    train_end = initial_train_size + fold * h
    test_start = train_end
    test_end = test_start + h
    train_df_fold = train_df_chronos.iloc[:train_end]
    test_df_fold = train_df_chronos.iloc[test_start:test_end]
    
    train_df_fold.reset_index(inplace=True)
    test_df_fold.reset_index(inplace=True)
    
    train_df_fold["timestamp"] = pd.to_datetime(train_df_fold["timestamp"])
    test_df_fold["timestamp"] = pd.to_datetime(test_df_fold["timestamp"])
    
    # Future timestamps 
    future_index = pd.date_range(
        start=train_df_fold["timestamp"].max() + pd.Timedelta("15min"),
        periods=h,
        freq="15min")
    
    # Preparo test set con known_covariates
    test_df_for_prediction = test_df_fold[test_df_fold["timestamp"].isin(future_index)].copy()
    test_df_for_prediction = test_df_for_prediction.set_index("timestamp")
    test_df_for_prediction = test_df_for_prediction.reindex(future_index)  # forza continuità
    test_df_for_prediction["item_id"] = train_df_fold["item_id"].iloc[0]  # supponiamo 1 sola serie
    test_df_for_prediction = test_df_for_prediction.reset_index().rename(columns={"index": "timestamp"})
    test_df_for_prediction = test_df_for_prediction.set_index(["item_id", "timestamp"])
    
    # Preparo il train con multindex
    train_df_fold = train_df_fold.set_index(["item_id", "timestamp"])
    print(f"Train: {train_df_fold.shape}, Test: {test_df_fold.shape}")
    
    # inizializzo il predictor
    predictor = TimeSeriesPredictor(
        prediction_length=h,
        target="target",
        known_covariates_names=known_covariates_names,
        eval_metric="MASE",
        freq="15min")
    
    # fit del modello
    predictor.fit(
        train_df_chronos,
        hyperparameters={
            "Chronos": [
                {
                    "model_path": "bolt_small",
                    "ag_args": {"name_suffix": "ZeroShot"}},
                {
                    "model_path": "bolt_small",
                    "covariate_regressor": "CAT",
                    "target_scaler": "robust",
                    "ag_args": {"name_suffix": "WithRegressor"},
                },
            ],
        },
        time_limit=600,
        enable_ensemble=True,  # Attiva ensemble per migliorare l'accuratezza
    )
    
    predictions = predictor.predict(train_df_fold, known_covariates=test_df_for_prediction)

    test_df_with_index = test_df_fold.set_index(["item_id", "timestamp"])

    # Debug
    print(f"Forma predictions: {predictions.shape}")
    print(f"Colonne predictions: {predictions.columns}")
    print(f"Numero di indici in predictions: {len(predictions.index)}")
    print(f"Numero di indici in test_df_with_index: {len(test_df_with_index.index)}")

    # trovo solo gli indici comuni
    common_indices = predictions.index.intersection(test_df_with_index.index)
    print(f"Indici comuni: {len(common_indices)}")

    # prendo solo gli indici comuni
    y_true = test_df_with_index.loc[common_indices, "target"]
    y_pred = predictions.loc[common_indices, 'mean'].to_numpy()

    mae = calcola_mae(y_true, y_pred)
    rmse = calcola_rmse(y_true, y_pred)
    
    results.append({'Fold': fold+1, 'MAE_chronos': mae, 'RMSE_chronos': rmse})
    print(f"Split {fold+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
# risultati finali
results_df_chronos = pd.DataFrame(results)
print("\n=== TimeSeriesCV Results ===")
results_df_chronos

In [0]:
# Estraggo le date
fold_data = []
train_df_chronos.reset_index(inplace=True)
for i in range(n_splits):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h
    
    # Ricava date da colonna 'ds' o dall'indice
    if 'timestamp' in train_df_chronos.columns:
        start_date = train_df_chronos['timestamp'].iloc[test_start]
        end_date = train_df_chronos['timestamp'].iloc[test_end - 1]
    else:
        start_date = train_df_chronos.index[test_start]
        end_date = train_df_chronos.index[test_end - 1]
    
    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]
    
    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)
fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])
# Stampa il dataframe prima della conversione per verificare i valori
print("Fold Intervals DataFrame (before conversion):")
fold_intervals_df

In [0]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_chronos['timestamp'], train_df_chronos['target'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot degli errori
ax = results_df_chronos.plot(x='Fold', y=['MAE_chronos', 'RMSE_chronos'], marker='o', title='Backtesting Errors')
ax.set_xticks(range(1, 6)) # Imposta le tacche sull'asse x da 1 a 5
plt.show()

Let's training on the whole dataset

In [0]:
# Fit del modello

# SOLO PER DATABRICKS
# sys.modules['sklearn.metrics._regression'].mean_absolute_error = sklearn.metrics.mean_absolute_error

predictor = TimeSeriesPredictor(
    prediction_length=len(test_df_chronos), 
    eval_metric="MASE",
    target="target",
    known_covariates_names=known_covariates_names,
    freq="15min"
)

train_df_chronos = TimeSeriesDataFrame(
    train_df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Aggiunta di più modelli nella configurazione per migliorare le performance
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {
                "model_path": "bolt_small",
                "ag_args": {"name_suffix": "ZeroShot"}},
            {
                "model_path": "bolt_small",
                "covariate_regressor": "CAT",
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            }
        ],
    },
    time_limit=600,
    enable_ensemble=True,  # Attiva ensemble per migliorare l'accuratezza
)

# Valutazione del modello in fase di training
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [0]:
# PREVISIONI 1 CLASSICHE
predictions = predictor.predict(
    train_df_chronos, 
    known_covariates=test_df_chronos
)
predictions.reset_index(inplace=True)
#prediction 0 shot
predictions_0shot = predictor.predict(
    train_df_chronos, 
    known_covariates=test_df_chronos,
    model="ChronosZeroShot[bolt_small]"
)
predictions_0shot.reset_index(inplace=True)

In [0]:
# # PREVISIONI 2 MULTISTEP DIRETTE ????
# # Separo le covariate in due gruppi
# known_ahead_covariates = ['giorno_settimana', 'ora_del_giorno']  # Queste sono realmente conosciute in anticipo

# lagged_covariates = [
#     'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION',
#        'lag_1', 'lag_2', 'lag_4',
#        'lag_24', 'lag_36', 'lag_48', 'lag_96', 'diff_1', 'diff_4', 'diff_96',
#        'day_sin', 'day_cos', 'hour_sin', 'hour_cos', 'rolling_mean_4',
#        'rolling_mean_48', 'rolling_mean_96']
# # Configurazione del predictor
# predictor = TimeSeriesPredictor(
#     prediction_length=len(test_df_chronos),  
#     eval_metric="MASE",
#     target="target",
#     known_covariates_names=known_ahead_covariates,
#     freq="15min"
# )

# test_known_covariates = test_df_chronos[known_ahead_covariates]

# # Fit del modello
# predictor.fit(
#     train_df_chronos,
#     hyperparameters={
#         "Chronos": [
#             {
#                 "model_path": "bolt_small",
#                 "ag_args": {"name_suffix": "ZeroShot"}
#             },
#             {
#                 "model_path": "bolt_small",
#                 "covariate_regressor": "CAT",
#                 "target_scaler": "robust",
#                 "ag_args": {"name_suffix": "WithRegressor"},
#             },
#         ],
#     },
#     time_limit=600,
#     enable_ensemble=True,
# )

# # Previsioni appropriate al contesto
# # Per un multi-step puro senza feedback
# predictions = predictor.predict(
#     train_df_chronos,
#     known_covariates=test_known_covariates  # Solo le covariate realmente note in anticipo
# )
# predictions.reset_index(inplace=True)

# predictions_0shot = predictor.predict(
#     train_df_chronos,
#     known_covariates=test_known_covariates,
#     model="ChronosZeroShot[bolt_small]"
# )
# predictions_0shot.reset_index(inplace=True)

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('target')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
predictions['mean'] = predictions['mean'] * (max_val - min_val) + min_val
predictions['0.1'] = predictions['0.1'] * (max_val - min_val) + min_val
predictions['0.5'] = predictions['0.5'] * (max_val - min_val) + min_val
predictions['0.9'] = predictions['0.9'] * (max_val - min_val) + min_val
predictions_0shot['mean'] = predictions_0shot['mean'] * (max_val - min_val) + min_val
predictions_0shot['0.5'] = predictions_0shot['0.5'] * (max_val - min_val) + min_val
predictions_0shot['0.9'] = predictions_0shot['0.9'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=train_df_chronos['timestamp'], y=train_df_original['target'], mode='lines', name='Training Data'))
# Dati di test

fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='With Regressors', line=dict(color='green')))
# PREVISIONI CHRONOS 0-shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='0-shot Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="CHRONOS forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_chronos['timestamp'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# metriche di valutazione
test_df_original.reset_index(inplace=True)
merged_df_chronos = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']],
        on=['timestamp', 'item_id']
)
metriche_chronos = calcola_metriche(merged_df_chronos['target'],merged_df_chronos['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos").round(5)
metriche_chronos

In [0]:
# metriche di valutazione 0 shot
merged_df_chronos_0shot = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions_0shot[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']],
        on=['timestamp', 'item_id']
)
metriche_chronos_0shot = calcola_metriche(merged_df_chronos_0shot['target'],merged_df_chronos_0shot['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos_0shot[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos 0 shot").round(5)
metriche_chronos_0shot

#### final evaluations

In [0]:
final_metrics = metriche_arima.join(metriche_ets).join(metriche_xgb).join(metriche_lstm).join(metriche_np).join(metriche_chronos).join(metriche_chronos_0shot).join(metriche_tft)
final_metrics

In [0]:
# FINAL PREDICTIONS PLOT FOR SOLAR ENERGY
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoARIMA'], 
    mode='lines', 
    name='ARIMA', 
    line=dict(color='grey')
))

# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoETS'], 
    mode='lines', 
    name='ETS', 
    line=dict(color='orange')
))

# PREVISIONI XGBOOST
fig.add_trace(go.Scatter(
    x=forecast_xgb['ds'], 
    y=forecast_xgb['forecast'], 
    mode='lines', 
    name='XGBoost', 
    line=dict(color='purple')
))

# PREVISIONI LSTM
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=Y_hat_df_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='pink')
))

# PREVISIONI NEURAL PROPHET
# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='aquamarine')
))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='yellow')))

# PREVISIONI CHRONOS
test_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS', line=dict(color='green')))
    
 # PREVISIONI CHRONOS 0 shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='CHRONOS 0-SHOT', line=dict(color='lightgreen')))
   
# Layout del grafico
fig.update_layout(
    title="Forecasts for White Noise", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
#sistemo i vari dataset prima del merge
cv_metrics_df_lstm.drop(columns=('Model'),inplace=True)
df_arima = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == 'AutoARIMA'][['Fold',
                                                                    'MAE_AutoARIMA','RMSE_AutoARIMA']]
df_ets = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == 'AutoETS'][['Fold',
                                                                    'MAE_AutoETS','RMSE_AutoETS']]
results_df_np.rename(columns={'split': 'Fold'}, inplace=True)
results_df_chronos.rename(columns={'split': 'Fold','MAE':'MAE_chronos','RMSE':'RMSE_chronos'}, inplace=True)
cv_metrics_df_xgboost.rename(columns={'MAE': 'MAE_XGB','RMSE':'RMSE_XGB'}, inplace=True)
df_arima.rename(columns={'MAE':'MAE_AutoARIMA','RMSE':'RMSE_AutoARIMA'}, inplace=True)
df_ets.rename(columns={'MAE':'MAE_AutoETS','RMSE':'RMSE_AutoETS'}, inplace=True)

In [0]:
# Final plot metrics TimeSeriesCV

# faccio il merge di tutte le tabelle dei vari CV
final_cv = df_ets.merge(cv_metrics_df_lstm, 
                            on="Fold",how="inner").merge(results_df_chronos, 
                            on="Fold",how="inner").merge(cv_metrics_df_xgboost[['MAE_XGB','RMSE_XGB','Fold']],
                            on="Fold",how="inner").merge(df_arima,
                            on="Fold",how="inner").merge(results_df_np,
                            on="Fold", how="inner").merge(cv_metrics_df_tft.drop(columns=['Model']),
                            on="Fold",how="inner")
# Rescaling CV metrics data
columns_to_modify = ['MAE_nprophet', 'RMSE_nprophet', 'MAE_TFT', 'RMSE_TFT',
                        'MAE_chronos', 'RMSE_chronos', 'MAE_XGB', 'RMSE_XGB']
for col in columns_to_modify:
    final_cv[f'{col}'] = final_cv[f'{col}'] * (max_val - min_val) + min_val

model_colors = {
    'AutoARIMA': '#808080',      # Grey
    'AutoETS': '#ff7f0e',        # Orange
    'XGB': '#9467bd',    # Purple
    'lstm': '#e377c2',       # Pink
    'nprophet': '#7fffd4',         # Aquamarine
    'TFT': '#ffff00',        # Yellow
    'chronos': '#2ca02c',    # Green
}


# Create the figure
fig = go.Figure()

# Add traces for each model - MAE (darker) and RMSE (lighter)
for model, color in model_colors.items():
    # Add MAE line (darker)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'MAE_{model}'],
        mode='lines+markers',
        name=f'{model} MAE',
        line=dict(color=color, width=3),
        marker=dict(size=8)
    ))
    
    # Add RMSE line (lighter with same color but different dash pattern)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'RMSE_{model}'],
        mode='lines+markers',
        name=f'{model} RMSE',
        line=dict(color=color, width=2, dash='dash'),
        marker=dict(size=6)
    ))

# Update layout
fig.update_layout(
    title='Model Performance Comparison (MAE and RMSE)',
    template='plotly_dark',
    xaxis=dict(title='Fold',
        tickmode='linear',
        tick0=1, dtick=1
    ),    yaxis=dict(
        title='Error Value'
    ),    legend=dict(
        orientation="v"
    ),    hovermode="closest")
fig.show()

In [0]:
# Confronto TFT/CHRONOS
tft_chronos = metriche_chronos.join(metriche_tft)
tft_chronos                                    

### **SOLAR ENERGY (MULTISTEP)**

In [ ]:
futr_exog_list = ['ora_del_giorno', 'giorno_settimana', 'day_sin','day_cos','hour_sin','hour_cos']
future_features = ['ds', 'y'] + futr_exog_list

In [ ]:
#preparazione dataset
df = df_solar.drop(columns=['AC_POWER'])
df = df.reset_index().rename(columns={'DATE_TIME': 'ds','DC_POWER':'y'})
df = df.dropna()
df['unique_id'] = 'serie_1'
h= 192

In [ ]:
# mancano due righe di notte, le fillo easy
row_to_copy = df[df['ds'] == '2020-06-17 06:00:00']
# Crea copie modificate per le 06:15 e 06:30
row_0615 = row_to_copy.copy()
row_0615['ds'] = pd.to_datetime('2020-06-17 06:15:00')

row_0630 = row_to_copy.copy()
row_0630['ds'] = pd.to_datetime('2020-06-17 06:30:00')

df = pd.concat([df, row_0615, row_0630], ignore_index=True)
df['ds'] = pd.to_datetime(df['ds'])
df = df.sort_values(by='ds').reset_index(drop=True)

In [ ]:
# Controllo gli zeri nel dataframe
df['ds'] = pd.to_datetime(df['ds'])

# 2. Definisci gli orari di interesse
start_time = pd.to_datetime('18:45').time()
end_time = pd.to_datetime('05:45').time()

# 3. Crea una maschera booleana per selezionare le righe nell'intervallo orario
mask = (df['ds'].dt.time >= start_time) | (df['ds'].dt.time <= end_time)

# 4. Applica la maschera al DataFrame per ottenere le righe nell'intervallo orario
df_notturno = df[mask]

# 5. Verifica se ci sono valori diversi da zero nella colonna 'y' nel subset notturno
valori_non_zero_notte = df_notturno[df_notturno['y'] != 0]

# 6. Stampa il risultato
if not valori_non_zero_notte.empty:
    print("Sono stati trovati valori diversi da zero nella colonna 'y' tra le 18:45 e le 05:45 (estremi inclusi):")
    print(valori_non_zero_notte)
else:
    print("Non sono stati trovati valori diversi da zero nella colonna 'y' tra le 18:45 e le 05:45 (estremi inclusi).")

In [ ]:
# funzione per creare feature
def create_time_series_features(df, target_col):
    df = df.copy()
    lags = [1, 2, 4, 24, 36, 48, 96]
    # Lag features (1-4)
    for lag in lags:
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    
    # Differencing features, 1, 4, 96
    df['diff_1'] = (df[target_col] - df[target_col].shift(1)).shift(1) #diff tra t-1 e t-2
    df['diff_4'] = (df[target_col] - df[target_col].shift(4)).shift(1) #diff tra t-1 e t-5
    df['diff_96'] = (df[target_col] - df[target_col].shift(96)).shift(1) #diff tra t-1 e t-97
    
    # Caratteristiche cicliche per rappresentare meglio la stagionalità
    df['day_sin'] = np.sin(2 * np.pi * df['giorno_settimana']/7)
    df['day_cos'] = np.cos(2 * np.pi * df['giorno_settimana']/7)
    df['hour_sin'] = np.sin(2 * np.pi * df['ora_del_giorno']/24)
    df['hour_cos'] = np.cos(2 * np.pi * df['ora_del_giorno']/24)
    
    # rolling means 
    df['rolling_mean_4'] = df[target_col].rolling(window=4).mean().shift(4) #ultima ora
    df['rolling_mean_48'] = df[target_col].rolling(window=6).mean().shift(1) #ultime 12 ore
    df['rolling_mean_96'] = df[target_col].rolling(window=12).mean().shift(1) #ultimo giorno
    
    return df

##### XGBOOST

In [ ]:
# df_xgb
df_xgb = create_time_series_features(df, target_col='y') 
df_xgb.drop(columns=['unique_id'],inplace=True)
df_xgb = df_xgb.dropna().reset_index(drop=True) #droppo i NaN creati dai lag
hist_exog_features = df_xgb.drop(columns=['ds', 'y'] + futr_exog_list).columns.tolist()
df_xgb = df_xgb[future_features]
print(df_xgb.columns)
df_xgb.head()

In [ ]:
# Split train/test
train_df_xgb = df_xgb.iloc[:-h]
test_df_xgb = df_xgb.iloc[-h:]

# Separazione features e target
feature_cols = [col for col in df_xgb.columns if col not in ['y', 'ds']]
X_train_xgb, y_train_xgb = train_df_xgb[feature_cols], train_df_xgb['y']
X_test_xgb, y_test_xgb = test_df_xgb[feature_cols], test_df_xgb['y']
X_train_xgb.shape, y_train_xgb.shape, X_test_xgb.shape, y_test_xgb.shape

In [ ]:
# train e test plot
fig = go.Figure()
#train dat
fig.add_trace(go.Scatter(x=df_xgb['ds'], y=y_train_xgb, mode='lines', name='Train'))
#test data 
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=y_test_xgb, mode='lines', name='Test'))
#layout
fig.update_layout(template='plotly_dark', title='Train and Test Data Plot')
fig.show()

In [ ]:
# Scaling dei dati

#features
scaler_x = MinMaxScaler()
X_train_scaled = scaler_x.fit_transform(X_train_xgb)
X_test_scaled = scaler_x.transform(X_test_xgb)

# target, non serve che scalo y_test
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train_xgb.values.reshape(-1, 1))

# Ricreo df con stessi indici e nomi
X_train_xgb = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train_xgb.index)
X_test_xgb = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test_xgb.index)
y_train_xgb = pd.Series(y_train_scaled.flatten(), name='y', index=y_train_xgb.index)

#aggiorno train_df_xgb
train_df_xgb[feature_cols] = X_train_xgb
train_df_xgb['y'] = y_train_xgb

#controllo le shape
X_train_xgb.shape, X_test_xgb.shape, y_train_xgb.shape, y_test_xgb.shape

Ottimizzazione Optuna (Bayesiana)

In [ ]:
# Optuna function
cv_metrics_log = []  # List globale per salvare metriche fold per fold

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'verbosity': 0,
        'seed': 42
    }

    tscv = TimeSeriesSplit(n_splits=5,test_size=len(test_df_xgb))
    y = train_df_xgb['y']
    df_xgb_feature = train_df_xgb.drop(columns=['y','ds'])
    all_rmse = []

    for fold, (train_idx, test_idx) in enumerate(tscv.split(df_xgb_feature)):
        X_train, X_test = df_xgb_feature.iloc[train_idx], df_xgb_feature.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        dtest = xgb.DMatrix(X_test.values, label=y_test.values)

        model = xgb.train(params, dtrain, num_boost_round=500,
                          evals=[(dtest, 'eval')],
                          early_stopping_rounds=30, verbose_eval=False)

        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mae = calcola_mae(y_test, preds)  # tua funzione custom

        all_rmse.append(rmse)

        # Recupera le date (se esiste colonna 'ds')
        start_date = df_xgb.iloc[test_idx]['ds'].min() if 'ds' in df_xgb.columns else None
        end_date = df_xgb.iloc[test_idx]['ds'].max() if 'ds' in df_xgb.columns else None

        # Logga le metriche della fold
        cv_metrics_log.append({
            'Trial': trial.number,
            'Fold': fold + 1,
            'MAE_XGB': mae,
            'RMSE_XGB': rmse,
            'start_date': start_date,
            'end_date': end_date,
            'Model': 'XGBoost'  # Puoi modificarlo se usi più modelli
        })
    return np.mean(all_rmse)

In [ ]:
# Optuna optimization
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50) #aumentare su databricks

In [ ]:
# Best parameters
print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)
print("Best trial:",study.best_trial.number)
besttrial = study.best_trial.number

In [ ]:
cv_metrics_log = pd.DataFrame(cv_metrics_log)
cv_metrics_log = cv_metrics_log[cv_metrics_log['Trial']==besttrial]
cv_metrics_log

Vediamo come ha performato nel miglior trial (best parameters)

In [ ]:
#PLOT CV
cv_metrics_df_xgboost = cv_metrics_log.copy()

# Stampa le metriche per modello
print("Metriche per modello:")
print(cv_metrics_df_xgboost)

# Calcola e stampa le metriche medie per modello
mean_metrics = cv_metrics_df_xgboost.mean(numeric_only=True).reset_index()

# Crea il DataFrame con gli intervalli delle fold
fold_intervals_df = cv_metrics_df_xgboost[['Fold', 'start_date', 'end_date']].drop_duplicates()

# Ora creiamo il grafico combinato
plt.figure(figsize=(16, 12))

# === PLOT 1: Serie temporale originale ===
plt.subplot(2, 1, 1)
#for unique_id in train_df_xgb['unique_id'].unique():
series_data = train_df_xgb.copy()
plt.plot(series_data['ds'], series_data['y'], label=f'Solar energy')

# Evidenzia le fold con colori
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)],
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10,
             bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# === PLOT 2: Metriche MAE e RMSE ===
plt.subplot(2, 1, 2)
ax1 = plt.gca()
ax2 = ax1.twinx()

# MAE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_XGB'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# RMSE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_XGB'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Legenda combinata
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


Training coi migliori parametri trovati da Optuna

In [ ]:
# Aggiorniamo il modello con i best params
best_params = study.best_params
best_params.update({'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 42})
import time
dtrain_xgb = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
start_time = time.time()
#fit 
model = xgb.train(best_params, dtrain_xgb, num_boost_round=500)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [ ]:
# Plot feature importance
xgb.plot_importance(model, importance_type='gain', max_num_features=10)
plt.title('Feature Importance (Gain)')
plt.tight_layout()
plt.show()

In [ ]:
dtest_xgb = xgb.DMatrix(X_test_xgb)
forecast_xgb = pd.DataFrame(model.predict(dtest_xgb),columns=['forecast'])
forecast_xgb['ds'] = test_df_xgb['ds'].values

Rescaling dei dati

In [ ]:
# RE-Scaling dei dati
forecast_xgb['forecast'] = scaler_y.inverse_transform(forecast_xgb[['forecast']])
train_df_xgb['y'] = scaler_y.inverse_transform(train_df_xgb[['y']])

In [ ]:
# Plot forecast
fig = go.Figure()
# Training data
fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=train_df_xgb['y'], mode='lines', name='Training'))
# Test data
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=test_df_xgb['y'], mode='lines', name='Test'))
# Forecast
fig.add_trace(go.Scatter(x=forecast_xgb['ds'], y=forecast_xgb['forecast'], mode='lines', 
                         name='Forecast', line=dict(color='deepskyblue')))
fig.update_layout(title='Forecast vs Real Data', template='plotly_dark')
fig.show()

Calcolo metriche

In [ ]:
# Calcola metriche
merged_df = pd.merge(test_df_xgb[['ds', 'y']], forecast_xgb, on='ds', how='left')
merged_df.dropna(inplace=True)
metriche_xgb = calcola_metriche(merged_df['y'],merged_df['forecast'],train_df_xgb['y'], 
                                modelname="XGBoost").round(10)
metriche_xgb

##### LSTM

In [ ]:
# divisione in train e test
df_lstm = create_time_series_features(df, target_col='y') 
df_lstm = df_lstm.dropna().reset_index(drop=True)

train_df_lstm = df_lstm.iloc[:-h]
test_df_lstm = df_lstm.iloc[-h:]
print(train_df_lstm.columns)
train_df_lstm.head()

In [ ]:
# Optuna function
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_lstm = pd.DataFrame()
def objective(trial):
    # Hyperparametri da ottimizzare
    encoder_n_layers = trial.suggest_int("encoder_n_layers", 1, 2,4)
    encoder_hidden_size = trial.suggest_categorical("encoder_hidden_size", [32, 64, 128, 256])
    decoder_hidden_size = trial.suggest_categorical("decoder_hidden_size", [32, 64, 128, 256])
    decoder_layers = trial.suggest_int("decoder_layers", 1, 2,4)
    input_size = trial.suggest_categorical("input_size", [int(h), int(h*2), int(h*5)])
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    # Definizione modello LSTM
    lstm = LSTM(
        h=len(test_df_lstm), 
        input_size=input_size,
        loss=DistributionLoss(distribution="Normal", level=[95]),
        scaler_type='robust',
        encoder_n_layers=encoder_n_layers,
        encoder_hidden_size=encoder_hidden_size,
        decoder_hidden_size=decoder_hidden_size,
        decoder_layers=decoder_layers,
        futr_exog_list=futr_exog_list,
        max_steps=50,
        batch_size=batch_size,
        learning_rate=learning_rate,
        recurrent=False
    )

    nf = NeuralForecast(models=[lstm], freq='15min')

    # cross-validation per tuning
    df_cv = nf.cross_validation(
        df=train_df_lstm,
        step_size=len(test_df_lstm),
        n_windows=5
    )

    # Calcola MAE per ogni fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'LSTM']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'LSTM',
            'MAE_lstm': mae,
            'RMSE_lstm': rmse
        })

    # Media MAE su tutte le fold
    cv_metrics_df_lstm = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_lstm['MAE_lstm'].mean()

    return mean_mae 

In [ ]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50) 

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
# salvo i best params trovati da Optuna
with open('pickles/LSTM_bestPar_SolarEnergy_multistep.pkl', 'wb') as file:
    pickle.dump(best_params, file)

Carico i best params

In [ ]:
with open('pickles/LSTM_bestPar_SolarEnergy_multistep.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [ ]:
# final model
best_lstm = LSTM(
    h=len(test_df_lstm),
    input_size=3*len(test_df_lstm),
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=100,
    #early_stop_patience_steps=20,
    batch_size=best_params['batch_size'],
    learning_rate=0.01,
    recurrent=False
)
nf = NeuralForecast(models=[best_lstm], freq='15min')

In [ ]:
# final CV
df_cv = nf.cross_validation(
    df=train_df_lstm,
    step_size=len(test_df_lstm),
    n_windows=5
)
with open('pickles/LSTM_CV_SolarEnergy_multistep.pkl', 'wb') as file:
     pickle.dump(df_cv, file)

with open('pickles/LSTM_CV_SolarEnergy_multistep.pkl', 'rb') as file:
    df_cv = pickle.load(file)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'LSTM']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'LSTM',
        'MAE_lstm': mae,
        'RMSE_lstm': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_lstm = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_lstm)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_lstm.groupby('Model').mean().reset_index()
print(mean_metrics)

In [ ]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_lstm['unique_id'].unique():
    series_data = train_df_lstm[train_df_lstm['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_lstm['Model'] = 'lstm'
# Plot MAE sul primo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_lstm'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_lstm'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


In [ ]:
best_params

In [ ]:
# Final training
final_model = LSTM(
    h=len(test_df_lstm),
    input_size=len(test_df_lstm),
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='standard',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=200,
    #early_stop_patience_steps=20,
    batch_size=best_params['batch_size'],
    learning_rate=0.01, #best_params['learning_rate']
    recurrent=True
)

nf = NeuralForecast(models=[final_model], freq='15min')

start_time = time.time()
#fit 
nf.fit(df=train_df_lstm, val_size=len(test_df_lstm))
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

Previsioni

In [ ]:
Y_hat_df_lstm = nf.predict(futr_df=test_df_lstm[['unique_id', 'ds'] + futr_exog_list])
# salvo previsioni in pickle
Y_hat_df_lstm.to_pickle('pickles/LSTM_solarEnergy_multistep.pkl')

In [ ]:
#carico le previsioni
Y_hat_df_lstm = pd.read_pickle('pickles/LSTM_solarEnergy_multistep.pkl')

In [ ]:
# Plot forecast
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_lstm['ds'], y=train_df_lstm['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_lstm['ds'], y=test_df_lstm['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM'], mode='lines', 
    name='MEAN Forecast', line=dict(color='green')))

#PREVISIONI MEDIANA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="LSTM forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_lstm['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")

fig.show()

In [ ]:
# Calcola metriche
merged_df_lstm = pd.merge(test_df_lstm[['ds', 'y']], Y_hat_df_lstm, on='ds', how='left')
merged_df_lstm.dropna(inplace=True)
metriche_lstm = calcola_metriche(merged_df_lstm['y'],merged_df_lstm['LSTM'],
                                 train_df_lstm['y'],modelname="LSTM").round(10)
metriche_lstm

##### NEURAL PROPHET

In [ ]:
df_np = create_time_series_features(df, target_col='y') 
df_np = df_np.dropna().reset_index(drop=True)
df_np = df_np[future_features]
train_df_np = df_np.iloc[:-h]
test_df_np = df_np.iloc[-h:]

In [ ]:
# Scaling dei dati
train_df_original = train_df_np.copy()
test_df_original = test_df_np.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_np['ds']
test_meta = test_df_np['ds']

feature_cols = [col for col in train_df_np.columns if col not in ['ds']]
train_idx = train_df_np.index
test_idx = test_df_np.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_np[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_np[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_np = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_np = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_np.shape, test_df_np.shape

TimeSeriesCV

In [ ]:
# CV
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

initial_train_size = len(train_df_np) - 5 * len(test_df_np)
h = len(test_df_np)
n_windows = 5

results = []

for i in range(n_windows):
    print(f"\n--- Fold {i+1}/{n_windows} ---")

    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    train_window = train_df_np.iloc[:train_end].copy()
    test_window = train_df_np.iloc[test_start:test_end].copy()

    print(f"Train: {train_window.shape}, Test: {test_window.shape}")

    # Modello NeuralProphet
    model = NeuralProphet(
        quantiles=[0.025, 0.975],
        learning_rate=0.01,
        batch_size=64,
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
        loss_func='Huber',
    )

    # Aggiungi regressori
    for reg in futr_exog_list:
        model.add_future_regressor(reg)

    # Fit
    _ = model.fit(train_window, freq="15min", epochs=100)

    # Predizione
    future_df = test_window.copy()  # Deve contenere anche i regressori futuri
    forecast = model.predict(future_df)

    # Metriche
    y_true = test_window['y'].values
    y_pred = forecast['yhat1'].values

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    results.append({'Fold': i+1, 'MAE_nprophet': mae, 'RMSE_nprophet': rmse})
    print(f"Split {i+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# Risultati CV
results_df_np = pd.DataFrame(results)
print("\n=== Backtest Results ===")
print(results_df_np)

# Plot Errori
results_df_np.plot(
    x='Fold',
    y=['MAE_nprophet', 'RMSE_nprophet'],
    marker='o',
    title='TimeSeriesCV Errors',
    figsize=(10, 5)
)
plt.grid(True)
plt.show()

In [ ]:
# Mi ricavo le date
initial_train_size = len(train_df_np)-5*len(test_df_np)
h = len(test_df_np)
n_windows = 5

# Crea lista per i dati delle fold
fold_data = []

for i in range(n_windows):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    # Ricava date da colonna 'ds' o dall'indice
    if 'ds' in train_df_np.columns:
        start_date = train_df_np['ds'].iloc[test_start]
        end_date = train_df_np['ds'].iloc[test_end - 1]
    else:
        start_date = np_df_train.index[test_start]
        end_date = np_df_train.index[test_end - 1]

    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]

    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)

# Converto le date in datetime se necessario
if all(isinstance(date, str) for date in fold_intervals_df['start_date']):
    fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
    fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])

print("Fold Intervals DataFrame:")
print(fold_intervals_df)

In [ ]:
# CV Plot
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_np['ds'], train_df_np['y'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'{row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

Now let's training on the whole dataset

In [ ]:
# FIT
# Fissa i seed per riproducibilità
import random
import time
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

np_df_train = train_df_np[future_features].copy()

# Modello senza regressori esterni
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.01, 
    batch_size=64,
    daily_seasonality=True,    # Impara la stagionalità dai dati
    weekly_seasonality=True,   # Impara pattern settimanali
    yearly_seasonality=True,   # Impara pattern annuali
    loss_func='Huber'
)
# Aggiungi solo i regressori deterministici
for reg in futr_exog_list:
    neuralprophet.add_future_regressor(reg)
start_time = time.time()
metrics_df = neuralprophet.fit(np_df_train, freq="15min",epochs=100)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [ ]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
forecast = neuralprophet.predict(test_df_np[future_features])
#salvo in pickle
forecast.to_pickle('pickles/NP_solarEnergy_multistep.pkl')

CARICO LE PREVISIONI DEL TEST SET

In [ ]:
# carico le previsioni
forecast_test = pd.read_pickle('pickles/NP_solarEnergy_multistep.pkl')

In [ ]:
#Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
forecast_test['yhat1'] = forecast_test['yhat1'] * (max_val - min_val) + min_val
forecast_test['yhat1 2.5%'] = forecast_test['yhat1 2.5%'] * (max_val - min_val) + min_val
forecast_test['yhat1 97.5%'] = forecast_test['yhat1 97.5%'] * (max_val - min_val) + min_val

In [ ]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))


# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [ ]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast_test['ds'], 'forecast': forecast_test['yhat1']})
merged_df = pd.merge(test_df_original[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
mae_val = mean_absolute_error(merged_df['y'], merged_df['forecast'])
metriche_np = calcola_metriche(merged_df['y'], merged_df['forecast'],train_df_original['y'],
                               modelname="NeuralProphet").round(10)
metriche_np

#### TFT 

In [ ]:
# Preparing dataset
df_tft = create_time_series_features(df, target_col='y') 
df_tft = df_tft.dropna().reset_index(drop=True)
hist_exog_features = df_tft.drop(columns=['unique_id', 'ds', 'y'] + futr_exog_list).columns.tolist()
# 2. Suddivisione in train/test
train_df_tft = df_tft.iloc[:-h]
test_df_tft = df_tft.iloc[-h:]
train_df_tft.shape, test_df_tft.shape

In [ ]:
# Scaling dei dati
#Salvo per dopo
train_df_original = train_df_tft.copy() 
test_df_original = test_df_tft.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_tft[['ds', 'unique_id']]
test_meta = test_df_tft[['ds', 'unique_id']]

feature_cols = [col for col in train_df_tft.columns if col not in ['ds', 'unique_id']]
train_idx = train_df_tft.index
test_idx = test_df_tft.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_tft[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_tft[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_tft = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_tft = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_tft.shape, test_df_tft.shape

##### TimeSeriesCV - fine tuning Optuna (LENTO)

In [0]:
# Optuna function
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_tft = pd.DataFrame()
h = len(test_df_tft)

def objective(trial):
    # Hyperparameters to optimize for TFT
    input_size = trial.suggest_categorical("input_size", [h, int(4*h), int(2*h)])
    hidden_size = trial.suggest_categorical("hidden_size", [32, 64, 128, 256])
    n_rnn_layers = trial.suggest_int("n_rnn_layers", 1, 4)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    grn_activation = trial.suggest_categorical("grn_activation", ["ReLU", "ELU", "Sigmoid"])
    
    # Define TFT model
    tft = TFT(
        h=h,  # forecast horizon
        input_size=input_size,
        hidden_size=hidden_size,
        n_rnn_layers=n_rnn_layers,
        rnn_type='lstm',
        grn_activation=grn_activation,
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=learning_rate,
        futr_exog_list=futr_exog_list,
        hist_exog_list=hist_exog_features,
        max_steps=50, #aumentare a 100
        val_check_steps=10,
        batch_size=batch_size,
        early_stop_patience_steps=-1,
        scaler_type="standard",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=64
)

    nf = NeuralForecast(models=[tft], freq='15min')

    # Cross-validation for tuning
    df_cv = nf.cross_validation(
        df=train_df_tft,
        step_size=h,
        n_windows=5
    )

    # Calculate MAE for each fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'TFT']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'TFT',
            'MAE_tft': mae,
            'RMSE_tft': rmse
        })

    # Average MAE across all folds
    cv_metrics_df_tft = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_tft['MAE_tft'].mean()

    return mean_mae

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)  #aumentare a 50 su databricks

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
# # Path nella repo Git
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_BestPars_SolarEnergy_multistep.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(best_params, f)

In [ ]:
# #carico i best params
with open("pickles/TFT_BestPars_SolarEnergy_multistep.pkl", 'rb') as file:
    best_params = pickle.load(file)

In [0]:
# final CV

# final model
best_tft = TFT(
        h=h,  
        input_size=int(h/2), 
        hidden_size=32,
        n_rnn_layers=best_params['n_rnn_layers'],
        rnn_type='lstm',
        grn_activation=best_params['grn_activation'],
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=best_params['learning_rate'],
        futr_exog_list=futr_exog_list,
        hist_exog_list = hist_exog_features,
        max_steps=200, #aumentare a 100
        val_check_steps=10,
        batch_size=8,
        early_stop_patience_steps=-1,
        scaler_type="standard",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=8
)
nf = NeuralForecast(models=[best_tft],freq="15min")
# final CV
df_cv = nf.cross_validation(
    df=train_df_tft,
    step_size=len(test_df_tft),
    n_windows=5
)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'TFT']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'TFT',
        'MAE_TFT': mae,
        'RMSE_TFT': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_tft = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_tft)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_tft.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# # Path nella repo Git
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_CV_SolarEnergy_multistep.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(cv_metrics_df_tft, f)

In [ ]:
# carico la CV
with open('pickles/TFT_CV_SolarEnergy_multistep.pkl', 'rb') as file:
    cv_metrics_df_tft = pickle.load(file)

In [ ]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_tft['unique_id'].unique():
    series_data = train_df_tft[train_df_tft['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_tft['Model'] = 'TFT'
# Plot MAE sul primo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_TFT'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_TFT'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


Final training

In [0]:
# Final training
import time
# final model
best_tft = TFT(
        h=h,  
        input_size=best_params['input_size']*2,
        hidden_size=best_params['hidden_size'],
        n_rnn_layers=best_params['n_rnn_layers'],
        rnn_type='lstm',
        grn_activation=best_params['grn_activation'],
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=best_params['learning_rate'],
        futr_exog_list=futr_exog_list,
        hist_exog_list=hist_exog_features,
        max_steps=500, #aumentare a 100
        val_check_steps=25,
        batch_size=best_params['batch_size'],
        early_stop_patience_steps=-1,
        scaler_type="standard",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=64
)
nf = NeuralForecast(models=[best_tft],freq="15min")

start_time = time.time()
#fit 
nf.fit(df=train_df_tft, val_size=len(test_df_tft))

end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Predict
Y_hat_df_tft = nf.predict(futr_df=test_df_tft[['unique_id','ds'] + futr_exog_list])

In [0]:
# #salvo le pred in pickle
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_pred_SolarEnergy_multistep.pkl"
# with open(save_path, 'wb') as file:
#     pickle.dump(Y_hat_df_tft, file)

In [ ]:
# #carico le pred
with open("pickles/TFT_pred_SolarEnergy_multistep.pkl", 'rb') as file:
    Y_hat_df_tft = pickle.load(file)

In [ ]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
Y_hat_df_tft['TFT'] = Y_hat_df_tft['TFT'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-median'] = Y_hat_df_tft['TFT-median'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-lo-90'] = Y_hat_df_tft['TFT-lo-90'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-hi-90'] = Y_hat_df_tft['TFT-hi-90'] * (max_val - min_val) + min_val

In [ ]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_tft['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_tft['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='lightgreen')))

# QUANTILI TFT
# Fascia tra P10 e P90
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_tft['ds']) + 
    list(Y_hat_df_tft['ds'][::-1]),
    y=list(Y_hat_df_tft['TFT-hi-90']) + list(Y_hat_df_tft['TFT-lo-90'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="TFT forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_tft['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [ ]:
# Calcola metriche
merged_df_tft = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_tft, on='ds', how='left')
merged_df_tft.rename(columns={'TFT-lo-90': '0.1','TFT-median':'0.5', 'TFT-hi-90':'0.9'},inplace=True)
merged_df_tft.dropna(inplace=True)
metriche_tft = calcola_metriche(merged_df_tft['y'],merged_df_tft['TFT'],train_df_original['y'],
                                y_pred_quantiles=merged_df_tft[['0.1','0.5','0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="TFT").round(10)
metriche_tft

##### TFT Explainability

In [0]:
tft_model = nf.models[0]

In [0]:
# # NON MI FA SALVARE IL MODELLO PERCHE' TROPPO PESANTE
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_model_SolarEnergy_multistep.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(tft_model, f)

In [0]:
# # # carico il modello tft
# with open('pickles/TFT_model_SolarEnergy_multistep.pkl', 'rb') as file:
#     tft_model = pickle.load(file)

In [0]:
attention = tft_model.attention_weights()
def plot_attention(
    self, plot: str = "time", output: str = "plot", width: int = 800, height: int = 400
):
    """
    Plot the attention weights.

    Args:
        plot (str, optional): The type of plot to generate. Can be one of the following:
            - 'time': Display the mean attention weights over time.
            - 'all': Display the attention weights for each horizon.
            - 'heatmap': Display the attention weights as a heatmap.
            - An integer in the range [1, model.h) to display the attention weights for a specific horizon.
        output (str, optional): The type of output to generate. Can be one of the following:
            - 'plot': Display the plot directly.
            - 'figure': Return the plot as a figure object.
        width (int, optional): Width of the plot in pixels. Default is 800.
        height (int, optional): Height of the plot in pixels. Default is 400.

    Returns:
        matplotlib.figure.Figure: If `output` is 'figure', the function returns the plot as a figure object.
    """

    attention = (
        self.mean_on_batch(self.interpretability_params["attn_wts"])
        .mean(dim=0)
        .cpu()
        .numpy()
    )

    fig, ax = plt.subplots(figsize=(width / 100, height / 100))

    if plot == "time":
        attention = attention[self.input_size :, :].mean(axis=0)
        ax.plot(np.arange(-self.input_size, self.h), attention)
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Mean Attention")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "all":
        for i in range(self.input_size, attention.shape[0]):
            ax.plot(
                np.arange(-self.input_size, self.h),
                attention[i, :],
                label=f"horizon {i-self.input_size+1}",
            )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Attention per horizon")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "heatmap":
        cax = ax.imshow(
            attention,
            aspect="auto",
            cmap="viridis",
            extent=[-self.input_size, self.h, -self.input_size, self.h],
        )
        fig.colorbar(cax)
        ax.set_title("Attention Heatmap")
        ax.set_xlabel("Attention (current time step)")
        ax.set_ylabel("Attention (previous time step)")

    elif isinstance(plot, int) and (plot in np.arange(1, self.h + 1)):
        i = self.input_size + plot - 1
        ax.plot(
            np.arange(-self.input_size, self.h),
            attention[i, :],
            label=f"horizon {plot}",
        )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title(f"Attention weight for horizon {plot}")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    else:
        raise ValueError(
            'plot has to be in ["time","all","heatmap"] or integer in range(1,model.h)'
        )

    plt.tight_layout()

    if output == "plot":
        plt.show()
    elif output == "figure":
        return fig
    else:
        raise ValueError(f"Invalid output: {output}. Expected 'plot' or 'figure'.")

In [0]:
# MEAN ATTENTION
plot_attention(tft_model, plot="time")
# pesi di attenzione medi nel tempo su tutti gli orizzonti di previsione.
# La curva blu rappresenta quanto il modello ha "guardato" (attention) 
# quel punto temporale in input durante la previsione.
# Valori più alti significano che il modello ritiene quel punto temporale
# particolarmente rilevante per fare le previsioni

In [0]:
# ATTENTION OF A SPECIFIC TIME STEPS
plot_attention(tft_model, plot=44)
# pesi di attenzione per ogni orizzonte di previsione separatamente.

In [0]:
plot_attention(tft_model, plot="heatmap")

In [0]:
# PAST VARIABLE IMPORTANCES
feature_importances = tft_model.feature_importances()
print(feature_importances.keys())
print("\n ==========PAST FEATURE IMPORTANCES==========")
feature_importances["Past variable importance over time"].mean().sort_values().plot(kind="barh")

Questo grafico mostra quanto il modello ha fatto affidamento sulle feature passate \
(quelle che il modello ha potuto osservare prima del punto di previsione). Serve a costruire il contesto dell'input

In [0]:
# FUTURE VARIABLE IMPORTANCES
print("\n ==========FUTURE FEATURE IMPORTANCES==========")
feature_importances["Future variable importance over time"].mean().sort_values().plot(kind="barh")

Mostra quanto il modello ha sfruttato le feature che sono servite a guidare le previsioni passo-passo nel decoder

DIFFERENZA TRA PAST E FUTURE: \
La diversa importanza riflette quanto ciascuna variabile aiuta nell’analisi del contesto passato (encoder -->> PAST) \
o nella generazione delle previsioni (decoder -->> FUTURE).

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME
df_importances = feature_importances["Past variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances.index))

for col in df_importances.columns:
    p = ax.bar(np.arange(-len(df_importances), 0), df_importances[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances[col]
ax.set_title("Past variable importance over time")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)

plt.show()

Questo grafico mostra QUANDO (e QUANTO) nel passato ogni feature è stata più utile per costruire il contesto predittivo.

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME
df_importances2 = feature_importances["Future variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances2.index))
for col in df_importances2.columns:
    p = ax.bar(np.arange(-len(df_importances2), 0), df_importances2[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances2[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.grid(True)
ax.legend()
plt.show()

Per ogni feature viene mostrato quanto è stata importante per il modello a ciascun orizzonte futuro

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances3 = feature_importances["Past variable importance over time"]
mean_attention = (
    nf.models[0]
    .attention_weights()[nf.models[0].input_size :, :]
    .mean(axis=0)[: nf.models[0].input_size]
)
df_importances3 = df_importances3.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances3.index))

for col in df_importances3.columns:
    p = ax.bar(np.arange(-len(df_importances3), 0), df_importances3[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances3[col]
ax.set_title("Past variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances3), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances4 = feature_importances["Future variable importance over time"]
mean_attention = (
    nf.models[0]
    .attention_weights()[nf.models[0].input_size :, :]
    .mean(axis=0)[: nf.models[0].input_size]
)
df_importances4 = df_importances4.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances4.index))

for col in df_importances4.columns:
    p = ax.bar(np.arange(-len(df_importances4), 0), df_importances4[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances4[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances4), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
nf.models[0].feature_importance_correlations()

#### CHRONOS

In [ ]:
# Preparing dataset
df_chronos = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos = df_chronos.dropna().reset_index(drop=True) 
df_chronos['item_id'] = df_chronos['unique_id'] 
df_chronos.drop(columns="unique_id", inplace=True) 
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Definizione delle covariate note
known_covariates_names=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id','target']]
feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

# Split train e test
train_df_chronos = df_chronos.iloc[:-h]
test_df_chronos = df_chronos.iloc[-h:]

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [ ]:
# Scaling dei dati
train_df_original = train_df_chronos.copy() 
test_df_original = test_df_chronos.copy()

train_idx = train_df_chronos.index
test_idx = test_df_chronos.index

feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

#scaler
scaler_x = MinMaxScaler()
train_df_chronos = pd.DataFrame(scaler_x.fit_transform(train_df_chronos[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_chronos = pd.DataFrame(scaler_x.transform(test_df_chronos[feature_cols]),
                            columns=feature_cols, index=test_idx)
#controllo le shape
train_df_chronos.shape, test_df_chronos.shape

TimeSeriesCV

In [ ]:
# TimeSeriesCV
h = len(test_df_chronos)  # prediction length
n_splits = 5
initial_train_size = len(train_df_chronos) - 5*h
results = []

for fold in range(n_splits):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")
    
    train_end = initial_train_size + fold * h
    test_start = train_end
    test_end = test_start + h
    train_df_fold = train_df_chronos.iloc[:train_end]
    test_df_fold = train_df_chronos.iloc[test_start:test_end]
    
    train_df_fold.reset_index(inplace=True)
    test_df_fold.reset_index(inplace=True)
    
    train_df_fold["timestamp"] = pd.to_datetime(train_df_fold["timestamp"])
    test_df_fold["timestamp"] = pd.to_datetime(test_df_fold["timestamp"])
    
    # Future timestamps 
    future_index = pd.date_range(
        start=train_df_fold["timestamp"].max() + pd.Timedelta(minutes=15),
        periods=h,
        freq="15min")
    
    # Preparo test set con known_covariates
    test_df_for_prediction = test_df_fold[test_df_fold["timestamp"].isin(future_index)].copy()
    test_df_for_prediction = test_df_for_prediction.set_index("timestamp")
    test_df_for_prediction = test_df_for_prediction.reindex(future_index)  # forza continuità
    test_df_for_prediction["item_id"] = train_df_fold["item_id"].iloc[0]  # supponiamo 1 sola serie
    test_df_for_prediction = test_df_for_prediction.reset_index().rename(columns={"index": "timestamp"})
    test_df_for_prediction = test_df_for_prediction.set_index(["item_id", "timestamp"])
    
    # Preparo il train con multindex
    train_df_fold = train_df_fold.set_index(["item_id", "timestamp"])
    print(f"Train: {train_df_fold.shape}, Test: {test_df_fold.shape}")
    
    # inizializzo il predictor
    predictor = TimeSeriesPredictor(
        prediction_length=h,
        target="target",
        known_covariates_names=known_covariates_names,
        eval_metric="MASE",
        freq="15min")
    
    # fit del modello
    predictor.fit(
        train_df_chronos,
        hyperparameters={
            "Chronos": [
                {
                    "model_path": "bolt_small",
                    "ag_args": {"name_suffix": "ZeroShot"}},
                {
                    "model_path": "bolt_small",
                    "covariate_regressor": "CAT",
                    "target_scaler": "robust",
                    "ag_args": {"name_suffix": "WithRegressor"},
                },
            ],
        },
        time_limit=600,
        enable_ensemble=True,  # Attiva ensemble per migliorare l'accuratezza
        )
    
    predictions = predictor.predict(train_df_fold, known_covariates=test_df_for_prediction)

    test_df_with_index = test_df_fold.set_index(["item_id", "timestamp"])

    # Debug
    print(f"Forma predictions: {predictions.shape}")
    print(f"Colonne predictions: {predictions.columns}")
    print(f"Numero di indici in predictions: {len(predictions.index)}")
    print(f"Numero di indici in test_df_with_index: {len(test_df_with_index.index)}")

    # trovo solo gli indici comuni
    common_indices = predictions.index.intersection(test_df_with_index.index)
    print(f"Indici comuni: {len(common_indices)}")

    # prendo solo gli indici comuni
    y_true = test_df_with_index.loc[common_indices, "target"]
    y_pred = predictions.loc[common_indices, 'mean'].to_numpy()

    mae = calcola_mae(y_true, y_pred)
    rmse = calcola_rmse(y_true, y_pred)
    
    results.append({'Fold': fold+1, 'MAE_chronos': mae, 'RMSE_chronos': rmse})
    print(f"Split {fold+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
# risultati finali
results_df_chronos = pd.DataFrame(results)
results_df_chronos.set_index('Fold',inplace=True)
print("\n=== TimeSeriesCV Results ===")
results_df_chronos

In [ ]:
# Estraggo le date
fold_data = []
train_df_chronos.reset_index(inplace=True)
for i in range(n_splits):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h
    
    # Ricava date da colonna 'ds' o dall'indice
    if 'timestamp' in train_df_chronos.columns:
        start_date = train_df_chronos['timestamp'].iloc[test_start]
        end_date = train_df_chronos['timestamp'].iloc[test_end - 1]
    else:
        start_date = train_df_chronos.index[test_start]
        end_date = train_df_chronos.index[test_end - 1]
    
    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]
    
    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)
fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])
# Stampa il dataframe prima della conversione per verificare i valori
print("Fold Intervals DataFrame (before conversion):")
fold_intervals_df

In [ ]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_chronos['timestamp'], train_df_chronos['target'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot degli errori
results_df_chronos.reset_index(inplace=True)
ax = results_df_chronos.plot(x='Fold', y=['MAE_chronos', 'RMSE_chronos'], marker='o', title='Backtesting Errors')
ax.set_xticks(range(1, 6)) # Imposta le tacche sull'asse x da 1 a 5
plt.show()

Let's training on the whole dataset

In [ ]:
# Preparo dataset per training
predictor = TimeSeriesPredictor(
    prediction_length=len(test_df_chronos), 
    eval_metric="MASE",
    target="target",
    known_covariates_names=futr_exog_list,
    freq="15min"
)
test_known_covariates = test_df_chronos[futr_exog_list]

In [ ]:
# Fit del modello
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {
                "model_path": "bolt_small",
                "ag_args": {"name_suffix": "ZeroShot"}
            },
            {
                "model_path": "bolt_mini",
                "covariate_regressor": "NN_TORCH",
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
        ],
    },
    time_limit=600,
    enable_ensemble=True,
)
# Valutazione del modello in fase di training
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [ ]:
# PREVISIONI MULTI STEP 
predictions = predictor.predict(
    train_df_chronos,
    known_covariates=test_known_covariates,  # Solo le covariate realmente note in anticipo
)
predictions.reset_index(inplace=True)

predictions_0shot = predictor.predict(
    train_df_chronos,
    known_covariates=test_known_covariates,
    model="ChronosZeroShot[bolt_small]"
)
predictions_0shot.reset_index(inplace=True)

In [ ]:
# Rescaling dei dati
target_col_position = feature_cols.index('target')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
predictions['mean'] = predictions['mean'] * (max_val - min_val) + min_val
predictions['0.1'] = predictions['0.1'] * (max_val - min_val) + min_val
predictions['0.5'] = predictions['0.5'] * (max_val - min_val) + min_val
predictions['0.9'] = predictions['0.9'] * (max_val - min_val) + min_val
predictions_0shot['mean'] = predictions_0shot['mean'] * (max_val - min_val) + min_val
predictions_0shot['0.5'] = predictions_0shot['0.5'] * (max_val - min_val) + min_val
predictions_0shot['0.9'] = predictions_0shot['0.9'] * (max_val - min_val) + min_val

In [ ]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=train_df_chronos['timestamp'], y=train_df_original['target'], mode='lines', name='Training Data'))
# Dati di test

fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='With Regressors', line=dict(color='green')))
# PREVISIONI CHRONOS 0-shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='0-shot Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="CHRONOS forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_chronos['timestamp'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [ ]:
# metriche di valutazione
test_df_original.reset_index(inplace=True)
merged_df_chronos = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']],
        on=['timestamp', 'item_id']
)
metriche_chronos = calcola_metriche(merged_df_chronos['target'],merged_df_chronos['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos").round(5)
metriche_chronos

In [ ]:
# metriche di valutazione 0 shot
merged_df_chronos_0shot = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions_0shot[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']],
        on=['timestamp', 'item_id']
)
metriche_chronos_0shot = calcola_metriche(merged_df_chronos_0shot['target'],merged_df_chronos_0shot['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos_0shot[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos 0 shot").round(5)
metriche_chronos_0shot

#### final evaluations

In [ ]:
final_metrics = metriche_xgb.join(metriche_lstm).join(metriche_np).join(metriche_chronos).join(metriche_chronos_0shot).join(metriche_tft)
final_metrics

In [ ]:
# FINAL PREDICTIONS PLOT FOR SOLAR ENERGY
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI XGBOOST
fig.add_trace(go.Scatter(
    x=forecast_xgb['ds'], 
    y=forecast_xgb['forecast'], 
    mode='lines', 
    name='XGBoost', 
    line=dict(color='purple')
))

# PREVISIONI LSTM
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=Y_hat_df_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='pink')
))

# PREVISIONI NEURAL PROPHET
# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='aquamarine')
))

# PREVISIONI MEDIA TFT
# fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
#     name='TFT Forecast', line=dict(color='yellow')))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS', line=dict(color='green')))
    
 # PREVISIONI CHRONOS 0 shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='CHRONOS 0-SHOT', line=dict(color='lightgreen')))
   
# Layout del grafico
fig.update_layout(
    title="Forecasts for White Noise", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
#sistemo i vari dataset prima del merge
results_df_tft.rename(columns={'split': 'Fold'}, inplace=True)
cv_metrics_df_lstm.drop(columns=('Model'),inplace=True)

In [ ]:
#sistemo i vari dataset prima del merge
cv_metrics_df_lstm.drop(columns=('Model'),inplace=True)
results_df_np.rename(columns={'split': 'Fold'}, inplace=True)
results_df_chronos.rename(columns={'split': 'Fold','MAE':'MAE_chronos','RMSE':'RMSE_chronos'}, inplace=True)
cv_metrics_df_xgboost.rename(columns={'MAE': 'MAE_XGB','RMSE':'RMSE_XGB'}, inplace=True)

In [ ]:
# Final plot metrics TimeSeriesCV

# faccio il merge di tutte le tabelle dei vari CV
final_cv = cv_metrics_df_lstm.merge(results_df_chronos, 
                            on="Fold",how="inner").merge(cv_metrics_df_xgboost[['MAE_XGB','RMSE_XGB','Fold']],
                            on="Fold",how="inner").merge(results_df_np,
                            on="Fold", how="inner").merge(cv_metrics_df_tft,
                            on="Fold",how="inner")
# Rescaling CV metrics data
columns_to_modify = ['MAE_nprophet', 'RMSE_nprophet', 'MAE_TFT', 'RMSE_TFT', #'MAE_lstm', 'RMSE_lstm',
                        'MAE_chronos', 'RMSE_chronos', 'MAE_XGB', 'RMSE_XGB']
for col in columns_to_modify:
    final_cv[f'{col}'] = final_cv[f'{col}'] * (max_val - min_val) + min_val

model_colors = {
    'XGB': '#9467bd',    # Purple
    'lstm': '#e377c2',       # Pink
    'nprophet': '#7fffd4',         # Aquamarine
    'TFT': '#ffff00',        # Yellow
    'chronos': '#2ca02c',    # Green
}


# Create the figure
fig = go.Figure()

# Add traces for each model - MAE (darker) and RMSE (lighter)
for model, color in model_colors.items():
    # Add MAE line (darker)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'MAE_{model}'],
        mode='lines+markers',
        name=f'{model} MAE',
        line=dict(color=color, width=3),
        marker=dict(size=8)
    ))
    
    # Add RMSE line (lighter with same color but different dash pattern)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'RMSE_{model}'],
        mode='lines+markers',
        name=f'{model} RMSE',
        line=dict(color=color, width=2, dash='dash'),
        marker=dict(size=6)
    ))

# Update layout
fig.update_layout(
    title='Model Performance Comparison (MAE and RMSE)',
    template='plotly_dark',
    xaxis=dict(title='Fold',
        tickmode='linear',
        tick0=1, dtick=1
    ),    yaxis=dict(
        title='Error Value'
    ),    legend=dict(
        orientation="v"
    ),    hovermode="closest")
fig.show()

In [ ]:
# Confronto TFT/CHRONOS
tft_chronos = metriche_chronos.join(metriche_tft)
tft_chronos                                    

### **OIL PRICES**

In [0]:
#preparazione dataset
oil_prices.dropna(inplace=True)
df = oil_prices.reset_index().rename(columns={'date': 'ds','price':'y'})
df['unique_id'] = 'serie_1'
df['ds'] = pd.to_datetime(df['ds'])

In [0]:
# funzione per creare feature
def create_time_series_features(df, target_col):
    df = df.copy()
    # features
    lags = [1, 2, 4, 12]
    # Lag features (1-4)
    for lag in lags:
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    
    df['diff_12'] = (df[target_col] - df[target_col].shift(12)).shift(1)
    
    df['change'] = (df[target_col] - df[target_col].shift(1)).shift(1)
    df['percentChange'] = df[target_col].pct_change().shift(1)

    df['month'] = df['ds'].dt.month
    df['quarter'] = df['ds'].dt.quarter
    df['year'] = df['ds'].dt.year
    
    # rolling means 
    df['rolling_mean_3'] = df[target_col].rolling(window=3).mean().shift(1) #ultimo trimestre
    df['rolling_mean_6'] = df[target_col].rolling(window=6).mean().shift(1) #ultima metà dell'anno
    df['rolling_mean_12'] = df[target_col].rolling(window=12).mean().shift(1) #ultimo anno
    df['rolling_std_3'] = df[target_col].rolling(window=3).std().shift(1)
    df['rolling_std_6'] = df[target_col].rolling(window=6).std().shift(1)
    df['rolling_std_12'] = df[target_col].rolling(window=12).std().shift(1)
    return df

##### ARIMA + ETS

In [0]:
# divisione in train e test
df_arima = df[['y','ds','unique_id']].iloc[13:] #droppo le prime 13 per rendere paragonabile con altri modelli che utilizzano features esogene
train_size = int(len(df_arima) * 0.9)
train_df = df_arima.iloc[:train_size]
test_df = df_arima.iloc[train_size:]
# queste due ci serviranno dopo
season_length = 12 # for monthly data = 12 
horizon = len(test_df) # number of predictions
train_df.shape, test_df.shape

In [0]:
# train e test plot
fig = go.Figure()
#train dat
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Train'))
#test data 
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Test'))
#layout
fig.update_layout(template='plotly_dark', title='Train and Test Data Plot')
fig.show()

In [0]:
# fit dei modelli
models = [
    AutoARIMA(season_length=season_length),
    AutoETS(season_length=season_length)
]
# creo il forecaster
sf = StatsForecast(models=models, freq='MS')
import time
start_time = time.time()
# fit
sf.fit(train_df)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")


In [0]:
#summary of the model ARIMA
result=sf.fitted_[0,0].model_
print(result.keys())
print("Parametri ARMA:",result['arma'])

In [0]:
arima_string(sf.fitted_[0,0].model_)

In [0]:
#summary of the model ETS
result2 = sf.fitted_[0,1].model_
print(result2.keys())
print("Modello:",result2['method'])

In [0]:
# let's forecast
Y_hat_df = sf.forecast(df=train_df, h=len(test_df), fitted=True, level=[95])
#see fitted values vs true values
values=sf.forecast_fitted_values() #qui viene aggiunta anche la vera y
values.head(2)

In [0]:
values.tail(2)

TimeSeriesCV

In [0]:
crossvalidation_df = sf.cross_validation(
    df=train_df,
    h=horizon, 
    step_size=len(test_df),
    n_windows=5 
)

In [0]:
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = crossvalidation_df['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = crossvalidation_df[crossvalidation_df['cutoff'] == fold]
    
    for model in ['AutoARIMA', 'AutoETS']:
        # Filtra i dati per il modello corrente
        model_data = fold_data[['y', f'{model}']]
        model_data = model_data.dropna()
        
        # Calcola MAE e RMSE
        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data[f'{model}'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data[f'{model}'])
        
        # Aggiungi i risultati
        cv_metrics.append({
            'Fold': i+1,
            'Model': model,
            f'MAE_{model}': mae,
            f'RMSE_{model}': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_arima_ets = pd.DataFrame(cv_metrics)

In [0]:
# PLOT CV
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = crossvalidation_df[crossvalidation_df['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df['unique_id'].unique():
    series_data = train_df[train_df['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# fold con colori diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

ax1 = plt.gca()
ax2 = ax1.twinx()

# Plot MAE 
for model in cv_metrics_df_arima_ets['Model'].unique():
    model_data = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == model]
    ax1.plot(model_data['Fold'], model_data[f'MAE_{model}'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE
for model in cv_metrics_df_arima_ets['Model'].unique():
    model_data = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == model]
    ax2.plot(model_data['Fold'], model_data[f'RMSE_{model}'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()

In [0]:
fold_intervals #EXPANDING WINDOW CROSS VALIDATION

Let's forecast on test set

In [0]:
# prediction on test set
test_pred = sf.predict(h=len(test_df), level=[95])
Y_hat_df = test_df.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])

In [0]:
# Calcola metriche
metriche_arima = calcola_metriche(Y_hat_df['y'],Y_hat_df['AutoARIMA'],train_df['y'], 
                                modelname="ARIMA").round(10)
metriche_ets = calcola_metriche(Y_hat_df['y'],Y_hat_df['AutoETS'],train_df['y'], 
                                modelname="ETS").round(10)
metriche_arima, metriche_ets

In [0]:
# Forecast plot
# Creiamo un dataframe completo che include training, test e previsioni future
full_df = pd.concat([train_df, test_df], axis=0)
# Aggiungiamo le previsioni al dataframe completo
forecast_df_etsA = full_df.merge(Y_hat_df, how='outer', on=['unique_id', 'ds'])

fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(x=test_pred['ds'], y=test_pred['AutoARIMA'], mode='lines', 
    name='AutoARIMA Forecast', line=dict(color='green')))
# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(x=test_pred['ds'], y=test_pred['AutoETS'], mode='lines', 
    name='AutoETS Forecast', line=dict(color='orange', dash='dot')))

# # Intervallo di confidenza ARIMA 95%
# fig.add_trace(go.Scatter(
#     x=list(test_pred['ds']) + 
#     list(test_pred['ds'][::-1]),
#     y=list(test_pred['AutoARIMA-hi-95']) + list(test_pred['AutoARIMA-lo-95'][::-1]),
#     fill='toself',
#     fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
#     line=dict(color='rgba(255,255,255,0)'),
#     hoverinfo="skip",
#     showlegend=True,
#     name='P10-P90 ARIMA'
# ))

# # Intervallo di confidenza ETS 95%
# fig.add_trace(go.Scatter(
#     x=list(test_pred['ds']) + 
#     list(test_pred['ds'][::-1]),
#     y=list(test_pred['AutoETS-hi-95']) + list(test_pred['AutoETS-lo-95'][::-1]),
#     fill='toself',
#     fillcolor='rgba(144, 228, 44, 0.3)',  
#     line=dict(color='rgba(255,255,255,0)'),
#     hoverinfo="skip",
#     showlegend=True,
#     name='P10-P90 ETS'
# ))

# Layout del grafico
fig.update_layout(title="Forecast Comparison: AutoARIMA vs AutoETS", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15)))
# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
sf.predict(h=51)

##### XGBOOST

In [0]:
# df_xgb
df_xgb = create_time_series_features(df, target_col='y') 
df_xgb.drop(columns=['unique_id'],inplace=True)
df_xgb = df_xgb.dropna().reset_index(drop=True) #droppo i NaN creati dai lag, quindi le prime 12 obs.
df_xgb['covid'] = (df_xgb['ds'] > pd.Timestamp('2020-02-01')).astype(int)
print(df_xgb.columns)
df_xgb.head()

In [0]:
# Split train/test
train_size = int(len(df_xgb) * 0.9)
train_df_xgb = df_xgb.iloc[:train_size]
test_df_xgb = df_xgb.iloc[train_size:]

# Separazione features e target
feature_cols = [col for col in df_xgb.columns if col not in ['y', 'ds']]
X_train_xgb, y_train_xgb = train_df_xgb[feature_cols], train_df_xgb['y']
X_test_xgb, y_test_xgb = test_df_xgb[feature_cols], test_df_xgb['y']

In [0]:
# Scaling dei dati

#features
scaler_x = MinMaxScaler()
X_train_scaled = scaler_x.fit_transform(X_train_xgb)
X_test_scaled = scaler_x.transform(X_test_xgb)

# target, non serve che scalo y_test
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train_xgb.values.reshape(-1, 1))

# Ricreo df con stessi indici e nomi
X_train_xgb = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train_xgb.index)
X_test_xgb = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test_xgb.index)
y_train_xgb = pd.Series(y_train_scaled.flatten(), name='y', index=y_train_xgb.index)

#aggiorno train_df_xgb
train_df_xgb[feature_cols] = X_train_xgb
train_df_xgb['y'] = y_train_xgb

#controllo le shape
X_train_xgb.shape, X_test_xgb.shape, y_train_xgb.shape, y_test_xgb.shape

Ottimizzazione Optuna (Bayesiana)

In [0]:
# Optuna function
cv_metrics_log = []  # List globale per salvare metriche fold per fold

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'verbosity': 0,
        'seed': 42
    }

    tscv = TimeSeriesSplit(n_splits=5,test_size=len(test_df_xgb))
    y = train_df_xgb['y']
    df_xgb_feature = train_df_xgb.drop(columns=['y','ds'])
    all_rmse = []

    for fold, (train_idx, test_idx) in enumerate(tscv.split(df_xgb_feature)):
        X_train, X_test = df_xgb_feature.iloc[train_idx], df_xgb_feature.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        dtest = xgb.DMatrix(X_test.values, label=y_test.values)

        model = xgb.train(params, dtrain, num_boost_round=500,
                          evals=[(dtest, 'eval')],
                          early_stopping_rounds=30, verbose_eval=False)

        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mae = calcola_mae(y_test, preds)  # tua funzione custom

        all_rmse.append(rmse)

        # Recupera le date (se esiste colonna 'ds')
        start_date = df_xgb.iloc[test_idx]['ds'].min() if 'ds' in df_xgb.columns else None
        end_date = df_xgb.iloc[test_idx]['ds'].max() if 'ds' in df_xgb.columns else None

        # Logga le metriche della fold
        cv_metrics_log.append({
            'Trial': trial.number,
            'Fold': fold + 1,
            'MAE_XGB': mae,
            'RMSE_XGB': rmse,
            'start_date': start_date,
            'end_date': end_date,
            'Model': 'XGBoost'  
        })
    return np.mean(all_rmse)

In [0]:
# Optuna optimization
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)

In [0]:
# Best parameters
print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)
print("Best trial:",study.best_trial.number)
besttrial = study.best_trial.number

In [0]:
cv_metrics_log = pd.DataFrame(cv_metrics_log)
cv_metrics_log = cv_metrics_log[cv_metrics_log['Trial']==besttrial]
cv_metrics_log

Vediamo come ha performato nel miglior trial

In [0]:
#PLOT CV
cv_metrics_df_xgboost = cv_metrics_log.copy()

# Stampa le metriche per modello
print("Metriche per modello:")
print(cv_metrics_df_xgboost)

# Calcola e stampa le metriche medie per modello
mean_metrics = cv_metrics_df_xgboost.mean(numeric_only=True).reset_index()

# Crea il DataFrame con gli intervalli delle fold
fold_intervals_df = cv_metrics_df_xgboost[['Fold', 'start_date', 'end_date']].drop_duplicates()

# Ora creiamo il grafico combinato
plt.figure(figsize=(16, 12))

# === PLOT 1: Serie temporale originale ===
plt.subplot(2, 1, 1)
series_data = train_df_xgb.copy()
plt.plot(series_data['ds'], series_data['y'], label='Oil Price - XGB, CV')

# Evidenzia le fold con colori
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)],
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10,
             bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# === PLOT 2: Metriche MAE e RMSE ===
plt.subplot(2, 1, 2)
ax1 = plt.gca()
ax2 = ax1.twinx()

# MAE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_XGB'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# RMSE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_XGB'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Legenda combinata
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


Ora facciamo training coi migliori parametri trovati da Optuna

In [0]:
# Addestriamo il modello con i best params
best_params = study.best_params
best_params.update({'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 42})

dtrain_xgb = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
import time
start_time = time.time()
#fit 
model = xgb.train(best_params, dtrain_xgb, num_boost_round=500)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Plot feature importance
xgb.plot_importance(model, importance_type='gain', max_num_features=10)
plt.title('Feature Importance (Gain)')
plt.tight_layout()
plt.show()

DIRECT MULTI-STEP FORECAST 

In [0]:
def direct_multistep_forecast(train_data, feature_cols, target_col, horizon=None):
    """
    Addestra modelli separati per ogni orizzonte temporale futuro
    """
    forecasts = []
    models = []
    
    # Crea dataframe per date future
    last_date = train_data['ds'].max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=horizon, freq='MS') #dopo modificare freq = M
    
    # Per ogni step futuro, addestra un modello dedicato
    for h in range(1, horizon+1):
        print(f"Training model for horizon {h}")
        
        # Prepara target con shift inverso per prevedere h passi avanti
        df_horizon = train_data.copy()
        df_horizon[f'y_horizon_{h}'] = df_horizon[target_col].shift(-h)
        df_horizon = df_horizon.dropna()
        
        # Prendi features e target per questo orizzonte
        X = df_horizon[feature_cols]
        y = df_horizon[f'y_horizon_{h}']
        
        # Split train/validation
        train_size = int(len(X) * 0.8)
        X_train, X_val = X.iloc[:train_size], X.iloc[train_size:]
        y_train, y_val = y.iloc[:train_size], y.iloc[train_size:]
        
        # Addestra modello
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        params = best_params
        model = xgb.train(
            params,
            dtrain,
            evals=[(dtrain, 'train'), (dval, 'val')],
            num_boost_round=100,
            early_stopping_rounds=20,
            verbose_eval=False
        )
        models.append(model)
        
    # Crea input per la previsione (l'ultimo punto noto)
    last_point = xgb.DMatrix(train_data.iloc[[-1]][feature_cols])
    
    # Prevedi ciascun orizzonte con il modello dedicato
    for i, model in enumerate(models):
        pred = model.predict(last_point)[0]
        forecasts.append(pred)
    return pd.DataFrame({'ds': future_dates, 'forecast': forecasts})

In [0]:
# # Forecasting
# forecast_xgb = direct_multistep_forecast(train_df_xgb,
#                                          feature_cols, 
#                                          target_col='y',
#                                          horizon=len(test_df_xgb)) #posso predire anche oltre

RECURSIVE FORECAST

In [0]:
dtest_xgb = xgb.DMatrix(X_test_xgb)
forecast_xgb = pd.DataFrame(model.predict(dtest_xgb),columns=['forecast'])
forecast_xgb['ds'] = test_df_xgb['ds'].values

Rescaling dei dati

In [0]:
# RE-Scaling dei dati
forecast_xgb['forecast'] = scaler_y.inverse_transform(forecast_xgb[['forecast']])
train_df_xgb['y'] = scaler_y.inverse_transform(train_df_xgb[['y']])

In [0]:
# Plot forecast
fig = go.Figure()
# Training data
fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=train_df_xgb['y'], mode='lines', name='Training'))
# Test data
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=test_df_xgb['y'], mode='lines', name='Test'))
# Forecast
fig.add_trace(go.Scatter(x=forecast_xgb['ds'], y=forecast_xgb['forecast'], mode='lines', 
                         name='Forecast', line=dict(color='deepskyblue')))
fig.update_layout(title='Forecast vs Real Data', template='plotly_dark')
fig.show()

Calcolo metriche

In [0]:
# Calcola metriche
merged_df = pd.merge(test_df_xgb[['ds', 'y']], forecast_xgb, on='ds', how='left')
merged_df.dropna(inplace=True)
metriche_xgb = calcola_metriche(merged_df['y'],merged_df['forecast'],train_df_xgb['y'],modelname="XGBoost")
metriche_xgb

##### LSTM

In [0]:
# divisione in train e test
df_lstm = create_time_series_features(df, target_col='y') 
df_lstm = df_lstm.dropna().reset_index(drop=True)

train_size = int(len(df_lstm) * 0.9)
train_df_lstm = df_lstm.iloc[:train_size]
test_df_lstm = df_lstm.iloc[train_size:]
print(train_df_lstm.columns)
train_df_lstm.head()

In [0]:
# Scaling dei dati
train_df_original = train_df_lstm.copy()
test_df_original = test_df_lstm.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_lstm[['ds', 'unique_id']]
test_meta = test_df_lstm[['ds', 'unique_id']]

feature_cols = [col for col in train_df_lstm.columns if col not in ['ds', 'unique_id']]
train_idx = train_df_lstm.index
test_idx = test_df_lstm.index

#scaler
scaler_x = MinMaxScaler()
train_df_lstm = pd.DataFrame(scaler_x.fit_transform(train_df_lstm[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_lstm = pd.DataFrame(scaler_x.transform(test_df_lstm[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_lstm = pd.concat([train_df_lstm, train_meta], axis=1)
test_df_lstm = pd.concat([test_df_lstm, test_meta], axis=1)

#controllo le shape
train_df_lstm.shape, test_df_lstm.shape

In [0]:
# Optuna function
futr_exog_list = train_df_lstm.drop(columns=['unique_id', 'ds', 'y']).columns.tolist()
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_lstm = pd.DataFrame()
def objective(trial):
    # Hyperparametri da ottimizzare
    encoder_n_layers = trial.suggest_int("encoder_n_layers", 1, 2,4)
    encoder_hidden_size = trial.suggest_categorical("encoder_hidden_size", [32, 64, 128, 256])
    decoder_hidden_size = trial.suggest_categorical("decoder_hidden_size", [32, 64, 128, 256])
    decoder_layers = trial.suggest_int("decoder_layers", 1, 2,4)
    input_size = trial.suggest_categorical("input_size", [12, 24, 36])
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    # Definizione modello LSTM
    lstm = LSTM(
        h=12,  # nel tuning
        input_size=input_size,
        loss=DistributionLoss(distribution="Normal", level=[95]),
        scaler_type='robust',
        encoder_n_layers=encoder_n_layers,
        encoder_hidden_size=encoder_hidden_size,
        decoder_hidden_size=decoder_hidden_size,
        decoder_layers=decoder_layers,
        futr_exog_list=futr_exog_list,
        max_steps=100,
        batch_size=batch_size,
        learning_rate=learning_rate,
        recurrent=False
    )

    nf = NeuralForecast(models=[lstm], freq='MS')

    # cross-validation per tuning
    df_cv = nf.cross_validation(
        df=train_df_lstm,
        step_size=12,
        n_windows=5
    )

    # Calcola MAE per ogni fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'LSTM']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'LSTM',
            'MAE_lstm': mae,
            'RMSE_lstm': rmse
        })

    # Media MAE su tutte le fold
    cv_metrics_df_lstm = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_lstm['MAE_lstm'].mean()

    return mean_mae 

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)  #aumentare a 50 su databricks

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
with open('pickles/LSTM_bestpars_OilPrice.pkl', 'wb') as file:
    pickle.dump(best_params, file)

In [0]:
# final model
best_lstm = LSTM(
    h=len(test_df_lstm),
    input_size=len(test_df_lstm),
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=150,
    #early_stop_patience_steps=20,
    batch_size=best_params['batch_size'],
    learning_rate=best_params['learning_rate'],
    recurrent=False
)
nf = NeuralForecast(models=[best_lstm], freq='MS')

In [0]:
# final CV
df_cv = nf.cross_validation(
    df=train_df_lstm,
    step_size=len(test_df_lstm),
    n_windows=5
)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'LSTM']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'LSTM',
        'MAE_lstm': mae,
        'RMSE_lstm': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_lstm = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_lstm)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_lstm.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_lstm['unique_id'].unique():
    series_data = train_df_lstm[train_df_lstm['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_lstm['Model'] = 'lstm'
# Plot MAE sul primo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_lstm'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_lstm'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


Carico i best params

In [0]:
# #carico i best params
with open('pickles/LSTM_bestpars_OilPrice.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [0]:
# Final training
best_params = study.best_trial.params

final_model = LSTM(
    h=len(test_df_lstm),
    input_size=len(test_df_lstm),
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=150,
    #early_stop_patience_steps=20,
    batch_size=best_params['batch_size'],
    learning_rate=best_params['learning_rate'],
    recurrent=False
)

nf = NeuralForecast(models=[final_model], freq='MS')

start_time = time.time()
#fit 
nf.fit(df=train_df_lstm, val_size=len(test_df_lstm))
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

Ora faccio predict

In [0]:
Y_hat_df_lstm = nf.predict(futr_df=test_df_lstm)

In [0]:
#Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
Y_hat_df_lstm['LSTM'] = Y_hat_df_lstm['LSTM'] * (max_val - min_val) + min_val
Y_hat_df_lstm['LSTM-median'] = Y_hat_df_lstm['LSTM-median'] * (max_val - min_val) + min_val
Y_hat_df_lstm['LSTM-lo-95'] = Y_hat_df_lstm['LSTM-lo-95'] * (max_val - min_val) + min_val
Y_hat_df_lstm['LSTM-hi-95'] = Y_hat_df_lstm['LSTM-hi-95'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_lstm['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_lstm['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM'], mode='lines', 
    name='MEAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIANA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='yellow')))

# Intervallo di confidenza 95%
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_lstm['ds']) + 
    list(Y_hat_df_lstm['ds'][::-1]),
    y=list(Y_hat_df_lstm['LSTM-hi-95']) + list(Y_hat_df_lstm['LSTM-lo-95'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="LSTM forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_lstm['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")

fig.show()

In [0]:
# Calcola metriche
merged_df_lstm = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_lstm, on='ds', how='left')
merged_df_lstm.dropna(inplace=True)
metriche_lstm = calcola_metriche(merged_df_lstm['y'],merged_df_lstm['LSTM'],train_df_original['y'],modelname="LSTM")
metriche_lstm

##### NEURAL PROPHET

In [0]:
df_np = create_time_series_features(df, target_col='y') 
df_np = df_np.dropna().reset_index(drop=True)
df_np = df_np.drop(columns=['unique_id'])
train_size = int(len(df_np) * 0.9)
train_df_np = df_np.iloc[:train_size]
test_df_np = df_np.iloc[train_size:]

In [0]:
# Scaling dei dati

train_df_original = train_df_np.copy()
test_df_original = test_df_np.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_np['ds']
test_meta = test_df_np['ds']

feature_cols = [col for col in train_df_np.columns if col not in ['ds']]
train_idx = train_df_np.index
test_idx = test_df_np.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_np[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_np[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_np = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_np = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_np.shape, test_df_np.shape

TimeSeriesCV

In [0]:
# preparazione dataset
np_df_train = train_df_np[['ds', 'y']].copy()
np_df_test = test_df_np[['ds', 'y']].copy()

# Aggiungi le colonne dei regressori
regressors = ['percentChange', 'change', 'lag_1', 'lag_2', 'lag_4', 'lag_12',
       'diff_12', 'month', 'quarter', 'year', 'rolling_mean_3',
       'rolling_mean_6', 'rolling_mean_12', 'rolling_std_3', 'rolling_std_6',
       'rolling_std_12']

for reg in regressors:
    np_df_train[reg] = train_df_np[reg]
    np_df_test[reg] = test_df_np[reg]

In [0]:
# Definisci il modello
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.01, 
    batch_size=64,
    daily_seasonality=True,
    weekly_seasonality=True,
    loss_func='Huber'
)

# Aggiungi i regressori
for reg in regressors:
    neuralprophet.add_future_regressor(reg)

In [0]:
# TIME SERIES CROSS VALIDATION
initial_train_size = len(train_df_np)-5*len(test_df_np)
h = len(test_df_np)
n_windows = 5

# Reset risultati
results = []

for i in range(n_windows):
    print(f"\n--- Fold {i+1}/{n_windows} ---")

    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    train_window = np_df_train.iloc[:train_end]
    test_window = np_df_train.iloc[test_start:test_end]

    print(f"Train: {train_window.shape}, Test: {test_window.shape}")

    # Modello
    model = NeuralProphet(
        quantiles=[0.025, 0.975],
        learning_rate=0.01,
        batch_size=64,
        daily_seasonality=True,
        weekly_seasonality=True,
        loss_func='Huber',
    )

    for reg in regressors:
        model.add_future_regressor(reg)

    # Fit
    _ = model.fit(train_window, freq="MS", epochs=150)

    # Previsione
    forecast = model.predict(test_window)

    # Metriche
    y_true = test_window['y'].values
    y_pred = forecast['yhat1'].values

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    results.append({'Fold': i+1, 'MAE_nprophet': mae, 'RMSE_nprophet': rmse})
    print(f"Split {i+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# Risultati
results_df_np = pd.DataFrame(results)
print("\n=== Backtest Results ===")
print(results_df_np)

# Plot
results_df_np.plot(x='Fold', y=['MAE_nprophet', 'RMSE_nprophet'], marker='o', title='TimeSeriesCV Errors')

In [0]:
# Mi ricavo le date
initial_train_size = len(train_df_np)-5*len(test_df_np)
h = len(test_df_np)
n_windows = 5

# Crea lista per i dati delle fold
fold_data = []

for i in range(n_windows):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    # Ricava date da colonna 'ds' o dall'indice
    if 'ds' in np_df_train.columns:
        start_date = np_df_train['ds'].iloc[test_start]
        end_date = np_df_train['ds'].iloc[test_end - 1]
    else:
        start_date = np_df_train.index[test_start]
        end_date = np_df_train.index[test_end - 1]

    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]

    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)

# Converto le date in datetime se necessario
if all(isinstance(date, str) for date in fold_intervals_df['start_date']):
    fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
    fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])

print("Fold Intervals DataFrame:")
print(fold_intervals_df)

In [0]:
# CV Plot
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_np['ds'], train_df_np['y'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

Now let's training on the whole dataset

In [0]:
# preparazione dataset
np_df_train = train_df_np[['ds', 'y']].copy()
np_df_test = test_df_np[['ds', 'y']].copy()

# Aggiungi le colonne dei regressori
regressors = ['percentChange', 'change', 'lag_1', 'lag_2', 'lag_4', 'lag_12',
       'diff_12', 'month', 'quarter', 'year', 'rolling_mean_3',
       'rolling_mean_6', 'rolling_mean_12', 'rolling_std_3', 'rolling_std_6',
       'rolling_std_12']

for reg in regressors:
    np_df_train[reg] = train_df_np[reg]
    np_df_test[reg] = test_df_np[reg]

In [0]:
# Definisco il modello

# Fissa i seed per riproducibilità
import random
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.01, 
    batch_size=64,
    daily_seasonality=True,
    weekly_seasonality=True,
    loss_func='Huber'
)

# Aggiungi i regressori
for reg in regressors:
    neuralprophet.add_future_regressor(reg)

In [0]:
# FIT
start_time = time.time()
#fit 
metrics_df = neuralprophet.fit(np_df_train, 
                               freq="MS",
                               epochs = 200)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
forecast_test = neuralprophet.predict(np_df_test)
forecast_test.columns

In [0]:
# # previsioni OUT-OF-SAMPLE, per ora non le utilizzo
# #utilizzando solo il mese, il trimestre e l'anno -->> non preciso
# new_neuralprophet = NeuralProphet(
#     quantiles=[0.025, 0.975],    
#     learning_rate=0.001, 
#     batch_size=64,
#     daily_seasonality=True,
#     weekly_seasonality=True,
#     loss_func='Huber'
# )
# new_neuralprophet.add_future_regressor("month")
# new_neuralprophet.add_future_regressor("quarter")
# new_neuralprophet.add_future_regressor("year")

# # Addestra il nuovo modello con solo le feature che hai disponibili per il futuro
# new_neuralprophet.fit(
#     df=np_df_train[['ds', 'y','month', 'quarter','year']],
#     freq='MS',
#     epoches=300
# )

# # Ora puoi fare previsioni senza la temperatura
# future_dates = pd.date_range(
#     start=df_np['ds'].max() + pd.Timedelta(days=1),
#     periods=36,
#     freq='MS'
# )

# future_regressors = pd.DataFrame({
#     'ds': future_dates,
#     'month': future_dates.month,
#     'quarter': future_dates.quarter,
#     'year': future_dates.year
# })

# future = new_neuralprophet.make_future_dataframe(
#     df=df_np[['ds', 'y', 'month', 'quarter','year']],
#     periods=36,
#     regressors_df=future_regressors
# )
# forecast_future = new_neuralprophet.predict(future)

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
forecast_test['yhat1'] = forecast_test['yhat1'] * (max_val - min_val) + min_val
forecast_test['yhat1 2.5%'] = forecast_test['yhat1 2.5%'] * (max_val - min_val) + min_val
forecast_test['yhat1 97.5%'] = forecast_test['yhat1 97.5%'] * (max_val - min_val) + min_val

In [0]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))

# # Out-of-sample forecast (future)
# fig.add_trace(go.Scatter(
#     x=forecast_future['ds'], 
#     y=forecast_future['yhat1'],
#     mode='lines', 
#     line=dict(color='green'),
#     showlegend=False,  # Evita duplicati nella legenda
#     name="Out-of-sample"
# ))

#Intervallo di confidenza al 95%
fig.add_trace(go.Scatter(
    x=list(forecast_test['ds']) + 
    list(forecast_test['ds'][::-1]), 
    y=list(forecast_test['yhat1 2.5%']) + list(forecast_test['yhat1 97.5%'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast_test['ds'], 'forecast': forecast_test['yhat1']})
merged_df = pd.merge(test_df_original[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
mae_val = mean_absolute_error(merged_df['y'], merged_df['forecast'])
metriche_np = calcola_metriche(merged_df['y'], merged_df['forecast'], train_df_original['y'],modelname="NeuralProphet")
metriche_np

#### TFT

In [0]:
# Preparing dataset
df_tft = create_time_series_features(df, target_col='y') 
df_tft = df_tft.dropna().reset_index(drop=True)
# 2. Suddivisione in train/test
train_size = int(len(df_tft) * 0.9)
train_df_tft = df_tft.iloc[:train_size]
test_df_tft = df_tft.iloc[train_size:]
train_df_tft.shape, test_df_tft.shape

In [0]:
# Scaling dei dati
#Salvo per dopo
train_df_original = train_df_tft.copy() 
test_df_original = test_df_tft.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_tft[['ds', 'unique_id']]
test_meta = test_df_tft[['ds', 'unique_id']]

feature_cols = [col for col in train_df_tft.columns if col not in ['ds', 'unique_id']]
train_idx = train_df_tft.index
test_idx = test_df_tft.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_tft[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_tft[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_tft = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_tft = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_tft.shape, test_df_tft.shape

##### TimeSeriesCV - fine tuning Optuna (TROPPO LENTO)

In [0]:
# Optuna function
futr_exog_list = train_df_tft.drop(columns=['unique_id', 'ds', 'y']).columns.tolist()
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_tft = pd.DataFrame()
h = len(test_df_tft)

def objective(trial):
    # Hyperparameters to optimize for TFT
    input_size = trial.suggest_categorical("input_size", [h, int(1.5*h), int(2*h)])
    hidden_size = trial.suggest_categorical("hidden_size", [32, 64, 128, 256])
    n_rnn_layers = trial.suggest_int("n_rnn_layers", 1, 4)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    grn_activation = trial.suggest_categorical("grn_activation", ["ReLU", "ELU", "Sigmoid"])
    
    # Define TFT model
    tft = TFT(
        h=h,  # forecast horizon
        input_size=input_size,
        hidden_size=hidden_size,
        n_rnn_layers=n_rnn_layers,
        rnn_type='lstm',
        grn_activation=grn_activation,
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=learning_rate,
        futr_exog_list=futr_exog_list,
        max_steps=40, #aumentare a 100
        val_check_steps=10,
        batch_size=batch_size,
        early_stop_patience_steps=-1,
        scaler_type="robust",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False
)

    nf = NeuralForecast(models=[tft], freq='MS')

    # Cross-validation for tuning
    df_cv = nf.cross_validation(
        df=train_df_tft,
        step_size=h,
        n_windows=5
    )

    # Calculate MAE for each fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'TFT']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'TFT',
            'MAE_tft': mae,
            'RMSE_tft': rmse
        })

    # Average MAE across all folds
    cv_metrics_df_tft = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_tft['MAE_tft'].mean()

    return mean_mae

In [0]:
# final model
h = len(test_df_tft)
nf = NeuralForecast(
    models=[
        TFT(
            h=h,
            input_size=h,
            hidden_size=60,  # deve essere divisibile per 4
            grn_activation="ReLU", #prima era ELU
            rnn_type="lstm",
            n_rnn_layers=4,  #consigliato fra 2 e 5; 
            one_rnn_initial_state=False,
            loss=DistributionLoss(distribution="StudentT", level=[90]),
            learning_rate=0.005, 
            futr_exog_list=train_df_tft.drop(columns=['unique_id', 'ds', 'y']).columns.tolist(),
            max_steps=50, #aumentare su databricks
            val_check_steps=10,
            batch_size=96, 
            #early_stop_patience_steps=15,  
            scaler_type="robust",
            enable_progress_bar=True,
            accelerator="auto",  # Utilizza GPU se disponibile
        ),
    ],
    freq='MS',
)

In [0]:
# final CV
df_cv = nf.cross_validation(
    df=train_df_tft,
    step_size=h,
    n_windows=5
)

In [0]:
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'TFT']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'TFT',
        'MAE_TFT': mae,
        'RMSE_TFT': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_tft = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_tft)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_tft.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
#series_data = train_df_lstm[train_df_lstm['unique_id'] == unique_id]
plt.plot(train_df_tft['ds'], train_df_tft['y'], label='Serie unica')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_tft['Model'] = 'TFT'
# Plot MAE sul primo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_TFT'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_TFT'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


##### Now let's training on the whole dataset

In [0]:
# model fit, LENTO
h = len(test_df_tft)
nf = NeuralForecast(
    models=[
        TFT(
            h=h,
            input_size=h,
            hidden_size=60,  # deve essere divisibile per 4
            grn_activation="ReLU", #prima era ELU
            rnn_type="lstm",
            n_rnn_layers=4,  #consigliato fra 2 e 5; 
            one_rnn_initial_state=False,
            loss=DistributionLoss(distribution="StudentT", level=[90]),
            learning_rate=0.005, 
            futr_exog_list=train_df_tft.drop(columns=['unique_id', 'ds', 'y']).columns.tolist(),
            max_steps=200, #aumentare su databricks
            val_check_steps=10,
            batch_size=96, 
            early_stop_patience_steps=15,  
            scaler_type="robust",
            enable_progress_bar=True,
            accelerator="auto",  # Utilizza GPU se disponibile
        ),
    ],
    freq='MS',
)
import time
start_time = time.time()
#fit 
nf.fit(df=train_df_tft, val_size=2*h)

end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Predict
Y_hat_df_tft = nf.predict(futr_df=test_df_tft)

In [0]:
# # Path nella repo Git
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_pred_OilPrice.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(Y_hat_df_tft, f)

In [0]:
# # Rescaling dei dati
# target_col_position = feature_cols.index('y')
# min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
# max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# # Trasformazione inversa manuale
# Y_hat_df_tft['TFT'] = Y_hat_df_tft['TFT'] * (max_val - min_val) + min_val
# Y_hat_df_tft['TFT-median'] = Y_hat_df_tft['TFT-median'] * (max_val - min_val) + min_val
# Y_hat_df_tft['TFT-lo-90'] = Y_hat_df_tft['TFT-lo-90'] * (max_val - min_val) + min_val
# Y_hat_df_tft['TFT-hi-90'] = Y_hat_df_tft['TFT-hi-90'] * (max_val - min_val) + min_val

In [0]:
# #carico le previsioni:
with open('pickles/TFT_pred_OilPrice.pkl', 'rb') as file:
    Y_hat_df_tft = pickle.load(file)

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_tft['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_tft['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='lightgreen')))

# QUANTILI TFT
# Fascia tra P10 e P90
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_tft['ds']) + 
    list(Y_hat_df_tft['ds'][::-1]),
    y=list(Y_hat_df_tft['TFT-hi-90']) + list(Y_hat_df_tft['TFT-lo-90'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="TFT forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_tft['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# Calcola metriche
merged_df_tft = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_tft, on='ds', how='left')
merged_df_tft.rename(columns={'TFT-lo-90': '0.1','TFT-median':'0.5', 'TFT-hi-90':'0.9'},inplace=True)
merged_df_tft.dropna(inplace=True)
metriche_tft = calcola_metriche(merged_df_tft['y'],merged_df_tft['TFT'],train_df_original['y'],
                                y_pred_quantiles=merged_df_tft[['0.1','0.5','0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="TFT").round(10)
metriche_tft

In [0]:
# carico le metriche cv di TFT (fatte su DBR)
with open('pickles/TFT_cv_metrics_df_tft.pkl', 'rb') as file:
    cv_metrics_df_tft = pickle.load(file)

##### TFT Explainability

In [0]:
# model_tft = nf.models[0]

In [0]:
# salvo il modello (ero in DBR)
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_cv_metrics_df_tft.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(cv_metrics_df_tft, f)
# salvo il modello
# torch.save(model_tft, 'TFT_model.pth')

In [0]:
# carico il modello
model_tft = torch.load('pickles/TFT_model.pth', map_location=torch.device('cpu'))

In [0]:
attention = model_tft.attention_weights()
def plot_attention(
    self, plot: str = "time", output: str = "plot", width: int = 800, height: int = 400
):
    """
    Plot the attention weights.

    Args:
        plot (str, optional): The type of plot to generate. Can be one of the following:
            - 'time': Display the mean attention weights over time.
            - 'all': Display the attention weights for each horizon.
            - 'heatmap': Display the attention weights as a heatmap.
            - An integer in the range [1, model.h) to display the attention weights for a specific horizon.
        output (str, optional): The type of output to generate. Can be one of the following:
            - 'plot': Display the plot directly.
            - 'figure': Return the plot as a figure object.
        width (int, optional): Width of the plot in pixels. Default is 800.
        height (int, optional): Height of the plot in pixels. Default is 400.

    Returns:
        matplotlib.figure.Figure: If `output` is 'figure', the function returns the plot as a figure object.
    """

    attention = (
        self.mean_on_batch(self.interpretability_params["attn_wts"])
        .mean(dim=0)
        .cpu()
        .numpy()
    )

    fig, ax = plt.subplots(figsize=(width / 100, height / 100))

    if plot == "time":
        attention = attention[self.input_size :, :].mean(axis=0)
        ax.plot(np.arange(-self.input_size, self.h), attention)
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Mean Attention")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "all":
        for i in range(self.input_size, attention.shape[0]):
            ax.plot(
                np.arange(-self.input_size, self.h),
                attention[i, :],
                label=f"horizon {i-self.input_size+1}",
            )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Attention per horizon")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "heatmap":
        cax = ax.imshow(
            attention,
            aspect="auto",
            cmap="viridis",
            extent=[-self.input_size, self.h, -self.input_size, self.h],
        )
        fig.colorbar(cax)
        ax.set_title("Attention Heatmap")
        ax.set_xlabel("Attention (current time step)")
        ax.set_ylabel("Attention (previous time step)")

    elif isinstance(plot, int) and (plot in np.arange(1, self.h + 1)):
        i = self.input_size + plot - 1
        ax.plot(
            np.arange(-self.input_size, self.h),
            attention[i, :],
            label=f"horizon {plot}",
        )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title(f"Attention weight for horizon {plot}")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    else:
        raise ValueError(
            'plot has to be in ["time","all","heatmap"] or integer in range(1,model.h)'
        )

    plt.tight_layout()

    if output == "plot":
        plt.show()
    elif output == "figure":
        return fig
    else:
        raise ValueError(f"Invalid output: {output}. Expected 'plot' or 'figure'.")

In [0]:
# MEAN ATTENTION
plot_attention(model_tft, plot="time")
# pesi di attenzione medi nel tempo su tutti gli orizzonti di previsione.
# La curva blu rappresenta quanto il modello ha "guardato" (attention) 
# quel punto temporale in input durante la previsione.
# Valori più alti significano che il modello ritiene quel punto temporale
# particolarmente rilevante per fare le previsioni

In [0]:
# ATTENTION OF A SPECIFIC TIME STEPS
plot_attention(nf.models[0], plot=44)
# pesi di attenzione per ogni orizzonte di previsione separatamente.

In [0]:
plot_attention(nf.models[0], plot="heatmap")

In [0]:
# PAST VARIABLE IMPORTANCES
feature_importances = model_tft.feature_importances()
print(feature_importances.keys())
print("\n ==========PAST FEATURE IMPORTANCES==========")
feature_importances["Past variable importance over time"].mean().sort_values().plot(kind="barh")

Questo grafico mostra quanto il modello ha fatto affidamento sulle feature passate \
(quelle che il modello ha potuto osservare prima del punto di previsione). Serve a costruire il contesto dell'input

In [0]:
# FUTURE VARIABLE IMPORTANCES
print("\n ==========FUTURE FEATURE IMPORTANCES==========")
feature_importances["Future variable importance over time"].mean().sort_values().plot(kind="barh")

Mostra quanto il modello ha sfruttato le feature che sono servite a guidare le previsioni passo-passo nel decoder

DIFFERENZA TRA PAST E FUTURE: \
La diversa importanza riflette quanto ciascuna variabile aiuta nell’analisi del contesto passato (encoder -->> PAST) \
o nella generazione delle previsioni (decoder -->> FUTURE).

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME
df_importances = feature_importances["Past variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances.index))

for col in df_importances.columns:
    p = ax.bar(np.arange(-len(df_importances), 0), df_importances[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances[col]
ax.set_title("Past variable importance over time")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)

plt.show()

Questo grafico mostra QUANDO (e QUANTO) nel passato ogni feature è stata più utile per costruire il contesto predittivo.

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME
df_importances2 = feature_importances["Future variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances2.index))
for col in df_importances2.columns:
    p = ax.bar(np.arange(-len(df_importances2), 0), df_importances2[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances2[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.grid(True)
ax.legend()
plt.show()

Per ogni feature viene mostrato quanto è stata importante per il modello a ciascun orizzonte futuro

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances3 = feature_importances["Past variable importance over time"]
mean_attention = (
    nf.models[0]
    .attention_weights()[nf.models[0].input_size :, :]
    .mean(axis=0)[: nf.models[0].input_size]
)
df_importances3 = df_importances3.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances3.index))

for col in df_importances3.columns:
    p = ax.bar(np.arange(-len(df_importances3), 0), df_importances3[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances3[col]
ax.set_title("Past variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances3), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances4 = feature_importances["Future variable importance over time"]
mean_attention = (
    nf.models[0]
    .attention_weights()[nf.models[0].input_size :, :]
    .mean(axis=0)[: nf.models[0].input_size]
)
df_importances4 = df_importances4.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances4.index))

for col in df_importances4.columns:
    p = ax.bar(np.arange(-len(df_importances4), 0), df_importances4[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances4[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances4), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
nf.models[0].feature_importance_correlations()

#### CHRONOS

item_id, timestamp, target (Chronos calcola tempo di fitting in automatico)

In [0]:
# Preparing dataset
df_chronos = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos = df_chronos.dropna().reset_index(drop=True) 
df_chronos['item_id'] = df_chronos['unique_id'] 
df_chronos.drop(columns="unique_id", inplace=True) 
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Definizione delle covariate note
known_covariates_names=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id','target']]
feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

# Split train e test
train_size = int(len(df_chronos) * 0.9)
train_df_chronos = df_chronos.iloc[:train_size]
test_df_chronos = df_chronos.iloc[train_size:]

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [0]:
# Scaling dei dati
train_df_original = train_df_chronos.copy() 
test_df_original = test_df_chronos.copy()

train_idx = train_df_chronos.index
test_idx = test_df_chronos.index

#scaler
scaler_x = MinMaxScaler()
train_df_chronos = pd.DataFrame(scaler_x.fit_transform(train_df_chronos[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_chronos = pd.DataFrame(scaler_x.transform(test_df_chronos[feature_cols]),
                            columns=feature_cols, index=test_idx)
#controllo le shape
train_df_chronos.shape, test_df_chronos.shape

TimeSeriesCV

In [0]:
# TimeSeriesCV
h = len(test_df_chronos)  # prediction length
n_splits = 5
initial_train_size = len(train_df_chronos) - 5*h
results = []

for fold in range(n_splits):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")
    
    train_end = initial_train_size + fold * h
    test_start = train_end
    test_end = test_start + h
    train_df_fold = train_df_chronos.iloc[:train_end]
    test_df_fold = train_df_chronos.iloc[test_start:test_end]
    
    train_df_fold.reset_index(inplace=True)
    test_df_fold.reset_index(inplace=True)
    
    train_df_fold["timestamp"] = pd.to_datetime(train_df_fold["timestamp"])
    test_df_fold["timestamp"] = pd.to_datetime(test_df_fold["timestamp"])
    
    # Future timestamps 
    future_index = pd.date_range(
        start=train_df_fold["timestamp"].max() + pd.Timedelta(days=1),
        periods=h,
        freq="MS")
    
    # Preparo test set con known_covariates
    test_df_for_prediction = test_df_fold[test_df_fold["timestamp"].isin(future_index)].copy()
    test_df_for_prediction = test_df_for_prediction.set_index("timestamp")
    test_df_for_prediction = test_df_for_prediction.reindex(future_index)  # forza continuità
    test_df_for_prediction["item_id"] = train_df_fold["item_id"].iloc[0]  # supponiamo 1 sola serie
    test_df_for_prediction = test_df_for_prediction.reset_index().rename(columns={"index": "timestamp"})
    test_df_for_prediction = test_df_for_prediction.set_index(["item_id", "timestamp"])
    
    # Preparo il train con multindex
    train_df_fold = train_df_fold.set_index(["item_id", "timestamp"])
    print(f"Train: {train_df_fold.shape}, Test: {test_df_fold.shape}")
    
    # inizializzo il predictor
    predictor = TimeSeriesPredictor(
        prediction_length=h,
        target="target",
        known_covariates_names=known_covariates_names,
        eval_metric="MASE",
        freq="MS")
    
    # fit del modello
    predictor.fit(
        train_df_chronos,
        hyperparameters={
            "Chronos": [
                {
                    "model_path": "bolt_small",
                    "ag_args": {"name_suffix": "ZeroShot"}},
                {
                    "model_path": "bolt_small",
                    "covariate_regressor": "CAT",
                    "target_scaler": "robust",
                    "ag_args": {"name_suffix": "WithRegressor"},
                },
            ],
        },
        time_limit=600,
        enable_ensemble=True,  # Attiva ensemble per migliorare l'accuratezza
        )
    
    predictions = predictor.predict(train_df_fold, known_covariates=test_df_for_prediction)

    test_df_with_index = test_df_fold.set_index(["item_id", "timestamp"])

    # Debug
    print(f"Forma predictions: {predictions.shape}")
    print(f"Colonne predictions: {predictions.columns}")
    print(f"Numero di indici in predictions: {len(predictions.index)}")
    print(f"Numero di indici in test_df_with_index: {len(test_df_with_index.index)}")

    # trovo solo gli indici comuni
    common_indices = predictions.index.intersection(test_df_with_index.index)
    print(f"Indici comuni: {len(common_indices)}")

    # prendo solo gli indici comuni
    y_true = test_df_with_index.loc[common_indices, "target"]
    y_pred = predictions.loc[common_indices, 'mean'].to_numpy()

    mae = calcola_mae(y_true, y_pred)
    rmse = calcola_rmse(y_true, y_pred)
    
    results.append({'Fold': fold+1, 'MAE_chronos': mae, 'RMSE_chronos': rmse})
    print(f"Split {fold+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
# risultati finali
results_df_chronos = pd.DataFrame(results)
results_df_chronos.set_index('Fold',inplace=True)
print("\n=== TimeSeriesCV Results ===")
results_df_chronos

In [0]:
# Estraggo le date
fold_data = []
train_df_chronos.reset_index(inplace=True)
for i in range(n_splits):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h
    
    # Ricava date da colonna 'ds' o dall'indice
    if 'timestamp' in train_df_chronos.columns:
        start_date = train_df_chronos['timestamp'].iloc[test_start]
        end_date = train_df_chronos['timestamp'].iloc[test_end - 1]
    else:
        start_date = train_df_chronos.index[test_start]
        end_date = train_df_chronos.index[test_end - 1]
    
    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]
    
    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)
fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])
# Stampa il dataframe prima della conversione per verificare i valori
print("Fold Intervals DataFrame (before conversion):")
fold_intervals_df

In [0]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_chronos['timestamp'], train_df_chronos['target'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot degli errori
results_df_chronos.reset_index(inplace=True)
ax = results_df_chronos.plot(x='Fold', y=['MAE_chronos','RMSE_chronos'], marker='o', title='TimeSeriesCV Errors')
ax.set_xticks(range(1, 6)) # Imposta le tacche sull'asse x da 1 a 5
plt.show()

In [0]:
results_df_chronos

Let's training on the whole dataset

In [0]:
# Fit del modello

# SOLO PER DATABRICKS
# sys.modules['sklearn.metrics._regression'].mean_absolute_error = sklearn.metrics.mean_absolute_error

predictor = TimeSeriesPredictor(
    prediction_length=len(test_df_chronos), 
    eval_metric="MASE",
    target="target",
    known_covariates_names=known_covariates_names
)

# Aggiunta di più modelli nella configurazione per migliorare le performance
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {
                "model_path": "bolt_small",
                "ag_args": {"name_suffix": "ZeroShot"}},
            {
                "model_path": "bolt_small",
                "covariate_regressor": "CAT",
                "target_scaler": "robust",
                "ag_args": {"name_suffix": "WithRegressor"},
            }
        ],
    },
    time_limit=600,
    enable_ensemble=True,  # Attiva ensemble per migliorare l'accuratezza
)

# Valutazione del modello in fase di training
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [0]:
# PREVISIONI CLASSICHE
# col miglior modello e col 0-shot
predictions = predictor.predict(
    train_df_chronos, 
    known_covariates=test_df_chronos
)
predictions.reset_index(inplace=True)

predictions_0shot = predictor.predict(
    train_df_chronos, 
    known_covariates=test_df_chronos,
    model="ChronosZeroShot[bolt_small]"
)
predictions_0shot.reset_index(inplace=True)


In [0]:
# # PREVISIONI MULTI STEP ????
# # Separa le covariate in due gruppi
# known_ahead_covariates = ['month', 'quarter', 'year']  # Queste sono realmente conosciute in anticipo

# lagged_covariates = [
#     'lag_1', 'lag_2', 'lag_4', 'lag_12',
#     'diff_12', 'change', 'percentChange',
#     'rolling_mean_3', 'rolling_mean_6', 'rolling_mean_12',
#     'rolling_std_3', 'rolling_std_6', 'rolling_std_12'
# ]
# # Configurazione del predictor
# predictor = TimeSeriesPredictor(
#     prediction_length=len(test_df_chronos),  # Ad esempio 6 per 6 mesi
#     eval_metric="MASE",
#     target="target",
#     known_covariates_names=known_ahead_covariates
# )

# test_known_covariates = test_df_chronos[known_ahead_covariates]

# # Fit del modello
# predictor.fit(
#     train_df_chronos,
#     hyperparameters={
#         "Chronos": [
#             {
#                 "model_path": "bolt_small",
#                 "ag_args": {"name_suffix": "ZeroShot"}
#             },
#             {
#                 "model_path": "bolt_small",
#                 "covariate_regressor": "CAT",
#                 "target_scaler": "robust",
#                 "ag_args": {"name_suffix": "WithRegressor"},
#             },
#         ],
#     },
#     time_limit=600,
#     enable_ensemble=True,
# )

# # Previsioni appropriate al contesto
# # Per un multi-step puro senza feedback
# predictions = predictor.predict(
#     train_df_chronos,
#     known_covariates=test_known_covariates  # Solo le covariate realmente note in anticipo
# )
# predictions.reset_index(inplace=True)

# predictions_0shot = predictor.predict(
#     train_df_chronos,
#     known_covariates=test_known_covariates,
#     model="ChronosZeroShot[bolt_small]"
# )
# predictions_0shot.reset_index(inplace=True)

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('target')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
predictions['mean'] = predictions['mean'] * (max_val - min_val) + min_val
predictions['0.1'] = predictions['0.1'] * (max_val - min_val) + min_val
predictions['0.5'] = predictions['0.5'] * (max_val - min_val) + min_val
predictions['0.9'] = predictions['0.9'] * (max_val - min_val) + min_val
predictions_0shot['mean'] = predictions_0shot['mean'] * (max_val - min_val) + min_val
predictions_0shot['0.5'] = predictions_0shot['0.5'] * (max_val - min_val) + min_val
predictions_0shot['0.9'] = predictions_0shot['0.9'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)

# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_chronos['timestamp'], y=train_df_original['target'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['0.5'], mode='lines', 
    name='P50', line=dict(color='green')))
# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='ChronosWithRegressor', line=dict(color='lightgreen')))
# QUANTILI CHRONOS
fig.add_trace(go.Scatter(
    x=list(test_df_chronos['timestamp']) + 
    list(test_df_chronos['timestamp'][::-1]),
    y=list(predictions['0.9']) + list(predictions['0.1'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# PREVISIONI CHRONOS 0 SHOT
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='0-shot Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="CHRONOS forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_chronos['timestamp'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# metriche di valutazione
test_df_original.reset_index(inplace=True)
merged_df_chronos = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)
metriche_chronos = calcola_metriche(merged_df_chronos['target'],merged_df_chronos['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos").round(5)
metriche_chronos

#### final evaluations

In [0]:
final_metrics = metriche_arima.join(metriche_ets).join(metriche_xgb).join(metriche_lstm).join(metriche_np).join(metriche_tft).join(metriche_chronos).round(4)
final_metrics

In [0]:
# FINAL PREDICTIONS PLOT FOR OIL PRICE
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['target'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoARIMA'], 
    mode='lines', 
    name='ARIMA', 
    line=dict(color='grey')
))

# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoETS'], 
    mode='lines', 
    name='ETS', 
    line=dict(color='orange')
))

# PREVISIONI XGBOOST
fig.add_trace(go.Scatter(
    x=forecast_xgb['ds'], 
    y=forecast_xgb['forecast'], 
    mode='lines', 
    name='XGBoost', 
    line=dict(color='purple')
))

# PREVISIONI LSTM
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=Y_hat_df_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='pink')
))

# PREVISIONI NEURAL PROPHET
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='aquamarine')
))


# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=forecast_test['ds'], 
                         #y=Y_hat_df_tft['TFT'], 
                         y=Y_hat_df_tft,
                         mode='lines', 
    name='TFT', line=dict(color='yellow')))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS', line=dict(color='green')))
    
# PREVISIONI CHRONOS 0 shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS 0-shot', line=dict(color='lightgreen')))

# Layout del grafico
fig.update_layout(
    title="Forecasts for White Noise", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
# sistemo i vari dataset prima del merge
#results_df_chronos.drop(columns=(['level_0', 'index']),inplace=True)
#cv_metrics_df_tft.rename(columns={'split': 'Fold'}, inplace=True)
cv_metrics_df_lstm.drop(columns=('Model'),inplace=True)
df_arima = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == 'AutoARIMA'][['Fold',
                                                                    'MAE_AutoARIMA','RMSE_AutoARIMA']]
df_ets = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == 'AutoETS'][['Fold',
                                                                    'MAE_AutoETS','RMSE_AutoETS']]

In [0]:
# Final plot metrics TimeSeriesCV

# faccio il merge di tutte le tabelle dei vari CV
final_cv = results_df_np.merge(cv_metrics_df_lstm, 
                            on="Fold",how="inner").merge(cv_metrics_df_tft, 
                            on="Fold",how="inner").merge(results_df_chronos, 
                            on="Fold",how="inner").merge(cv_metrics_df_xgboost[['MAE_XGB','RMSE_XGB','Fold']],
                            on="Fold",how="inner").merge(df_arima,
                            on="Fold",how="inner").merge(df_ets,
                            on="Fold", how="inner")
# Rescaling CV metrics data
columns_to_modify = ['MAE_nprophet', 'RMSE_nprophet', 'MAE_lstm', 'RMSE_lstm', 'MAE_TFT', 'RMSE_TFT',
                        'MAE_chronos', 'RMSE_chronos', 'MAE_XGB', 'RMSE_XGB']
for col in columns_to_modify:
    final_cv[f'{col}'] = final_cv[f'{col}'] * (max_val - min_val) + min_val

model_colors = {
    'AutoARIMA': '#808080',      # Grey
    'AutoETS': '#ff7f0e',        # Orange
    'XGB': '#9467bd',    # Purple
    'lstm': '#e377c2',       # Pink
    'nprophet': '#7fffd4',         # Aquamarine
    'TFT': '#ffff00',        # Yellow
    'chronos': '#2ca02c',    # Green
}


# Create the figure
fig = go.Figure()

# Add traces for each model - MAE (darker) and RMSE (lighter)
for model, color in model_colors.items():
    # Add MAE line (darker)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'MAE_{model}'],
        mode='lines+markers',
        name=f'{model} MAE',
        line=dict(color=color, width=3),
        marker=dict(size=8)
    ))
    
    # Add RMSE line (lighter with same color but different dash pattern)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'RMSE_{model}'],
        mode='lines+markers',
        name=f'{model} RMSE',
        line=dict(color=color, width=2, dash='dash'),
        marker=dict(size=6)
    ))

# Update layout
fig.update_layout(
    title='Model Performance Comparison (MAE and RMSE)',
    template='plotly_dark',
    xaxis=dict(title='Fold',
        tickmode='linear',
        tick0=1, dtick=1
    ),    yaxis=dict(
        title='Error Value'
    ),    legend=dict(
        orientation="v"
    ),    hovermode="closest")
fig.show()

In [0]:
# Confronto TFT/CHRONOS
tft_chronos = metriche_chronos.join(metriche_tft)
tft_chronos.round(4)                                

### **OIL PRICE (multistep)**

H=12 mesi 

In [0]:
# preparazione dataset
oil_prices.dropna(inplace=True)
df = oil_prices.reset_index().rename(columns={'date': 'ds','price':'y'})
df['unique_id'] = 'serie_1'
df['ds'] = pd.to_datetime(df['ds'])

In [0]:
# funzione per creare feature
def create_time_series_features(df, target_col):
    df = df.copy()
    # features
    lags = [1, 2, 4, 12]
    # Lag features (1-4)
    for lag in lags:
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    
    df['diff_12'] = (df[target_col] - df[target_col].shift(12)).shift(1)
    
    df['change'] = (df[target_col] - df[target_col].shift(1)).shift(1)
    df['percentChange'] = df[target_col].pct_change().shift(1)

    df['month'] = df['ds'].dt.month
    df['quarter'] = df['ds'].dt.quarter
    df['year'] = df['ds'].dt.year
    
    # rolling means 
    df['rolling_mean_3'] = df[target_col].rolling(window=3).mean().shift(1) #ultimo trimestre
    df['rolling_mean_6'] = df[target_col].rolling(window=6).mean().shift(1) #ultima metà dell'anno
    df['rolling_mean_12'] = df[target_col].rolling(window=12).mean().shift(1) #ultimo anno
    df['rolling_std_3'] = df[target_col].rolling(window=3).std().shift(1)
    df['rolling_std_6'] = df[target_col].rolling(window=6).std().shift(1)
    df['rolling_std_12'] = df[target_col].rolling(window=12).std().shift(1)
    return df

In [0]:
h = 12
future_features = ['ds', 'y','month', 'quarter', 'year']
futr_exog_list = ['month', 'quarter', 'year']

##### XGBOOST

In [0]:
# df_xgb
df_xgb = create_time_series_features(df, target_col='y') 
hist_exog_features = df_xgb.drop(columns=['unique_id', 'ds', 'y'] + futr_exog_list).columns.tolist()
df_xgb.drop(columns=['unique_id'],inplace=True)
df_xgb = df_xgb.dropna().reset_index(drop=True) #droppo i NaN creati dai lag, quindi le prime 12 obs.
df_xgb['covid'] = (df_xgb['ds'] > pd.Timestamp('2020-02-01')).astype(int)

# STEP 2: Prepara DMatrix per previsione
df_xgb = df_xgb[future_features]
print(df_xgb.columns)
df_xgb.head()

In [0]:
# Split train/test
# train tutto tranne le ultime 12 osservazioni
train_df_xgb = df_xgb.iloc[:-h]
test_df_xgb = df_xgb.iloc[-h:]

# Separazione features e target
feature_cols = [col for col in df_xgb.columns if col not in ['y', 'ds']]
X_train_xgb, y_train_xgb = train_df_xgb[feature_cols], train_df_xgb['y']
X_test_xgb, y_test_xgb = test_df_xgb[feature_cols], test_df_xgb['y']

In [0]:
# train e test plot
fig = go.Figure()
#train dat
fig.add_trace(go.Scatter(x=df_xgb['ds'], y=y_train_xgb, mode='lines', name='Train'))
#test data 
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=y_test_xgb, mode='lines', name='Test'))
#layout
fig.update_layout(template='plotly_dark', title='Train and Test Data Plot')
fig.show()

In [0]:
# Scaling dei dati
#features
scaler_x = MinMaxScaler()
X_train_scaled = scaler_x.fit_transform(X_train_xgb)
X_test_scaled = scaler_x.transform(X_test_xgb)

# target, non serve che scalo y_test
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train_xgb.values.reshape(-1, 1))

# Ricreo df con stessi indici e nomi
X_train_xgb = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train_xgb.index)
X_test_xgb = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test_xgb.index)
y_train_xgb = pd.Series(y_train_scaled.flatten(), name='y', index=y_train_xgb.index)

#aggiorno train_df_xgb
train_df_xgb[feature_cols] = X_train_xgb
train_df_xgb['y'] = y_train_xgb

#controllo le shape
X_train_xgb.shape, X_test_xgb.shape, y_train_xgb.shape, y_test_xgb.shape

Ottimizzazione Optuna (Bayesiana)

In [0]:
# Optuna function
cv_metrics_log = []  # List globale per salvare metriche fold per fold

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'verbosity': 0,
        'seed': 42
    }

    tscv = TimeSeriesSplit(n_splits=5,test_size=len(test_df_xgb))
    y = train_df_xgb['y']
    df_xgb_feature = train_df_xgb.drop(columns=['y','ds'])
    all_rmse = []

    for fold, (train_idx, test_idx) in enumerate(tscv.split(df_xgb_feature)):
        X_train, X_test = df_xgb_feature.iloc[train_idx], df_xgb_feature.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        dtest = xgb.DMatrix(X_test.values, label=y_test.values)

        model = xgb.train(params, dtrain, num_boost_round=500,
                          evals=[(dtest, 'eval')],
                          early_stopping_rounds=30, verbose_eval=False)

        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mae = calcola_mae(y_test, preds)  

        all_rmse.append(rmse)

        # Recupera le date (se esiste colonna 'ds')
        start_date = df_xgb.iloc[test_idx]['ds'].min() if 'ds' in df_xgb.columns else None
        end_date = df_xgb.iloc[test_idx]['ds'].max() if 'ds' in df_xgb.columns else None

        # Logga le metriche della fold
        cv_metrics_log.append({
            'Trial': trial.number,
            'Fold': fold + 1,
            'MAE_XGB': mae,
            'RMSE_XGB': rmse,
            'start_date': start_date,
            'end_date': end_date,
            'Model': 'XGBoost'  
        })
    return np.mean(all_rmse)

In [0]:
# Optuna optimization
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)

In [0]:
# Best parameters
print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)
print("Best trial:",study.best_trial.number)
besttrial = study.best_trial.number

In [0]:
cv_metrics_log = pd.DataFrame(cv_metrics_log)
cv_metrics_log = cv_metrics_log[cv_metrics_log['Trial']==besttrial]
cv_metrics_log

Vediamo come ha performato nel miglior trial

In [0]:
#PLOT CV
cv_metrics_df_xgboost = cv_metrics_log.copy()

# Stampa le metriche per modello
print("Metriche per modello:")
print(cv_metrics_df_xgboost)

# Calcola e stampa le metriche medie per modello
mean_metrics = cv_metrics_df_xgboost.mean(numeric_only=True).reset_index()

# Crea il DataFrame con gli intervalli delle fold
fold_intervals_df = cv_metrics_df_xgboost[['Fold', 'start_date', 'end_date']].drop_duplicates()

# Ora creiamo il grafico combinato
plt.figure(figsize=(16, 12))

# === PLOT 1: Serie temporale originale ===
plt.subplot(2, 1, 1)
series_data = train_df_xgb.copy()
plt.plot(series_data['ds'], series_data['y'], label='Oil Price - XGB, CV')

# Evidenzia le fold con colori
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)],
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'{row["Fold"]}', ha='center', fontsize=10,
             bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# === PLOT 2: Metriche MAE e RMSE ===
plt.subplot(2, 1, 2)
ax1 = plt.gca()
ax2 = ax1.twinx()

# MAE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_XGB'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# RMSE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_XGB'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Legenda combinata
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


Ora facciamo training coi migliori parametri trovati da Optuna

In [0]:
# Addestriamo il modello con i best params
best_params = study.best_params
best_params.update({'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 42})

dtrain_xgb = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
start_time = time.time()
#fit 
model = xgb.train(best_params, dtrain_xgb, num_boost_round=500)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot feature importance
fig, ax = plt.subplots()
xgb.plot_importance(model, importance_type='gain', max_num_features=10, ax=ax)
for text in ax.texts:
    text.set_visible(False)
plt.title('Feature Importance (Gain)')
plt.tight_layout()
plt.show()

In [0]:
dtest_xgb = xgb.DMatrix(X_test_xgb)
forecast_xgb = pd.DataFrame(model.predict(dtest_xgb),columns=['forecast'])
forecast_xgb['ds'] = test_df_xgb['ds'].values

Rescaling dei dati

In [0]:
# RE-Scaling dei dati
forecast_xgb['forecast'] = scaler_y.inverse_transform(forecast_xgb[['forecast']])
train_df_xgb['y'] = scaler_y.inverse_transform(train_df_xgb[['y']])

In [0]:
# Plot forecast
fig = go.Figure()
# Training data
fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=train_df_xgb['y'], mode='lines', name='Training'))
# Test data
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=test_df_xgb['y'], mode='lines', name='Test'))
# Forecast
fig.add_trace(go.Scatter(x=forecast_xgb['ds'], y=forecast_xgb['forecast'], mode='lines', 
                         name='Forecast', line=dict(color='deepskyblue')))
fig.update_layout(title='Forecast vs Real Data', template='plotly_dark')
fig.show()

Calcolo metriche

In [0]:
# Calcola metriche
merged_df = pd.merge(test_df_xgb[['ds', 'y']], forecast_xgb, on='ds', how='left')
merged_df.dropna(inplace=True)
metriche_xgb = calcola_metriche(merged_df['y'],merged_df['forecast'],train_df_xgb['y'],modelname="XGBoost")
metriche_xgb

##### LSTM

In [0]:
# divisione in train e test
df_lstm = create_time_series_features(df, target_col='y') 
df_lstm = df_lstm.dropna().reset_index(drop=True)

train_df_lstm = df_lstm.iloc[:-h]
test_df_lstm = df_lstm.iloc[-h:]
print(train_df_lstm.columns)
train_df_lstm.head()

In [0]:
# Scaling dei dati
train_df_original = train_df_lstm.copy()
test_df_original = test_df_lstm.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_lstm[['ds', 'unique_id']]
test_meta = test_df_lstm[['ds', 'unique_id']]

feature_cols = [col for col in train_df_lstm.columns if col not in ['ds', 'unique_id']]
train_idx = train_df_lstm.index
test_idx = test_df_lstm.index

#scaler
scaler_x = MinMaxScaler()
train_df_lstm = pd.DataFrame(scaler_x.fit_transform(train_df_lstm[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_lstm = pd.DataFrame(scaler_x.transform(test_df_lstm[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_lstm = pd.concat([train_df_lstm, train_meta], axis=1)
test_df_lstm = pd.concat([test_df_lstm, test_meta], axis=1)

#controllo le shape
train_df_lstm.shape, test_df_lstm.shape

In [0]:
# Optuna function
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_lstm = pd.DataFrame()
def objective(trial):
    # Hyperparametri da ottimizzare
    encoder_n_layers = trial.suggest_int("encoder_n_layers", 1, 2,4)
    encoder_hidden_size = trial.suggest_categorical("encoder_hidden_size", [32, 64, 128, 256])
    decoder_hidden_size = trial.suggest_categorical("decoder_hidden_size", [32, 64, 128, 256])
    decoder_layers = trial.suggest_int("decoder_layers", 1, 2,4)
    input_size = trial.suggest_categorical("input_size", [12, 24, 36])
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    # Definizione modello LSTM
    lstm = LSTM(
        h=len(test_df_lstm), 
        input_size=input_size,
        loss=DistributionLoss(distribution="Normal", level=[95]),
        scaler_type='robust',
        encoder_n_layers=encoder_n_layers,
        encoder_hidden_size=encoder_hidden_size,
        decoder_hidden_size=decoder_hidden_size,
        decoder_layers=decoder_layers,
        futr_exog_list=futr_exog_list,
        max_steps=100,
        batch_size=batch_size,
        learning_rate=learning_rate,
        recurrent=False
    )

    nf = NeuralForecast(models=[lstm], freq='MS')

    # cross-validation per tuning
    df_cv = nf.cross_validation(
        df=train_df_lstm,
        step_size=len(test_df_lstm),
        n_windows=5
    )

    # Calcola MAE per ogni fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'LSTM']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'LSTM',
            'MAE_lstm': mae,
            'RMSE_lstm': rmse
        })

    # Media MAE su tutte le fold
    cv_metrics_df_lstm = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_lstm['MAE_lstm'].mean()

    return mean_mae 

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)  #aumentare a 50 su databricks

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
with open('pickles/LSTM_bestpars_OilPrice_multistep.pkl', 'wb') as file:
    pickle.dump(best_params, file)

Carico i best params

In [0]:
with open('pickles/LSTM_bestpars_OilPrice_multistep.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [0]:
# final model
best_lstm = LSTM(
    h=len(test_df_lstm),
    input_size=len(test_df_lstm),
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=150,
    #early_stop_patience_steps=20,
    batch_size=best_params['batch_size'],
    learning_rate=best_params['learning_rate'],
    recurrent=False
)
nf = NeuralForecast(models=[best_lstm], freq='MS')

In [0]:
# final CV
df_cv = nf.cross_validation(
    df=train_df_lstm,
    step_size=len(test_df_lstm),
    n_windows=5
)
with open('pickles/LSTM_cv_OilPrice_multistep.pkl', 'wb') as file:
    pickle.dump(best_params, file)
# with open('pickles/LSTM_cv_OilPrice_multistep.pkl', 'rb') as file:
#     best_params = pickle.load(file)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'LSTM']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'LSTM',
        'MAE_lstm': mae,
        'RMSE_lstm': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_lstm = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_lstm)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_lstm.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_lstm['unique_id'].unique():
    series_data = train_df_lstm[train_df_lstm['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_lstm['Model'] = 'lstm'
# Plot MAE sul primo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_lstm'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_lstm'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


In [0]:
# Final training
final_model = LSTM(
    h=len(test_df_lstm),
    input_size=len(test_df_lstm),
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=150,
    #early_stop_patience_steps=20,
    batch_size=best_params['batch_size'],
    learning_rate=best_params['learning_rate'],
    recurrent=False
)

nf = NeuralForecast(models=[final_model], freq='MS')

start_time = time.time()
#fit 
nf.fit(df=train_df_lstm, val_size=len(test_df_lstm))
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

Ora faccio predict

In [0]:
Y_hat_df_lstm = nf.predict(futr_df=test_df_lstm[['unique_id', 'ds'] + futr_exog_list])

In [0]:
#Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
Y_hat_df_lstm['LSTM'] = Y_hat_df_lstm['LSTM'] * (max_val - min_val) + min_val
Y_hat_df_lstm['LSTM-median'] = Y_hat_df_lstm['LSTM-median'] * (max_val - min_val) + min_val
Y_hat_df_lstm['LSTM-lo-95'] = Y_hat_df_lstm['LSTM-lo-95'] * (max_val - min_val) + min_val
Y_hat_df_lstm['LSTM-hi-95'] = Y_hat_df_lstm['LSTM-hi-95'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_lstm['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_lstm['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM'], mode='lines', 
    name='MEAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIANA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='yellow')))

# Intervallo di confidenza 95%
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_lstm['ds']) + 
    list(Y_hat_df_lstm['ds'][::-1]),
    y=list(Y_hat_df_lstm['LSTM-hi-95']) + list(Y_hat_df_lstm['LSTM-lo-95'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="LSTM forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_lstm['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")

fig.show()

In [0]:
# Calcola metriche
merged_df_lstm = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_lstm, on='ds', how='left')
merged_df_lstm.dropna(inplace=True)
metriche_lstm = calcola_metriche(merged_df_lstm['y'],merged_df_lstm['LSTM'],train_df_original['y'],modelname="LSTM")
metriche_lstm

##### NEURAL PROPHET

In [0]:
df_np = create_time_series_features(df, target_col='y') 
df_np = df_np.dropna().reset_index(drop=True)
df_np = df_np[future_features]
train_df_np = df_np.iloc[:-h]
test_df_np = df_np.iloc[-h:]

In [0]:
# Scaling dei dati
train_df_original = train_df_np.copy()
test_df_original = test_df_np.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_np['ds']
test_meta = test_df_np['ds']

feature_cols = [col for col in train_df_np.columns if col not in ['ds']]
train_idx = train_df_np.index
test_idx = test_df_np.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_np[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_np[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_np = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_np = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_np.shape, test_df_np.shape

TimeSeriesCV

In [0]:
# CV
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

initial_train_size = len(train_df_np) - 5 * len(test_df_np)
h = len(test_df_np)
n_windows = 5

results = []

for i in range(n_windows):
    print(f"\n--- Fold {i+1}/{n_windows} ---")

    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    train_window = train_df_np.iloc[:train_end].copy()
    test_window = train_df_np.iloc[test_start:test_end].copy()

    print(f"Train: {train_window.shape}, Test: {test_window.shape}")

    # Modello NeuralProphet
    model = NeuralProphet(
        quantiles=[0.025, 0.975],
        learning_rate=0.01,
        batch_size=64,
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
        loss_func='Huber',
    )

    # Aggiungi regressori
    for reg in futr_exog_list:
        model.add_future_regressor(reg)

    # Fit
    _ = model.fit(train_window, freq="MS", epochs=100)

    # Predizione
    future_df = test_window.copy()  # Deve contenere anche i regressori futuri
    forecast = model.predict(future_df)

    # Metriche
    y_true = test_window['y'].values
    y_pred = forecast['yhat1'].values

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    results.append({'Fold': i+1, 'MAE_nprophet': mae, 'RMSE_nprophet': rmse})
    print(f"Split {i+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# Risultati CV
results_df_np = pd.DataFrame(results)
print("\n=== Backtest Results ===")
print(results_df_np)

# with open('pickles/NP_cv_OilPrice_multistep.pkl', 'wb') as file:
#     pickle.dump(results_df_np, file)
with open('pickles/NP_cv_OilPrice_multistep.pkl', 'rb') as file:
    results_df_np = pickle.load(file)

# Plot Errori
results_df_np.plot(
    x='Fold',
    y=['MAE_nprophet', 'RMSE_nprophet'],
    marker='o',
    title='TimeSeriesCV Errors',
    figsize=(10, 5)
)
plt.grid(True)
plt.show()

In [0]:
# Mi ricavo le date
initial_train_size = len(train_df_np)-5*len(test_df_np)
h = len(test_df_np)
n_windows = 5

# Crea lista per i dati delle fold
fold_data = []

for i in range(n_windows):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    # Ricava date da colonna 'ds' o dall'indice
    if 'ds' in np_df_train.columns:
        start_date = np_df_train['ds'].iloc[test_start]
        end_date = np_df_train['ds'].iloc[test_end - 1]
    else:
        start_date = np_df_train.index[test_start]
        end_date = np_df_train.index[test_end - 1]

    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]

    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)

# Converto le date in datetime se necessario
if all(isinstance(date, str) for date in fold_intervals_df['start_date']):
    fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
    fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])

print("Fold Intervals DataFrame:")
print(fold_intervals_df)

In [0]:
# CV Plot
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_np['ds'], train_df_np['y'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'{row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

Now let's training on the whole dataset

In [0]:
# FIT
# Fissa i seed per riproducibilità
import random
import time
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

np_df_train = train_df_np[future_features].copy()

# Modello senza regressori esterni
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.01, 
    batch_size=64,
    daily_seasonality=True,    # Impara la stagionalità dai dati
    weekly_seasonality=True,   # Impara pattern settimanali
    yearly_seasonality=True,   # Impara pattern annuali
    loss_func='Huber'
)
# Aggiungi solo i regressori deterministici
deterministic_features = ['month','quarter','year']
for reg in deterministic_features:
    neuralprophet.add_future_regressor(reg)
start_time = time.time()
metrics_df = neuralprophet.fit(np_df_train, freq="MS",epochs=100)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
forecast = neuralprophet.predict(test_df_np[future_features])

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
forecast['yhat1'] = forecast['yhat1'] * (max_val - min_val) + min_val
forecast['yhat1 2.5%'] = forecast['yhat1 2.5%'] * (max_val - min_val) + min_val
forecast['yhat1 97.5%'] = forecast['yhat1 97.5%'] * (max_val - min_val) + min_val

In [0]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast['ds'], 
    y=forecast['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))

# #Intervallo di confidenza al 95%
# fig.add_trace(go.Scatter(
#     x=list(forecast['ds']) + 
#     list(forecast['ds'][::-1]), 
#     y=list(forecast['yhat1 2.5%']) + list(forecast_test['yhat1 97.5%'][::-1]),
#     fill='toself',
#     fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
#     line=dict(color='rgba(255,255,255,0)'),
#     hoverinfo="skip",
#     showlegend=True,
#     name='P10-P90'
# ))

# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast['ds'], 'forecast': forecast['yhat1']})
merged_df = pd.merge(test_df_original[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
mae_val = mean_absolute_error(merged_df['y'], merged_df['forecast'])
metriche_np = calcola_metriche(merged_df['y'], merged_df['forecast'], train_df_original['y'],modelname="NeuralProphet")
metriche_np

#### TFT

In [0]:
# Preparing dataset
df_tft = create_time_series_features(df, target_col='y') 
df_tft = df_tft.dropna().reset_index(drop=True)
# 2. Suddivisione in train/test
train_df_tft = df_tft.iloc[:-h]
test_df_tft = df_tft.iloc[-h:]
train_df_tft.shape, test_df_tft.shape

In [0]:
# Scaling dei dati
#Salvo per dopo
train_df_original = train_df_tft.copy() 
test_df_original = test_df_tft.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_tft[['ds', 'unique_id']]
test_meta = test_df_tft[['ds', 'unique_id']]

feature_cols = [col for col in train_df_tft.columns if col not in ['ds', 'unique_id']]
train_idx = train_df_tft.index
test_idx = test_df_tft.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_tft[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_tft[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_tft = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_tft = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_tft.shape, test_df_tft.shape

##### TimeSeriesCV - fine tuning Optuna (TROPPO LENTO)

In [0]:
# Optuna function
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_tft = pd.DataFrame()
h = len(test_df_tft)

def objective(trial):
    # Hyperparameters to optimize for TFT
    input_size = trial.suggest_categorical("input_size", [int(h*6), int(4*h), int(2*h)])
    hidden_size = trial.suggest_categorical("hidden_size", [32, 64, 128, 256])
    n_rnn_layers = trial.suggest_int("n_rnn_layers", 1, 4)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    grn_activation = trial.suggest_categorical("grn_activation", ["ReLU", "ELU", "Sigmoid"])
    
    # Define TFT model
    tft=TFT(
            h=h,
            input_size=input_size,
            hidden_size=hidden_size,
            n_rnn_layers=n_rnn_layers,
            grn_activation=grn_activation, #prima era ELU
            rnn_type="lstm",
            one_rnn_initial_state=False,
            loss=DistributionLoss(distribution="StudentT", level=[90]),
            learning_rate=learning_rate, 
            hist_exog_list=hist_exog_features,
            futr_exog_list=futr_exog_list, #queste dovranno essere nel test
            max_steps=30, 
            val_check_steps=10,
            batch_size=batch_size, 
            #early_stop_patience_steps=15,  
            scaler_type="robust",
            enable_progress_bar=True,
            accelerator="auto",  
        )
    
    # nf
    nf = NeuralForecast(models=[tft], freq='MS')

    # Cross-validation for tuning
    df_cv = nf.cross_validation(
        df=train_df_tft,
        step_size=h,
        n_windows=5
    )

    # Calculate MAE for each fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'TFT']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'TFT',
            'MAE_tft': mae,
            'RMSE_tft': rmse
        })

    # Average MAE across all folds
    cv_metrics_df_tft = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_tft['MAE_tft'].mean()

    return mean_mae

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)  #aumentare a 50 su databricks

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
# salvo i best params
# with open('pickles/TFT_bestpars_OilPrice_multistep.pkl', 'wb') as file:
#     pickle.dump(best_params, file)

In [0]:
# carico i best params
with open('pickles/TFT_bestpars_OilPrice_multistep.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [0]:
# final model
best_tft = TFT(
        h=h,  
        input_size=best_params['input_size'],
        hidden_size=best_params['hidden_size'],
        n_rnn_layers=best_params['n_rnn_layers'],
        rnn_type='lstm',
        grn_activation=best_params['grn_activation'],
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=best_params['learning_rate'],
        futr_exog_list=futr_exog_list,
        hist_exog_list=hist_exog_features,
        max_steps=50, #aumentare a 100
        val_check_steps=10,
        batch_size=best_params['batch_size'],
        early_stop_patience_steps=-1,
        scaler_type="standard",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False
)
nf = NeuralForecast(models=[best_tft],freq="MS")

In [0]:
# # final CV
# df_cv = nf.cross_validation(
#     df=train_df_tft,
#     step_size=len(test_df_tft),
#     n_windows=5
# )
# # salvo le cv
# with open('pickles/TFT_cv_OilPrice_multistep.pkl', 'wb') as file:
#     pickle.dump(df_cv, file)

# carico le cv
with open('pickles/TFT_cv_OilPrice_multistep.pkl', 'rb') as file:
    df_cv = pickle.load(file)

# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'TFT']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'TFT',
        'MAE_tft': mae,
        'RMSE_tft': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_tft = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_tft)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_tft.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_tft['unique_id'].unique():
    series_data = train_df_tft[train_df_tft['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'{row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_tft['Model'] = 'TFT'
# Plot MAE sul primo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_tft'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_tft'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()

##### Now let's training on the whole dataset

In [0]:
# final training with best params
best_tft = TFT(
        h=h,  
        input_size=best_params['input_size'],
        hidden_size=best_params['hidden_size'],
        n_rnn_layers=best_params['n_rnn_layers'],
        rnn_type='lstm',
        grn_activation=best_params['grn_activation'],
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=best_params['learning_rate'],
        futr_exog_list=futr_exog_list,
        hist_exog_list=hist_exog_features,
        max_steps=50, #aumentare a 100
        val_check_steps=10,
        batch_size=best_params['batch_size'],
        early_stop_patience_steps=-1,
        scaler_type="standard",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False
)
nf = NeuralForecast(models=[best_tft],freq="MS")
start_time = time.time()
#fit 
nf.fit(df=train_df_tft, val_size=best_params['input_size'])
end_time = time.time()

training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")
tft_model = nf.models[0]

In [0]:
# # training senza best params
# h = len(test_df_tft)
# nf = NeuralForecast(
#     models=[
#         TFT(
#             h=h,
#             input_size=h*5,
#             hidden_size=60,  # deve essere divisibile per 4
#             grn_activation="ReLU", #prima era ELU
#             rnn_type="lstm",
#             n_rnn_layers=4,  #consigliato fra 2 e 5; 
#             one_rnn_initial_state=False,
#             loss=DistributionLoss(distribution="StudentT", level=[90]),
#             learning_rate=0.005, 
#             hist_exog_list=hist_exog_features,
#             futr_exog_list=futr_exog_list, #queste dovranno essere nel test
#             max_steps=50, 
#             val_check_steps=10,
#             batch_size=96, 
#             early_stop_patience_steps=15,  
#             scaler_type="robust",
#             enable_progress_bar=True,
#             accelerator="auto",  
#         ),
#     ],
#     freq='MS',
# )

# start_time = time.time()
# #fit 
# nf.fit(df=train_df_tft, val_size=2*h)

# end_time = time.time()
# training_time = end_time - start_time
# print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Predict
Y_hat_df_tft = nf.predict(futr_df=test_df_tft[['unique_id','ds'] + futr_exog_list])

In [0]:
# # salvo le prediction
# with open('pickles/TFT_pred_OilPrice_multistep.pkl', 'wb') as file:
#     pickle.dump(Y_hat_df_tft, file)

In [0]:
# carico le prediction
with open('pickles/TFT_pred_OilPrice_multistep.pkl', 'rb') as file:
    Y_hat_df_tft = pickle.load(file)

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
Y_hat_df_tft['TFT'] = Y_hat_df_tft['TFT'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-median'] = Y_hat_df_tft['TFT-median'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-lo-90'] = Y_hat_df_tft['TFT-lo-90'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-hi-90'] = Y_hat_df_tft['TFT-hi-90'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_tft['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_tft['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='lightgreen')))

# QUANTILI TFT
# Fascia tra P10 e P90
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_tft['ds']) + 
    list(Y_hat_df_tft['ds'][::-1]),
    y=list(Y_hat_df_tft['TFT-hi-90']) + list(Y_hat_df_tft['TFT-lo-90'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="TFT forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_tft['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# Calcola metriche
merged_df_tft = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_tft, on='ds', how='left')
merged_df_tft.rename(columns={'TFT-lo-90': '0.1','TFT-median':'0.5', 'TFT-hi-90':'0.9'},inplace=True)
merged_df_tft.dropna(inplace=True)
metriche_tft = calcola_metriche(merged_df_tft['y'],merged_df_tft['TFT'],train_df_original['y'],
                                y_pred_quantiles=merged_df_tft[['0.1','0.5','0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="TFT").round(10)
metriche_tft

##### TFT Explainability

In [0]:
# # salvo il modello tft
# with open('pickles/TFT_model_OilPrice_multistep.pkl', 'wb') as file:
#     pickle.dump(tft_model, file)

In [0]:
# carico il modello tft
with open('pickles/TFT_model_OilPrice_multistep.pkl', 'rb') as file:
    tft_model = pickle.load(file)

In [0]:
attention = tft_model.attention_weights()
def plot_attention(
    self, plot: str = "time", output: str = "plot", width: int = 800, height: int = 400
):
    """
    Plot the attention weights.

    Args:
        plot (str, optional): The type of plot to generate. Can be one of the following:
            - 'time': Display the mean attention weights over time.
            - 'all': Display the attention weights for each horizon.
            - 'heatmap': Display the attention weights as a heatmap.
            - An integer in the range [1, model.h) to display the attention weights for a specific horizon.
        output (str, optional): The type of output to generate. Can be one of the following:
            - 'plot': Display the plot directly.
            - 'figure': Return the plot as a figure object.
        width (int, optional): Width of the plot in pixels. Default is 800.
        height (int, optional): Height of the plot in pixels. Default is 400.

    Returns:
        matplotlib.figure.Figure: If `output` is 'figure', the function returns the plot as a figure object.
    """

    attention = (
        self.mean_on_batch(self.interpretability_params["attn_wts"])
        .mean(dim=0)
        .cpu()
        .numpy()
    )

    fig, ax = plt.subplots(figsize=(width / 100, height / 100))

    if plot == "time":
        attention = attention[self.input_size :, :].mean(axis=0)
        ax.plot(np.arange(-self.input_size, self.h), attention)
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Mean Attention")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "all":
        for i in range(self.input_size, attention.shape[0]):
            ax.plot(
                np.arange(-self.input_size, self.h),
                attention[i, :],
                label=f"horizon {i-self.input_size+1}",
            )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Attention per horizon")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "heatmap":
        cax = ax.imshow(
            attention,
            aspect="auto",
            cmap="viridis",
            extent=[-self.input_size, self.h, -self.input_size, self.h],
        )
        fig.colorbar(cax)
        ax.set_title("Attention Heatmap")
        ax.set_xlabel("Attention (current time step)")
        ax.set_ylabel("Attention (previous time step)")

    elif isinstance(plot, int) and (plot in np.arange(1, self.h + 1)):
        i = self.input_size + plot - 1
        ax.plot(
            np.arange(-self.input_size, self.h),
            attention[i, :],
            label=f"horizon {plot}",
        )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title(f"Attention weight for horizon {plot}")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    else:
        raise ValueError(
            'plot has to be in ["time","all","heatmap"] or integer in range(1,model.h)'
        )

    plt.tight_layout()

    if output == "plot":
        plt.show()
    elif output == "figure":
        return fig
    else:
        raise ValueError(f"Invalid output: {output}. Expected 'plot' or 'figure'.")

In [0]:
# MEAN ATTENTION
plot_attention(tft_model, plot="time")
# pesi di attenzione medi nel tempo su tutti gli orizzonti di previsione.
# La curva blu rappresenta quanto il modello ha "guardato" (attention) 
# quel punto temporale in input durante la previsione.
# Valori più alti significano che il modello ritiene quel punto temporale
# particolarmente rilevante per fare le previsioni

In [0]:
# ATTENTION OF A SPECIFIC TIME STEPS
plot_attention(tft_model, plot=4)
# pesi di attenzione per ogni orizzonte di previsione separatamente.

In [0]:
plot_attention(tft_model, plot="heatmap")

In [0]:
# PAST VARIABLE IMPORTANCES
feature_importances = tft_model.feature_importances()
print(feature_importances.keys())
print("\n ==========PAST FEATURE IMPORTANCES==========")
feature_importances["Past variable importance over time"].mean().sort_values().plot(kind="barh")

Questo grafico mostra quanto il modello ha fatto affidamento sulle feature passate \
(quelle che il modello ha potuto osservare prima del punto di previsione). Serve a costruire il contesto dell'input

In [0]:
# FUTURE VARIABLE IMPORTANCES
print("\n ==========FUTURE FEATURE IMPORTANCES==========")
feature_importances["Future variable importance over time"].mean().sort_values().plot(kind="barh")

Mostra quanto il modello ha sfruttato le feature che sono servite a guidare le previsioni passo-passo nel decoder

DIFFERENZA TRA PAST E FUTURE: \
La diversa importanza riflette quanto ciascuna variabile aiuta nell’analisi del contesto passato (encoder -->> PAST) \
o nella generazione delle previsioni (decoder -->> FUTURE).

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME
df_importances = feature_importances["Past variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances.index))

for col in df_importances.columns:
    p = ax.bar(np.arange(-len(df_importances), 0), df_importances[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances[col]
ax.set_title("Past variable importance over time")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)

plt.show()

Questo grafico mostra QUANDO (e QUANTO) nel passato ogni feature è stata più utile per costruire il contesto predittivo.

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME
df_importances2 = feature_importances["Future variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances2.index))
for col in df_importances2.columns:
    p = ax.bar(np.arange(-len(df_importances2), 0), df_importances2[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances2[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.grid(True)
ax.legend()
plt.show()

Per ogni feature viene mostrato quanto è stata importante per il modello a ciascun orizzonte futuro

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances3 = feature_importances["Past variable importance over time"]
mean_attention = (
    tft_model
    .attention_weights()[tft_model.input_size :, :]
    .mean(axis=0)[: tft_model.input_size]
)
df_importances3 = df_importances3.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances3.index))

for col in df_importances3.columns:
    p = ax.bar(np.arange(-len(df_importances3), 0), df_importances3[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances3[col]
ax.set_title("Past variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances3), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
print(df_importances4.shape)  # (n_time_steps, n_variables)
print(mean_attention.shape)   # Deve essere (n_variables,)

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION

# Estrai l'importanza delle future variables
df_importances4 = feature_importances["Future variable importance over time"]

# Estrai tutte le attention weights mediate nel tempo
full_attention = tft_model.attention_weights()[tft_model.input_size:, :].mean(axis=0)

# Estrai solo la parte relativa ai 12 time steps futuri
n_future_steps = df_importances4.shape[0]
mean_attention_future = full_attention[-n_future_steps:]

# Applica la ponderazione riga per riga (sul tempo)
df_importances4 = df_importances4.multiply(mean_attention_future, axis=0)

# Plot
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances4.index))

for col in df_importances4.columns:
    ax.bar(
        np.arange(1, len(df_importances4) + 1), 
        df_importances4[col].values, 
        width=0.6, 
        label=col, 
        bottom=bottom
    )
    bottom += df_importances4[col].values

# Titoli e assi
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Future time steps")
ax.grid(True)

# Linea media di attenzione
ax.plot(
    np.arange(1, len(df_importances4) + 1),
    mean_attention_future,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention"
)
ax.legend()
plt.show()

In [0]:
tft_model.feature_importance_correlations()

#### CHRONOS

item_id, timestamp, target (Chronos calcola tempo di fitting in automatico)

In [0]:
# Preparing dataset
df_chronos = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos = df_chronos.dropna().reset_index(drop=True) 
df_chronos['item_id'] = df_chronos['unique_id'] 
df_chronos.drop(columns="unique_id", inplace=True) 
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Definizione delle covariate note
known_covariates_names=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id','target']]
feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

# Split train e test
train_df_chronos = df_chronos.iloc[:-h]
test_df_chronos = df_chronos.iloc[-h:]

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [0]:
# Scaling dei dati
train_df_original = train_df_chronos.copy() 
test_df_original = test_df_chronos.copy()

train_idx = train_df_chronos.index
test_idx = test_df_chronos.index

#scaler
scaler_x = MinMaxScaler()
train_df_chronos = pd.DataFrame(scaler_x.fit_transform(train_df_chronos[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_chronos = pd.DataFrame(scaler_x.transform(test_df_chronos[feature_cols]),
                            columns=feature_cols, index=test_idx)
#controllo le shape
train_df_chronos.shape, test_df_chronos.shape

TimeSeriesCV

In [0]:
# TimeSeriesCV
h = len(test_df_chronos)  # prediction length
n_splits = 5
initial_train_size = len(train_df_chronos) - 5*h
results = []

for fold in range(n_splits):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")
    
    train_end = initial_train_size + fold * h
    test_start = train_end
    test_end = test_start + h
    train_df_fold = train_df_chronos.iloc[:train_end]
    test_df_fold = train_df_chronos.iloc[test_start:test_end]
    
    train_df_fold.reset_index(inplace=True)
    test_df_fold.reset_index(inplace=True)
    
    train_df_fold["timestamp"] = pd.to_datetime(train_df_fold["timestamp"])
    test_df_fold["timestamp"] = pd.to_datetime(test_df_fold["timestamp"])
    
    # Future timestamps 
    future_index = pd.date_range(
        start=train_df_fold["timestamp"].max() + pd.Timedelta(days=1),
        periods=h,
        freq="MS")
    
    # Preparo test set con known_covariates
    test_df_for_prediction = test_df_fold[test_df_fold["timestamp"].isin(future_index)].copy()
    test_df_for_prediction = test_df_for_prediction.set_index("timestamp")
    test_df_for_prediction = test_df_for_prediction.reindex(future_index)  # forza continuità
    test_df_for_prediction["item_id"] = train_df_fold["item_id"].iloc[0]  # supponiamo 1 sola serie
    test_df_for_prediction = test_df_for_prediction.reset_index().rename(columns={"index": "timestamp"})
    test_df_for_prediction = test_df_for_prediction.set_index(["item_id", "timestamp"])
    
    # Preparo il train con multindex
    train_df_fold = train_df_fold.set_index(["item_id", "timestamp"])
    print(f"Train: {train_df_fold.shape}, Test: {test_df_fold.shape}")
    
    # inizializzo il predictor
    predictor = TimeSeriesPredictor(
        prediction_length=h,
        target="target",
        known_covariates_names=known_covariates_names,
        eval_metric="MASE",
        freq="MS")
    
    # fit del modello
    predictor.fit(
        train_df_chronos,
        hyperparameters={
            "Chronos": [
                {
                    "model_path": "bolt_small",
                    "ag_args": {"name_suffix": "ZeroShot"}},
                {
                    "model_path": "bolt_small",
                    "covariate_regressor": "CAT",
                    "target_scaler": "robust",
                    "ag_args": {"name_suffix": "WithRegressor"},
                },
            ],
        },
        time_limit=600,
        enable_ensemble=True,  # Attiva ensemble per migliorare l'accuratezza
        )
    
    predictions = predictor.predict(train_df_fold, known_covariates=test_df_for_prediction)

    test_df_with_index = test_df_fold.set_index(["item_id", "timestamp"])

    # Debug
    print(f"Forma predictions: {predictions.shape}")
    print(f"Colonne predictions: {predictions.columns}")
    print(f"Numero di indici in predictions: {len(predictions.index)}")
    print(f"Numero di indici in test_df_with_index: {len(test_df_with_index.index)}")

    # trovo solo gli indici comuni
    common_indices = predictions.index.intersection(test_df_with_index.index)
    print(f"Indici comuni: {len(common_indices)}")

    # prendo solo gli indici comuni
    y_true = test_df_with_index.loc[common_indices, "target"]
    y_pred = predictions.loc[common_indices, 'mean'].to_numpy()

    mae = calcola_mae(y_true, y_pred)
    rmse = calcola_rmse(y_true, y_pred)
    
    results.append({'Fold': fold+1, 'MAE_chronos': mae, 'RMSE_chronos': rmse})
    print(f"Split {fold+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
# risultati finali
results_df_chronos = pd.DataFrame(results)
results_df_chronos.set_index('Fold',inplace=True)
print("\n=== TimeSeriesCV Results ===")
results_df_chronos

In [0]:
# Estraggo le date
fold_data = []
train_df_chronos.reset_index(inplace=True)
for i in range(n_splits):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h
    
    # Ricava date da colonna 'ds' o dall'indice
    if 'timestamp' in train_df_chronos.columns:
        start_date = train_df_chronos['timestamp'].iloc[test_start]
        end_date = train_df_chronos['timestamp'].iloc[test_end - 1]
    else:
        start_date = train_df_chronos.index[test_start]
        end_date = train_df_chronos.index[test_end - 1]
    
    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]
    
    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)
fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])
# Stampa il dataframe prima della conversione per verificare i valori
print("Fold Intervals DataFrame (before conversion):")
fold_intervals_df

In [0]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_chronos['timestamp'], train_df_chronos['target'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'{row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot degli errori
results_df_chronos.reset_index(inplace=True)
ax = results_df_chronos.plot(x='Fold', y=['MAE_chronos','RMSE_chronos'], marker='o', title='TimeSeriesCV Errors')
ax.set_xticks(range(1, 6)) # Imposta le tacche sull'asse x da 1 a 5
plt.show()

In [0]:
results_df_chronos

Let's training on the whole dataset

In [0]:
# Preparo dataset per training
predictor = TimeSeriesPredictor(
    prediction_length=len(test_df_chronos), 
    eval_metric="MASE",
    target="target",
    known_covariates_names=futr_exog_list
)
test_known_covariates = test_df_chronos[futr_exog_list]

In [0]:
# Fit del modello
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {
                "model_path": "bolt_small",
                "ag_args": {"name_suffix": "ZeroShot"}
            },
            {
                "model_path": "bolt_small",
                "covariate_regressor": "CAT",
                "target_scaler": "robust",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
        ],
    },
    time_limit=600,
    enable_ensemble=True,
)

In [0]:
# PREVISIONI MULTI STEP 
predictions = predictor.predict(
    train_df_chronos,
    known_covariates=test_known_covariates  # Solo le covariate realmente note in anticipo
)
predictions.reset_index(inplace=True)

predictions_0shot = predictor.predict(
    train_df_chronos,
    known_covariates=test_known_covariates,
    model="ChronosZeroShot[bolt_small]"
)
predictions_0shot.reset_index(inplace=True)

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('target')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
predictions['mean'] = predictions['mean'] * (max_val - min_val) + min_val
predictions['0.1'] = predictions['0.1'] * (max_val - min_val) + min_val
predictions['0.5'] = predictions['0.5'] * (max_val - min_val) + min_val
predictions['0.9'] = predictions['0.9'] * (max_val - min_val) + min_val
predictions_0shot['mean'] = predictions_0shot['mean'] * (max_val - min_val) + min_val
predictions_0shot['0.5'] = predictions_0shot['0.5'] * (max_val - min_val) + min_val
predictions_0shot['0.9'] = predictions_0shot['0.9'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)

# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_chronos['timestamp'], y=train_df_original['target'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['0.5'], mode='lines', 
    name='P50', line=dict(color='green')))
# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='ChronosWithRegressor', line=dict(color='lightgreen')))
# QUANTILI CHRONOS
fig.add_trace(go.Scatter(
    x=list(test_df_chronos['timestamp']) + 
    list(test_df_chronos['timestamp'][::-1]),
    y=list(predictions['0.9']) + list(predictions['0.1'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# PREVISIONI CHRONOS 0 SHOT
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='0-shot Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="CHRONOS forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_chronos['timestamp'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# metriche di valutazione
test_df_original.reset_index(inplace=True)
merged_df_chronos = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)
metriche_chronos = calcola_metriche(merged_df_chronos['target'],merged_df_chronos['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos").round(5)
metriche_chronos

In [0]:
# metriche di valutazione
test_df_original.reset_index(inplace=True)
merged_df_chronos = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions_0shot[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)
metriche_chronos_0shot = calcola_metriche(merged_df_chronos['target'],merged_df_chronos['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos_0shot").round(5)
metriche_chronos_0shot

#### final evaluations

In [0]:
final_metrics = metriche_arima.join(metriche_ets).join(metriche_xgb).join(metriche_lstm).join(metriche_np).join(metriche_tft).join(metriche_chronos).join(metriche_chronos_0shot).round(4)
final_metrics

In [0]:
# FINAL PREDICTIONS PLOT FOR OIL PRICE
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoARIMA'], 
    mode='lines', 
    name='ARIMA', 
    line=dict(color='grey')
))

# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoETS'], 
    mode='lines', 
    name='ETS', 
    line=dict(color='orange')
))

# PREVISIONI XGBOOST
fig.add_trace(go.Scatter(
    x=forecast_xgb['ds'], 
    y=forecast_xgb['forecast'], 
    mode='lines', 
    name='XGBoost', 
    line=dict(color='purple')
))

# PREVISIONI LSTM
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=Y_hat_df_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='pink')
))

# PREVISIONI NEURAL PROPHET
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=forecast['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='aquamarine')
))


# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], 
                         y=Y_hat_df_tft['TFT'], 
                         #y=Y_hat_df_tft_TFT,
                         mode='lines', 
    name='TFT', line=dict(color='yellow')))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS', line=dict(color='green')))
    
# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS 0-shot', line=dict(color='lightgreen')))

# Layout del grafico
fig.update_layout(
    title="Forecasts for White Noise", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
#sistemo i vari dataset prima del merge
cv_metrics_df_lstm.drop(columns=('Model'),inplace=True)
df_arima = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == 'AutoARIMA'][['Fold',
                                                                    'MAE','RMSE']]
df_ets = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == 'AutoETS'][['Fold',
                                                                    'MAE','RMSE']]
results_df_np.rename(columns={'split': 'Fold'}, inplace=True)
results_df_chronos.rename(columns={'split': 'Fold','MAE':'MAE_chronos','RMSE':'RMSE_chronos'}, inplace=True)
cv_metrics_df_xgboost.rename(columns={'MAE': 'MAE_XGB','RMSE':'RMSE_XGB'}, inplace=True)
df_arima.rename(columns={'MAE':'MAE_AutoARIMA','RMSE':'RMSE_AutoARIMA'}, inplace=True)
df_ets.rename(columns={'MAE':'MAE_AutoETS','RMSE':'RMSE_AutoETS'}, inplace=True)

In [0]:
# Final plot metrics TimeSeriesCV

# faccio il merge di tutte le tabelle dei vari CV
final_cv = df_ets.merge(cv_metrics_df_lstm, 
                            on="Fold",how="inner").merge(results_df_chronos, 
                            on="Fold",how="inner").merge(cv_metrics_df_xgboost[['MAE_XGB','RMSE_XGB','Fold']],
                            on="Fold",how="inner").merge(df_arima,
                            on="Fold",how="inner").merge(results_df_np,
                            on="Fold", how="inner").merge(cv_metrics_df_tft,
                            on="Fold",how="inner")
# Rescaling CV metrics data
columns_to_modify = ['MAE_nprophet', 'RMSE_nprophet', 'MAE_tft', 'RMSE_tft', 'MAE_lstm', 'RMSE_lstm',
                        'MAE_chronos', 'RMSE_chronos', 'MAE_XGB', 'RMSE_XGB']
for col in columns_to_modify:
    final_cv[f'{col}'] = final_cv[f'{col}'] * (max_val - min_val) + min_val

model_colors = {
    'AutoARIMA': '#808080',      # Grey
    'AutoETS': '#ff7f0e',        # Orange
    'XGB': '#9467bd',    # Purple
    'lstm': '#e377c2',       # Pink
    'nprophet': '#7fffd4',         # Aquamarine
    'tft': '#ffff00',        # Yellow
    'chronos': '#2ca02c',    # Green
}


# Create the figure
fig = go.Figure()

# Add traces for each model - MAE (darker) and RMSE (lighter)
for model, color in model_colors.items():
    # Add MAE line (darker)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'MAE_{model}'],
        mode='lines+markers',
        name=f'{model} MAE',
        line=dict(color=color, width=3),
        marker=dict(size=8)
    ))
    
    # Add RMSE line (lighter with same color but different dash pattern)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'RMSE_{model}'],
        mode='lines+markers',
        name=f'{model} RMSE',
        line=dict(color=color, width=2, dash='dash'),
        marker=dict(size=6)
    ))

# Update layout
fig.update_layout(
    title='Model Performance Comparison (MAE and RMSE)',
    template='plotly_dark',
    xaxis=dict(title='Fold',
        tickmode='linear',
        tick0=1, dtick=1
    ),    yaxis=dict(
        title='Error Value'
    ),    legend=dict(
        orientation="v"
    ),    hovermode="closest")
fig.show()

In [0]:
# Confronto TFT/CHRONOS
tft_chronos = metriche_chronos.join(metriche_tft)
tft_chronos                                    

### **OUTLIERS data**

In [0]:
#preparazione dataset
df = out.reset_index().rename(columns={'date': 'ds','total_daily_sales':'y'})
df['unique_id'] = 'serie_1'
df['ds'] = pd.to_datetime(df['ds'])

In [0]:
# funzione per creare feature
def create_time_series_features(df, target_col):
    df = df.copy()
    # Lag features (1-7)
    for lag in range(1, 8):
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    # Differencing features, 1 e 7
    df['diff_1'] = (df[target_col] - df[target_col].shift(1)).shift(1)
    df['diff_1'] = (df[target_col] - df[target_col].shift(7)).shift(1)
    
    # Caratteristiche temporali
    df['dayofweek'] = df['ds'].dt.dayofweek
    df['month'] = df['ds'].dt.month
    df['quarter'] = df['ds'].dt.quarter
    
    # Caratteristiche cicliche per rappresentare meglio la stagionalità
    df['day_sin'] = np.sin(2 * np.pi * df['dayofweek']/7)
    df['day_cos'] = np.cos(2 * np.pi * df['dayofweek']/7)
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    
    # rolling means 
    df['rolling_mean_3'] = df[target_col].rolling(window=3).mean().shift(1) #ultimoi 3 giorni
    df['rolling_mean_7'] = df[target_col].rolling(window=7).mean().shift(1) #ultima settimana
    df['rolling_std_3'] = df[target_col].rolling(window=3).std().shift(1)
    df['rolling_std_7'] = df[target_col].rolling(window=7).std().shift(1)
    
    return df

##### ARIMA + ETS

In [0]:
# divisione in train e test
df_arima = df[['y','ds','unique_id']].iloc[8:]
train_size = int(len(df_arima) * 0.9)
train_df = df_arima.iloc[:train_size]
test_df = df_arima.iloc[train_size:]
# queste due ci serviranno dopo
season_length = 7 # for monthly data = 12 
horizon = len(test_df) # number of predictions
train_df.shape, test_df.shape

In [0]:
# train e test plot
fig = go.Figure()
#train dat
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Train'))
#test data 
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Test'))
#layout
fig.update_layout(template='plotly_dark', title='Train and Test Data Plot')
fig.show()

In [0]:
# definisco i modelli
models = [
    AutoARIMA(season_length=season_length),
    AutoETS(season_length=season_length)
]
# creo il forecaster
sf = StatsForecast(models=models, freq='D')
import time
start_time = time.time()
# fit
sf.fit(train_df)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
#summary of the model ARIMA
result=sf.fitted_[0,0].model_
print(result.keys())
print("Parametri ARMA:",result['arma'])

In [0]:
arima_string(sf.fitted_[0,0].model_)

In [0]:
#summary of the model ETS
result2 = sf.fitted_[0,1].model_
print(result2.keys())
print("Modello:",result2['method'])

In [0]:
# let's forecast
Y_hat_df = sf.forecast(df=train_df, h=len(test_df), fitted=True, level=[95])
#see fitted values vs true values
values=sf.forecast_fitted_values() #qui viene aggiunta anche la vera y
values.head(2)

In [0]:
#CROSS VALIDATION
crossvalidation_df = sf.cross_validation(
    df=train_df,
    h=horizon, 
    step_size=len(test_df),
    n_windows=5
)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = crossvalidation_df['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = crossvalidation_df[crossvalidation_df['cutoff'] == fold]
    
    for model in ['AutoARIMA', 'AutoETS']:
        # Filtra i dati per il modello corrente
        model_data = fold_data[['y', f'{model}']]
        model_data = model_data.dropna()
        
        # Calcola MAE e RMSE
        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data[f'{model}'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data[f'{model}'])
        
        # Aggiungi i risultati
        cv_metrics.append({
            'Fold': i+1,
            'Model': model,
            'MAE': mae,
            'RMSE': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_arima_ets = pd.DataFrame(cv_metrics)

In [0]:
cv_metrics_df_arima_ets

In [0]:
# PLOT CV
#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_arima_ets)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_arima_ets.groupby('Model').mean().reset_index()
print(mean_metrics[['Model', 'MAE', 'RMSE']])

# Ora creiamo un grafico che combina la serie temporale con le metriche di errore per visualizzare dove si verificano gli errori maggiori

# Prima dobbiamo identificare gli intervalli temporali per ogni fold
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = crossvalidation_df[crossvalidation_df['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df['unique_id'].unique():
    series_data = train_df[train_df['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

# Plot MAE sul primo asse
for model in cv_metrics_df_arima_ets['Model'].unique():
    model_data = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_arima_ets['Model'].unique():
    model_data = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()

In [0]:
fold_intervals #EXPANDING WINDOW CROSS VALIDATION

Let's forecast on test set

In [0]:
# prediction on test set
test_pred = sf.forecast(df=train_df, h=len(test_df), level=[95], fitted=True)
pred = test_df.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])

In [0]:
# Calcola metriche
metriche_arima = calcola_metriche(pred['y'],pred['AutoARIMA'],train_df['y'], 
                                modelname="ARIMA").round(10)
metriche_ets = calcola_metriche(pred['y'],pred['AutoETS'],train_df['y'], 
                                modelname="ETS").round(10)
metriche_arima, metriche_ets

In [0]:
# Forecast plot
# Creiamo un dataframe completo che include training, test e previsioni future
full_df = pd.concat([train_df, test_df], axis=0)
# Aggiungiamo le previsioni al dataframe completo
forecast_df_etsA = full_df.merge(pred, how='outer', on=['unique_id', 'ds'])

fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(x=test_pred['ds'], y=test_pred['AutoARIMA'], mode='lines', 
    name='AutoARIMA Forecast', line=dict(color='green')))
# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(x=test_pred['ds'], y=test_pred['AutoETS'], mode='lines', 
    name='AutoETS Forecast', line=dict(color='orange', dash='dot')))

# Layout del grafico
fig.update_layout(title="Forecast Comparison: AutoARIMA vs AutoETS", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15)))
# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

##### XGBOOST

In [0]:
# df_xgb
df_xgb = create_time_series_features(df, target_col='y') 
df_xgb.drop(columns=['unique_id'],inplace=True)
df_xgb = df_xgb.dropna().reset_index(drop=True) #droppo i NaN creati dai lag, quindi le prime 7 obs.
# print(df_xgb.columns)
# df_xgb.head()

In [0]:
# Split train/test
train_size = int(len(df_xgb) * 0.9)
train_df_xgb = df_xgb.iloc[:train_size]
test_df_xgb = df_xgb.iloc[train_size:]

# Separazione features e target
feature_cols = [col for col in df_xgb.columns if col not in ['y', 'ds']]
X_train_xgb, y_train_xgb = train_df_xgb[feature_cols], train_df_xgb['y']
X_test_xgb, y_test_xgb = test_df_xgb[feature_cols], test_df_xgb['y']
X_train_xgb.shape, y_train_xgb.shape, X_test_xgb.shape, y_test_xgb.shape

In [0]:
# Scaling dei dati

#features
scaler_x = MinMaxScaler()
X_train_scaled = scaler_x.fit_transform(X_train_xgb)
X_test_scaled = scaler_x.transform(X_test_xgb)

# target, non serve che scalo y_test
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train_xgb.values.reshape(-1, 1))

# Ricreo df con stessi indici e nomi
X_train_xgb = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train_xgb.index)
X_test_xgb = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test_xgb.index)
y_train_xgb = pd.Series(y_train_scaled.flatten(), name='y', index=y_train_xgb.index)

#aggiorno train_df_xgb
train_df_xgb[feature_cols] = X_train_xgb
train_df_xgb['y'] = y_train_xgb

#controllo le shape
X_train_xgb.shape, X_test_xgb.shape, y_train_xgb.shape, y_test_xgb.shape

Ottimizzazione Optuna (Bayesiana)

In [0]:
# Optuna function
cv_metrics_log = []  # List globale per salvare metriche fold per fold

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'verbosity': 0,
        'seed': 42
    }

    tscv = TimeSeriesSplit(n_splits=5,test_size=len(test_df_xgb))
    y = train_df_xgb['y']
    df_xgb_feature = train_df_xgb.drop(columns=['y','ds'])
    all_rmse = []

    for fold, (train_idx, test_idx) in enumerate(tscv.split(df_xgb_feature)):
        X_train, X_test = df_xgb_feature.iloc[train_idx], df_xgb_feature.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        dtest = xgb.DMatrix(X_test.values, label=y_test.values)

        model = xgb.train(params, dtrain, num_boost_round=500,
                          evals=[(dtest, 'eval')],
                          early_stopping_rounds=30, verbose_eval=False)

        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mae = calcola_mae(y_test, preds)  # tua funzione custom

        all_rmse.append(rmse)

        # Recupera le date (se esiste colonna 'ds')
        start_date = df_xgb.iloc[test_idx]['ds'].min() if 'ds' in df_xgb.columns else None
        end_date = df_xgb.iloc[test_idx]['ds'].max() if 'ds' in df_xgb.columns else None

        # Logga le metriche della fold
        cv_metrics_log.append({
            'Trial': trial.number,
            'Fold': fold + 1,
            'MAE': mae,
            'RMSE': rmse,
            'start_date': start_date,
            'end_date': end_date,
            'Model': 'XGBoost'  
        })
    return np.mean(all_rmse)

In [0]:
# Optuna optimization
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50) #aumentare in databricks

In [0]:
# Best parameters
print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)
print("Best trial:",study.best_trial.number)
besttrial = study.best_trial.number

In [0]:
cv_metrics_log = pd.DataFrame(cv_metrics_log)
cv_metrics_log = cv_metrics_log[cv_metrics_log['Trial']==besttrial]
cv_metrics_log

Vediamo come ha performato nel miglior trial

In [0]:
#PLOT CV
cv_metrics_df_xgboost = cv_metrics_log.copy()

# Stampa le metriche per modello
print("Metriche per modello:")
print(cv_metrics_df_xgboost)

# Calcola e stampa le metriche medie per modello
mean_metrics = cv_metrics_df_xgboost.mean(numeric_only=True).reset_index()

# Crea il DataFrame con gli intervalli delle fold
fold_intervals_df = cv_metrics_df_xgboost[['Fold', 'start_date', 'end_date']].drop_duplicates()

# Ora creiamo il grafico combinato
plt.figure(figsize=(16, 12))

# === PLOT 1: Serie temporale originale ===
plt.subplot(2, 1, 1)
#for unique_id in train_df_xgb['unique_id'].unique():
series_data = train_df_xgb.copy()
plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)],
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10,
             bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# === PLOT 2: Metriche MAE e RMSE ===
plt.subplot(2, 1, 2)
ax1 = plt.gca()
ax2 = ax1.twinx()

# MAE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# RMSE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Legenda combinata
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


Training coi migliori parametri trovati da Optuna

In [0]:
# Addestriamo il modello con i best params
best_params = study.best_params
best_params.update({'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 42})

dtrain_xgb = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
start_time = time.time()
#fit 
model = xgb.train(best_params, dtrain_xgb, num_boost_round=500)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Plot feature importance
ax = xgb.plot_importance(model, importance_type='gain', max_num_features=10)
plt.title('Feature Importance (Gain)')
for text in ax.texts:
    text.set_visible(False)
plt.tight_layout()
plt.show()

MULTI-STEP FORECAST

In [0]:
def direct_multistep_forecast(train_data, feature_cols, target_col, horizon=None):
    """
    Addestra modelli separati per ogni orizzonte temporale futuro
    """
    forecasts = []
    models = []
    
    # Crea dataframe per date future
    last_date = train_data['ds'].max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=horizon, freq='D')
    
    # Per ogni step futuro, addestra un modello dedicato
    for h in range(1, horizon+1):
        print(f"Training model for horizon {h}")
        
        # Prepara target con shift inverso per prevedere h passi avanti
        df_horizon = train_data.copy()
        df_horizon[f'y_horizon_{h}'] = df_horizon[target_col].shift(-h)
        df_horizon = df_horizon.dropna()
        
        # Prendi features e target per questo orizzonte
        X = df_horizon[feature_cols]
        y = df_horizon[f'y_horizon_{h}']
        
        # Split train/validation
        train_size = int(len(X) * 0.8)
        X_train, X_val = X.iloc[:train_size], X.iloc[train_size:]
        y_train, y_val = y.iloc[:train_size], y.iloc[train_size:]
        
        # Addestra modello
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        params = best_params
        model = xgb.train(
            params,
            dtrain,
            evals=[(dtrain, 'train'), (dval, 'val')],
            num_boost_round=100,
            early_stopping_rounds=20,
            verbose_eval=False
        )
        models.append(model)
        
    # Crea input per la previsione (l'ultimo punto noto)
    last_point = xgb.DMatrix(train_data.iloc[[-1]][feature_cols])
    
    # Prevedi ciascun orizzonte con il modello dedicato
    for i, model in enumerate(models):
        pred = model.predict(last_point)[0]
        forecasts.append(pred)
    return pd.DataFrame({'ds': future_dates, 'forecast': forecasts})

In [0]:
# # Forecasting
# forecast_xgb = direct_multistep_forecast(train_df_xgb, feature_cols, target_col='y',horizon=len(test_df_xgb))
# print(f"Previsioni per i prossimi {(len(test_df_xgb))} step:", forecast_xgb.head()) 

RECURSIVE FORECAST

In [0]:
dtest_xgb = xgb.DMatrix(X_test_xgb)
forecast_xgb = pd.DataFrame(model.predict(dtest_xgb),columns=['forecast'])
forecast_xgb['ds'] = test_df_xgb['ds'].values

Rescaling dei dati

In [0]:
# RE-Scaling dei dati
forecast_xgb['forecast'] = scaler_y.inverse_transform(forecast_xgb[['forecast']])
train_df_xgb['y'] = scaler_y.inverse_transform(train_df_xgb[['y']])

In [0]:
# Plot forecast
fig = go.Figure()
# Training data
fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=train_df_xgb['y'], mode='lines', name='Training'))
# Test data
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=test_df_xgb['y'], mode='lines', name='Test'))
# Forecast
fig.add_trace(go.Scatter(x=forecast_xgb['ds'], y=forecast_xgb['forecast'], mode='lines', 
                         name='Forecast', line=dict(color='deepskyblue')))
fig.update_layout(title='Forecast vs Real Data', template='plotly_dark')
fig.show()

Calcolo metriche

In [0]:
# Calcola metriche
merged_df = pd.merge(test_df_xgb[['ds', 'y']], forecast_xgb, on='ds', how='left')
merged_df.dropna(inplace=True)
metriche_xgb = calcola_metriche(merged_df['y'],merged_df['forecast'],train_df_xgb['y'],modelname="XGBoost")
metriche_xgb

##### LSTM (neuralforecast)

In [0]:
# divisione in train e test
df_lstm = create_time_series_features(df, target_col='y') 
df_lstm = df_lstm.dropna().reset_index(drop=True)

train_size = int(len(df_lstm) * 0.9)
train_df_lstm = df_lstm.iloc[:train_size]
test_df_lstm = df_lstm.iloc[train_size:]
print(train_df_lstm.columns)
train_df_lstm.head()

In [0]:
train_df_lstm.shape, test_df_lstm.shape

In [0]:
# Optuna function
futr_exog_list = train_df_lstm.drop(columns=['unique_id', 'ds', 'y']).columns.tolist()
folds = pd.DataFrame()
h = len(test_df_lstm)
cv_metrics_df_lstm = pd.DataFrame()
def objective(trial):
    cv_metrics = []
    # Hyperparametri da ottimizzare
    input_size = trial.suggest_categorical("input_size", [h, int(2.5*h)] )
    encoder_n_layers = trial.suggest_int("encoder_n_layers", 1, 3)
    encoder_hidden_size = trial.suggest_categorical("encoder_hidden_size", [32, 64, 128])
    decoder_hidden_size = trial.suggest_categorical("decoder_hidden_size", [32, 64, 128])
    decoder_layers = trial.suggest_int("decoder_layers", 1, 3)
    learning_rate = trial.suggest_float("learning_rate", 1e-3, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    # Fisso il seed per riproducibilità
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

    # Definizione modello LSTM
    lstm = LSTM(
        h=h,  # nel tuning
        input_size=input_size,
        loss=DistributionLoss(distribution="Normal", level=[95]),
        scaler_type='robust',
        encoder_n_layers=encoder_n_layers,
        encoder_hidden_size=encoder_hidden_size,
        decoder_hidden_size=decoder_hidden_size,
        decoder_layers=decoder_layers,
        futr_exog_list=futr_exog_list,
        max_steps=50, #aumentare a 150
        batch_size=batch_size,
        learning_rate=learning_rate,
        recurrent=False
    )

    nf = NeuralForecast(models=[lstm], freq='D')

    # cross-validation per tuning
    df_cv = nf.cross_validation(
        df=train_df_lstm,
        step_size=h,
        n_windows=5
    )

    # Calcola MAE per ogni fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'LSTM']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'LSTM',
            'MAE_lstm': mae,
            'RMSE_lstm': rmse
        })

    # Media MAE su tutte le fold
    cv_metrics_df_lstm = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_lstm['MAE_lstm'].mean()

    return mean_mae 

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50) 

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
# salvo i best params trovati da Optuna
# with open('pickles/LSTM_bestPar_Outliers.pkl', 'wb') as file:
#     pickle.dump(best_params, file)

Carico i best params

In [0]:
with open('pickles/LSTM_bestPar_Outliers.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [0]:
# final model
best_lstm = LSTM(
    h=len(test_df_lstm),
    input_size=best_params['input_size'],
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=100,
    #early_stop_patience_steps=15,
    batch_size=best_params['batch_size'],
    learning_rate=best_params['learning_rate'],
    recurrent=False
)
nf = NeuralForecast(models=[best_lstm], freq='D')

In [0]:
# # final CV
# df_cv = nf.cross_validation(
#     df=train_df_lstm,
#     step_size=len(test_df_lstm),
#     n_windows=5
# )
# with open('pickles/LSTM_CV_Outliers.pkl', 'wb') as file:
#     pickle.dump(df_cv, file)

with open('pickles/LSTM_CV_Outliers.pkl', 'rb') as file:
    df_cv = pickle.load(file)

# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'LSTM']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'LSTM',
        'MAE_lstm': mae,
        'RMSE_lstm': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_lstm = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_lstm)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_lstm.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_lstm['unique_id'].unique():
    series_data = train_df_lstm[train_df_lstm['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_lstm['Model'] = 'lstm'
# Plot MAE sul primo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_lstm'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_lstm'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


In [0]:
# Final training

# Fisso il seed per riproducibilità
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

final_model = LSTM(
    h=len(test_df_lstm),
    input_size=best_params['input_size'],
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=100, #aumentare a 150
    early_stop_patience_steps=15,
    batch_size=best_params['batch_size'],
    learning_rate=best_params['learning_rate'], 
    recurrent=False
)

nf = NeuralForecast(models=[final_model], freq='D')
start_time = time.time()
#fit 
nf.fit(df=train_df_lstm, val_size=len(test_df_lstm))
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

Previsioni

In [0]:
Y_hat_df_lstm = nf.predict(futr_df=test_df_lstm)
# salvo previsioni in pickle
#Y_hat_df_lstm.to_pickle('pickles/LSTM_solarEnergy.pkl')

In [0]:
# #carico le previsioni
Y_hat_df_lstm = pd.read_pickle('pickles/LSTM_solarEnergy.pkl')

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_lstm['ds'], y=train_df_lstm['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_lstm['ds'], y=test_df_lstm['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM'], mode='lines', 
    name='MEAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIANA LSTM
# fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM-median'], mode='lines', 
#     name='MEDIAN Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="LSTM forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_lstm['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")

fig.show()

In [0]:
# Calcola metriche
merged_df_lstm = pd.merge(test_df_lstm[['ds', 'y']], Y_hat_df_lstm, on='ds', how='left')
merged_df_lstm.dropna(inplace=True)
metriche_lstm = calcola_metriche(merged_df_lstm['y'],merged_df_lstm['LSTM'],train_df_lstm['y'],modelname="LSTM")
metriche_lstm

##### NEURAL PROPHET

In [0]:
df_np = create_time_series_features(df, target_col='y') 
df_np = df_np.drop(columns=['unique_id'])
df_np = df_np.dropna().reset_index(drop=True)
train_size = int(len(df_np) * 0.9)
train_df_np = df_np.iloc[:train_size]
test_df_np = df_np.iloc[train_size:]

In [0]:
# Scaling dei dati

train_df_original = train_df_np.copy()
test_df_original = test_df_np.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_np['ds']
test_meta = test_df_np['ds']

feature_cols = [col for col in train_df_np.columns if col not in ['ds']]
train_idx = train_df_np.index
test_idx = test_df_np.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_np[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_np[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_np = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_np = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_np.shape, test_df_np.shape

In [0]:
df_np.columns

TimeSeriesCV

In [0]:
# preparazione dataset
np_df_train = train_df_np[['ds', 'y']].copy()
np_df_test = test_df_np[['ds', 'y']].copy()

# Aggiungi le colonne dei regressori
regressors = ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6',
       'lag_7', 'diff_1', 'dayofweek', 'month', 'quarter', 'day_sin',
       'day_cos', 'month_sin', 'month_cos', 'rolling_mean_3', 'rolling_mean_7',
       'rolling_std_3', 'rolling_std_7']

for reg in regressors:
    np_df_train[reg] = train_df_np[reg]
    np_df_test[reg] = test_df_np[reg]
# Definisci il modello
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.001, 
    batch_size=32,
    daily_seasonality=True,
    weekly_seasonality=True,
    loss_func='Huber'
)

# Aggiungi i regressori
for reg in regressors:
    neuralprophet.add_future_regressor(reg)

In [0]:
# TIME SERIES CROSS VALIDATION
initial_train_size = len(train_df_np)-5*len(test_df_np)
h = len(test_df_np)
n_windows = 5

# Verifica
required_length = initial_train_size + h * n_windows
if len(np_df_train) < required_length:
    raise ValueError("Dataset troppo corto per questo schema di cross-validation.")

# Reset risultati
results = []

for i in range(n_windows):
    print(f"\n--- Fold {i+1}/{n_windows} ---")

    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    train_window = np_df_train.iloc[:train_end]
    test_window = np_df_train.iloc[test_start:test_end]

    print(f"Train: {train_window.shape}, Test: {test_window.shape}")

    # Modello
    model = NeuralProphet(
        quantiles=[0.025, 0.975],
        learning_rate=0.01,
        batch_size=32,
        daily_seasonality=True,
        weekly_seasonality=True,
        loss_func='Huber',
        global_normalization=True,
        unknown_data_normalization=True
    )

    for reg in regressors:
        model.add_future_regressor(reg)

    # Fit
    _ = model.fit(train_window, freq="15min", epochs=100)

    # Previsione
    forecast = model.predict(test_window)

    # Metriche
    y_true = test_window['y'].values
    y_pred = forecast['yhat1'].values

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    results.append({'split': i+1, 'MAE_np': mae, 'RMSE_np': rmse})
    print(f"Split {i+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# Risultati
results_df_np = pd.DataFrame(results)
print("\n=== Backtest Results ===")
print(results_df_np)

# Plot
results_df_np.plot(x='split', y=['MAE_np', 'RMSE_np'], marker='o', title='Backtesting Errors')


In [0]:
results_df_np

In [0]:
# Mi ricavo le date
initial_train_size = len(train_df_np)-5*len(test_df_np)
h = len(test_df_np)
n_windows = 5

# Crea lista per i dati delle fold
fold_data = []

for i in range(n_windows):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    # Ricava date da colonna 'ds' o dall'indice
    if 'ds' in np_df_train.columns:
        start_date = np_df_train['ds'].iloc[test_start]
        end_date = np_df_train['ds'].iloc[test_end - 1]
    else:
        start_date = np_df_train.index[test_start]
        end_date = np_df_train.index[test_end - 1]

    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]

    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)

# Converto le date in datetime se necessario
if all(isinstance(date, str) for date in fold_intervals_df['start_date']):
    fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
    fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])

print("Fold Intervals DataFrame:")
print(fold_intervals_df)

In [0]:
# CV PLOT
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_np['ds'], train_df_np['y'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

Now let's training on the whole dataset

In [0]:
# Definisco il modello

# Fisso i seed per riproducibilità
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.01, #0.005
    batch_size=32,
    daily_seasonality=True,
    weekly_seasonality=True,
    loss_func='Huber'
)

# Aggiungi i regressori
for reg in regressors:
    neuralprophet.add_future_regressor(reg)

In [0]:
# FIT
start_time = time.time()
metrics_df = neuralprophet.fit(np_df_train, freq="D", epochs=300)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
forecast_test = neuralprophet.predict(np_df_test)
forecast_test.columns

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
forecast_test['yhat1'] = forecast_test['yhat1'] * (max_val - min_val) + min_val
forecast_test['yhat1 2.5%'] = forecast_test['yhat1 2.5%'] * (max_val - min_val) + min_val
forecast_test['yhat1 97.5%'] = forecast_test['yhat1 97.5%'] * (max_val - min_val) + min_val

In [0]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_original['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_original['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))

# Out-of-sample forecast (future)
# fig.add_trace(go.Scatter(
#     x=forecast_future['ds'], 
#     y=forecast_future['yhat1'],
#     mode='lines', 
#     line=dict(color='green'),
#     showlegend=False,  # Evita duplicati nella legenda
#     name="Out-of-sample"
# ))


# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast_test['ds'], 'forecast': forecast_test['yhat1']})
merged_df = pd.merge(test_df_original[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
mae_val = mean_absolute_error(merged_df['y'], merged_df['forecast'])
metriche_np = calcola_metriche(merged_df['y'], merged_df['forecast'], 
                               train_df_original['y'], modelname="NeuralProphet")
metriche_np

#### TFT

In [0]:
# Preparing dataset
df_tft = create_time_series_features(df, target_col='y') 
df_tft = df_tft.iloc[8:]
# 2. Suddivisione in train/test
train_size = int(len(df_tft) * 0.9)
train_df_tft = df_tft.iloc[:train_size]
test_df_tft = df_tft.iloc[train_size:]
train_df_tft.shape, test_df_tft.shape

In [0]:
# Scaling dei dati
#Salvo per dopo
train_df_original = train_df_tft.copy() 
test_df_original = test_df_tft.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_tft[['ds','unique_id']]
test_meta = test_df_tft[['ds','unique_id']]

feature_cols = [col for col in train_df_tft.columns if col not in ['ds','unique_id']]
train_idx = train_df_tft.index
test_idx = test_df_tft.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_tft[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_tft[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_tft = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_tft = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_tft.shape, test_df_tft.shape

#####  TimeSeriesCV - fine tuning Optuna (LENTO)

In [0]:
# Optuna function
futr_exog_list = train_df_tft.drop(columns=['unique_id', 'ds', 'y']).columns.tolist()
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_tft = pd.DataFrame()
h = len(test_df_tft)

def objective(trial):
    # Hyperparameters to optimize for TFT
    input_size = trial.suggest_categorical("input_size", [h, int(1.5*h), int(2*h)])
    hidden_size = trial.suggest_categorical("hidden_size", [32, 64, 128, 256])
    n_rnn_layers = trial.suggest_int("n_rnn_layers", 1, 4)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    grn_activation = trial.suggest_categorical("grn_activation", ["ReLU", "ELU", "Sigmoid"])
    
    # Define TFT model
    tft = TFT(
        h=h,  # forecast horizon
        input_size=input_size,
        hidden_size=hidden_size,
        n_rnn_layers=n_rnn_layers,
        rnn_type='lstm',
        grn_activation=grn_activation,
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=learning_rate,
        futr_exog_list=futr_exog_list,
        max_steps=50, 
        val_check_steps=10,
        batch_size=batch_size,
        early_stop_patience_steps=-1,
        scaler_type="robust",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=64
)

    nf = NeuralForecast(models=[tft], freq='D')

    # Cross-validation for tuning
    df_cv = nf.cross_validation(
        df=train_df_tft,
        step_size=h,
        n_windows=5
    )

    # Calculate MAE for each fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'TFT']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'TFT',
            'MAE_tft': mae,
            'RMSE_tft': rmse
        })

    # Average MAE across all folds
    cv_metrics_df_tft = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_tft['MAE_tft'].mean()

    return mean_mae

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)  #aumentare a 50 su databricks

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params
# #salvo i best params in pickle
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_bestPars_Outliers.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(best_params, f)

In [0]:
# #carico i best params
with open('pickles/TFT_bestPars_Outliers.pkl', 'rb') as file:
    best_params = pickle.load(file)

La CV è troppo lenta, si blocca!

In [0]:
# final CV

# final model
best_tft = TFT(
        h=h,  
        input_size=int(h/3), #best_params['input_size']
        hidden_size=4, #best_params['hidden_size']
        n_rnn_layers=1, #best_params['n_rnn_layers']
        rnn_type='lstm',
        grn_activation=best_params['grn_activation'],
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=best_params['learning_rate'],
        futr_exog_list=futr_exog_list,
        max_steps=50, #aumentare a 100
        val_check_steps=10,
        batch_size=int(best_params['batch_size'] / 4),
        early_stop_patience_steps=-1,
        scaler_type="robust",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False
)
nf = NeuralForecast(models=[best_tft],freq="D")

# final CV
df_cv = nf.cross_validation(
    df=train_df_tft[200:],
    step_size=int(h/2),
    n_windows=5
)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'TFT']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'TFT',
        'MAE_TFT': mae,
        'RMSE_TFT': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_tft = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_tft)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_tft.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/cv_metrics_df_tft.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(cv_metrics_df_tft, f)

In [0]:
# carico le metriche di CV
with open('pickles/Outliers_cv_metrics_df_tft.pkl', 'rb') as file:
    cv_metrics_df_tft = pickle.load(file)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_tft['ds'], train_df_tft['y'], label=f'Serie unica')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_tft['Model'] = 'TFT'
# Plot MAE sul primo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_TFT'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_TFT'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


Final training

In [0]:
# Final training
best_tft = TFT(
        h=h,  
        input_size=best_params['input_size'],
        hidden_size=best_params['hidden_size'],
        n_rnn_layers=best_params['n_rnn_layers'],
        rnn_type='lstm',
        grn_activation=best_params['grn_activation'],
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=best_params['learning_rate'],
        futr_exog_list=futr_exog_list,
        max_steps=100, 
        val_check_steps=25,
        batch_size=best_params['batch_size'],
        early_stop_patience_steps=-1,
        scaler_type="robust",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=64        
)
nf = NeuralForecast(models=[best_tft],freq="D")
import time
start_time = time.time()
#fit 
nf.fit(df=train_df_tft, val_size=len(test_df_tft))

end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Predict
Y_hat_df_tft = nf.predict(futr_df=test_df_tft)
#salvo in pickle
# with open('pickles/TFT_Outliers.pkl', 'wb') as file:
#     pickle.dump(Y_hat_df_tft, file)

In [0]:
# #carico le previsioni:
with open('pickles/TFT_Outliers.pkl', 'rb') as file:
    Y_hat_df_tft = pickle.load(file)

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
Y_hat_df_tft['TFT'] = Y_hat_df_tft['TFT'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-median'] = Y_hat_df_tft['TFT-median'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-lo-90'] = Y_hat_df_tft['TFT-lo-90'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-hi-90'] = Y_hat_df_tft['TFT-hi-90'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_tft['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_tft['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='lightgreen')))

# QUANTILI TFT
# Fascia tra P10 e P90
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_tft['ds']) + 
    list(Y_hat_df_tft['ds'][::-1]),
    y=list(Y_hat_df_tft['TFT-hi-90']) + list(Y_hat_df_tft['TFT-lo-90'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="TFT forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_tft['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# Calcola metriche
merged_df_tft = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_tft, on='ds', how='left')
merged_df_tft.rename(columns={'TFT-lo-90': '0.1','TFT-median':'0.5', 'TFT-hi-90':'0.9'},inplace=True)
merged_df_tft.dropna(inplace=True)
metriche_tft = calcola_metriche(merged_df_tft['y'],merged_df_tft['TFT'],train_df_original['y'],
                                y_pred_quantiles=merged_df_tft[['0.1','0.5','0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="TFT").round(10)
metriche_tft

##### TFT Explainability

Questo modello non me lo faceva scaricare perchè troppo pesante

In [0]:
# model_tft = nf.models[0]
# #salvo il modello (ero in DBR)
# torch.save(model_tft, 'TFT_model_Outliers.pkl')

In [0]:
# # carico il modello
# model_tft = torch.load('pickles/TFT_model.pth', map_location=torch.device('cpu'))

In [0]:
attention = nf.models[0].attention_weights()
def plot_attention(
    self, plot: str = "time", output: str = "plot", width: int = 800, height: int = 400
):
    """
    Plot the attention weights.

    Args:
        plot (str, optional): The type of plot to generate. Can be one of the following:
            - 'time': Display the mean attention weights over time.
            - 'all': Display the attention weights for each horizon.
            - 'heatmap': Display the attention weights as a heatmap.
            - An integer in the range [1, model.h) to display the attention weights for a specific horizon.
        output (str, optional): The type of output to generate. Can be one of the following:
            - 'plot': Display the plot directly.
            - 'figure': Return the plot as a figure object.
        width (int, optional): Width of the plot in pixels. Default is 800.
        height (int, optional): Height of the plot in pixels. Default is 400.

    Returns:
        matplotlib.figure.Figure: If `output` is 'figure', the function returns the plot as a figure object.
    """

    attention = (
        self.mean_on_batch(self.interpretability_params["attn_wts"])
        .mean(dim=0)
        .cpu()
        .numpy()
    )

    fig, ax = plt.subplots(figsize=(width / 100, height / 100))

    if plot == "time":
        attention = attention[self.input_size :, :].mean(axis=0)
        ax.plot(np.arange(-self.input_size, self.h), attention)
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Mean Attention")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "all":
        for i in range(self.input_size, attention.shape[0]):
            ax.plot(
                np.arange(-self.input_size, self.h),
                attention[i, :],
                label=f"horizon {i-self.input_size+1}",
            )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Attention per horizon")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "heatmap":
        cax = ax.imshow(
            attention,
            aspect="auto",
            cmap="viridis",
            extent=[-self.input_size, self.h, -self.input_size, self.h],
        )
        fig.colorbar(cax)
        ax.set_title("Attention Heatmap")
        ax.set_xlabel("Attention (current time step)")
        ax.set_ylabel("Attention (previous time step)")

    elif isinstance(plot, int) and (plot in np.arange(1, self.h + 1)):
        i = self.input_size + plot - 1
        ax.plot(
            np.arange(-self.input_size, self.h),
            attention[i, :],
            label=f"horizon {plot}",
        )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title(f"Attention weight for horizon {plot}")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    else:
        raise ValueError(
            'plot has to be in ["time","all","heatmap"] or integer in range(1,model.h)'
        )

    plt.tight_layout()

    if output == "plot":
        plt.show()
    elif output == "figure":
        return fig
    else:
        raise ValueError(f"Invalid output: {output}. Expected 'plot' or 'figure'.")

In [0]:
# MEAN ATTENTION
plot_attention(nf.models[0], plot="time")
# pesi di attenzione medi nel tempo su tutti gli orizzonti di previsione.
# La curva blu rappresenta quanto il modello ha "guardato" (attention) 
# quel punto temporale in input durante la previsione.
# Valori più alti significano che il modello ritiene quel punto temporale
# particolarmente rilevante per fare le previsioni

In [0]:
# PAST VARIABLE IMPORTANCES
feature_importances = nf.models[0].feature_importances()
print(feature_importances.keys())
print("\n ==========PAST FEATURE IMPORTANCES==========")
feature_importances["Past variable importance over time"].mean().sort_values().plot(kind="barh")

Questo grafico mostra quanto il modello ha fatto affidamento sulle feature passate \
(quelle che il modello ha potuto osservare prima del punto di previsione). Serve a costruire il contesto dell'input

In [0]:
# FUTURE VARIABLE IMPORTANCES
print("\n ==========FUTURE FEATURE IMPORTANCES==========")
feature_importances["Future variable importance over time"].mean().sort_values().plot(kind="barh")

Mostra quanto il modello ha sfruttato le feature che sono servite a guidare le previsioni passo-passo nel decoder

DIFFERENZA TRA PAST E FUTURE: \
La diversa importanza riflette quanto ciascuna variabile aiuta nell’analisi del contesto passato (encoder -->> PAST) \
o nella generazione delle previsioni (decoder -->> FUTURE).

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME
df_importances = feature_importances["Past variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances.index))

for col in df_importances.columns:
    p = ax.bar(np.arange(-len(df_importances), 0), df_importances[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances[col]
ax.set_title("Past variable importance over time")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)

plt.show()

Questo grafico mostra QUANDO (e QUANTO) nel passato ogni feature è stata più utile per costruire il contesto predittivo.

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME
df_importances2 = feature_importances["Future variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances2.index))
for col in df_importances2.columns:
    p = ax.bar(np.arange(-len(df_importances2), 0), df_importances2[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances2[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.grid(True)
ax.legend()
plt.show()

Per ogni feature viene mostrato quanto è stata importante per il modello a ciascun orizzonte futuro

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances3 = feature_importances["Past variable importance over time"]
mean_attention = (
    nf.models[0]
    .attention_weights()[nf.models[0].input_size :, :]
    .mean(axis=0)[: nf.models[0].input_size]
)
df_importances3 = df_importances3.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances3.index))

for col in df_importances3.columns:
    p = ax.bar(np.arange(-len(df_importances3), 0), df_importances3[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances3[col]
ax.set_title("Past variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances3), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances4 = feature_importances["Future variable importance over time"]
mean_attention = (
    nf.models[0]
    .attention_weights()[nf.models[0].input_size :, :]
    .mean(axis=0)[: nf.models[0].input_size]
)
df_importances4 = df_importances4.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances4.index))

for col in df_importances4.columns:
    p = ax.bar(np.arange(-len(df_importances4), 0), df_importances4[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances4[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances4), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
nf.models[0].feature_importance_correlations()

#### CHRONOS

In [0]:
# Preparing dataset
df_chronos = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos = df_chronos.dropna().reset_index(drop=True) 
df_chronos['item_id'] = df_chronos['unique_id'] 
df_chronos.drop(columns="unique_id", inplace=True) 
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Definizione delle covariate note
known_covariates_names=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id','target']]
feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

# Split train e test
train_size = int(len(df_chronos) * 0.9)
train_df_chronos = df_chronos.iloc[:train_size]
test_df_chronos = df_chronos.iloc[train_size:]

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [0]:
 # Scaling dei dati
train_df_original = train_df_chronos.copy() 
test_df_original = test_df_chronos.copy()

train_idx = train_df_chronos.index
test_idx = test_df_chronos.index

#scaler
scaler_x = MinMaxScaler()
train_df_chronos = pd.DataFrame(scaler_x.fit_transform(train_df_chronos[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_chronos = pd.DataFrame(scaler_x.transform(test_df_chronos[feature_cols]),
                            columns=feature_cols, index=test_idx)
#controllo le shape
train_df_chronos.shape, test_df_chronos.shape

TimeSeriesCV

In [0]:
#TimeSeriesCV
h = len(test_df_chronos)  # prediction length
n_splits = 5
initial_train_size = len(train_df_chronos) - 5*h
results = []

for fold in range(n_splits):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")
    
    train_end = initial_train_size + fold * h
    test_start = train_end
    test_end = test_start + h
    train_df_fold = train_df_chronos.iloc[:train_end]
    test_df_fold = train_df_chronos.iloc[test_start:test_end]
    
    train_df_fold.reset_index(inplace=True)
    test_df_fold.reset_index(inplace=True)
    
    train_df_fold["timestamp"] = pd.to_datetime(train_df_fold["timestamp"])
    test_df_fold["timestamp"] = pd.to_datetime(test_df_fold["timestamp"])
    
    # Future timestamps 
    future_index = pd.date_range(
        start=train_df_fold["timestamp"].max() + pd.Timedelta(days=1),
        periods=h,
        freq="D")
    
    # Preparo test set con known_covariates
    test_df_for_prediction = test_df_fold[test_df_fold["timestamp"].isin(future_index)].copy()
    test_df_for_prediction = test_df_for_prediction.set_index("timestamp")
    test_df_for_prediction = test_df_for_prediction.reindex(future_index)  # forza continuità
    test_df_for_prediction["item_id"] = train_df_fold["item_id"].iloc[0]  # supponiamo 1 sola serie
    test_df_for_prediction = test_df_for_prediction.reset_index().rename(columns={"index": "timestamp"})
    test_df_for_prediction = test_df_for_prediction.set_index(["item_id", "timestamp"])
    
    # Preparo il train con multindex
    train_df_fold = train_df_fold.set_index(["item_id", "timestamp"])
    print(f"Train: {train_df_fold.shape}, Test: {test_df_fold.shape}")
    
    # inizializzo il predictor
    predictor = TimeSeriesPredictor(
        prediction_length=h,
        target="target",
        known_covariates_names=known_covariates_names,
        eval_metric="MASE",
        freq="D")
    
    # fit del modello
    predictor.fit(train_df_fold,
        hyperparameters={
            "Chronos": [
                {"model_path": "bolt_small", "ag_args": {"name_suffix": "ZeroShot"}},
                {
                    "model_path": "bolt_small",
                    "covariate_regressor": "CAT",
                    "target_scaler": "standard",
                    "ag_args": {"name_suffix": "WithRegressor"},
                },
                {
                    "model_path": "bolt_base",
                    "covariate_regressor": "MLP",
                    "target_scaler": "robust",
                    "ag_args": {"name_suffix": "MLP_Regressor"},
                },
            ]
        },
        time_limit=600,
        enable_ensemble=True,)
    
    predictions = predictor.predict(train_df_fold, known_covariates=test_df_for_prediction)

    test_df_with_index = test_df_fold.set_index(["item_id", "timestamp"])

    # Debug
    print(f"Forma predictions: {predictions.shape}")
    print(f"Colonne predictions: {predictions.columns}")
    print(f"Numero di indici in predictions: {len(predictions.index)}")
    print(f"Numero di indici in test_df_with_index: {len(test_df_with_index.index)}")

    # trovo solo gli indici comuni
    common_indices = predictions.index.intersection(test_df_with_index.index)
    print(f"Indici comuni: {len(common_indices)}")

    # prendo solo gli indici comuni
    y_true = test_df_with_index.loc[common_indices, "target"]
    y_pred = predictions.loc[common_indices, 'mean'].to_numpy()

    mae = calcola_mae(y_true, y_pred)
    rmse = calcola_rmse(y_true, y_pred)
    
    results.append({'split': fold+1, 'MAE': mae, 'RMSE': rmse})
    print(f"Split {fold+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
# risultati finali
results_df_chronos = pd.DataFrame(results)
results_df_chronos.set_index('split',inplace=True)
print("\n=== Backtest Results ===")
results_df_chronos

In [0]:
# Estraggo le date
fold_data = []
train_df_chronos.reset_index(inplace=True)
for i in range(n_splits):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h
    
    # Ricava date da colonna 'ds' o dall'indice
    if 'timestamp' in train_df_chronos.columns:
        start_date = train_df_chronos['timestamp'].iloc[test_start]
        end_date = train_df_chronos['timestamp'].iloc[test_end - 1]
    else:
        start_date = train_df_chronos.index[test_start]
        end_date = train_df_chronos.index[test_end - 1]
    
    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]
    
    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)
fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])
# Stampa il dataframe prima della conversione per verificare i valori
print("Fold Intervals DataFrame (before conversion):")
fold_intervals_df

In [0]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_chronos['timestamp'], train_df_chronos['target'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot degli errori
results_df_chronos.reset_index(inplace=True)
ax = results_df_chronos.plot(x='split', y=['MAE','RMSE'], marker='o', title='TimeSeriesCV Errors')
ax.set_xticks(range(1, 6)) # Imposta le tacche sull'asse x da 1 a 5
plt.show()

Let's training on the whole dataset

In [0]:
# Fit del modello

# SOLO PER DATABRICKS
# sys.modules['sklearn.metrics._regression'].mean_absolute_error = sklearn.metrics.mean_absolute_error

predictor = TimeSeriesPredictor(
    prediction_length=len(test_df_chronos), 
    eval_metric="MASE",
    target="target",
    known_covariates_names=known_covariates_names,
    freq = "D"
)

# Aggiunta di più modelli nella configurazione per migliorare le performance
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": 
            [
            {
                "model_path": "bolt_small",
                "ag_args": {"name_suffix": "ZeroShot"}},
            {
                "model_path": "bolt_base",
                "covariate_regressor": "NN_TORCH",
                "target_scaler": "robust",
                "ag_args": {"name_suffix": "WithRegressor"},
            }
        ],
    },
    time_limit=600,
    enable_ensemble=True,  
)

# Valutazione del modello
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [0]:
# PREVISIONI CLASSICHE
# col miglior modello e col 0-shot
predictions = predictor.predict(
    train_df_chronos, 
    known_covariates=test_df_chronos
)
predictions.reset_index(inplace=True)

predictions_0shot = predictor.predict(
    train_df_chronos, 
    known_covariates=test_df_chronos,
    model="ChronosZeroShot[bolt_small]"
)
predictions_0shot.reset_index(inplace=True)

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('target')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
predictions['mean'] = predictions['mean'] * (max_val - min_val) + min_val
predictions['0.1'] = predictions['0.1'] * (max_val - min_val) + min_val
predictions['0.5'] = predictions['0.5'] * (max_val - min_val) + min_val
predictions['0.9'] = predictions['0.9'] * (max_val - min_val) + min_val
predictions_0shot['mean'] = predictions_0shot['mean'] * (max_val - min_val) + min_val
predictions_0shot['0.1'] = predictions_0shot['0.1'] * (max_val - min_val) + min_val
predictions_0shot['0.5'] = predictions_0shot['0.5'] * (max_val - min_val) + min_val
predictions_0shot['0.9'] = predictions_0shot['0.9'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)

# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_chronos['timestamp'], y=train_df_original['target'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['0.5'], mode='lines', 
    name='P50', line=dict(color='green')))
# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='ChronosWithRegressor', line=dict(color='lightgreen')))
# QUANTILI CHRONOS
fig.add_trace(go.Scatter(
    x=list(test_df_chronos['timestamp']) + 
    list(test_df_chronos['timestamp'][::-1]),
    y=list(predictions['0.9']) + list(predictions['0.1'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# PREVISIONI CHRONOS 0 SHOT
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='0-shot Forecast', line=dict(color='yellow')))
# QUANTILI CHRONOS 0 SHOT
fig.add_trace(go.Scatter(
    x=list(test_df_chronos['timestamp']) + 
    list(test_df_chronos['timestamp'][::-1]),
    y=list(predictions_0shot['0.9']) + list(predictions_0shot['0.1'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90 0shot'
))

# Layout del grafico
fig.update_layout(title="CHRONOS forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_chronos['timestamp'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# metriche di valutazione
test_df_original.reset_index(inplace=True)
merged_df_chronos = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)
metriche_chronos = calcola_metriche(merged_df_chronos['target'],merged_df_chronos['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos").round(5)
metriche_chronos

In [0]:
# metriche 0 shot
merged_df_chronos_0shot = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions_0shot[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)
metriche_chronos_0shot = calcola_metriche(merged_df_chronos_0shot['target'],merged_df_chronos_0shot['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos_0shot[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos 0-shot")
pd.set_option('display.float_format', '{:.4f}'.format)
metriche_chronos_0shot

Il modello Chronos 0 shot funziona meglio, perchè? 
1) Il modello 0 shot funziona molto bene con dataset piccoli (il regressor CAT ha poco su cui lavorare)
2) Il modello 0 shot funziona molto bene con dataset regolari ad alta stagionalità
3) Le covariate sono deboli e/o altamente correlate fra di loro
Il gap di performance è basso, questo a dimostrazione del fatto che i due modelli performano in maniera abbastanza simile, ma le covariate portano leggermente fuori strada il modello ChronosWithRegressor

#### final evaluations

In [0]:
final_metrics = metriche_arima.join(metriche_ets).join(metriche_xgb).join(metriche_lstm).join(metriche_np).join(metriche_chronos).join(metriche_chronos_0shot).join(metriche_tft).round(4)
final_metrics

In [0]:
# FINAL PLOT for outlier data
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoARIMA'], 
    mode='lines', 
    name='ARIMA', 
    line=dict(color='grey')
))

# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoETS'], 
    mode='lines', 
    name='ETS', 
    line=dict(color='orange')
))

# PREVISIONI XGBOOST
fig.add_trace(go.Scatter(
    x=forecast_xgb['ds'], 
    y=forecast_xgb['forecast'], 
    mode='lines', 
    name='XGB', 
    line=dict(color='purple')
))

# PREVISIONI LSTM
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=Y_hat_df_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='pink')
))

# PREVISIONI NEURAL PROPHET
# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet ', 
    line=dict(color='aquamarine')
))

    
# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=forecast_test['ds'], 
                         y=Y_hat_df_tft['TFT'],
                         mode='lines', 
    name='TFT', line=dict(color='yellow')))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='ChronosWithRegressor', line=dict(color='lightgreen')))

# PREVISIONI CHRONOS 0 SHOT
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='Chronos 0 shot', line=dict(color='green')))

# Layout del grafico
fig.update_layout(
    title="Forecasts for White Noise", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
# sistemo i vari dataset prima del merge
results_df_np.rename(columns={'split': 'Fold'}, inplace=True)
results_df_chronos.rename(columns={'split': 'Fold','MAE':'MAE_chronos','RMSE':'RMSE_chronos'}, inplace=True)
cv_metrics_df_lstm.drop(columns=('Model'),inplace=True)
cv_metrics_df_xgboost.rename(columns={'MAE': 'MAE_XGB','RMSE':'RMSE_XGB'}, inplace=True)
df_arima = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == 'AutoARIMA'][['Fold',
                                                                    'MAE','RMSE']]
df_ets = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == 'AutoETS'][['Fold',
                                                                    'MAE','RMSE']]
df_arima.rename(columns={'MAE':'MAE_AutoARIMA','RMSE':'RMSE_AutoARIMA'}, inplace=True)
df_ets.rename(columns={'MAE':'MAE_AutoETS','RMSE':'RMSE_AutoETS'}, inplace=True)
# aggiusto i valori minimi e massimi (senza outlier)
jan_2017 = train_df_lstm[(train_df_lstm['ds'].dt.year == 2017) & (train_df_lstm['ds'].dt.month == 1)]
# Calcola il valore massimo del target in gennaio 2017
max_val = jan_2017['y'].max()
min_val = 0

In [0]:
# Final plot metrics TimeSeriesCV

# faccio il merge di tutte le tabelle dei vari CV
final_cv = results_df_np.merge(cv_metrics_df_lstm, 
                            on="Fold",how="inner").merge(results_df_chronos,
                            on="Fold",how="inner").merge(cv_metrics_df_xgboost[['MAE_XGB','RMSE_XGB','Fold']],
                            on="Fold",how="inner").merge(df_arima,
                            on="Fold",how="inner").merge(df_ets,
                            on="Fold", how="inner").merge(cv_metrics_df_tft.drop(columns='Model'), 
                            on="Fold",how="inner")
# Rescaling CV metrics data
columns_to_modify = ['MAE_np', 'RMSE_np', 'MAE_TFT', 'RMSE_TFT',
                        'MAE_chronos', 'RMSE_chronos', 'MAE_XGB', 'RMSE_XGB']
for col in columns_to_modify:
    final_cv[f'{col}'] = final_cv[f'{col}'] * (max_val - min_val) + min_val

model_colors = {
    'AutoARIMA': '#808080',      # Grey
    'AutoETS': '#ff7f0e',        # Orange
    'XGB': '#9467bd',    # Purple
    'lstm': '#e377c2',       # Pink
    'np': '#7fffd4',         # Aquamarine
    'TFT': '#ffff00',        # Yellow
    'chronos': '#2ca02c',    # Green
}


# Create the figure
fig = go.Figure()

# Add traces for each model - MAE (darker) and RMSE (lighter)
for model, color in model_colors.items():
    # Add MAE line (darker)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'MAE_{model}'],
        mode='lines+markers',
        name=f'{model} MAE',
        line=dict(color=color, width=3),
        marker=dict(size=8)
    ))
    
    # Add RMSE line (lighter with same color but different dash pattern)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'RMSE_{model}'],
        mode='lines+markers',
        name=f'{model} RMSE',
        line=dict(color=color, width=2, dash='dash'),
        marker=dict(size=6)
    ))

# Update layout
fig.update_layout(
    title='Model Performance Comparison (MAE and RMSE)',
    template='plotly_dark',
    xaxis=dict(title='Fold',
        tickmode='linear',
        tick0=1, dtick=1
    ),    yaxis=dict(
        title='Error Value'
    ),    legend=dict(
        orientation="v"
    ),    hovermode="closest")
fig.show()

In [0]:
# Confronto TFT/CHRONOS
tft_chronos = metriche_chronos.join(metriche_tft)
tft_chronos.round(4)   

### **OUTLIERS DATA (multistep)**

H = 90 gg 

In [0]:
#preparazione dataset
df = out.reset_index().rename(columns={'date': 'ds','total_daily_sales':'y'})
df['unique_id'] = 'serie_1'
df['ds'] = pd.to_datetime(df['ds'])

In [0]:
# funzione per creare feature
def create_time_series_features(df, target_col):
    df = df.copy()
    # Lag features (1-7)
    for lag in range(1, 8):
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    # Differencing features, 1 e 7
    df['diff_1'] = (df[target_col] - df[target_col].shift(1)).shift(1)
    df['diff_1'] = (df[target_col] - df[target_col].shift(7)).shift(1)
    
    # Caratteristiche temporali
    df['dayofweek'] = df['ds'].dt.dayofweek
    df['month'] = df['ds'].dt.month
    df['quarter'] = df['ds'].dt.quarter
    
    # Caratteristiche cicliche per rappresentare meglio la stagionalità
    df['day_sin'] = np.sin(2 * np.pi * df['dayofweek']/7)
    df['day_cos'] = np.cos(2 * np.pi * df['dayofweek']/7)
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    
    # rolling means 
    df['rolling_mean_3'] = df[target_col].rolling(window=3).mean().shift(1) #ultimoi 3 giorni
    df['rolling_mean_7'] = df[target_col].rolling(window=7).mean().shift(1) #ultima settimana
    df['rolling_std_3'] = df[target_col].rolling(window=3).std().shift(1)
    df['rolling_std_7'] = df[target_col].rolling(window=7).std().shift(1)
    
    df['covid'] = (df['ds'] > pd.Timestamp('2020-02-01')).astype(int)
        
    return df

In [0]:
h = 90
futr_exog_list = ['dayofweek', 'month', 'quarter', 'day_sin',
       'day_cos', 'month_sin', 'month_cos', 'covid']
future_features = ['ds', 'y'] + futr_exog_list

##### XGBOOST

In [0]:
# df_xgb
df_xgb = create_time_series_features(df, target_col='y') 
hist_exog_features = df_xgb.drop(columns=['unique_id', 'ds', 'y'] + futr_exog_list).columns.tolist()
df_xgb.drop(columns=['unique_id'],inplace=True)
df_xgb = df_xgb.dropna().reset_index(drop=True) #droppo i NaN creati dai lag, quindi le prime 12 obs.
# STEP 2: Prepara DMatrix per previsione
df_xgb = df_xgb[future_features]
print(df_xgb.columns)

In [0]:
# Split train/test
# train tutto tranne le ultime 12 osservazioni
train_df_xgb = df_xgb.iloc[:-h]
test_df_xgb = df_xgb.iloc[-h:]

# Separazione features e target
feature_cols = [col for col in df_xgb.columns if col not in ['y', 'ds']]
X_train_xgb, y_train_xgb = train_df_xgb[feature_cols], train_df_xgb['y']
X_test_xgb, y_test_xgb = test_df_xgb[feature_cols], test_df_xgb['y']

In [0]:
# train e test plot
fig = go.Figure()
#train dat
fig.add_trace(go.Scatter(x=df_xgb['ds'], y=y_train_xgb, mode='lines', name='Train'))
#test data 
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=y_test_xgb, mode='lines', name='Test'))
#layout
fig.update_layout(template='plotly_dark', title='Train and Test Data Plot')
fig.show()

In [0]:
# Scaling dei dati
#features
scaler_x = MinMaxScaler()
X_train_scaled = scaler_x.fit_transform(X_train_xgb)
X_test_scaled = scaler_x.transform(X_test_xgb)

# target, non serve che scalo y_test
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train_xgb.values.reshape(-1, 1))

# Ricreo df con stessi indici e nomi
X_train_xgb = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train_xgb.index)
X_test_xgb = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test_xgb.index)
y_train_xgb = pd.Series(y_train_scaled.flatten(), name='y', index=y_train_xgb.index)

#aggiorno train_df_xgb
train_df_xgb[feature_cols] = X_train_xgb
train_df_xgb['y'] = y_train_xgb

#controllo le shape
X_train_xgb.shape, X_test_xgb.shape, y_train_xgb.shape, y_test_xgb.shape

Ottimizzazione Optuna (Bayesiana)

In [0]:
# Optuna function
cv_metrics_log = []  # List globale per salvare metriche fold per fold

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'verbosity': 0,
        'seed': 42
    }

    tscv = TimeSeriesSplit(n_splits=5,test_size=len(test_df_xgb))
    y = train_df_xgb['y']
    df_xgb_feature = train_df_xgb.drop(columns=['y','ds'])
    all_rmse = []

    for fold, (train_idx, test_idx) in enumerate(tscv.split(df_xgb_feature)):
        X_train, X_test = df_xgb_feature.iloc[train_idx], df_xgb_feature.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        dtest = xgb.DMatrix(X_test.values, label=y_test.values)

        model = xgb.train(params, dtrain, num_boost_round=500,
                          evals=[(dtest, 'eval')],
                          early_stopping_rounds=30, verbose_eval=False)

        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mae = calcola_mae(y_test, preds)  

        all_rmse.append(rmse)

        # Recupera le date (se esiste colonna 'ds')
        start_date = df_xgb.iloc[test_idx]['ds'].min() if 'ds' in df_xgb.columns else None
        end_date = df_xgb.iloc[test_idx]['ds'].max() if 'ds' in df_xgb.columns else None

        # Logga le metriche della fold
        cv_metrics_log.append({
            'Trial': trial.number,
            'Fold': fold + 1,
            'MAE_XGB': mae,
            'RMSE_XGB': rmse,
            'start_date': start_date,
            'end_date': end_date,
            'Model': 'XGBoost'  
        })
    return np.mean(all_rmse)

In [0]:
# Optuna optimization
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)

In [0]:
# Best parameters
print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)
print("Best trial:",study.best_trial.number)
besttrial = study.best_trial.number

In [0]:
cv_metrics_log = pd.DataFrame(cv_metrics_log)
cv_metrics_log = cv_metrics_log[cv_metrics_log['Trial']==besttrial]
cv_metrics_log

Vediamo come ha performato nel miglior trial

In [0]:
#PLOT CV
cv_metrics_df_xgboost = cv_metrics_log.copy()

# Stampa le metriche per modello
print("Metriche per modello:")
print(cv_metrics_df_xgboost)

# Calcola e stampa le metriche medie per modello
mean_metrics = cv_metrics_df_xgboost.mean(numeric_only=True).reset_index()

# Crea il DataFrame con gli intervalli delle fold
fold_intervals_df = cv_metrics_df_xgboost[['Fold', 'start_date', 'end_date']].drop_duplicates()

# Ora creiamo il grafico combinato
plt.figure(figsize=(16, 12))

# === PLOT 1: Serie temporale originale ===
plt.subplot(2, 1, 1)
series_data = train_df_xgb.copy()
plt.plot(series_data['ds'], series_data['y'], label='Oil Price - XGB, CV')

# Evidenzia le fold con colori
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)],
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'{row["Fold"]}', ha='center', fontsize=10,
             bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# === PLOT 2: Metriche MAE e RMSE ===
plt.subplot(2, 1, 2)
ax1 = plt.gca()
ax2 = ax1.twinx()

# MAE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_XGB'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# RMSE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_XGB'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Legenda combinata
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


Ora facciamo training coi migliori parametri trovati da Optuna

In [0]:
# Addestriamo il modello con i best params
best_params = study.best_params
best_params.update({'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 42})
import time
dtrain_xgb = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
start_time = time.time()
#fit 
model = xgb.train(best_params, dtrain_xgb, num_boost_round=500)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot feature importance
fig, ax = plt.subplots()
xgb.plot_importance(model, importance_type='gain', max_num_features=10, ax=ax)
for text in ax.texts:
    text.set_visible(False)
plt.title('Feature Importance (Gain)')
plt.tight_layout()
plt.show()

In [0]:
dtest_xgb = xgb.DMatrix(X_test_xgb)
forecast_xgb = pd.DataFrame(model.predict(dtest_xgb),columns=['forecast'])
forecast_xgb['ds'] = test_df_xgb['ds'].values

Rescaling dei dati

In [0]:
# RE-Scaling dei dati
forecast_xgb['forecast'] = scaler_y.inverse_transform(forecast_xgb[['forecast']])
train_df_xgb['y'] = scaler_y.inverse_transform(train_df_xgb[['y']])

In [0]:
# Plot forecast
fig = go.Figure()
# Training data
fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=train_df_xgb['y'], mode='lines', name='Training'))
# Test data
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=test_df_xgb['y'], mode='lines', name='Test'))
# Forecast
fig.add_trace(go.Scatter(x=forecast_xgb['ds'], y=forecast_xgb['forecast'], mode='lines', 
                         name='Forecast', line=dict(color='deepskyblue')))
fig.update_layout(title='Forecast vs Real Data', template='plotly_dark')
fig.show()

Calcolo metriche

In [0]:
# Calcola metriche
merged_df = pd.merge(test_df_xgb[['ds', 'y']], forecast_xgb, on='ds', how='left')
merged_df.dropna(inplace=True)
metriche_xgb = calcola_metriche(merged_df['y'],merged_df['forecast'],train_df_xgb['y'],modelname="XGBoost")
metriche_xgb

##### LSTM

In [0]:
# divisione in train e test
df_lstm = create_time_series_features(df, target_col='y') 
df_lstm = df_lstm.dropna().reset_index(drop=True)

train_df_lstm = df_lstm.iloc[:-h]
test_df_lstm = df_lstm.iloc[-h:]
print(train_df_lstm.columns)
train_df_lstm.head()

In [0]:
# Scaling dei dati
train_df_original = train_df_lstm.copy()
test_df_original = test_df_lstm.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_lstm[['ds', 'unique_id']]
test_meta = test_df_lstm[['ds', 'unique_id']]

feature_cols = [col for col in train_df_lstm.columns if col not in ['ds', 'unique_id']]
train_idx = train_df_lstm.index
test_idx = test_df_lstm.index

#scaler
scaler_x = MinMaxScaler()
train_df_lstm = pd.DataFrame(scaler_x.fit_transform(train_df_lstm[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_lstm = pd.DataFrame(scaler_x.transform(test_df_lstm[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_lstm = pd.concat([train_df_lstm, train_meta], axis=1)
test_df_lstm = pd.concat([test_df_lstm, test_meta], axis=1)

#controllo le shape
train_df_lstm.shape, test_df_lstm.shape

In [0]:
# Optuna function
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_lstm = pd.DataFrame()
def objective(trial):
    # Hyperparametri da ottimizzare
    encoder_n_layers = trial.suggest_int("encoder_n_layers", 1, 2,4)
    encoder_hidden_size = trial.suggest_categorical("encoder_hidden_size", [32, 64, 128, 256])
    decoder_hidden_size = trial.suggest_categorical("decoder_hidden_size", [32, 64, 128, 256])
    decoder_layers = trial.suggest_int("decoder_layers", 1, 2,4)
    input_size = trial.suggest_categorical("input_size", [int(h), int(h*2), int(h*5)])
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    # Definizione modello LSTM
    lstm = LSTM(
        h=len(test_df_lstm), 
        input_size=input_size,
        loss=DistributionLoss(distribution="Normal", level=[95]),
        scaler_type='robust',
        encoder_n_layers=encoder_n_layers,
        encoder_hidden_size=encoder_hidden_size,
        decoder_hidden_size=decoder_hidden_size,
        decoder_layers=decoder_layers,
        futr_exog_list=futr_exog_list,
        max_steps=50,
        batch_size=batch_size,
        learning_rate=learning_rate,
        recurrent=False
    )

    nf = NeuralForecast(models=[lstm], freq='D')

    # cross-validation per tuning
    df_cv = nf.cross_validation(
        df=train_df_lstm,
        step_size=len(test_df_lstm),
        n_windows=5
    )

    # Calcola MAE per ogni fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'LSTM']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'LSTM',
            'MAE_lstm': mae,
            'RMSE_lstm': rmse
        })

    # Media MAE su tutte le fold
    cv_metrics_df_lstm = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_lstm['MAE_lstm'].mean()

    return mean_mae 

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)  #aumentare a 50 su databricks

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
with open('pickles/LSTM_bestpars_OutliersData_multistep.pkl', 'wb') as file:
    pickle.dump(best_params, file)

Carico i best params

In [0]:
with open('pickles/LSTM_bestpars_OutliersData_multistep.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [0]:
# final model
best_lstm = LSTM(
    h=len(test_df_lstm),
    input_size=len(test_df_lstm),
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=150,
    #early_stop_patience_steps=20,
    batch_size=best_params['batch_size'],
    learning_rate=best_params['learning_rate'],
    recurrent=False
)
nf = NeuralForecast(models=[best_lstm], freq='D')

In [0]:
# final CV
# df_cv = nf.cross_validation(
#     df=train_df_lstm,
#     step_size=len(test_df_lstm),
#     n_windows=5
# )
# with open('pickles/LSTM_cv_OilPrice_multistep.pkl', 'wb') as file:
#     pickle.dump(best_params, file)
with open('pickles/LSTM_cv_OilPrice_multistep.pkl', 'rb') as file:
    best_params = pickle.load(file)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'LSTM']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'LSTM',
        'MAE_lstm': mae,
        'RMSE_lstm': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_lstm = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_lstm)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_lstm.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_lstm['unique_id'].unique():
    series_data = train_df_lstm[train_df_lstm['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_lstm['Model'] = 'lstm'
# Plot MAE sul primo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_lstm'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_lstm'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


In [0]:
# Final training
final_model = LSTM(
    h=len(test_df_lstm),
    input_size=len(test_df_lstm),
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=150,
    #early_stop_patience_steps=20,
    batch_size=best_params['batch_size'],
    learning_rate=best_params['learning_rate'],
    recurrent=False
)

nf = NeuralForecast(models=[final_model], freq='D')

start_time = time.time()
#fit 
nf.fit(df=train_df_lstm, val_size=len(test_df_lstm))
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

Ora faccio predict

In [0]:
Y_hat_df_lstm = nf.predict(futr_df=test_df_lstm[['unique_id', 'ds'] + futr_exog_list])

In [0]:
# salvo in pickle
with open('pickles/LSTM_pred_Outliers_multistep.pkl', 'wb') as file:
    pickle.dump(Y_hat_df_lstm, file)

In [0]:
# carico da pickle
with open('pickles/LSTM_pred_Outliers_multistep.pkl', 'rb') as file:
    Y_hat_df_lstm = pickle.load(file)

In [0]:
#Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
Y_hat_df_lstm['LSTM'] = Y_hat_df_lstm['LSTM'] * (max_val - min_val) + min_val
Y_hat_df_lstm['LSTM-median'] = Y_hat_df_lstm['LSTM-median'] * (max_val - min_val) + min_val
Y_hat_df_lstm['LSTM-lo-95'] = Y_hat_df_lstm['LSTM-lo-95'] * (max_val - min_val) + min_val
Y_hat_df_lstm['LSTM-hi-95'] = Y_hat_df_lstm['LSTM-hi-95'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_lstm['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_lstm['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM'], mode='lines', 
    name='MEAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIANA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='yellow')))

# Intervallo di confidenza 95%
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_lstm['ds']) + 
    list(Y_hat_df_lstm['ds'][::-1]),
    y=list(Y_hat_df_lstm['LSTM-hi-95']) + list(Y_hat_df_lstm['LSTM-lo-95'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="LSTM forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_lstm['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")

fig.show()

In [0]:
# Calcola metriche
merged_df_lstm = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_lstm, on='ds', how='left')
merged_df_lstm.dropna(inplace=True)
metriche_lstm = calcola_metriche(merged_df_lstm['y'],merged_df_lstm['LSTM'],train_df_original['y'],modelname="LSTM")
metriche_lstm

##### NEURAL PROPHET

In [0]:
df_np = create_time_series_features(df, target_col='y') 
df_np = df_np.dropna().reset_index(drop=True)
df_np = df_np[future_features]
train_df_np = df_np.iloc[:-h]
test_df_np = df_np.iloc[-h:]

In [0]:
# Scaling dei dati
train_df_original = train_df_np.copy()
test_df_original = test_df_np.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_np['ds']
test_meta = test_df_np['ds']

feature_cols = [col for col in train_df_np.columns if col not in ['ds']]
train_idx = train_df_np.index
test_idx = test_df_np.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_np[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_np[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_np = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_np = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_np.shape, test_df_np.shape

TimeSeriesCV

In [0]:
# CV
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

initial_train_size = len(train_df_np) - 5 * len(test_df_np)
h = len(test_df_np)
n_windows = 5

results = []

for i in range(n_windows):
    print(f"\n--- Fold {i+1}/{n_windows} ---")

    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    train_window = train_df_np.iloc[:train_end].copy()
    test_window = train_df_np.iloc[test_start:test_end].copy()

    print(f"Train: {train_window.shape}, Test: {test_window.shape}")

    # Modello NeuralProphet
    model = NeuralProphet(
        quantiles=[0.025, 0.975],
        learning_rate=0.01,
        batch_size=64,
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
        loss_func='Huber',
    )

    # Aggiungi regressori
    for reg in futr_exog_list:
        model.add_future_regressor(reg)

    # Fit
    _ = model.fit(train_window, freq="D", epochs=100)

    # Predizione
    future_df = test_window.copy()  # Deve contenere anche i regressori futuri
    forecast = model.predict(future_df.drop(columns=['covid']))

    # Metriche
    y_true = test_window['y'].values
    y_pred = forecast['yhat1'].values

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    results.append({'Fold': i+1, 'MAE_nprophet': mae, 'RMSE_nprophet': rmse})
    print(f"Split {i+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# Risultati CV
results_df_np = pd.DataFrame(results)
print("\n=== Backtest Results ===")
print(results_df_np)

with open('pickles/NP_cv_Outliers_multistep.pkl', 'wb') as file:
    pickle.dump(results_df_np, file)
with open('pickles/NP_cv_Outliers_multistep.pkl', 'rb') as file:
    results_df_np = pickle.load(file)

# Plot Errori
results_df_np.plot(
    x='Fold',
    y=['MAE_nprophet', 'RMSE_nprophet'],
    marker='o',
    title='TimeSeriesCV Errors',
    figsize=(10, 5)
)
plt.grid(True)
plt.show()

In [0]:
# Mi ricavo le date
initial_train_size = len(train_df_np)-5*len(test_df_np)
h = len(test_df_np)
n_windows = 5

# Crea lista per i dati delle fold
fold_data = []

for i in range(n_windows):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    # Ricava date da colonna 'ds' o dall'indice
    if 'ds' in train_df_np.columns:
        start_date = train_df_np['ds'].iloc[test_start]
        end_date = train_df_np['ds'].iloc[test_end - 1]
    else:
        start_date = np_df_train.index[test_start]
        end_date = np_df_train.index[test_end - 1]

    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]

    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)

# Converto le date in datetime se necessario
if all(isinstance(date, str) for date in fold_intervals_df['start_date']):
    fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
    fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])

print("Fold Intervals DataFrame:")
print(fold_intervals_df)

In [0]:
# CV Plot
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_np['ds'], train_df_np['y'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'{row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

Now let's training on the whole dataset

In [0]:
# FIT
# Fissa i seed per riproducibilità
import random
import time
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

np_df_train = train_df_np[future_features].copy()

# Modello senza regressori esterni
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.01, 
    batch_size=64,
    daily_seasonality=True,    # Impara la stagionalità dai dati
    weekly_seasonality=True,   # Impara pattern settimanali
    yearly_seasonality=True,   # Impara pattern annuali
    loss_func='Huber'
)
# Aggiungi solo i regressori deterministici
for reg in futr_exog_list:
    neuralprophet.add_future_regressor(reg)
start_time = time.time()
metrics_df = neuralprophet.fit(np_df_train, freq="D",epochs=100)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
forecast = neuralprophet.predict(test_df_np[future_features].drop(columns=['covid']))

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
forecast['yhat1'] = forecast['yhat1'] * (max_val - min_val) + min_val
forecast['yhat1 2.5%'] = forecast['yhat1 2.5%'] * (max_val - min_val) + min_val
forecast['yhat1 97.5%'] = forecast['yhat1 97.5%'] * (max_val - min_val) + min_val

In [0]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast['ds'], 
    y=forecast['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))

#Intervallo di confidenza al 95%
fig.add_trace(go.Scatter(
    x=list(forecast['ds']) + 
    list(forecast['ds'][::-1]), 
    y=list(forecast['yhat1 2.5%']) + list(forecast['yhat1 97.5%'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast['ds'], 'forecast': forecast['yhat1']})
merged_df = pd.merge(test_df_original[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
mae_val = mean_absolute_error(merged_df['y'], merged_df['forecast'])
metriche_np = calcola_metriche(merged_df['y'], merged_df['forecast'], train_df_original['y'],modelname="NeuralProphet")
metriche_np

#### TFT

In [0]:
# Preparing dataset
df_tft = create_time_series_features(df, target_col='y') 
df_tft = df_tft.dropna().reset_index(drop=True)
# 2. Suddivisione in train/test
train_df_tft = df_tft.iloc[:-h]
test_df_tft = df_tft.iloc[-h:]
train_df_tft.shape, test_df_tft.shape

In [0]:
# Scaling dei dati
#Salvo per dopo
train_df_original = train_df_tft.copy() 
test_df_original = test_df_tft.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_tft[['ds', 'unique_id']]
test_meta = test_df_tft[['ds', 'unique_id']]

feature_cols = [col for col in train_df_tft.columns if col not in ['ds', 'unique_id']]
train_idx = train_df_tft.index
test_idx = test_df_tft.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_tft[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_tft[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_tft = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_tft = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_tft.shape, test_df_tft.shape

##### TimeSeriesCV - fine tuning Optuna (TROPPO LENTO)

In [0]:
# Optuna function
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_tft = pd.DataFrame()
h = len(test_df_tft)

def objective(trial):
    # Hyperparameters to optimize for TFT
    input_size = trial.suggest_categorical("input_size", [int(h/2), int(h)])
    hidden_size = trial.suggest_categorical("hidden_size", [16, 32, 64, 128])
    n_rnn_layers = trial.suggest_int("n_rnn_layers", 1, 4)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    grn_activation = trial.suggest_categorical("grn_activation", ["ReLU", "ELU", "Sigmoid"])
    
    # Define TFT model
    tft=TFT(
            h=h,
            input_size=input_size,
            hidden_size=hidden_size,
            n_rnn_layers=n_rnn_layers,
            grn_activation=grn_activation, #prima era ELU
            rnn_type="lstm",
            one_rnn_initial_state=False,
            loss=DistributionLoss(distribution="StudentT", level=[90]),
            learning_rate=learning_rate, 
            hist_exog_list=hist_exog_features,
            futr_exog_list=futr_exog_list, #queste dovranno essere nel test
            max_steps=40, 
            val_check_steps=10,
            batch_size=batch_size, 
            #early_stop_patience_steps=15,  
            scaler_type="standard",
            enable_progress_bar=True,
            accelerator="auto",  
        )
    
    # nf
    nf = NeuralForecast(models=[tft], freq='D')

    # Cross-validation for tuning
    df_cv = nf.cross_validation(
        df=train_df_tft,
        step_size=h,
        n_windows=5
    )

    # Calculate MAE for each fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'TFT']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'TFT',
            'MAE_tft': mae,
            'RMSE_tft': rmse
        })

    # Average MAE across all folds
    cv_metrics_df_tft = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_tft['MAE_tft'].mean()

    return mean_mae

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)  #aumentare su databricks

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
# salvo i best params
with open('pickles/TFT_bestpars_Outliers_multistep.pkl', 'wb') as file:
    pickle.dump(best_params, file)

In [0]:
# carico i best params
with open('pickles/TFT_bestpars_Outliers_multistep.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [0]:
# final model
best_tft = TFT(
        h=h,  
        input_size=best_params['input_size'],
        hidden_size=best_params['hidden_size'],
        n_rnn_layers=best_params['n_rnn_layers'],
        rnn_type='lstm',
        grn_activation=best_params['grn_activation'],
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=best_params['learning_rate'],
        futr_exog_list=futr_exog_list,
        hist_exog_list=hist_exog_features,
        max_steps=100, #aumentare a 100
        val_check_steps=25,
        batch_size=best_params['batch_size'],
        early_stop_patience_steps=-1,
        scaler_type="standard",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False
)
nf = NeuralForecast(models=[best_tft],freq="D")

In [0]:
# # final CV
# df_cv = nf.cross_validation(
#     df=train_df_tft,
#     step_size=len(test_df_tft),
#     n_windows=5
# )
# # salvo le cv
# with open('pickles/TFT_cv_Outliers_multistep.pkl', 'wb') as file:
#     pickle.dump(df_cv, file)

# carico le cv
with open('pickles/TFT_cv_Outliers_multistep.pkl', 'rb') as file:
    df_cv = pickle.load(file)

# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'TFT']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'TFT',
        'MAE_tft': mae,
        'RMSE_tft': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_tft = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_tft)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_tft.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_tft['unique_id'].unique():
    series_data = train_df_tft[train_df_tft['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'{row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_tft['Model'] = 'TFT'
# Plot MAE sul primo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_tft'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_tft'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()

##### Now let's training on the whole dataset

In [0]:
best_params

In [0]:
# final training with best params
import time
best_tft = TFT(
        h=h,  
        input_size=int(best_params['input_size']*2),
        hidden_size=best_params['hidden_size'],
        n_rnn_layers=2, #best_params['n_rnn_layers']
        rnn_type='lstm',
        grn_activation=best_params['grn_activation'],
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=0.01, #best_params['learning_rate'], 
        futr_exog_list=futr_exog_list,
        hist_exog_list=hist_exog_features,
        max_steps=80, #aumentare a 100
        val_check_steps=25,
        batch_size=best_params['batch_size'],
        early_stop_patience_steps=-1,
        scaler_type="standard",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=16
)
nf = NeuralForecast(models=[best_tft],freq="D")
start_time = time.time()
#fit 
nf.fit(df=train_df_tft, val_size=best_params['input_size'])
end_time = time.time()

training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")
tft_model = nf.models[0]

In [0]:
# # training senza best params
# h = len(test_df_tft)
# nf = NeuralForecast(
#     models=[
#         TFT(
#             h=h,
#             input_size=h*5,
#             hidden_size=60,  # deve essere divisibile per 4
#             grn_activation="ReLU", #prima era ELU
#             rnn_type="lstm",
#             n_rnn_layers=4,  #consigliato fra 2 e 5; 
#             one_rnn_initial_state=False,
#             loss=DistributionLoss(distribution="StudentT", level=[90]),
#             learning_rate=0.005, 
#             hist_exog_list=hist_exog_features,
#             futr_exog_list=futr_exog_list, #queste dovranno essere nel test
#             max_steps=50, 
#             val_check_steps=10,
#             batch_size=96, 
#             early_stop_patience_steps=15,  
#             scaler_type="robust",
#             enable_progress_bar=True,
#             accelerator="auto",  
#         ),
#     ],
#     freq='MS',
# )

# start_time = time.time()
# #fit 
# nf.fit(df=train_df_tft, val_size=2*h)

# end_time = time.time()
# training_time = end_time - start_time
# print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Predict
Y_hat_df_tft = nf.predict(futr_df=test_df_tft[['unique_id','ds'] + futr_exog_list])

In [0]:
# salvo le prediction
# with open('pickles/TFT_pred_Outliers_multistep.pkl', 'wb') as file:
#     pickle.dump(Y_hat_df_tft, file)

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
Y_hat_df_tft['TFT'] = Y_hat_df_tft['TFT'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-median'] = Y_hat_df_tft['TFT-median'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-lo-90'] = Y_hat_df_tft['TFT-lo-90'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-hi-90'] = Y_hat_df_tft['TFT-hi-90'] * (max_val - min_val) + min_val

In [0]:
# # carico le prediction, MASE = 0.6893
with open('pickles/TFT_pred_Outliers_multistep.pkl', 'rb') as file:
    Y_hat_df_tft = pickle.load(file)

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_tft['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_tft['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='lightgreen')))

# QUANTILI TFT
# Fascia tra P10 e P90
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_tft['ds']) + 
    list(Y_hat_df_tft['ds'][::-1]),
    y=list(Y_hat_df_tft['TFT-hi-90']) + list(Y_hat_df_tft['TFT-lo-90'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="TFT forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_tft['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# Calcola metriche
merged_df_tft = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_tft, on='ds', how='left')
merged_df_tft.rename(columns={'TFT-lo-90': '0.1','TFT-median':'0.5', 'TFT-hi-90':'0.9'},inplace=True)
merged_df_tft.dropna(inplace=True)
metriche_tft = calcola_metriche(merged_df_tft['y'],merged_df_tft['TFT'],train_df_original['y'],
                                y_pred_quantiles=merged_df_tft[['0.1','0.5','0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="TFT").round(10)
metriche_tft

##### TFT Explainability

In [0]:
# salvo il modello tft
# with open('pickles/TFT_model_Outliers_multistep.pkl', 'wb') as file:
#     pickle.dump(tft_model, file)

In [0]:
# # carico il modello tft
with open('pickles/TFT_model_Outliers_multistep.pkl', 'rb') as file:
    tft_model = pickle.load(file)

In [0]:
attention = tft_model.attention_weights()
def plot_attention(
    self, plot: str = "time", output: str = "plot", width: int = 800, height: int = 400
):
    """
    Plot the attention weights.

    Args:
        plot (str, optional): The type of plot to generate. Can be one of the following:
            - 'time': Display the mean attention weights over time.
            - 'all': Display the attention weights for each horizon.
            - 'heatmap': Display the attention weights as a heatmap.
            - An integer in the range [1, model.h) to display the attention weights for a specific horizon.
        output (str, optional): The type of output to generate. Can be one of the following:
            - 'plot': Display the plot directly.
            - 'figure': Return the plot as a figure object.
        width (int, optional): Width of the plot in pixels. Default is 800.
        height (int, optional): Height of the plot in pixels. Default is 400.

    Returns:
        matplotlib.figure.Figure: If `output` is 'figure', the function returns the plot as a figure object.
    """

    attention = (
        self.mean_on_batch(self.interpretability_params["attn_wts"])
        .mean(dim=0)
        .cpu()
        .numpy()
    )

    fig, ax = plt.subplots(figsize=(width / 100, height / 100))

    if plot == "time":
        attention = attention[self.input_size :, :].mean(axis=0)
        ax.plot(np.arange(-self.input_size, self.h), attention)
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Mean Attention")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "all":
        for i in range(self.input_size, attention.shape[0]):
            ax.plot(
                np.arange(-self.input_size, self.h),
                attention[i, :],
                label=f"horizon {i-self.input_size+1}",
            )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Attention per horizon")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "heatmap":
        cax = ax.imshow(
            attention,
            aspect="auto",
            cmap="viridis",
            extent=[-self.input_size, self.h, -self.input_size, self.h],
        )
        fig.colorbar(cax)
        ax.set_title("Attention Heatmap")
        ax.set_xlabel("Attention (current time step)")
        ax.set_ylabel("Attention (previous time step)")

    elif isinstance(plot, int) and (plot in np.arange(1, self.h + 1)):
        i = self.input_size + plot - 1
        ax.plot(
            np.arange(-self.input_size, self.h),
            attention[i, :],
            label=f"horizon {plot}",
        )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title(f"Attention weight for horizon {plot}")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    else:
        raise ValueError(
            'plot has to be in ["time","all","heatmap"] or integer in range(1,model.h)'
        )

    plt.tight_layout()

    if output == "plot":
        plt.show()
    elif output == "figure":
        return fig
    else:
        raise ValueError(f"Invalid output: {output}. Expected 'plot' or 'figure'.")

In [0]:
# MEAN ATTENTION
plot_attention(tft_model, plot="time")
# pesi di attenzione medi nel tempo su tutti gli orizzonti di previsione.
# La curva blu rappresenta quanto il modello ha "guardato" (attention) 
# quel punto temporale in input durante la previsione.
# Valori più alti significano che il modello ritiene quel punto temporale
# particolarmente rilevante per fare le previsioni

In [0]:
# ATTENTION OF A SPECIFIC TIME STEPS
plot_attention(tft_model, plot=4)
# pesi di attenzione per ogni orizzonte di previsione separatamente.

In [0]:
plot_attention(tft_model, plot="heatmap")

In [0]:
# PAST VARIABLE IMPORTANCES
feature_importances = tft_model.feature_importances()
print(feature_importances.keys())
print("\n ==========PAST FEATURE IMPORTANCES==========")
feature_importances["Past variable importance over time"].mean().sort_values().plot(kind="barh")

Questo grafico mostra quanto il modello ha fatto affidamento sulle feature passate \
(quelle che il modello ha potuto osservare prima del punto di previsione). Serve a costruire il contesto dell'input

In [0]:
# FUTURE VARIABLE IMPORTANCES
print("\n ==========FUTURE FEATURE IMPORTANCES==========")
feature_importances["Future variable importance over time"].mean().sort_values().plot(kind="barh")

Mostra quanto il modello ha sfruttato le feature che sono servite a guidare le previsioni passo-passo nel decoder

DIFFERENZA TRA PAST E FUTURE: \
La diversa importanza riflette quanto ciascuna variabile aiuta nell’analisi del contesto passato (encoder -->> PAST) \
o nella generazione delle previsioni (decoder -->> FUTURE).

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME
df_importances = feature_importances["Past variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances.index))

for col in df_importances.columns:
    p = ax.bar(np.arange(-len(df_importances), 0), df_importances[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances[col]
ax.set_title("Past variable importance over time")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)

plt.show()

Questo grafico mostra QUANDO (e QUANTO) nel passato ogni feature è stata più utile per costruire il contesto predittivo.

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME
df_importances2 = feature_importances["Future variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances2.index))
for col in df_importances2.columns:
    p = ax.bar(np.arange(-len(df_importances2), 0), df_importances2[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances2[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.grid(True)
ax.legend()
plt.show()

Per ogni feature viene mostrato quanto è stata importante per il modello a ciascun orizzonte futuro

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances3 = feature_importances["Past variable importance over time"]
mean_attention = (
    tft_model
    .attention_weights()[tft_model.input_size :, :]
    .mean(axis=0)[: tft_model.input_size]
)
df_importances3 = df_importances3.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances3.index))

for col in df_importances3.columns:
    p = ax.bar(np.arange(-len(df_importances3), 0), df_importances3[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances3[col]
ax.set_title("Past variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances3), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
print(df_importances4.shape)  # (n_time_steps, n_variables)
print(mean_attention.shape)   # Deve essere (n_variables,)

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION

# Estrai l'importanza delle future variables
df_importances4 = feature_importances["Future variable importance over time"]

# Estrai tutte le attention weights mediate nel tempo
full_attention = tft_model.attention_weights()[tft_model.input_size:, :].mean(axis=0)

# Estrai solo la parte relativa ai 12 time steps futuri
n_future_steps = df_importances4.shape[0]
mean_attention_future = full_attention[-n_future_steps:]

# Applica la ponderazione riga per riga (sul tempo)
df_importances4 = df_importances4.multiply(mean_attention_future, axis=0)

# Plot
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances4.index))

for col in df_importances4.columns:
    ax.bar(
        np.arange(1, len(df_importances4) + 1), 
        df_importances4[col].values, 
        width=0.6, 
        label=col, 
        bottom=bottom
    )
    bottom += df_importances4[col].values

# Titoli e assi
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Future time steps")
ax.grid(True)

# Linea media di attenzione
ax.plot(
    np.arange(1, len(df_importances4) + 1),
    mean_attention_future,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention"
)
ax.legend()
plt.show()

In [0]:
tft_model.feature_importance_correlations()

#### CHRONOS

item_id, timestamp, target (Chronos calcola tempo di fitting in automatico)

In [0]:
# Preparing dataset
df_chronos = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos = df_chronos.dropna().reset_index(drop=True) 
df_chronos['item_id'] = df_chronos['unique_id'] 
df_chronos.drop(columns="unique_id", inplace=True) 
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Definizione delle covariate note
known_covariates_names=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id','target']]
feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

# Split train e test
train_df_chronos = df_chronos.iloc[:-h]
test_df_chronos = df_chronos.iloc[-h:]

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [0]:
# Scaling dei dati
train_df_original = train_df_chronos.copy() 
test_df_original = test_df_chronos.copy()

train_idx = train_df_chronos.index
test_idx = test_df_chronos.index

#scaler
scaler_x = MinMaxScaler()
train_df_chronos = pd.DataFrame(scaler_x.fit_transform(train_df_chronos[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_chronos = pd.DataFrame(scaler_x.transform(test_df_chronos[feature_cols]),
                            columns=feature_cols, index=test_idx)
#controllo le shape
train_df_chronos.shape, test_df_chronos.shape

TimeSeriesCV

In [0]:
# TimeSeriesCV
h = len(test_df_chronos)  # prediction length
n_splits = 5
initial_train_size = len(train_df_chronos) - 5*h
results = []

for fold in range(n_splits):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")
    
    train_end = initial_train_size + fold * h
    test_start = train_end
    test_end = test_start + h
    train_df_fold = train_df_chronos.iloc[:train_end]
    test_df_fold = train_df_chronos.iloc[test_start:test_end]
    
    train_df_fold.reset_index(inplace=True)
    test_df_fold.reset_index(inplace=True)
    
    train_df_fold["timestamp"] = pd.to_datetime(train_df_fold["timestamp"])
    test_df_fold["timestamp"] = pd.to_datetime(test_df_fold["timestamp"])
    
    # Future timestamps 
    future_index = pd.date_range(
        start=train_df_fold["timestamp"].max() + pd.Timedelta(days=1),
        periods=h,
        freq="D")
    
    # Preparo test set con known_covariates
    test_df_for_prediction = test_df_fold[test_df_fold["timestamp"].isin(future_index)].copy()
    test_df_for_prediction = test_df_for_prediction.set_index("timestamp")
    test_df_for_prediction = test_df_for_prediction.reindex(future_index)  # forza continuità
    test_df_for_prediction["item_id"] = train_df_fold["item_id"].iloc[0]  # supponiamo 1 sola serie
    test_df_for_prediction = test_df_for_prediction.reset_index().rename(columns={"index": "timestamp"})
    test_df_for_prediction = test_df_for_prediction.set_index(["item_id", "timestamp"])
    
    # Preparo il train con multindex
    train_df_fold = train_df_fold.set_index(["item_id", "timestamp"])
    print(f"Train: {train_df_fold.shape}, Test: {test_df_fold.shape}")
    
    # inizializzo il predictor
    predictor = TimeSeriesPredictor(
        prediction_length=h,
        target="target",
        known_covariates_names=known_covariates_names,
        eval_metric="MASE",
        freq="D")
    
    # fit del modello
    predictor.fit(
        train_df_chronos,
        hyperparameters={
            "Chronos": [
                {
                    "model_path": "bolt_small",
                    "ag_args": {"name_suffix": "ZeroShot"}},
                {
                    "model_path": "bolt_small",
                    "covariate_regressor": "CAT",
                    "target_scaler": "robust",
                    "ag_args": {"name_suffix": "WithRegressor"},
                },
            ],
        },
        time_limit=600,
        enable_ensemble=True,  # Attiva ensemble per migliorare l'accuratezza
        )
    
    predictions = predictor.predict(train_df_fold, known_covariates=test_df_for_prediction)

    test_df_with_index = test_df_fold.set_index(["item_id", "timestamp"])

    # Debug
    print(f"Forma predictions: {predictions.shape}")
    print(f"Colonne predictions: {predictions.columns}")
    print(f"Numero di indici in predictions: {len(predictions.index)}")
    print(f"Numero di indici in test_df_with_index: {len(test_df_with_index.index)}")

    # trovo solo gli indici comuni
    common_indices = predictions.index.intersection(test_df_with_index.index)
    print(f"Indici comuni: {len(common_indices)}")

    # prendo solo gli indici comuni
    y_true = test_df_with_index.loc[common_indices, "target"]
    y_pred = predictions.loc[common_indices, 'mean'].to_numpy()

    mae = calcola_mae(y_true, y_pred)
    rmse = calcola_rmse(y_true, y_pred)
    
    results.append({'Fold': fold+1, 'MAE_chronos': mae, 'RMSE_chronos': rmse})
    print(f"Split {fold+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
# risultati finali
results_df_chronos = pd.DataFrame(results)
results_df_chronos.set_index('Fold',inplace=True)
print("\n=== TimeSeriesCV Results ===")
results_df_chronos

In [0]:
# Estraggo le date
fold_data = []
train_df_chronos.reset_index(inplace=True)
for i in range(n_splits):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h
    
    # Ricava date da colonna 'ds' o dall'indice
    if 'timestamp' in train_df_chronos.columns:
        start_date = train_df_chronos['timestamp'].iloc[test_start]
        end_date = train_df_chronos['timestamp'].iloc[test_end - 1]
    else:
        start_date = train_df_chronos.index[test_start]
        end_date = train_df_chronos.index[test_end - 1]
    
    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]
    
    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)
fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])
# Stampa il dataframe prima della conversione per verificare i valori
print("Fold Intervals DataFrame (before conversion):")
fold_intervals_df

In [0]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_chronos['timestamp'], train_df_chronos['target'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'{row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot degli errori
results_df_chronos.reset_index(inplace=True)
ax = results_df_chronos.plot(x='Fold', y=['MAE_chronos','RMSE_chronos'], marker='o', title='TimeSeriesCV Errors')
ax.set_xticks(range(1, 6)) # Imposta le tacche sull'asse x da 1 a 5
plt.show()

Let's training on the whole dataset

In [0]:
# Preparo dataset per training
predictor = TimeSeriesPredictor(
    prediction_length=len(test_df_chronos), 
    eval_metric="MASE",
    target="target",
    known_covariates_names=futr_exog_list,
    freq="D"
)
test_known_covariates = test_df_chronos[futr_exog_list]

In [0]:
# Fit del modello
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {
                "model_path": "bolt_small",
                "ag_args": {"name_suffix": "ZeroShot"}
            },
            {
                "model_path": "bolt_mini",
                "covariate_regressor": "NN_TORCH",
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
        ],
    },
    time_limit=600,
    enable_ensemble=True,
)
# Valutazione del modello in fase di training
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [0]:
# PREVISIONI MULTI STEP 
predictions = predictor.predict(
    train_df_chronos,
    known_covariates=test_known_covariates,  # Solo le covariate realmente note in anticipo
    model="ChronosWithRegressor[bolt_mini]"
)
predictions.reset_index(inplace=True)

predictions_0shot = predictor.predict(
    train_df_chronos,
    known_covariates=test_known_covariates,
    #model="ChronosZeroShot[bolt_small]"
)
predictions_0shot.reset_index(inplace=True)

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('target')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
predictions['mean'] = predictions['mean'] * (max_val - min_val) + min_val
predictions['0.1'] = predictions['0.1'] * (max_val - min_val) + min_val
predictions['0.5'] = predictions['0.5'] * (max_val - min_val) + min_val
predictions['0.9'] = predictions['0.9'] * (max_val - min_val) + min_val
predictions_0shot['mean'] = predictions_0shot['mean'] * (max_val - min_val) + min_val
predictions_0shot['0.5'] = predictions_0shot['0.5'] * (max_val - min_val) + min_val
predictions_0shot['0.9'] = predictions_0shot['0.9'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)

# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_chronos['timestamp'], y=train_df_original['target'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['0.5'], mode='lines', 
    name='P50', line=dict(color='green')))
# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='ChronosWithRegressor', line=dict(color='lightgreen')))
# QUANTILI CHRONOS
fig.add_trace(go.Scatter(
    x=list(test_df_chronos['timestamp']) + 
    list(test_df_chronos['timestamp'][::-1]),
    y=list(predictions['0.9']) + list(predictions['0.1'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# PREVISIONI CHRONOS 0 SHOT
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='0-shot Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="CHRONOS forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_chronos['timestamp'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# metriche di valutazione
test_df_original.reset_index(inplace=True)
merged_df_chronos = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)
metriche_chronos = calcola_metriche(merged_df_chronos['target'],merged_df_chronos['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos").round(5)
metriche_chronos

In [0]:
# metriche di valutazione 0 shot
#test_df_original.reset_index(inplace=True)
merged_df_chronos = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions_0shot[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)
metriche_chronos_0shot = calcola_metriche(merged_df_chronos['target'],merged_df_chronos['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos_0shot").round(5)
metriche_chronos_0shot

Il modello 0 shot è il migliore

#### final evaluations

In [0]:
final_metrics = metriche_xgb.join(metriche_lstm).join(metriche_np).join(metriche_tft).join(metriche_chronos).join(metriche_chronos_0shot).round(4)
final_metrics

In [0]:
# FINAL PREDICTIONS PLOT FOR OIL PRICE
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))


# PREVISIONI XGBOOST
fig.add_trace(go.Scatter(
    x=forecast_xgb['ds'], 
    y=forecast_xgb['forecast'], 
    mode='lines', 
    name='XGBoost', 
    line=dict(color='purple')
))

# PREVISIONI LSTM
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=Y_hat_df_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='pink')
))

# PREVISIONI NEURAL PROPHET
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=forecast['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='aquamarine')
))


# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], 
                         y=Y_hat_df_tft['TFT'], 
                         #y=Y_hat_df_tft_TFT,
                         mode='lines', 
    name='TFT', line=dict(color='yellow')))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS', line=dict(color='green')))
    
# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS 0-shot', line=dict(color='lightgreen')))

# Layout del grafico
fig.update_layout(
    title="Forecasts for White Noise", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
#sistemo i vari dataset prima del merge
cv_metrics_df_lstm.drop(columns=('Model'),inplace=True)
results_df_np.rename(columns={'split': 'Fold'}, inplace=True)
results_df_chronos.rename(columns={'split': 'Fold','MAE':'MAE_chronos','RMSE':'RMSE_chronos'}, inplace=True)
cv_metrics_df_xgboost.rename(columns={'MAE': 'MAE_XGB','RMSE':'RMSE_XGB'}, inplace=True)

In [0]:
# Final plot metrics TimeSeriesCV

# faccio il merge di tutte le tabelle dei vari CV
final_cv = cv_metrics_df_lstm.merge(results_df_chronos, 
                            on="Fold",how="inner").merge(cv_metrics_df_xgboost[['MAE_XGB','RMSE_XGB','Fold']],
                            on="Fold",how="inner").merge(results_df_np,
                            on="Fold", how="inner").merge(cv_metrics_df_tft,
                            on="Fold",how="inner")
# Rescaling CV metrics data
columns_to_modify = ['MAE_nprophet', 'RMSE_nprophet', 'MAE_tft', 'RMSE_tft', 'MAE_lstm', 'RMSE_lstm',
                        'MAE_chronos', 'RMSE_chronos', 'MAE_XGB', 'RMSE_XGB']
for col in columns_to_modify:
    final_cv[f'{col}'] = final_cv[f'{col}'] * (max_val - min_val) + min_val

model_colors = {
    'XGB': '#9467bd',    # Purple
    'lstm': '#e377c2',       # Pink
    'nprophet': '#7fffd4',         # Aquamarine
    'tft': '#ffff00',        # Yellow
    'chronos': '#2ca02c',    # Green
}


# Create the figure
fig = go.Figure()

# Add traces for each model - MAE (darker) and RMSE (lighter)
for model, color in model_colors.items():
    # Add MAE line (darker)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'MAE_{model}'],
        mode='lines+markers',
        name=f'{model} MAE',
        line=dict(color=color, width=3),
        marker=dict(size=8)
    ))
    
    # Add RMSE line (lighter with same color but different dash pattern)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'RMSE_{model}'],
        mode='lines+markers',
        name=f'{model} RMSE',
        line=dict(color=color, width=2, dash='dash'),
        marker=dict(size=6)
    ))

# Update layout
fig.update_layout(
    title='Model Performance Comparison (MAE and RMSE)',
    template='plotly_dark',
    xaxis=dict(title='Fold',
        tickmode='linear',
        tick0=1, dtick=1
    ),    yaxis=dict(
        title='Error Value'
    ),    legend=dict(
        orientation="v"
    ),    hovermode="closest")
fig.show()

In [0]:
# Confronto TFT/CHRONOS
tft_chronos = metriche_chronos_0shot.join(metriche_tft)
tft_chronos                                    

### **MISSING data**

In [0]:
#preparazione dataset
df = missdata.reset_index().rename(columns={'date': 'ds','total_daily_sales':'y'})
df['ds'] = pd.to_datetime(df['ds'])
df_np = create_time_series_features(df, target_col='y') 
train_size = int(len(df_np) * 0.9)
train_df_np = df_np.iloc[:train_size]
test_df_np = df_np.iloc[train_size:]

In [0]:
#ci servirà dopo per valutazione
#Questo è il dataset vero, senza distorsioni (prima di creare i NaN)
missing = loaded_data['favorita_train']
missing = missing[['sales']]
#sommo tutte le vendite di un singolo store per ogni giorno
missing = missing.groupby(['date']).sum().reset_index()
missing = missing.rename(columns={"sales": "total_daily_sales"})
missing.set_index("date", inplace=True)
test_completo = missing[train_size:]
train_completo = missing[:train_size]

##### NEURAL PROPHET 

In [0]:
# preparazione dataset
np_df_train = train_df_np[['ds', 'y']].copy()
np_df_test = test_df_np[['ds', 'y']].copy()

# Aggiungi le colonne dei regressori
regressors = ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6',
       'lag_7', 'diff_1', 'dayofweek', 'month', 'quarter', 'day_sin',
       'day_cos', 'month_sin', 'month_cos']

for reg in regressors:
    np_df_train[reg] = train_df_np[reg]
    np_df_test[reg] = test_df_np[reg]
 
# Fissa i seed per riproducibilità
import random
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)    
    
# Definisci il modello
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.01, 
    batch_size=32,
    daily_seasonality=True,
    weekly_seasonality=True,
    loss_func='Huber'
)

# Aggiungi i regressori
for reg in regressors:
    neuralprophet.add_future_regressor(reg)

Non si riesce a fare la cross validation con i NaN

Now let's training on the whole dataset

In [0]:
# FIT
start_time = time.time()
metrics_df = neuralprophet.fit(np_df_train, freq="D", epochs=150)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
forecast_test = neuralprophet.predict(np_df_test)

In [0]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_np['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_np['y'], mode='lines', name='Testing Data'))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))

# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)
fig.show()

In [0]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast_test['ds'], 'forecast': forecast_test['yhat1']})
merged_df = pd.merge(test_df_np[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
metriche_np = calcola_metriche(y_true = test_completo['total_daily_sales'], y_pred = merged_df['forecast'],
                               y_train = train_completo['total_daily_sales'], modelname="NeuralProphet")
metriche_np

Nonostante i NaN fa un ottimo lavoro

##### CHRONOS

In [0]:
# Preparing dataset
df = missdata.reset_index().rename(columns={'date': 'ds','total_daily_sales':'y'})
df['ds'] = pd.to_datetime(df['ds'])
df_chronos = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos['item_id'] = "serie_unica"
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)
df_chronos = df_chronos.iloc[8:]
# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Definizione delle covariate note
known_covariates_names=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id','target']]
feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

# Split train e test
train_size = int(len(df_chronos) * 0.9)
train_df_chronos = df_chronos.iloc[:train_size]
test_df_chronos = df_chronos.iloc[train_size:]

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [0]:
# Fit del modello

# SOLO PER DATABRICKS
# sys.modules['sklearn.metrics._regression'].mean_absolute_error = sklearn.metrics.mean_absolute_error

predictor = TimeSeriesPredictor(
    prediction_length=len(test_df_chronos), 
    eval_metric="MASE",
    target="target",
    known_covariates_names=known_covariates_names,
    freq="D"
)

# Aggiunta di più modelli nella configurazione per migliorare le performance
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {
                "model_path": "bolt_base",
                "ag_args": {"name_suffix": "ZeroShot"}},
            {
                "model_path": "bolt_small",
                "covariate_regressor": "CAT",
                "target_scaler": "robust",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
            {
                "model_path": "bolt_mini",
                "covariate_regressor": "NN_TORCH", 
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            }
        ],
    },
    time_limit=600,
    enable_ensemble=True,  # Attiva ensemble per migliorare l'accuratezza
)

# Valutazione del modello in fase di training
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [0]:
# PREVISIONI CLASSICHE
# col miglior modello e col 0-shot
predictions = predictor.predict(
    train_df_chronos, 
    known_covariates=test_df_chronos
)
predictions.reset_index(inplace=True)

predictions_0shot = predictor.predict(
    train_df_chronos, 
    known_covariates=test_df_chronos,
    model="ChronosZeroShot[bolt_base]"
)
predictions_0shot.reset_index(inplace=True)

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
train_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=train_df_chronos['timestamp'], y=train_df_chronos['target'], mode='lines', name='Training Data'))
# Dati di test
test_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=test_df_chronos['target'], mode='lines', name='Testing Data'))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='ChronosWithRegressors', line=dict(color='green')))
# PREVISIONI CHRONOS 0 shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='0 shot', line=dict(color='lightgreen')))

# Layout del grafico
fig.update_layout(title="CHRONOS forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_chronos['timestamp'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
#calcola metriche
# metriche di valutazione
merged_df_chronos = pd.merge(
        test_df_chronos[['timestamp', 'target', 'item_id']], 
        predictions[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)

test_completo2 = test_completo.iloc[1:] 
metriche_chronos = calcola_metriche(y_true = test_completo2['total_daily_sales'], 
                                    y_pred = merged_df_chronos['mean'],
                                    y_train = train_completo['total_daily_sales'], 
                                    y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                    quantiles=[0.1, 0.5, 0.9], 
                                    modelname="Chronos").round(5)
metriche_chronos

In [0]:
#calcola metriche 0 shot
# metriche di valutazione
merged_df_chronos = pd.merge(
        test_df_chronos[['timestamp', 'target', 'item_id']], 
        predictions_0shot[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)
metriche_chronos_0shot = calcola_metriche(y_true = test_completo2['total_daily_sales'], 
                                    y_pred = merged_df_chronos['mean'],
                                    y_train = train_completo['total_daily_sales'], 
                                    y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                    quantiles=[0.1, 0.5, 0.9], 
                                    modelname="Chronos 0 shot").round(5)
metriche_chronos_0shot

### **MISSING DATA (multistep)**

In [0]:
#preparazione dataset
df = missdata.reset_index().rename(columns={'date': 'ds','total_daily_sales':'y'})
df['ds'] = pd.to_datetime(df['ds'])
df_np = create_time_series_features(df, target_col='y') 
df_np = df_np[future_features]
train_df_np = df_np.iloc[:-h]
test_df_np = df_np.iloc[-h:]

In [0]:
#ci servirà dopo per valutazione
#Questo è il dataset vero, senza distorsioni (prima di creare i NaN)
missing = loaded_data['favorita_train']
missing = missing[['sales']]
#sommo tutte le vendite di un singolo store per ogni giorno
missing = missing.groupby(['date']).sum().reset_index()
missing = missing.rename(columns={"sales": "total_daily_sales"})
missing.set_index("date", inplace=True)
test_completo = missing[-h:]
train_completo = missing[:-h]

##### NEURAL PROPHET 

In [0]:
# preparazione dataset
np_df_train = train_df_np[future_features].copy()
 
# Fissa i seed per riproducibilità
import random
import time
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)    
    
# Definisci il modello
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.01, 
    batch_size=32,
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True,
    loss_func='Huber'
)

# Aggiungi i regressori
for reg in futr_exog_list:
    neuralprophet.add_future_regressor(reg)

Non si riesce a fare la cross validation con i NaN

Now let's training on the whole dataset

In [0]:
# FIT
start_time = time.time()
metrics_df = neuralprophet.fit(np_df_train, freq="D", epochs=150)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
forecast_test = neuralprophet.predict(test_df_np[future_features].drop(columns=['covid']))

In [0]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_np['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_np['y'], mode='lines', name='Testing Data'))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))

# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)
fig.show()

In [0]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast_test['ds'], 'forecast': forecast_test['yhat1']})
merged_df = pd.merge(test_df_np[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
metriche_np = calcola_metriche(y_true = test_completo['total_daily_sales'], y_pred = merged_df['forecast'],
                               y_train = train_completo['total_daily_sales'], modelname="NeuralProphet")
metriche_np

Nonostante i NaN fa un ottimo lavoro

##### CHRONOS

In [0]:
# Preparing dataset
df = missdata.reset_index().rename(columns={'date': 'ds','total_daily_sales':'y'})
df['ds'] = pd.to_datetime(df['ds'])
df_chronos = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos['item_id'] = "serie_unica"
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)
df_chronos = df_chronos.iloc[8:]
# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Definizione delle covariate note
known_covariates_names=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id','target']]
feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

# Split train e test
train_df_chronos = df_chronos.iloc[:-h]
test_df_chronos = df_chronos.iloc[-h:]

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [0]:
# Fit del modello

# SOLO PER DATABRICKS
# sys.modules['sklearn.metrics._regression'].mean_absolute_error = sklearn.metrics.mean_absolute_error

predictor = TimeSeriesPredictor(
    prediction_length=len(test_df_chronos), 
    eval_metric="MASE",
    target="target",
    known_covariates_names=futr_exog_list,
    freq="D"
)

# Aggiunta di più modelli nella configurazione per migliorare le performance
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {
                "model_path": "bolt_base",
                "ag_args": {"name_suffix": "ZeroShot"}},
            {
                "model_path": "bolt_small",
                "covariate_regressor": "CAT",
                "target_scaler": "robust",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
            {
                "model_path": "bolt_mini",
                "covariate_regressor": "NN_TORCH", 
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            }
        ],
    },
    time_limit=600,
    enable_ensemble=True,  # Attiva ensemble per migliorare l'accuratezza
)

# Valutazione del modello in fase di training
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [0]:
# PREVISIONI CLASSICHE
test_known_covariates = test_df_chronos[futr_exog_list]
# col miglior modello e col 0-shot
predictions = predictor.predict(
    train_df_chronos, 
    known_covariates=test_known_covariates
)
predictions.reset_index(inplace=True)

predictions_0shot = predictor.predict(
    train_df_chronos, 
    known_covariates=test_known_covariates,
    model="ChronosZeroShot[bolt_base]"
)
predictions_0shot.reset_index(inplace=True)

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
train_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=train_df_chronos['timestamp'], y=train_df_chronos['target'], mode='lines', name='Training Data'))
# Dati di test
test_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=test_df_chronos['target'], mode='lines', name='Testing Data'))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='ChronosWithRegressors', line=dict(color='green')))
# PREVISIONI CHRONOS 0 shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='0 shot', line=dict(color='lightgreen')))

# Layout del grafico
fig.update_layout(title="CHRONOS forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_chronos['timestamp'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
#calcola metriche
# metriche di valutazione
merged_df_chronos = pd.merge(
        test_df_chronos[['timestamp', 'target', 'item_id']], 
        predictions[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)
metriche_chronos = calcola_metriche(y_true = test_completo['total_daily_sales'], 
                                    y_pred = merged_df_chronos['mean'],
                                    y_train = train_completo['total_daily_sales'], 
                                    y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                    quantiles=[0.1, 0.5, 0.9], 
                                    modelname="Chronos").round(5)
metriche_chronos

In [0]:
#calcola metriche 0 shot
# metriche di valutazione
merged_df_chronos = pd.merge(
        test_df_chronos[['timestamp', 'target', 'item_id']], 
        predictions_0shot[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)
metriche_chronos_0shot = calcola_metriche(y_true = test_completo['total_daily_sales'], 
                                    y_pred = merged_df_chronos['mean'],
                                    y_train = train_completo['total_daily_sales'], 
                                    y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                    quantiles=[0.1, 0.5, 0.9], 
                                    modelname="Chronos 0 shot").round(5)
metriche_chronos_0shot

### **ELECTRICITY DEMAND**

In [0]:
#preparazione dataset
df = df_elec.reset_index().rename(columns={'DATE_TIME': 'ds','Demand':'y'})
df['unique_id'] = 'serie_1'
df['ds'] = pd.to_datetime(df['ds'])

In [0]:
# funzione per creare feature
def create_time_series_features(df, target_col):
    df = df.copy()
    lags = [1, 2, 4, 24, 36, 48, 96]
    # Lag features (1-4)
    for lag in lags:
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    
    # Differencing features, 1, 4, 48
    df['diff_1'] = (df[target_col] - df[target_col].shift(1)).shift(1)
    df['diff_4'] = (df[target_col] - df[target_col].shift(4)).shift(1)
    df['diff_48'] = (df[target_col] - df[target_col].shift(48)).shift(1)
    df['diff_96'] = (df[target_col] - df[target_col].shift(96)).shift(1)
    
    # Caratteristiche cicliche per rappresentare meglio la stagionalità
    df['day_sin'] = np.sin(2 * np.pi * df['giorno_settimana']/7)
    df['day_cos'] = np.cos(2 * np.pi * df['giorno_settimana']/7)
    df['hour_sin'] = np.sin(2 * np.pi * df['ora_del_giorno']/24)
    df['hour_cos'] = np.cos(2 * np.pi * df['ora_del_giorno']/24)
    df['month_sin'] = np.sin(2 * np.pi * df['mese_del_anno']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['mese_del_anno']/12)
    
    # rolling means 
    df['rolling_mean_4'] = df[target_col].rolling(window=4).mean().shift(1) #ultime 2 ore
    df['rolling_mean_24'] = df[target_col].rolling(window=24).mean().shift(1) #ultime 12 ore
    df['rolling_mean_48'] = df[target_col].rolling(window=48).mean().shift(1) #ultimo giorno
    df['rolling_mean_96'] = df[target_col].rolling(window=96).mean().shift(1) #ultimi 2 giorni
    df['rolling_std_4'] = df[target_col].rolling(window=4).std().shift(1)
    df['rolling_std_24'] = df[target_col].rolling(window=24).mean().shift(1) 
    df['rolling_std_48'] = df[target_col].rolling(window=48).std().shift(1)
    df['rolling_std_48'] = df[target_col].rolling(window=96).mean().shift(1) 
    return df

##### ARIMA + ETS 

In [0]:
# divisione in train e test
df_arima = df[['y','ds','unique_id']].iloc[97:]
train_size = int(len(df_arima) * 0.9)
train_df = df_arima.iloc[:train_size]
test_df = df_arima.iloc[train_size:]
# queste due ci serviranno dopo
season_length = 48 # i have half hourly data 
horizon = len(test_df) # number of predictions
train_df.shape, test_df.shape

In [0]:
# train e test plot
fig = go.Figure()
#train dat
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Train'))
#test data 
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Test'))
#layout
fig.update_layout(template='plotly_dark', title='Train and Test Data Plot')
fig.show()

In [0]:
# definisco i modelli
models = [
    AutoARIMA(), # ci dovrebbe andare season_length=season_length, ma ci mette 30min+
    AutoETS() #season_length=season_length
]
import time
# creo il forecaster
sf = StatsForecast(models=models, freq='30min')
start_time = time.time()
# fit
sf.fit(train_df)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
#summary of the model ARIMA
result=sf.fitted_[0,0].model_
print(result.keys())
print("Parametri ARMA:",result['arma'])

In [0]:
arima_string(sf.fitted_[0,0].model_)

In [0]:
#summary of the model ETS
result2 = sf.fitted_[0,1].model_
print(result2.keys())
print("Modello:",result2['method'])

In [0]:
# let's forecast
Y_hat_df = sf.forecast(df=train_df, h=len(test_df), fitted=True, level=[95])
#see fitted values vs true values
values=sf.forecast_fitted_values() #qui viene aggiunta anche la vera y
#salvo i valori fittati nel training in un pickle
# with open('pickles/ARIMAETS_fit_elecDemand.pkl', 'wb') as file:
#     pickle.dump(values, file)
values.head(2)

In [0]:
values.tail(2)

In [0]:
# carico i valori fittati se ne avessi bisogno
# with open('ARIMAETS_fit_elecDemand.pkl', 'rb') as file:
#     values = pickle.load(file)

Cross Validation

In [0]:
#CROSS VALIDATION
crossvalidation_df = sf.cross_validation(
    df=train_df,
    h=horizon, 
    step_size=len(test_df),
    n_windows=5
)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = crossvalidation_df['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = crossvalidation_df[crossvalidation_df['cutoff'] == fold]
    
    for model in ['AutoARIMA', 'AutoETS']:
        # Filtra i dati per il modello corrente
        model_data = fold_data[['y', f'{model}']]
        model_data = model_data.dropna()
        
        # Calcola MAE e RMSE
        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data[f'{model}'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data[f'{model}'])
        
        # Aggiungi i risultati
        cv_metrics.append({
            'Fold': i+1,
            'Model': model,
            f'MAE_{model}': mae,
            f'RMSE_{model}': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_arima_ets = pd.DataFrame(cv_metrics)

In [0]:
# PLOT CV
#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_arima_ets)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_arima_ets.groupby('Model').mean().reset_index()
print(mean_metrics)

# Ora creiamo un grafico che combina la serie temporale con le metriche di errore per visualizzare dove si verificano gli errori maggiori

# Prima dobbiamo identificare gli intervalli temporali per ogni fold
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = crossvalidation_df[crossvalidation_df['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df['unique_id'].unique():
    series_data = train_df[train_df['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

# Plot MAE sul primo asse
for model in cv_metrics_df_arima_ets['Model'].unique():
    model_data = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == model]
    ax1.plot(model_data['Fold'], model_data[f'MAE_{model}'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_arima_ets['Model'].unique():
    model_data = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == model]
    ax2.plot(model_data['Fold'], model_data[f'RMSE_{model}'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()

In [0]:
fold_intervals #EXPANDING WINDOW CROSS VALIDATION

Let's forecast on test set

In [0]:
# prediction on test set
test_pred = sf.forecast(df=train_df, h=len(test_df)+96, level=[95])
Y_hat_df = test_df.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])

In [0]:
# Calcola metriche
metriche_arima = calcola_metriche(Y_hat_df['y'],Y_hat_df['AutoARIMA'],train_df['y'], 
                                modelname="ARIMA").round(5)
metriche_ets = calcola_metriche(Y_hat_df['y'],Y_hat_df['AutoETS'],train_df['y'], 
                                modelname="ETS").round(5)
metriche_arima, metriche_ets

In [0]:
# Forecast plot
# Creiamo un dataframe completo che include training, test e previsioni future
full_df = pd.concat([train_df, test_df], axis=0)
# Aggiungiamo le previsioni al dataframe completo
forecast_df_etsA = full_df.merge(Y_hat_df, how='outer', on=['unique_id', 'ds'])

fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Training Data'))

# Dati di test
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(x=test_pred['ds'], y=test_pred['AutoARIMA'], mode='lines', 
    name='AutoARIMA Forecast', line=dict(color='green')))

# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(x=test_pred['ds'], y=test_pred['AutoETS'], mode='lines', 
    name='AutoETS Forecast', line=dict(color='orange', dash='dot')))

# # Intervallo di confidenza AUTOARIMA 
# if 'AutoARIMA-lo-95' in forecast_df_etsA.columns and 'AutoARIMA-hi-95' in forecast_df_etsA.columns:
#     fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoARIMA-lo-95'], 
#         mode='lines', line_color='green', showlegend=False))
    
#     fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoARIMA-hi-95'],
#         mode='lines', fill='tonexty', fillcolor='rgba(200,200,200,0.2)',
#         line_color='green', name='95% CI AutoARIMA'
#     ))

# # Intervallo di confidenza AUTOETS 
# if 'AutoETS-lo-95' in forecast_df_etsA.columns and 'AutoETS-hi-95' in forecast_df_etsA.columns:
#     fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoETS-lo-95'], 
#         mode='lines', line_color='orange', showlegend=False))
    
#     fig.add_trace(go.Scatter(x=forecast_df_etsA['ds'], y=forecast_df_etsA['AutoETS-hi-95'],
#         mode='lines', fill='tonexty', fillcolor='rgba(255,165,0,0.15)',
#         line_color='orange', name='95% CI AutoETS'))

# Layout del grafico
fig.update_layout(title="Forecast Comparison: AutoARIMA vs AutoETS", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

##### XGBOOST

In [0]:
# df_xgb
df_xgb = create_time_series_features(df, target_col='y') 
df_xgb.drop(columns=['unique_id'],inplace=True)
df_xgb = df_xgb.dropna().reset_index(drop=True) #droppo i NaN creati dai lag
print(df_xgb.columns)
df_xgb.head()

In [0]:
# Split train/test
train_size = int(len(df_xgb) * 0.9)
train_df_xgb = df_xgb.iloc[:train_size]
test_df_xgb = df_xgb.iloc[train_size:]

# Separazione features e target
feature_cols = [col for col in df_xgb.columns if col not in ['y', 'ds']]
X_train_xgb, y_train_xgb = train_df_xgb[feature_cols], train_df_xgb['y']
X_test_xgb, y_test_xgb = test_df_xgb[feature_cols], test_df_xgb['y']
X_train_xgb.shape, y_train_xgb.shape, X_test_xgb.shape, y_test_xgb.shape

In [0]:
# Scaling dei dati

#features
scaler_x = MinMaxScaler()
X_train_scaled = scaler_x.fit_transform(X_train_xgb)
X_test_scaled = scaler_x.transform(X_test_xgb)

# target, non serve che scalo y_test
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train_xgb.values.reshape(-1, 1))

# Ricreo df con stessi indici e nomi
X_train_xgb = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train_xgb.index)
X_test_xgb = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test_xgb.index)
y_train_xgb = pd.Series(y_train_scaled.flatten(), name='y', index=y_train_xgb.index)

#aggiorno train_df_xgb
train_df_xgb[feature_cols] = X_train_xgb
train_df_xgb['y'] = y_train_xgb

#controllo le shape
X_train_xgb.shape, X_test_xgb.shape, y_train_xgb.shape, y_test_xgb.shape

Ottimizzazione Optuna (Bayesiana)

In [0]:
# Optuna function
cv_metrics_log = []  # List globale per salvare metriche fold per fold

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'verbosity': 0,
        'seed': 42
    }

    tscv = TimeSeriesSplit(n_splits=5,test_size=len(test_df_xgb))
    y = train_df_xgb['y']
    df_xgb_feature = train_df_xgb.drop(columns=['y','ds'])
    all_rmse = []

    for fold, (train_idx, test_idx) in enumerate(tscv.split(df_xgb_feature)):
        X_train, X_test = df_xgb_feature.iloc[train_idx], df_xgb_feature.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        dtest = xgb.DMatrix(X_test.values, label=y_test.values)

        model = xgb.train(params, dtrain, num_boost_round=500,
                          evals=[(dtest, 'eval')],
                          early_stopping_rounds=30, verbose_eval=False)

        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mae = calcola_mae(y_test, preds)  # tua funzione custom

        all_rmse.append(rmse)

        # Recupera le date (se esiste colonna 'ds')
        start_date = df_xgb.iloc[test_idx]['ds'].min() if 'ds' in df_xgb.columns else None
        end_date = df_xgb.iloc[test_idx]['ds'].max() if 'ds' in df_xgb.columns else None

        # Logga le metriche della fold
        cv_metrics_log.append({
            'Trial': trial.number,
            'Fold': fold + 1,
            'MAE_XGB': mae,
            'RMSE_XGB': rmse,
            'start_date': start_date,
            'end_date': end_date,
            'Model': 'XGBoost'  
        })
    return np.mean(all_rmse)

In [0]:
# Optuna optimization
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50) #aumentare su databricks

In [0]:
# Best parameters
print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)
print("Best trial:",study.best_trial.number)
besttrial = study.best_trial.number

In [0]:
cv_metrics_log = pd.DataFrame(cv_metrics_log)
cv_metrics_log = cv_metrics_log[cv_metrics_log['Trial']==besttrial]
cv_metrics_log

Vediamo come ha performato nel miglior trial

In [0]:
#PLOT CV
cv_metrics_df_xgboost = cv_metrics_log.copy()

# Stampa le metriche per modello
print("Metriche per modello:")
print(cv_metrics_df_xgboost)

# Calcola e stampa le metriche medie per modello
mean_metrics = cv_metrics_df_xgboost.mean(numeric_only=True).reset_index()

# Crea il DataFrame con gli intervalli delle fold
fold_intervals_df = cv_metrics_df_xgboost[['Fold', 'start_date', 'end_date']].drop_duplicates()

# Ora creiamo il grafico combinato
plt.figure(figsize=(16, 12))

# === PLOT 1: Serie temporale originale ===
plt.subplot(2, 1, 1)
#for unique_id in train_df_xgb['unique_id'].unique():
series_data = train_df_xgb.copy()
plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)],
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10,
             bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# === PLOT 2: Metriche MAE e RMSE ===
plt.subplot(2, 1, 2)
ax1 = plt.gca()
ax2 = ax1.twinx()

# MAE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_XGB'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# RMSE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_XGB'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Legenda combinata
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


Training coi migliori parametri trovati da Optuna

In [0]:
# Addestriamo il modello con i best params
best_params = study.best_params
best_params.update({'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 42})

dtrain_xgb = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
start_time = time.time()
#fit 
model = xgb.train(best_params, dtrain_xgb, num_boost_round=500)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Plot feature importance
ax = xgb.plot_importance(model, importance_type='gain', max_num_features=10)
for text in ax.texts:
    text.set_visible(False)
plt.title('Feature Importance (Gain)')
plt.tight_layout()
plt.show()

MULTI-STEP FORECAST

In [0]:
def direct_multistep_forecast(train_data, feature_cols, target_col, horizon=None):
    """
    Addestra modelli separati per ogni orizzonte temporale futuro
    """
    forecasts = []
    models = []
    
    # Crea dataframe per date future
    last_date = train_data['ds'].max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta(minutes=30), periods=horizon, freq='30min') #dopo modificare
    
    # Per ogni step futuro, addestra un modello dedicato
    for h in range(1, horizon+1):
        print(f"Training model for horizon {h}")
        
        # Prepara target con shift inverso per prevedere h passi avanti
        df_horizon = train_data.copy()
        df_horizon[f'y_horizon_{h}'] = df_horizon[target_col].shift(-h)
        df_horizon = df_horizon.dropna()
        
        # Prendi features e target per questo orizzonte
        X = df_horizon[feature_cols]
        y = df_horizon[f'y_horizon_{h}']
        
        # Split train/validation
        train_size = int(len(X) * 0.8)
        X_train, X_val = X.iloc[:train_size], X.iloc[train_size:]
        y_train, y_val = y.iloc[:train_size], y.iloc[train_size:]
        
        # Addestra modello
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        params = best_params
        model = xgb.train(
            params,
            dtrain,
            evals=[(dtrain, 'train'), (dval, 'val')],
            num_boost_round=100,
            early_stopping_rounds=20,
            verbose_eval=False
        )
        models.append(model)
        
    # Crea input per la previsione (l'ultimo punto noto)
    last_point = xgb.DMatrix(train_data.iloc[[-1]][feature_cols])
    
    # Prevedi ciascun orizzonte con il modello dedicato
    for i, model in enumerate(models):
        pred = model.predict(last_point)[0]
        forecasts.append(pred)
    return pd.DataFrame({'ds': future_dates, 'forecast': forecasts})

In [0]:
# #Forecasting, took 8/9 minutes
# forecast_xgb = direct_multistep_forecast(train_df_xgb, feature_cols, target_col='y',horizon=len(test_df_xgb)+96)
# with open('pickles/XGB_elecDemand.pkl', 'wb') as file:
#     pickle.dump(forecast_xgb, file)

In [0]:
# # carico le previsioni
# with open('pickles/XGB_elecDemand.pkl', 'rb') as file:
#     forecast_xgb = pickle.load(file)

RECURSIVE FORECAST

In [0]:
dtest_xgb = xgb.DMatrix(X_test_xgb)
forecast_xgb = pd.DataFrame(model.predict(dtest_xgb),columns=['forecast'])
forecast_xgb['ds'] = test_df_xgb['ds'].values

Rescaling dei dati

In [0]:
# RE-Scaling dei dati
forecast_xgb['forecast'] = scaler_y.inverse_transform(forecast_xgb[['forecast']])
train_df_xgb['y'] = scaler_y.inverse_transform(train_df_xgb[['y']])

In [0]:
# Plot forecast
fig = go.Figure()
# Training data
fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=train_df_xgb['y'], mode='lines', name='Training'))
# Test data
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=test_df_xgb['y'], mode='lines', name='Test'))
# Forecast
fig.add_trace(go.Scatter(x=forecast_xgb['ds'], y=forecast_xgb['forecast'], mode='lines', 
                         name='Forecast', line=dict(color='deepskyblue')))
# Forecast dirette
fig.update_layout(title='Forecast vs Real Data', template='plotly_dark')
fig.show()

Calcolo metriche

In [0]:
# Calcola metriche
merged_df = pd.merge(test_df_xgb[['ds', 'y']], forecast_xgb, on='ds', how='left')
merged_df.dropna(inplace=True)
metriche_xgb = calcola_metriche(merged_df['y'],merged_df['forecast'],
                                train_df_xgb['y'], modelname="XGBoost")
metriche_xgb

##### LSTM (neuralforecast)

In [0]:
# divisione in train e test
df_lstm = create_time_series_features(df, target_col='y') 
df_lstm = df_lstm.dropna().reset_index(drop=True)

train_size = int(len(df_lstm) * 0.9)
train_df_lstm = df_lstm.iloc[:train_size]
test_df_lstm = df_lstm.iloc[train_size:]
print(train_df_lstm.columns)
train_df_lstm.head()

In [0]:
train_df_lstm.shape, test_df_lstm.shape

In [0]:
# Optuna function
futr_exog_list = train_df_lstm.drop(columns=['unique_id', 'ds', 'y']).columns.tolist()
folds = pd.DataFrame()
h = len(test_df_lstm)
cv_metrics_df_lstm = pd.DataFrame()
def objective(trial):
    cv_metrics = []
    # Hyperparametri da ottimizzare
    input_size = trial.suggest_categorical("input_size", [h, int(2.5*h)] )
    encoder_n_layers = trial.suggest_int("encoder_n_layers", 1, 4)
    encoder_hidden_size = trial.suggest_categorical("encoder_hidden_size", [32, 256])
    decoder_hidden_size = trial.suggest_categorical("decoder_hidden_size", [32, 256])
    decoder_layers = trial.suggest_int("decoder_layers", 1, 4)
    learning_rate = trial.suggest_float("learning_rate", 1e-3, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 128])

    # Fisso il seed per riproducibilità
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

    # Definizione modello LSTM
    lstm = LSTM(
        h=h,  # nel tuning
        input_size=input_size,
        loss=DistributionLoss(distribution="Normal", level=[95]),
        scaler_type='robust',
        encoder_n_layers=encoder_n_layers,
        encoder_hidden_size=encoder_hidden_size,
        decoder_hidden_size=decoder_hidden_size,
        decoder_layers=decoder_layers,
        futr_exog_list=futr_exog_list,
        max_steps=20, 
        batch_size=batch_size,
        learning_rate=learning_rate,
        recurrent=False
    )

    nf = NeuralForecast(models=[lstm], freq='30min')

    # cross-validation per tuning
    df_cv = nf.cross_validation(
        df=train_df_lstm,
        step_size=h,
        n_windows=5
    )

    # Calcola MAE per ogni fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'LSTM']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'LSTM',
            'MAE_lstm': mae,
            'RMSE_lstm': rmse
        })

    # Media MAE su tutte le fold
    cv_metrics_df_lstm = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_lstm['MAE_lstm'].mean()

    return mean_mae 

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50) 

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
# # salvo i best params trovati da Optuna
# with open('pickles/LSTM_bestPar_ElecDemand.pkl', 'wb') as file:
#     pickle.dump(best_params, file)

Carico i best params

In [0]:
with open('pickles/LSTM_bestPar_ElecDemand.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [0]:
# final model
best_lstm = LSTM(
    h=len(test_df_lstm),
    input_size=best_params['input_size'],
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=50,
    #early_stop_patience_steps=20,
    batch_size=best_params['batch_size'],
    learning_rate=best_params['learning_rate'],
    recurrent=False
)
nf = NeuralForecast(models=[best_lstm], freq='30min')

In [0]:
# final CV
# df_cv = nf.cross_validation(
#     df=train_df_lstm,
#     step_size=len(test_df_lstm),
#     n_windows=5
# )
# with open('pickles/LSTM_CV_ElecDemand.pkl', 'wb') as file:
#      pickle.dump(df_cv, file)

with open('pickles/LSTM_CV_ElecDemand.pkl', 'rb') as file:
    df_cv = pickle.load(file)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'LSTM']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'LSTM',
        'MAE_lstm': mae,
        'RMSE_lstm': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_lstm = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_lstm)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_lstm.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_lstm['unique_id'].unique():
    series_data = train_df_lstm[train_df_lstm['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_lstm['Model'] = 'lstm'
# Plot MAE sul primo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_lstm'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_lstm'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


In [0]:
# Final training

# Fisso il seed per riproducibilità
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

final_model = LSTM(
    h=len(test_df_lstm),
    input_size=best_params['input_size'],
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=50, #aumentare a 150
    early_stop_patience_steps=15,
    batch_size=256,
    learning_rate=best_params['learning_rate'], 
    recurrent=False
)

nf = NeuralForecast(models=[final_model], freq='30min')
start_time = time.time()
#fit 
nf.fit(df=train_df_lstm, val_size=len(test_df_lstm))
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

Previsioni

In [0]:
#predictions
#  Y_hat_df_lstm = nf.predict(futr_df=test_df_lstm)
#salvo in pickle
# with open('pickles/LSTM_elecDemand.pkl', 'wb') as file:
#     pickle.dump(Y_hat_df_lstm, file)

In [0]:
#carico le previsioni:
with open('pickles/LSTM_elecDemand.pkl', 'rb') as file:
    Y_hat_df_lstm = pickle.load(file)
#queste previsioni sono state generate con 'max_steps' = 50, ma ci mette 15 minuti

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_lstm['ds'], y=train_df_lstm['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_lstm['ds'], y=test_df_lstm['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM'], mode='lines', 
    name='LSTM MEAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIANA LSTM
# fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM-median'], mode='lines', 
#     name='MEDIAN Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="LSTM forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_lstm['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")

fig.show()

In [0]:
# Calcola metriche
merged_df_lstm = pd.merge(test_df_lstm[['ds', 'y']], Y_hat_df_lstm, on='ds', how='left')
merged_df_lstm.dropna(inplace=True)
metriche_lstm = calcola_metriche(merged_df_lstm['y'],merged_df_lstm['LSTM'],train_df_lstm['y'],modelname="LSTM")
metriche_lstm

##### NEURAL PROPHET

In [0]:
df_np = create_time_series_features(df, target_col='y') 
df_np = df_np.dropna().reset_index(drop=True)
df_np = df_np.drop(columns=['unique_id'])
df_np['settimana_del_anno'] = df_np['settimana_del_anno'].astype(int)
df_np.columns

In [0]:
train_size = int(len(df_np) * 0.9)
train_df_np = df_np.iloc[:train_size]
test_df_np = df_np.iloc[train_size:]

In [0]:
# Scaling dei dati
train_df_original = train_df_np.copy()
test_df_original = test_df_np.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_np['ds']
test_meta = test_df_np['ds']

feature_cols = [col for col in train_df_np.columns if col not in ['ds']]
train_idx = train_df_np.index
test_idx = test_df_np.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_np[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_np[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_np = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_np = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_np.shape, test_df_np.shape

In [0]:
# preparazione dataset
np_df_train = train_df_np[['ds', 'y']].copy()
np_df_test = test_df_np[['ds', 'y']].copy()

# Aggiungi le colonne dei regressori
regressors = ['Workday', 'Temperature', 'giorno_settimana', 'ora_del_giorno',
       'settimana_del_mese', 'settimana_del_anno', 'giorno_del_mese',
       'mese_del_anno', 'lag_1', 'lag_2', 'lag_4', 'lag_24', 'lag_36',
       'lag_48', 'lag_96', 'diff_1', 'diff_4', 'diff_48', 'diff_96', 'day_sin',
       'day_cos', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
       'rolling_mean_4', 'rolling_mean_24', 'rolling_mean_48',
       'rolling_mean_96', 'rolling_std_4', 'rolling_std_24', 'rolling_std_48']

for reg in regressors:
    np_df_train[reg] = train_df_np[reg]
    np_df_test[reg] = test_df_np[reg]
# Definisci il modello
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.001, 
    batch_size=32,
    daily_seasonality=True,
    weekly_seasonality=True,
    loss_func='Huber'
)

# Aggiungi i regressori
for reg in regressors:
    neuralprophet.add_future_regressor(reg)

In [0]:
# TIME SERIES CROSS VALIDATION
initial_train_size = len(train_df_np)-5*len(test_df_np)
h = len(test_df_np)
n_windows = 5

# Reset risultati
results = []

for i in range(n_windows):
    print(f"\n--- Fold {i+1}/{n_windows} ---")

    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    train_window = np_df_train.iloc[:train_end]
    test_window = np_df_train.iloc[test_start:test_end]

    print(f"Train: {train_window.shape}, Test: {test_window.shape}")

    # Modello
    model = NeuralProphet(
        quantiles=[0.025, 0.975],
        learning_rate=0.001,
        batch_size=64,
        daily_seasonality=True,
        weekly_seasonality=True,
        loss_func='Huber',
        global_normalization=True,
        unknown_data_normalization=True
    )

    for reg in regressors:
        model.add_future_regressor(reg)

    # Fit
    _ = model.fit(train_window, freq="30min", epochs=100)

    # Previsione
    forecast = model.predict(test_window)

    # Metriche
    y_true = test_window['y'].values
    y_pred = forecast['yhat1'].values

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    results.append({'split': i+1, 'MAE_np': mae, 'RMSE_np': rmse})
    print(f"Split {i+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# Risultati
results_df_np = pd.DataFrame(results)
print("\n=== Backtest Results ===")
print(results_df_np)

# Plot
results_df_np.plot(x='split', y=['MAE_np', 'RMSE_np'], marker='o', title='Backtesting Errors')

In [0]:
# Salvo results df CV
# with open('pickles/NP_CV_ElecDemand.pkl', 'wb') as file:
#     pickle.dump(results_df_np, file)

In [0]:
results_df_np

In [0]:
# Carico i risultati della CV
with open('pickles/NP_CV_ElecDemand.pkl', 'rb') as file:
    results_df_np = pickle.load(file)
results_df_np.plot(x='split', y=['MAE_np', 'RMSE_np'], marker='o', title='Backtesting Errors')

In [0]:
# Mi ricavo le date
initial_train_size = len(train_df_np)-5*len(test_df_np)
h = len(test_df_np)
n_windows = 5

# Crea lista per i dati delle fold
fold_data = []

for i in range(n_windows):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    # Ricava date da colonna 'ds' o dall'indice
    if 'ds' in np_df_train.columns:
        start_date = np_df_train['ds'].iloc[test_start]
        end_date = np_df_train['ds'].iloc[test_end - 1]
    else:
        start_date = np_df_train.index[test_start]
        end_date = np_df_train.index[test_end - 1]

    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]

    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)

# Converto le date in datetime se necessario
if all(isinstance(date, str) for date in fold_intervals_df['start_date']):
    fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
    fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])

print("Fold Intervals DataFrame:")
print(fold_intervals_df)

In [0]:
# CV PLOT
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_np['ds'], train_df_np['y'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

Now let's training on the whole dataset

In [0]:
# preparazione dataset
np_df_train = train_df_np[['ds', 'y']].copy()
np_df_test = test_df_np[['ds', 'y']].copy()

# Aggiungi le colonne dei regressori
regressors = ['Workday', 'Temperature', 'giorno_settimana', 'ora_del_giorno',
       'settimana_del_mese', 'settimana_del_anno', 'giorno_del_mese',
       'mese_del_anno', 'lag_1', 'lag_2', 'lag_4', 'lag_24', 'lag_36',
       'lag_48', 'lag_96', 'diff_1', 'diff_4', 'diff_48', 'diff_96', 'day_sin',
       'day_cos', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
       'rolling_mean_4', 'rolling_mean_24', 'rolling_mean_48',
       'rolling_mean_96', 'rolling_std_4', 'rolling_std_24', 'rolling_std_48']

for reg in regressors:
    np_df_train[reg] = train_df_np[reg]
    np_df_test[reg] = test_df_np[reg]
    

# Fisso il seed per riproducibilità
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


# Definisci il modello
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],
    learning_rate=0.001,
    batch_size=64,
    daily_seasonality=True,
    weekly_seasonality=True,
    loss_func='Huber',
)

# Aggiungi i regressori
for reg in regressors:
    neuralprophet.add_future_regressor(reg)

In [0]:
# FIT
start_time = time.time()
metrics_df = neuralprophet.fit(np_df_train, freq="30min", epochs=100)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
# forecast_test = neuralprophet.predict(np_df_test)
# #salvo le prediction sul test in locale
# with open('pickles/NP_elecDemand.pkl', 'wb') as file:
#     pickle.dump(forecast_test, file)

CARICO LE PREVISIONI DEL TEST SET

In [0]:
# carico le previsioni del test
with open('pickles/NP_elecDemand.pkl', 'rb') as file:
    forecast_test = pickle.load(file)

In [0]:
#Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
forecast_test['yhat1'] = forecast_test['yhat1'] * (max_val - min_val) + min_val
forecast_test['yhat1 2.5%'] = forecast_test['yhat1 2.5%'] * (max_val - min_val) + min_val
forecast_test['yhat1 97.5%'] = forecast_test['yhat1 97.5%'] * (max_val - min_val) + min_val

In [0]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))

# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast_test['ds'], 'forecast': forecast_test['yhat1']})
merged_df = pd.merge(test_df_original[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
mae_val = mean_absolute_error(merged_df['y'], merged_df['forecast'])
metriche_np = calcola_metriche(merged_df['y'], merged_df['forecast'],train_df_original['y'],
                               modelname="NeuralProphet").round(10)
metriche_np

#### TFT (Colab)

In [0]:
# Preparing dataset
df_tft = create_time_series_features(df, target_col='y') 
df_tft = df_tft.dropna().reset_index(drop=True)
# 2. Suddivisione in train/test
train_size = int(len(df_tft) * 0.9)
train_df_tft = df_tft.iloc[:train_size]
test_df_tft = df_tft.iloc[train_size:]
train_df_tft.shape, test_df_tft.shape

In [0]:
# Scaling dei dati
#Salvo per dopo
train_df_original = train_df_tft.copy() 
test_df_original = test_df_tft.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_tft[['ds', 'unique_id']]
test_meta = test_df_tft[['ds', 'unique_id']]

feature_cols = [col for col in train_df_tft.columns if col not in ['ds', 'unique_id']]
train_idx = train_df_tft.index
test_idx = test_df_tft.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_tft[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_tft[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_tft = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_tft = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_tft.shape, test_df_tft.shape

#####  TimeSeriesCV - fine tuning Optuna (TROPPO LENTO)

In [0]:
# Optuna function
futr_exog_list = train_df_tft.drop(columns=['unique_id', 'ds', 'y']).columns.tolist()
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_tft = pd.DataFrame()
h = len(test_df_tft)

def objective(trial):
    # Hyperparameters to optimize for TFT
    input_size = trial.suggest_categorical("input_size", [h, int(1.5*h), int(h/2)])
    hidden_size = trial.suggest_categorical("hidden_size", [32, 64, 128, 256])
    n_rnn_layers = trial.suggest_int("n_rnn_layers", 1, 4)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    grn_activation = trial.suggest_categorical("grn_activation", ["ReLU", "ELU", "Sigmoid"])
    
    # Define TFT model
    tft = TFT(
        h=int(h/5),  # forecast horizon
        input_size=input_size,
        hidden_size=hidden_size,
        n_rnn_layers=n_rnn_layers,
        rnn_type='lstm',
        grn_activation=grn_activation,
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=learning_rate,
        futr_exog_list=futr_exog_list,
        max_steps=50, 
        val_check_steps=10,
        batch_size=batch_size,
        early_stop_patience_steps=-1,
        scaler_type="robust",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=128
)

    nf = NeuralForecast(models=[tft], freq='30min')

    # Cross-validation for tuning
    df_cv = nf.cross_validation(
        df=train_df_tft[5000:],
        step_size=int(h/5),
        n_windows=5
    )

    # Calculate MAE for each fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'TFT']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'TFT',
            'MAE_tft': mae,
            'RMSE_tft': rmse
        })

    # Average MAE across all folds
    cv_metrics_df_tft = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_tft['MAE_tft'].mean()

    return mean_mae

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=10)  #aumentare a 50 su databricks

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params
#salvo i best params in pickle
with open('pickles/TFT_bestPars_ElecDemand.pkl', 'wb') as file:
    pickle.dump(best_params, file)

In [0]:
# #carico i best params
# with open('pickles/TFT_bestPars_ElecDemand.pkl', 'rb') as file:
#     best_params = pickle.load(file)
# with open('pickles/TFT_ElecDemand_best.pkl', 'rb') as file:
#     best_params = pickle.load(file)

In [0]:
# carico le metriche di CV (fatte in colab)
cv_metrics_df_tft = pd.read_pickle("pickles/TFT_ElecDemand_metrics_cv.pkl")
cv_metrics_df_tft.plot(x='Fold', y=['MAE_TFT', 'RMSE_TFT'], marker='o', title='Backtesting Errors')
cv_metrics_df_tft

La CV è troppo lenta, si blocca!

In [0]:
# final CV

# final model
best_tft = TFT(
        h=h,  
        input_size=best_params['input_size'],
        hidden_size=best_params['hidden_size'],
        n_rnn_layers=best_params['n_rnn_layers'],
        rnn_type='lstm',
        grn_activation=best_params['grn_activation'],
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=best_params['learning_rate'],
        futr_exog_list=futr_exog_list,
        max_steps=50, #aumentare a 100
        val_check_steps=10,
        batch_size=batch_size,
        early_stop_patience_steps=-1,
        scaler_type="robust",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False
)
nf = NeuralForecast(models=[best_tft],freq="30min")

# final CV
df_cv = nf.cross_validation(
    df=train_df_tft,
    step_size=len(test_df_tft),
    n_windows=5
)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'LSTM']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'LSTM',
        'MAE_lstm': mae,
        'RMSE_lstm': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_tft = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_tft)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_tft.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_lstm['unique_id'].unique():
    series_data = train_df_lstm[train_df_lstm['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_lstm['Model'] = 'lstm'
# Plot MAE sul primo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_lstm'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_lstm'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


##### TFT training

In [0]:
# # Final training con best_params
# best_tft = TFT(
#         h=h,  
#         input_size=best_params['input_size'],
#         hidden_size=best_params['hidden_size'],
#         n_rnn_layers=best_params['n_rnn_layers'],
#         rnn_type='lstm',
#         grn_activation=best_params['grn_activation'],
#         loss=DistributionLoss(distribution="StudentT", level=[90]),
#         learning_rate=best_params['learning_rate'],
#         futr_exog_list=futr_exog_list,
#         max_steps=50, #aumentare a 100
#         val_check_steps=25,
#         batch_size=best_params['batch_size'],
#         early_stop_patience_steps=-1,
#         scaler_type="robust",
#         enable_progress_bar=True,
#         accelerator="auto",
#         one_rnn_initial_state=False,
#         windows_batch_size=64        
# )
# nf = NeuralForecast(models=[best_tft],freq="30min")
# nf.fit(df=train_df_tft, val_size=len(test_df_tft))

In [0]:
# TRAINING FATTO SU GPT T4 GOOGLE COLAB
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

h = len(test_df_tft)
futr_exog_list = train_df_tft.drop(columns=['unique_id', 'ds', 'y']).columns.tolist()

# Configurazione conservativa per la memoria
best_tft = TFT(
    h=h,  
    input_size=168,
    hidden_size=16, 
    n_rnn_layers=2,
    rnn_type='lstm',
    grn_activation='ReLU',
    loss=DistributionLoss(distribution="StudentT", level=[90]),
    learning_rate=0.005,
    futr_exog_list=futr_exog_list,
    max_steps=200,
    val_check_steps=50,
    batch_size=32,  
    early_stop_patience_steps=-1,
    scaler_type="standard",
    enable_progress_bar=True,
    accelerator="auto",
    one_rnn_initial_state=False,
    windows_batch_size=16,  
    start_padding_enabled=True
)
nf = NeuralForecast(models=[best_tft], freq="30min")

# Pulire la memoria GPU prima di iniziare
torch.cuda.empty_cache()
gc.collect()

start_time = time.time()
#fit 
nf.fit(df=train_df_tft, val_size=len(test_df_tft))

end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi") 

  | Name                    | Type                     | Params | Mode \
-----------------------------------------------------------------------------\
0 | loss                    | DistributionLoss         | 3      | train\
1 | padder_train            | ConstantPad1d            | 0      | train\
2 | scaler                  | TemporalNorm             | 0      | train\
3 | embedding               | TFTEmbedding             | 1.1 K  | train\
4 | temporal_encoder        | TemporalCovariateEncoder | 136 K  | train\
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 4.3 K  | train\
6 | output_adapter          | Linear                   | 51     | train\
-----------------------------------------------------------------------------\
141 K     Trainable params\
3         Non-trainable params\
141 K     Total params\
0.566     Total estimated model params size (MB)\
584       Modules in train mode\

In [0]:
# Predict
Y_hat_df_tft = nf.predict(futr_df=test_df_tft)

In [0]:
Y_hat_df_tft = pd.read_csv('pickles/TFT_ElecDemand.csv')
Y_hat_df_tft['ds'] = pd.to_datetime(Y_hat_df_tft['ds'])

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
Y_hat_df_tft['TFT'] = Y_hat_df_tft['TFT'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-median'] = Y_hat_df_tft['TFT-median'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-lo-90'] = Y_hat_df_tft['TFT-lo-90'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-hi-90'] = Y_hat_df_tft['TFT-hi-90'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_tft['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_tft['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='lightgreen')))

# QUANTILI TFT
# Fascia tra P10 e P90
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_tft['ds']) + 
    list(Y_hat_df_tft['ds'][::-1]),
    y=list(Y_hat_df_tft['TFT-hi-90']) + list(Y_hat_df_tft['TFT-lo-90'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="TFT forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_tft['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# Calcola metriche
merged_df_tft = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_tft, on='ds', how='left')
merged_df_tft.rename(columns={'TFT-lo-90': '0.1','TFT-median':'0.5', 'TFT-hi-90':'0.9'},inplace=True)
merged_df_tft.dropna(inplace=True)
metriche_tft = calcola_metriche(merged_df_tft['y'],merged_df_tft['TFT'],train_df_original['y'],
                                y_pred_quantiles=merged_df_tft[['0.1','0.5','0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="TFT").round(10)
metriche_tft

##### TFT Explainability

In [0]:
best_tft = torch.load('pickles/best_tft_ElecDemand_model.pth', map_location=torch.device('cpu'))

In [0]:
attention = best_tft.attention_weights()
def plot_attention(
    self, plot: str = "time", output: str = "plot", width: int = 800, height: int = 400
):
    """
    Plot the attention weights.

    Args:
        plot (str, optional): The type of plot to generate. Can be one of the following:
            - 'time': Display the mean attention weights over time.
            - 'all': Display the attention weights for each horizon.
            - 'heatmap': Display the attention weights as a heatmap.
            - An integer in the range [1, model.h) to display the attention weights for a specific horizon.
        output (str, optional): The type of output to generate. Can be one of the following:
            - 'plot': Display the plot directly.
            - 'figure': Return the plot as a figure object.
        width (int, optional): Width of the plot in pixels. Default is 800.
        height (int, optional): Height of the plot in pixels. Default is 400.

    Returns:
        matplotlib.figure.Figure: If `output` is 'figure', the function returns the plot as a figure object.
    """

    attention = (
        self.mean_on_batch(self.interpretability_params["attn_wts"])
        .mean(dim=0)
        .cpu()
        .numpy()
    )

    fig, ax = plt.subplots(figsize=(width / 100, height / 100))

    if plot == "time":
        attention = attention[self.input_size :, :].mean(axis=0)
        ax.plot(np.arange(-self.input_size, self.h), attention)
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Mean Attention")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "all":
        for i in range(self.input_size, attention.shape[0]):
            ax.plot(
                np.arange(-self.input_size, self.h),
                attention[i, :],
                label=f"horizon {i-self.input_size+1}",
            )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Attention per horizon")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "heatmap":
        cax = ax.imshow(
            attention,
            aspect="auto",
            cmap="viridis",
            extent=[-self.input_size, self.h, -self.input_size, self.h],
        )
        fig.colorbar(cax)
        ax.set_title("Attention Heatmap")
        ax.set_xlabel("Attention (current time step)")
        ax.set_ylabel("Attention (previous time step)")

    elif isinstance(plot, int) and (plot in np.arange(1, self.h + 1)):
        i = self.input_size + plot - 1
        ax.plot(
            np.arange(-self.input_size, self.h),
            attention[i, :],
            label=f"horizon {plot}",
        )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title(f"Attention weight for horizon {plot}")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    else:
        raise ValueError(
            'plot has to be in ["time","all","heatmap"] or integer in range(1,model.h)'
        )

    plt.tight_layout()

    if output == "plot":
        plt.show()
    elif output == "figure":
        return fig
    else:
        raise ValueError(f"Invalid output: {output}. Expected 'plot' or 'figure'.")

In [0]:
# MEAN ATTENTION
plot_attention(best_tft, plot="time")
# pesi di attenzione medi nel tempo su tutti gli orizzonti di previsione.
# La curva blu rappresenta quanto il modello ha "guardato" (attention) 
# quel punto temporale in input durante la previsione.
# Valori più alti significano che il modello ritiene quel punto temporale
# particolarmente rilevante per fare le previsioni

In [0]:
# ATTENTION OF A SPECIFIC TIME STEPS
plot_attention(best_tft, plot=44)
# pesi di attenzione per ogni orizzonte di previsione separatamente.

In [0]:
plot_attention(best_tft, plot="heatmap")

In [0]:
# PAST VARIABLE IMPORTANCES
feature_importances = best_tft.feature_importances()
print(feature_importances.keys())
print("\n ==========PAST FEATURE IMPORTANCES==========")
feature_importances["Past variable importance over time"].mean().sort_values().plot(kind="barh")

Questo grafico mostra quanto il modello ha fatto affidamento sulle feature passate \
(quelle che il modello ha potuto osservare prima del punto di previsione). Serve a costruire il contesto dell'input

In [0]:
# FUTURE VARIABLE IMPORTANCES
print("\n ==========FUTURE FEATURE IMPORTANCES==========")
feature_importances["Future variable importance over time"].mean().sort_values().plot(kind="barh")

Mostra quanto il modello ha sfruttato le feature che sono servite a guidare le previsioni passo-passo nel decoder

DIFFERENZA TRA PAST E FUTURE: \
La diversa importanza riflette quanto ciascuna variabile aiuta nell’analisi del contesto passato (encoder -->> PAST) \
o nella generazione delle previsioni (decoder -->> FUTURE).

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME
df_importances = feature_importances["Past variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances.index))

for col in df_importances.columns:
    p = ax.bar(np.arange(-len(df_importances), 0), df_importances[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances[col]
ax.set_title("Past variable importance over time")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)

plt.show()

Questo grafico mostra QUANDO (e QUANTO) nel passato ogni feature è stata più utile per costruire il contesto predittivo.

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME
df_importances2 = feature_importances["Future variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances2.index))
for col in df_importances2.columns:
    p = ax.bar(np.arange(-len(df_importances2), 0), df_importances2[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances2[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.grid(True)
ax.legend()
plt.show()

Per ogni feature viene mostrato quanto è stata importante per il modello a ciascun orizzonte futuro

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances3 = feature_importances["Past variable importance over time"]
mean_attention = (
    best_tft
    .attention_weights()[best_tft.input_size :, :]
    .mean(axis=0)[: best_tft.input_size]
)
df_importances3 = df_importances3.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances3.index))

for col in df_importances3.columns:
    p = ax.bar(np.arange(-len(df_importances3), 0), df_importances3[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances3[col]
ax.set_title("Past variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances3), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances4 = feature_importances["Future variable importance over time"]
mean_attention = (
    best_tft
    .attention_weights()[best_tft.input_size :, :]
    .mean(axis=0)[: best_tft.input_size]
)
df_importances4 = df_importances4.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances4.index))

for col in df_importances4.columns:
    p = ax.bar(np.arange(-len(df_importances4), 0), df_importances4[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances4[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances4), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
best_tft.feature_importance_correlations()

#### CHRONOS

In [0]:
# Preparing dataset
df_chronos = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos = df_chronos.dropna().reset_index(drop=True) 
df_chronos['item_id'] = df_chronos['unique_id'] 
df_chronos.drop(columns="unique_id", inplace=True) 
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)
df_chronos.columns

In [0]:
# Definizione delle covariate note
known_covariates_names = [
'Workday', 'Temperature', 'giorno_settimana',
       'ora_del_giorno', 'settimana_del_mese', 'settimana_del_anno',
       'giorno_del_mese', 'mese_del_anno', 'lag_1', 'lag_2', 'lag_4', 'lag_24',
       'lag_36', 'lag_48', 'lag_96', 'diff_1', 'diff_4', 'diff_48', 'diff_96',
       'day_sin', 'day_cos', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
       'rolling_mean_4', 'rolling_mean_24', 'rolling_mean_48',
       'rolling_mean_96', 'rolling_std_4', 'rolling_std_24', 'rolling_std_48']

# Split train e test
train_size = int(len(df_chronos) * 0.9)
train_df_chronos = df_chronos.iloc[:train_size]
test_df_chronos = df_chronos.iloc[train_size:]

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [0]:
# Scaling dei dati
train_df_original = train_df_chronos.copy() 
test_df_original = test_df_chronos.copy()

train_idx = train_df_chronos.index
test_idx = test_df_chronos.index

feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

#scaler
scaler_x = MinMaxScaler()
train_df_chronos = pd.DataFrame(scaler_x.fit_transform(train_df_chronos[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_chronos = pd.DataFrame(scaler_x.transform(test_df_chronos[feature_cols]),
                            columns=feature_cols, index=test_idx)
#controllo le shape
train_df_chronos.shape, test_df_chronos.shape

TimeSeriesCV

In [0]:
#TimeSeriesCV
h = len(test_df_chronos)  # prediction length
n_splits = 5
initial_train_size = len(train_df_chronos) - 5*h
results = []

for fold in range(n_splits):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")
    
    train_end = initial_train_size + fold * h
    test_start = train_end
    test_end = test_start + h
    train_df_fold = train_df_chronos.iloc[:train_end]
    test_df_fold = train_df_chronos.iloc[test_start:test_end]
    
    train_df_fold.reset_index(inplace=True)
    test_df_fold.reset_index(inplace=True)
    
    train_df_fold["timestamp"] = pd.to_datetime(train_df_fold["timestamp"])
    test_df_fold["timestamp"] = pd.to_datetime(test_df_fold["timestamp"])
    
    # Future timestamps 
    future_index = pd.date_range(
        start=train_df_fold["timestamp"].max() + pd.Timedelta("30min"),
        periods=h,
        freq="30min")
    
    # Preparo test set con known_covariates
    test_df_for_prediction = test_df_fold[test_df_fold["timestamp"].isin(future_index)].copy()
    test_df_for_prediction = test_df_for_prediction.set_index("timestamp")
    test_df_for_prediction = test_df_for_prediction.reindex(future_index)  # forza continuità
    test_df_for_prediction["item_id"] = train_df_fold["item_id"].iloc[0]  # supponiamo 1 sola serie
    test_df_for_prediction = test_df_for_prediction.reset_index().rename(columns={"index": "timestamp"})
    test_df_for_prediction = test_df_for_prediction.set_index(["item_id", "timestamp"])
    
    # Preparo il train con multindex
    train_df_fold = train_df_fold.set_index(["item_id", "timestamp"])
    print(f"Train: {train_df_fold.shape}, Test: {test_df_fold.shape}")
    
    # inizializzo il predictor
    predictor = TimeSeriesPredictor(
        prediction_length=h,
        target="target",
        known_covariates_names=known_covariates_names,
        eval_metric="MASE",
        freq="30min")
    
    # fit del modello
    predictor.fit(train_df_fold,
        hyperparameters={
            "Chronos": [
                {"model_path": "bolt_small", "ag_args": {"name_suffix": "ZeroShot"}},
                {
                    "model_path": "bolt_small",
                    "covariate_regressor": "CAT",
                    "target_scaler": "standard",
                    "ag_args": {"name_suffix": "WithRegressor"},
                }
            ]
        },
        time_limit=600,
        enable_ensemble=True,)
    
    predictions = predictor.predict(train_df_fold, known_covariates=test_df_for_prediction)

    test_df_with_index = test_df_fold.set_index(["item_id", "timestamp"])

    # Debug
    print(f"Forma predictions: {predictions.shape}")
    print(f"Colonne predictions: {predictions.columns}")
    print(f"Numero di indici in predictions: {len(predictions.index)}")
    print(f"Numero di indici in test_df_with_index: {len(test_df_with_index.index)}")

    # trovo solo gli indici comuni
    common_indices = predictions.index.intersection(test_df_with_index.index)
    print(f"Indici comuni: {len(common_indices)}")

    # prendo solo gli indici comuni
    y_true = test_df_with_index.loc[common_indices, "target"]
    y_pred = predictions.loc[common_indices, 'mean'].to_numpy()

    mae = calcola_mae(y_true, y_pred)
    rmse = calcola_rmse(y_true, y_pred)
    
    results.append({'split': fold+1, 'MAE_chronos': mae, 'RMSE_chronos': rmse})
    print(f"Split {fold+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
# risultati finali
results_df_chronos = pd.DataFrame(results)
results_df_chronos.set_index('split',inplace=True)
print("\n=== Backtest Results ===")
#salvo results_df_chronos
results_df_chronos.to_pickle('pickles/CHRONOS_CV_ElecDemand.pkl')
results_df_chronos

In [0]:
#carico results_df_chronos
results_df_chronos = pd.read_pickle('pickles/CHRONOS_CV_ElecDemand.pkl')

In [0]:
# Estraggo le date
fold_data = []
train_df_chronos.reset_index(inplace=True)
n_splits = 5
for i in range(n_splits):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h
    
    if 'timestamp' in train_df_chronos.columns:
        start_date = train_df_chronos['timestamp'].iloc[test_start]
        end_date = train_df_chronos['timestamp'].iloc[test_end - 1]
    else:
        start_date = train_df_chronos.index[test_start]
        end_date = train_df_chronos.index[test_end - 1]
    
    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]
    
    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# creo dataframe finale
fold_intervals_df = pd.DataFrame(fold_data)
fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])

print("Fold Intervals DataFrame (before conversion):")
fold_intervals_df

In [0]:
results_df_chronos

In [0]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_chronos['timestamp'], train_df_chronos['target'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot degli errori
results_df_chronos.reset_index(inplace=True)
ax = results_df_chronos.plot(x='split', y=['MAE_chronos','RMSE_chronos'], marker='o', title='TimeSeriesCV Errors')
ax.set_xticks(range(1, 6)) # Imposta le tacche sull'asse x da 1 a 5
plt.show()

Let's training on the whole dataset

In [0]:
# Fit del modello

# SOLO PER DATABRICKS
# sys.modules['sklearn.metrics._regression'].mean_absolute_error = sklearn.metrics.mean_absolute_error

predictor = TimeSeriesPredictor(
    prediction_length=len(test_df_chronos), 
    eval_metric="MASE",
    target="target",
    known_covariates_names=known_covariates_names,
    freq="30min"
)

train_df_chronos = TimeSeriesDataFrame(
    train_df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Aggiunta di più modelli nella configurazione per migliorare le performance
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {
                "model_path": "bolt_base",
                "ag_args": {"name_suffix": "ZeroShot"}},
            {
                "model_path": "bolt_small",
                "covariate_regressor": "CAT",
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
            {
                "model_path": "bolt_mini",
                "covariate_regressor": "NN_TORCH", 
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            }
        ],
    },
    time_limit=600,
    enable_ensemble=True, 
)

# Valutazione del modello in fase di training
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [0]:
# PREVISIONI 1 CLASSICHE
#train_df_chronos.set_index(['item_id','timestamp'],inplace=True)
predictions = predictor.predict(
    train_df_chronos, 
    known_covariates=test_df_chronos
)
predictions.reset_index(inplace=True)
#prediction 0 shot
predictions_0shot = predictor.predict(
    train_df_chronos, 
    known_covariates=test_df_chronos,
    model="ChronosZeroShot[bolt_base]"
)
predictions_0shot.reset_index(inplace=True)

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('target')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
predictions['mean'] = predictions['mean'] * (max_val - min_val) + min_val
predictions['0.1'] = predictions['0.1'] * (max_val - min_val) + min_val
predictions['0.5'] = predictions['0.5'] * (max_val - min_val) + min_val
predictions['0.9'] = predictions['0.9'] * (max_val - min_val) + min_val
predictions_0shot['mean'] = predictions_0shot['mean'] * (max_val - min_val) + min_val
predictions_0shot['0.5'] = predictions_0shot['0.5'] * (max_val - min_val) + min_val
predictions_0shot['0.9'] = predictions_0shot['0.9'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=train_df_chronos['timestamp'], y=train_df_original['target'], mode='lines', name='Training Data'))
# Dati di test

fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='With Regressors', line=dict(color='green')))
# PREVISIONI CHRONOS 0-shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='0-shot Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="CHRONOS forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_chronos['timestamp'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# metriche di valutazione
test_df_original.reset_index(inplace=True)
merged_df_chronos = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']],
        on=['timestamp', 'item_id']
)
metriche_chronos = calcola_metriche(merged_df_chronos['target'],merged_df_chronos['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos").round(5)
metriche_chronos

In [0]:
# metriche di valutazione 0 shot
merged_df_chronos_0shot = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions_0shot[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']],
        on=['timestamp', 'item_id']
)
metriche_chronos_0shot = calcola_metriche(merged_df_chronos_0shot['target'],merged_df_chronos_0shot['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos_0shot[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos 0 shot").round(5)
metriche_chronos_0shot

#### final evaluations

In [0]:
final_metrics = metriche_arima.join(metriche_ets).join(metriche_xgb).join(metriche_lstm).join(metriche_np).join(metriche_chronos).join(metriche_chronos_0shot).join(metriche_tft).round(4)
final_metrics

In [0]:
# FINAL PREDICTIONS PLOT FOR OIL PRICE
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoARIMA'], 
    mode='lines', 
    name='ARIMA', 
    line=dict(color='grey')
))

# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoETS'], 
    mode='lines', 
    name='ETS', 
    line=dict(color='orange')
))

# PREVISIONI XGBOOST
fig.add_trace(go.Scatter(
    x=forecast_xgb['ds'], 
    y=forecast_xgb['forecast'], 
    mode='lines', 
    name='XGBoost', 
    line=dict(color='purple')
))

# PREVISIONI LSTM
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=Y_hat_df_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='pink')
))

# PREVISIONI NEURAL PROPHET
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='aquamarine')
))


# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=forecast_test['ds'], 
                         #y=Y_hat_df_tft['TFT'], 
                         y=Y_hat_df_tft,
                         mode='lines', 
    name='TFT', line=dict(color='yellow')))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS', line=dict(color='green')))
    
# PREVISIONI CHRONOS 0 shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS 0-shot', line=dict(color='lightgreen')))

# Layout del grafico
fig.update_layout(
    title="Forecasts for Electricity demand", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
#sistemo i vari dataset prima del merge
cv_metrics_df_lstm.drop(columns=('Model'),inplace=True)
df_arima = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == 'AutoARIMA'][['Fold',
                                                                    'MAE_AutoARIMA','RMSE_AutoARIMA']]
df_ets = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == 'AutoETS'][['Fold',
                                                                    'MAE_AutoETS','RMSE_AutoETS']]
results_df_np.rename(columns={'split': 'Fold'}, inplace=True)
results_df_chronos.rename(columns={'split': 'Fold','MAE':'MAE_chronos','RMSE':'RMSE_chronos'}, inplace=True)
cv_metrics_df_xgboost.rename(columns={'MAE': 'MAE_XGB','RMSE':'RMSE_XGB'}, inplace=True)
df_arima.rename(columns={'MAE':'MAE_AutoARIMA','RMSE':'RMSE_AutoARIMA'}, inplace=True)
df_ets.rename(columns={'MAE':'MAE_AutoETS','RMSE':'RMSE_AutoETS'}, inplace=True)

In [0]:
# Final plot metrics TimeSeriesCV

# faccio il merge di tutte le tabelle dei vari CV
final_cv = df_ets.merge(cv_metrics_df_lstm, 
                            on="Fold",how="inner").merge(results_df_chronos, 
                            on="Fold",how="inner").merge(cv_metrics_df_xgboost[['MAE_XGB','RMSE_XGB','Fold']],
                            on="Fold",how="inner").merge(df_arima,
                            on="Fold",how="inner").merge(results_df_np,
                            on="Fold", how="inner").merge(cv_metrics_df_tft,
                            on="Fold",how="inner")
# Rescaling CV metrics data
columns_to_modify = ['MAE_np', 'RMSE_np', 'MAE_TFT', 'RMSE_TFT', #'MAE_lstm', 'RMSE_lstm', #'MAE_AutoETS', 'RMSE_AutoETS',
                        'MAE_chronos', 'RMSE_chronos', 'MAE_XGB', 'RMSE_XGB']
for col in columns_to_modify:
    final_cv[f'{col}'] = final_cv[f'{col}'] * (max_val - min_val) + min_val

model_colors = {
    'AutoARIMA': '#808080',      # Grey
    'AutoETS': '#ff7f0e',        # Orange
    'XGB': '#9467bd',    # Purple
    'lstm': '#e377c2',       # Pink
    'np': '#7fffd4',         # Aquamarine
    'TFT': '#ffff00',        # Yellow
    'chronos': '#2ca02c',    # Green
}


# Create the figure
fig = go.Figure()

# Add traces for each model - MAE (darker) and RMSE (lighter)
for model, color in model_colors.items():
    # Add MAE line (darker)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'MAE_{model}'],
        mode='lines+markers',
        name=f'{model} MAE',
        line=dict(color=color, width=3),
        marker=dict(size=8)
    ))
    
    # Add RMSE line (lighter with same color but different dash pattern)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'RMSE_{model}'],
        mode='lines+markers',
        name=f'{model} RMSE',
        line=dict(color=color, width=2, dash='dash'),
        marker=dict(size=6)
    ))

# Update layout
fig.update_layout(
    title='Model Performance Comparison (MAE and RMSE)',
    template='plotly_dark',
    xaxis=dict(title='Fold',
        tickmode='linear',
        tick0=1, dtick=1
    ),    yaxis=dict(
        title='Error Value'
    ),    legend=dict(
        orientation="v"
    ),    hovermode="closest")
fig.show()

In [0]:
# Confronto TFT/CHRONOS
tft_chronos = metriche_chronos.join(metriche_tft)
tft_chronos                                    

### **ELECTRICITY DEMAND (MULTISTEP)**

In [ ]:
futr_exog_list = ['Workday', 'ora_del_giorno', 'giorno_settimana', 'settimana_del_mese',
                  'settimana_del_anno','giorno_del_mese','mese_del_anno',
                  'day_sin','day_cos','hour_sin','hour_cos', 'month_sin','month_cos',]
future_features = ['ds', 'y'] + futr_exog_list

In [ ]:
#preparazione dataset
df = df_elec.reset_index().rename(columns={'DATE_TIME': 'ds','Demand':'y'})
df['unique_id'] = 'serie_1'
df['ds'] = pd.to_datetime(df['ds'])
h = 336 #la settimana successiva (24*2*7)

In [ ]:
# funzione per creare feature
def create_time_series_features(df, target_col):
    df = df.copy()
    lags = [1, 2, 4, 24, 36, 48, 96]
    # Lag features (1-4)
    for lag in lags:
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    
    # Differencing features, 1, 4, 48
    df['diff_1'] = (df[target_col] - df[target_col].shift(1)).shift(1)
    df['diff_4'] = (df[target_col] - df[target_col].shift(4)).shift(1)
    df['diff_48'] = (df[target_col] - df[target_col].shift(48)).shift(1)
    df['diff_96'] = (df[target_col] - df[target_col].shift(96)).shift(1)
    
    # Caratteristiche cicliche per rappresentare meglio la stagionalità
    df['day_sin'] = np.sin(2 * np.pi * df['giorno_settimana']/7)
    df['day_cos'] = np.cos(2 * np.pi * df['giorno_settimana']/7)
    df['hour_sin'] = np.sin(2 * np.pi * df['ora_del_giorno']/24)
    df['hour_cos'] = np.cos(2 * np.pi * df['ora_del_giorno']/24)
    df['month_sin'] = np.sin(2 * np.pi * df['mese_del_anno']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['mese_del_anno']/12)
    
    # rolling means 
    df['rolling_mean_4'] = df[target_col].rolling(window=4).mean().shift(1) #ultime 2 ore
    df['rolling_mean_24'] = df[target_col].rolling(window=24).mean().shift(1) #ultime 12 ore
    df['rolling_mean_48'] = df[target_col].rolling(window=48).mean().shift(1) #ultimo giorno
    df['rolling_mean_96'] = df[target_col].rolling(window=96).mean().shift(1) #ultimi 2 giorni
    df['rolling_std_4'] = df[target_col].rolling(window=4).std().shift(1)
    df['rolling_std_24'] = df[target_col].rolling(window=24).mean().shift(1) 
    df['rolling_std_48'] = df[target_col].rolling(window=48).std().shift(1)
    df['rolling_std_48'] = df[target_col].rolling(window=96).mean().shift(1) 
    return df

##### XGBOOST

In [ ]:
# df_xgb
df_xgb = create_time_series_features(df, target_col='y') 
hist_exog_features = df_xgb.drop(columns=['unique_id', 'ds', 'y'] + futr_exog_list).columns.tolist()
df_xgb.drop(columns=['unique_id'],inplace=True)
df_xgb = df_xgb.dropna().reset_index(drop=True) #droppo i NaN creati dai lag, quindi le prime 12 obs.
# STEP 2: Prepara DMatrix per previsione
df_xgb = df_xgb[future_features]
print(df_xgb.columns)

In [ ]:
# Split train/test
# train tutto tranne le ultime 12 osservazioni
train_df_xgb = df_xgb.iloc[:-h]
test_df_xgb = df_xgb.iloc[-h:]

# Separazione features e target
feature_cols = [col for col in df_xgb.columns if col not in ['y', 'ds']]
X_train_xgb, y_train_xgb = train_df_xgb[feature_cols], train_df_xgb['y']
X_test_xgb, y_test_xgb = test_df_xgb[feature_cols], test_df_xgb['y']

In [ ]:
# train e test plot
fig = go.Figure()
#train dat
fig.add_trace(go.Scatter(x=df_xgb['ds'], y=y_train_xgb, mode='lines', name='Train'))
#test data 
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=y_test_xgb, mode='lines', name='Test'))
#layout
fig.update_layout(template='plotly_dark', title='Train and Test Data Plot')
fig.show()

In [ ]:
# Scaling dei dati

#features
scaler_x = MinMaxScaler()
X_train_scaled = scaler_x.fit_transform(X_train_xgb)
X_test_scaled = scaler_x.transform(X_test_xgb)

# target, non serve che scalo y_test
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train_xgb.values.reshape(-1, 1))

# Ricreo df con stessi indici e nomi
X_train_xgb = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train_xgb.index)
X_test_xgb = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test_xgb.index)
y_train_xgb = pd.Series(y_train_scaled.flatten(), name='y', index=y_train_xgb.index)

#aggiorno train_df_xgb
train_df_xgb[feature_cols] = X_train_xgb
train_df_xgb['y'] = y_train_xgb

#controllo le shape
X_train_xgb.shape, X_test_xgb.shape, y_train_xgb.shape, y_test_xgb.shape

Ottimizzazione Optuna (Bayesiana)

In [ ]:
# Optuna function
cv_metrics_log = []  # List globale per salvare metriche fold per fold

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'verbosity': 0,
        'seed': 42
    }

    tscv = TimeSeriesSplit(n_splits=5,test_size=len(test_df_xgb))
    y = train_df_xgb['y']
    df_xgb_feature = train_df_xgb.drop(columns=['y','ds'])
    all_rmse = []

    for fold, (train_idx, test_idx) in enumerate(tscv.split(df_xgb_feature)):
        X_train, X_test = df_xgb_feature.iloc[train_idx], df_xgb_feature.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        dtest = xgb.DMatrix(X_test.values, label=y_test.values)

        model = xgb.train(params, dtrain, num_boost_round=500,
                          evals=[(dtest, 'eval')],
                          early_stopping_rounds=30, verbose_eval=False)

        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mae = calcola_mae(y_test, preds)  

        all_rmse.append(rmse)

        # Recupera le date (se esiste colonna 'ds')
        start_date = df_xgb.iloc[test_idx]['ds'].min() if 'ds' in df_xgb.columns else None
        end_date = df_xgb.iloc[test_idx]['ds'].max() if 'ds' in df_xgb.columns else None

        # Logga le metriche della fold
        cv_metrics_log.append({
            'Trial': trial.number,
            'Fold': fold + 1,
            'MAE_XGB': mae,
            'RMSE_XGB': rmse,
            'start_date': start_date,
            'end_date': end_date,
            'Model': 'XGBoost'  
        })
    return np.mean(all_rmse)

In [ ]:
# Optuna optimization
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50) #aumentare su databricks

In [ ]:
# Best parameters
print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)
print("Best trial:",study.best_trial.number)
besttrial = study.best_trial.number

In [ ]:
cv_metrics_log = pd.DataFrame(cv_metrics_log)
cv_metrics_log = cv_metrics_log[cv_metrics_log['Trial']==besttrial]
cv_metrics_log

Vediamo come ha performato nel miglior trial

In [ ]:
#PLOT CV
cv_metrics_df_xgboost = cv_metrics_log.copy()

# Stampa le metriche per modello
print("Metriche per modello:")
print(cv_metrics_df_xgboost)

# Calcola e stampa le metriche medie per modello
mean_metrics = cv_metrics_df_xgboost.mean(numeric_only=True).reset_index()

# Crea il DataFrame con gli intervalli delle fold
fold_intervals_df = cv_metrics_df_xgboost[['Fold', 'start_date', 'end_date']].drop_duplicates()

# Ora creiamo il grafico combinato
plt.figure(figsize=(16, 12))

# === PLOT 1: Serie temporale originale ===
plt.subplot(2, 1, 1)
#for unique_id in train_df_xgb['unique_id'].unique():
series_data = train_df_xgb.copy()
plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)],
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'{row["Fold"]}', ha='center', fontsize=10,
             bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# === PLOT 2: Metriche MAE e RMSE ===
plt.subplot(2, 1, 2)
ax1 = plt.gca()
ax2 = ax1.twinx()

# MAE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_XGB'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# RMSE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_XGB'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Legenda combinata
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


Training coi migliori parametri trovati da Optuna

In [ ]:
# Addestriamo il modello con i best params
best_params = study.best_params
best_params.update({'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 42})

dtrain_xgb = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
start_time = time.time()
#fit 
model = xgb.train(best_params, dtrain_xgb, num_boost_round=500)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [ ]:
# Plot feature importance
ax = xgb.plot_importance(model, importance_type='gain', max_num_features=10)
for text in ax.texts:
    text.set_visible(False)
plt.title('Feature Importance (Gain)')
plt.tight_layout()
plt.show()

In [ ]:
dtest_xgb = xgb.DMatrix(X_test_xgb)
forecast_xgb = pd.DataFrame(model.predict(dtest_xgb),columns=['forecast'])
forecast_xgb['ds'] = test_df_xgb['ds'].values

Rescaling dei dati

In [ ]:
# RE-Scaling dei dati
forecast_xgb['forecast'] = scaler_y.inverse_transform(forecast_xgb[['forecast']])
train_df_xgb['y'] = scaler_y.inverse_transform(train_df_xgb[['y']])

In [ ]:
# Plot forecast
fig = go.Figure()
# Training data
fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=train_df_xgb['y'], mode='lines', name='Training'))
# Test data
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=test_df_xgb['y'], mode='lines', name='Test'))
# Forecast
fig.add_trace(go.Scatter(x=forecast_xgb['ds'], y=forecast_xgb['forecast'], mode='lines', 
                         name='Forecast', line=dict(color='deepskyblue')))
# Forecast dirette
fig.update_layout(title='Forecast vs Real Data', template='plotly_dark')
fig.show()

Calcolo metriche

In [ ]:
# Calcola metriche
merged_df = pd.merge(test_df_xgb[['ds', 'y']], forecast_xgb, on='ds', how='left')
merged_df.dropna(inplace=True)
metriche_xgb = calcola_metriche(merged_df['y'],merged_df['forecast'],
                                train_df_xgb['y'], modelname="XGBoost")
metriche_xgb

##### LSTM

In [ ]:
# divisione in train e test
df_lstm = create_time_series_features(df, target_col='y') 
df_lstm = df_lstm.dropna().reset_index(drop=True)

train_df_lstm = df_lstm.iloc[:-h]
test_df_lstm = df_lstm.iloc[-h:]
print(train_df_lstm.columns)
train_df_lstm.head()

In [ ]:
train_df_lstm.shape, test_df_lstm.shape

In [ ]:
# Optuna function
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_lstm = pd.DataFrame()
def objective(trial):
    # Hyperparametri da ottimizzare
    encoder_n_layers = trial.suggest_int("encoder_n_layers", 1, 2,4)
    encoder_hidden_size = trial.suggest_categorical("encoder_hidden_size", [32, 64, 128, 256])
    decoder_hidden_size = trial.suggest_categorical("decoder_hidden_size", [32, 64, 128, 256])
    decoder_layers = trial.suggest_int("decoder_layers", 1, 2,4)
    input_size = trial.suggest_categorical("input_size", [int(h), int(h*2), int(h*5)])
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    # Definizione modello LSTM
    lstm = LSTM(
        h=len(test_df_lstm), 
        input_size=input_size,
        loss=DistributionLoss(distribution="Normal", level=[95]),
        scaler_type='robust',
        encoder_n_layers=encoder_n_layers,
        encoder_hidden_size=encoder_hidden_size,
        decoder_hidden_size=decoder_hidden_size,
        decoder_layers=decoder_layers,
        futr_exog_list=futr_exog_list,
        max_steps=50,
        batch_size=batch_size,
        learning_rate=learning_rate,
        recurrent=False
    )

    nf = NeuralForecast(models=[lstm], freq='30min')

    # cross-validation per tuning
    df_cv = nf.cross_validation(
        df=train_df_lstm,
        step_size=len(test_df_lstm),
        n_windows=5
    )

    # Calcola MAE per ogni fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'LSTM']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'LSTM',
            'MAE_lstm': mae,
            'RMSE_lstm': rmse
        })

    # Media MAE su tutte le fold
    cv_metrics_df_lstm = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_lstm['MAE_lstm'].mean()

    return mean_mae 

In [ ]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50) 

In [ ]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [ ]:
# # salvo i best params trovati da Optuna
with open('pickles/LSTM_bestPar_ElecDemand_multistep.pkl', 'wb') as file:
    pickle.dump(best_params, file)

Carico i best params

In [ ]:
with open('pickles/LSTM_bestPar_ElecDemand_multistep.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [ ]:
# final model
best_lstm = LSTM(
    h=len(test_df_lstm),
    input_size=best_params['input_size'],
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=50,
    #early_stop_patience_steps=20,
    batch_size=best_params['batch_size'],
    learning_rate=best_params['learning_rate'],
    recurrent=False
)
nf = NeuralForecast(models=[best_lstm], freq='30min')

In [ ]:
# final CV
# df_cv = nf.cross_validation(
#     df=train_df_lstm,
#     step_size=len(test_df_lstm),
#     n_windows=5
# )
# with open('pickles/LSTM_CV_ElecDemand_multistep.pkl', 'wb') as file:
#      pickle.dump(df_cv, file)

with open('pickles/LSTM_CV_ElecDemand_multistep.pkl', 'rb') as file:
    df_cv = pickle.load(file)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'LSTM']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'LSTM',
        'MAE_lstm': mae,
        'RMSE_lstm': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_lstm = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
cv_metrics_df_lstm.iloc[1:2,2:] = cv_metrics_df_lstm.iloc[1:2,2:]/1000
print(cv_metrics_df_lstm)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_lstm.groupby('Model').mean().reset_index()
print(mean_metrics)

In [ ]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_lstm['unique_id'].unique():
    series_data = train_df_lstm[train_df_lstm['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_lstm['Model'] = 'lstm'
# Plot MAE sul primo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_lstm'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_lstm'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


In [ ]:
best_params

In [ ]:
# Final training

# Fisso il seed per riproducibilità
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

final_model = LSTM(
    h=len(test_df_lstm),
    input_size=int(best_params['input_size']*5),
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='standard',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=200, #aumentare a 150
    early_stop_patience_steps=25,
    batch_size=128,
    learning_rate=0.032, #best_params['learning_rate']
    recurrent=True
)

nf = NeuralForecast(models=[final_model], freq='30min')
start_time = time.time()
#fit 
nf.fit(df=train_df_lstm, val_size=len(test_df_lstm))
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

Previsioni

In [ ]:
#predictions
Y_hat_df_lstm = nf.predict(futr_df=test_df_lstm[['unique_id', 'ds'] + futr_exog_list])
#salvo in pickle
with open('pickles/LSTM_elecDemand_multistep.pkl', 'wb') as file:
    pickle.dump(Y_hat_df_lstm, file)

In [ ]:
#carico le previsioni:
with open('pickles/LSTM_elecDemand_multistep.pkl', 'rb') as file:
    Y_hat_df_lstm = pickle.load(file)

In [ ]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_lstm['ds'], y=train_df_lstm['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_lstm['ds'], y=test_df_lstm['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM'], mode='lines', 
    name='LSTM MEAN Forecast', line=dict(color='green')))

# Layout del grafico
fig.update_layout(title="LSTM forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_lstm['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")

fig.show()

In [ ]:
# Calcola metriche
merged_df_lstm = pd.merge(test_df_lstm[['ds', 'y']], Y_hat_df_lstm, on='ds', how='left')
merged_df_lstm.dropna(inplace=True)
metriche_lstm = calcola_metriche(merged_df_lstm['y'],merged_df_lstm['LSTM'],train_df_lstm['y'],modelname="LSTM")
metriche_lstm

##### NEURAL PROPHET

In [ ]:
df_np = create_time_series_features(df, target_col='y') 
df_np = df_np.dropna().reset_index(drop=True)
df_np = df_np[future_features]
train_df_np = df_np.iloc[:-h]
test_df_np = df_np.iloc[-h:]

In [ ]:
# Scaling dei dati
train_df_original = train_df_np.copy()
test_df_original = test_df_np.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_np['ds']
test_meta = test_df_np['ds']

feature_cols = [col for col in train_df_np.columns if col not in ['ds']]
train_idx = train_df_np.index
test_idx = test_df_np.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_np[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_np[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_np = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_np = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_np.shape, test_df_np.shape

In [ ]:
# CV
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

initial_train_size = len(train_df_np) - 5 * len(test_df_np)
h = len(test_df_np)
n_windows = 5

results = []

for i in range(n_windows):
    print(f"\n--- Fold {i+1}/{n_windows} ---")

    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    train_window = train_df_np.iloc[:train_end].copy()
    test_window = train_df_np.iloc[test_start:test_end].copy()

    print(f"Train: {train_window.shape}, Test: {test_window.shape}")

    # Modello NeuralProphet
    model = NeuralProphet(
        quantiles=[0.025, 0.975],
        learning_rate=0.01,
        batch_size=64,
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
        loss_func='Huber',
    )

    # Aggiungi regressori
    for reg in futr_exog_list:
        model.add_future_regressor(reg)

    # Fit
    _ = model.fit(train_window, freq="30min", epochs=50)

    # Predizione
    future_df = test_window.copy()  # Deve contenere anche i regressori futuri
    forecast = model.predict(future_df)

    # Metriche
    y_true = test_window['y'].values
    y_pred = forecast['yhat1'].values

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    results.append({'Fold': i+1, 'MAE_nprophet': mae, 'RMSE_nprophet': rmse})
    print(f"Split {i+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# Risultati CV
results_df_np = pd.DataFrame(results)
print("\n=== Backtest Results ===")
print(results_df_np)

# Plot Errori
results_df_np.plot(
    x='Fold',
    y=['MAE_nprophet', 'RMSE_nprophet'],
    marker='o',
    title='TimeSeriesCV Errors',
    figsize=(10, 5)
)
plt.grid(True)
plt.show()

In [ ]:
#Salvo results df CV
with open('pickles/NP_CV_ElecDemand_multistep.pkl', 'wb') as file:
    pickle.dump(results_df_np, file)

In [ ]:
# Carico i risultati della CV
with open('pickles/NP_CV_ElecDemand_multistep.pkl', 'rb') as file:
    results_df_np = pickle.load(file)
    results_df_np = results_df_np.reset_index(drop=True)
results_df_np.plot(x='Fold', y=['MAE_nprophet', 'RMSE_nprophet'], marker='o', title='Backtesting Errors')

Now let's training on the whole dataset

In [ ]:
# FIT
# Fissa i seed per riproducibilità
import random
import time
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

np_df_train = train_df_np[future_features].copy()

# Modello senza regressori esterni
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.02, 
    batch_size=128,
    daily_seasonality=True,    # Impara la stagionalità dai dati
    weekly_seasonality=True,   # Impara pattern settimanali
    yearly_seasonality=True,   # Impara pattern annuali
    loss_func='Huber'
)
# Aggiungi solo i regressori deterministici
for reg in futr_exog_list:
    neuralprophet.add_future_regressor(reg)
start_time = time.time()
metrics_df = neuralprophet.fit(np_df_train, freq="30min",epochs=100)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [ ]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
forecast_test = neuralprophet.predict(test_df_np[future_features])
#salvo le prediction sul test in locale
with open('pickles/NP_elecDemand_multistep.pkl', 'wb') as file:
    pickle.dump(forecast_test, file)

CARICO LE PREVISIONI DEL TEST SET

In [ ]:
# carico le previsioni del test
with open('pickles/NP_elecDemand_multistep.pkl', 'rb') as file:
    forecast_test = pickle.load(file)

In [ ]:
#Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
forecast_test['yhat1'] = forecast_test['yhat1'] * (max_val - min_val) + min_val
forecast_test['yhat1 2.5%'] = forecast_test['yhat1 2.5%'] * (max_val - min_val) + min_val
forecast_test['yhat1 97.5%'] = forecast_test['yhat1 97.5%'] * (max_val - min_val) + min_val

In [ ]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))

# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [ ]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast_test['ds'], 'forecast': forecast_test['yhat1']})
merged_df = pd.merge(test_df_original[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
mae_val = mean_absolute_error(merged_df['y'], merged_df['forecast'])
metriche_np = calcola_metriche(merged_df['y'], merged_df['forecast'],train_df_original['y'],
                               modelname="NeuralProphet").round(10)
metriche_np

#### TFT

In [ ]:
# Preparing dataset
df_tft = create_time_series_features(df, target_col='y') 
df_tft = df_tft.dropna().reset_index(drop=True)
hist_exog_features = df_tft.drop(columns=['unique_id', 'ds', 'y'] + futr_exog_list).columns.tolist()
# 2. Suddivisione in train/test
train_df_tft = df_tft.iloc[:-h]
test_df_tft = df_tft.iloc[-h:]
train_df_tft.shape, test_df_tft.shape

In [ ]:
# Scaling dei dati
#Salvo per dopo
train_df_original = train_df_tft.copy() 
test_df_original = test_df_tft.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_tft[['ds', 'unique_id']]
test_meta = test_df_tft[['ds', 'unique_id']]

feature_cols = [col for col in train_df_tft.columns if col not in ['ds', 'unique_id']]
train_idx = train_df_tft.index
test_idx = test_df_tft.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_tft[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_tft[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_tft = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_tft = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_tft.shape, test_df_tft.shape

#####  TimeSeriesCV - fine tuning Optuna (TROPPO LENTO)

In [0]:
# Optuna function
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_tft = pd.DataFrame()
h = len(test_df_tft)

def objective(trial):
    # Hyperparameters to optimize for TFT
    input_size = trial.suggest_categorical("input_size", [h, int(h/2)])
    hidden_size = trial.suggest_categorical("hidden_size", [32, 64, 128, 256])
    n_rnn_layers = trial.suggest_int("n_rnn_layers", 1, 3)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    grn_activation = trial.suggest_categorical("grn_activation", ["ReLU", "ELU", "Sigmoid"])
    
    # Define TFT model
    tft = TFT(
        h=h,  # forecast horizon
        input_size=input_size,
        hidden_size=hidden_size,
        n_rnn_layers=n_rnn_layers,
        rnn_type='lstm',
        grn_activation=grn_activation,
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=learning_rate,
        futr_exog_list=futr_exog_list,
        hist_exog_list=hist_exog_features,
        max_steps=50, #aumentare a 100
        val_check_steps=10,
        batch_size=batch_size,
        early_stop_patience_steps=-1,
        scaler_type="standard",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=32
)

    nf = NeuralForecast(models=[tft], freq='15min')

    # Cross-validation for tuning
    df_cv = nf.cross_validation(
        df=train_df_tft,
        step_size=h,
        n_windows=5
    )

    # Calculate MAE for each fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'TFT']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'TFT',
            'MAE_tft': mae,
            'RMSE_tft': rmse
        })

    # Average MAE across all folds
    cv_metrics_df_tft = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_tft['MAE_tft'].mean()

    return mean_mae

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=50)  #aumentare a 50 su databricks

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
# # Path nella repo Git
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_bestPars_ElecDemand_multistep.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(best_params, f)

In [ ]:
#carico i best params
with open('pickles/TFT_bestPars_ElecDemand_multistep.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [ ]:
# final CV

# final model
best_tft = TFT(
        h=h,  
        input_size=best_params['input_size'], 
        hidden_size=32,
        n_rnn_layers=best_params['n_rnn_layers'],
        rnn_type='lstm',
        grn_activation=best_params['grn_activation'],
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=best_params['learning_rate'],
        futr_exog_list=futr_exog_list,
        hist_exog_list = hist_exog_features,
        max_steps=200, #aumentare a 100
        val_check_steps=10,
        batch_size=8,
        early_stop_patience_steps=-1,
        scaler_type="standard",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=8
)
nf = NeuralForecast(models=[best_tft],freq="30min")
# final CV
df_cv = nf.cross_validation(
    df=train_df_tft,
    step_size=len(test_df_tft),
    n_windows=5
)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'TFT']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'TFT',
        'MAE_TFT': mae,
        'RMSE_TFT': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_tft = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_tft)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_tft.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# # Path nella repo Git
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_CV_ElecDemand_multistep.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(cv_metrics_df_tft, f)

In [ ]:
# carico la CV
with open('pickles/TFT_CV_ElecDemand_multistep.pkl', 'rb') as file:
    cv_metrics_df_tft = pickle.load(file)

In [0]:
cv_metrics_df_tft

In [ ]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_tft['unique_id'].unique():
    series_data = train_df_tft[train_df_tft['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_tft['Model'] = 'TFT'
# Plot MAE sul primo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_TFT'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_tft['Model'].unique():
    model_data = cv_metrics_df_tft[cv_metrics_df_tft['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_TFT'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()

##### TFT training

In [0]:
# Final training
import time
import torch
import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()

# final model
best_tft = TFT(
        h=h,  
        input_size=best_params['input_size']*6,
        hidden_size=best_params['hidden_size'],
        n_rnn_layers=best_params['n_rnn_layers'],
        rnn_type='lstm',
        grn_activation=best_params['grn_activation'],
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=best_params['learning_rate'],
        futr_exog_list=futr_exog_list,
        hist_exog_list=hist_exog_features,
        max_steps=200, 
        val_check_steps=50,
        batch_size=best_params['batch_size'],
        early_stop_patience_steps=25,
        scaler_type="robust",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=64
)
nf = NeuralForecast(models=[best_tft],freq="30min")

start_time = time.time()
#fit 
nf.fit(df=train_df_tft, val_size=len(test_df_tft))

end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Predict
Y_hat_df_tft = nf.predict(futr_df=test_df_tft[['unique_id','ds'] + futr_exog_list])

In [0]:
# #salvo le pred in pickle
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_pred_ElecDemand_multistep.pkl"
# with open(save_path, 'wb') as file:
#     pickle.dump(Y_hat_df_tft, file)

In [ ]:
# #carico le pred
with open("pickles/TFT_pred_ElecDemand_multistep.pkl", 'rb') as file:
    Y_hat_df_tft = pickle.load(file)

In [ ]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
Y_hat_df_tft['TFT'] = Y_hat_df_tft['TFT'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-median'] = Y_hat_df_tft['TFT-median'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-lo-90'] = Y_hat_df_tft['TFT-lo-90'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-hi-90'] = Y_hat_df_tft['TFT-hi-90'] * (max_val - min_val) + min_val

In [ ]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_tft['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_tft['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='lightgreen')))

# QUANTILI TFT
# Fascia tra P10 e P90
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_tft['ds']) + 
    list(Y_hat_df_tft['ds'][::-1]),
    y=list(Y_hat_df_tft['TFT-hi-90']) + list(Y_hat_df_tft['TFT-lo-90'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="TFT forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_tft['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [ ]:
# Calcola metriche
merged_df_tft = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_tft, on='ds', how='left')
merged_df_tft.rename(columns={'TFT-lo-90': '0.1','TFT-median':'0.5', 'TFT-hi-90':'0.9'},inplace=True)
merged_df_tft.dropna(inplace=True)
metriche_tft = calcola_metriche(merged_df_tft['y'],merged_df_tft['TFT'],train_df_original['y'],
                                y_pred_quantiles=merged_df_tft[['0.1','0.5','0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="TFT").round(10)
metriche_tft

##### TFT Explainability

In [0]:
best_tft = nf.models[0]

In [0]:
# # NON MI FA SALVARE IL MODELLO PERCHE' TROPPO PESANTE
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_model_ElecDemand_multistep.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(best_tft, f)

In [0]:
# # # carico il modello tft
# with open('pickles/TFT_model_ElecDemand_multistep.pkl', 'rb') as file:
#     best_tft = pickle.load(file)

In [0]:
attention = best_tft.attention_weights()
def plot_attention(
    self, plot: str = "time", output: str = "plot", width: int = 800, height: int = 400
):
    """
    Plot the attention weights.

    Args:
        plot (str, optional): The type of plot to generate. Can be one of the following:
            - 'time': Display the mean attention weights over time.
            - 'all': Display the attention weights for each horizon.
            - 'heatmap': Display the attention weights as a heatmap.
            - An integer in the range [1, model.h) to display the attention weights for a specific horizon.
        output (str, optional): The type of output to generate. Can be one of the following:
            - 'plot': Display the plot directly.
            - 'figure': Return the plot as a figure object.
        width (int, optional): Width of the plot in pixels. Default is 800.
        height (int, optional): Height of the plot in pixels. Default is 400.

    Returns:
        matplotlib.figure.Figure: If `output` is 'figure', the function returns the plot as a figure object.
    """

    attention = (
        self.mean_on_batch(self.interpretability_params["attn_wts"])
        .mean(dim=0)
        .cpu()
        .numpy()
    )

    fig, ax = plt.subplots(figsize=(width / 100, height / 100))

    if plot == "time":
        attention = attention[self.input_size :, :].mean(axis=0)
        ax.plot(np.arange(-self.input_size, self.h), attention)
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Mean Attention")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "all":
        for i in range(self.input_size, attention.shape[0]):
            ax.plot(
                np.arange(-self.input_size, self.h),
                attention[i, :],
                label=f"horizon {i-self.input_size+1}",
            )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Attention per horizon")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "heatmap":
        cax = ax.imshow(
            attention,
            aspect="auto",
            cmap="viridis",
            extent=[-self.input_size, self.h, -self.input_size, self.h],
        )
        fig.colorbar(cax)
        ax.set_title("Attention Heatmap")
        ax.set_xlabel("Attention (current time step)")
        ax.set_ylabel("Attention (previous time step)")

    elif isinstance(plot, int) and (plot in np.arange(1, self.h + 1)):
        i = self.input_size + plot - 1
        ax.plot(
            np.arange(-self.input_size, self.h),
            attention[i, :],
            label=f"horizon {plot}",
        )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title(f"Attention weight for horizon {plot}")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    else:
        raise ValueError(
            'plot has to be in ["time","all","heatmap"] or integer in range(1,model.h)'
        )

    plt.tight_layout()

    if output == "plot":
        plt.show()
    elif output == "figure":
        return fig
    else:
        raise ValueError(f"Invalid output: {output}. Expected 'plot' or 'figure'.")

In [0]:
# MEAN ATTENTION
plot_attention(best_tft, plot="time")
# pesi di attenzione medi nel tempo su tutti gli orizzonti di previsione.
# La curva blu rappresenta quanto il modello ha "guardato" (attention) 
# quel punto temporale in input durante la previsione.
# Valori più alti significano che il modello ritiene quel punto temporale
# particolarmente rilevante per fare le previsioni

In [0]:
# ATTENTION OF A SPECIFIC TIME STEPS
plot_attention(best_tft, plot=44)
# pesi di attenzione per ogni orizzonte di previsione separatamente.

In [0]:
plot_attention(best_tft, plot="heatmap")

In [0]:
# PAST VARIABLE IMPORTANCES
feature_importances = best_tft.feature_importances()
print(feature_importances.keys())
print("\n ==========PAST FEATURE IMPORTANCES==========")
feature_importances["Past variable importance over time"].mean().sort_values().plot(kind="barh")

Questo grafico mostra quanto il modello ha fatto affidamento sulle feature passate \
(quelle che il modello ha potuto osservare prima del punto di previsione). Serve a costruire il contesto dell'input

In [0]:
# FUTURE VARIABLE IMPORTANCES
print("\n ==========FUTURE FEATURE IMPORTANCES==========")
feature_importances["Future variable importance over time"].mean().sort_values().plot(kind="barh")

Mostra quanto il modello ha sfruttato le feature che sono servite a guidare le previsioni passo-passo nel decoder

DIFFERENZA TRA PAST E FUTURE: \
La diversa importanza riflette quanto ciascuna variabile aiuta nell’analisi del contesto passato (encoder -->> PAST) \
o nella generazione delle previsioni (decoder -->> FUTURE).

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME
df_importances = feature_importances["Past variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances.index))

for col in df_importances.columns:
    p = ax.bar(np.arange(-len(df_importances), 0), df_importances[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances[col]
ax.set_title("Past variable importance over time")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)

plt.show()

Questo grafico mostra QUANDO (e QUANTO) nel passato ogni feature è stata più utile per costruire il contesto predittivo.

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME
df_importances2 = feature_importances["Future variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances2.index))
for col in df_importances2.columns:
    p = ax.bar(np.arange(-len(df_importances2), 0), df_importances2[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances2[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.grid(True)
ax.legend()
plt.show()

Per ogni feature viene mostrato quanto è stata importante per il modello a ciascun orizzonte futuro

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances3 = feature_importances["Past variable importance over time"]
mean_attention = (
    best_tft
    .attention_weights()[best_tft.input_size :, :]
    .mean(axis=0)[: best_tft.input_size]
)
df_importances3 = df_importances3.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances3.index))

for col in df_importances3.columns:
    p = ax.bar(np.arange(-len(df_importances3), 0), df_importances3[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances3[col]
ax.set_title("Past variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances3), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances4 = feature_importances["Future variable importance over time"]
mean_attention = (
    best_tft
    .attention_weights()[best_tft.input_size :, :]
    .mean(axis=0)[: best_tft.input_size]
)
df_importances4 = df_importances4.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances4.index))

for col in df_importances4.columns:
    p = ax.bar(np.arange(-len(df_importances4), 0), df_importances4[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances4[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances4), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
best_tft.feature_importance_correlations()

#### CHRONOS

In [ ]:
# Preparing dataset
df_chronos = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos = df_chronos.dropna().reset_index(drop=True) 
df_chronos['item_id'] = df_chronos['unique_id'] 
df_chronos.drop(columns="unique_id", inplace=True) 
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Definizione delle covariate note
known_covariates_names=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id','target']]
feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

# Split train e test
train_df_chronos = df_chronos.iloc[:-h]
test_df_chronos = df_chronos.iloc[-h:]

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [ ]:
# Scaling dei dati
train_df_original = train_df_chronos.copy() 
test_df_original = test_df_chronos.copy()

train_idx = train_df_chronos.index
test_idx = test_df_chronos.index

feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

#scaler
scaler_x = MinMaxScaler()
train_df_chronos = pd.DataFrame(scaler_x.fit_transform(train_df_chronos[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_chronos = pd.DataFrame(scaler_x.transform(test_df_chronos[feature_cols]),
                            columns=feature_cols, index=test_idx)
#controllo le shape
train_df_chronos.shape, test_df_chronos.shape

TimeSeriesCV

In [ ]:
#TimeSeriesCV
h = len(test_df_chronos)  # prediction length
n_splits = 5
initial_train_size = len(train_df_chronos) - 5*h
results = []

for fold in range(n_splits):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")
    
    train_end = initial_train_size + fold * h
    test_start = train_end
    test_end = test_start + h
    train_df_fold = train_df_chronos.iloc[:train_end]
    test_df_fold = train_df_chronos.iloc[test_start:test_end]
    
    train_df_fold.reset_index(inplace=True)
    test_df_fold.reset_index(inplace=True)
    
    train_df_fold["timestamp"] = pd.to_datetime(train_df_fold["timestamp"])
    test_df_fold["timestamp"] = pd.to_datetime(test_df_fold["timestamp"])
    
    # Future timestamps 
    future_index = pd.date_range(
        start=train_df_fold["timestamp"].max() + pd.Timedelta("30min"),
        periods=h,
        freq="30min")
    
    # Preparo test set con known_covariates
    test_df_for_prediction = test_df_fold[test_df_fold["timestamp"].isin(future_index)].copy()
    test_df_for_prediction = test_df_for_prediction.set_index("timestamp")
    test_df_for_prediction = test_df_for_prediction.reindex(future_index)  # forza continuità
    test_df_for_prediction["item_id"] = train_df_fold["item_id"].iloc[0]  # supponiamo 1 sola serie
    test_df_for_prediction = test_df_for_prediction.reset_index().rename(columns={"index": "timestamp"})
    test_df_for_prediction = test_df_for_prediction.set_index(["item_id", "timestamp"])
    
    # Preparo il train con multindex
    train_df_fold = train_df_fold.set_index(["item_id", "timestamp"])
    print(f"Train: {train_df_fold.shape}, Test: {test_df_fold.shape}")
    
    # inizializzo il predictor
    predictor = TimeSeriesPredictor(
        prediction_length=h,
        target="target",
        known_covariates_names=known_covariates_names,
        eval_metric="MASE",
        freq="30min")
    
    # fit del modello
    predictor.fit(train_df_fold,
        hyperparameters={
            "Chronos": [
                {"model_path": "bolt_small", "ag_args": {"name_suffix": "ZeroShot"}},
                {
                    "model_path": "bolt_small",
                    "covariate_regressor": "CAT",
                    "target_scaler": "standard",
                    "ag_args": {"name_suffix": "WithRegressor"},
                }
            ]
        },
        time_limit=600,
        enable_ensemble=True,)
    
    predictions = predictor.predict(train_df_fold, known_covariates=test_df_for_prediction)

    test_df_with_index = test_df_fold.set_index(["item_id", "timestamp"])

    # Debug
    print(f"Forma predictions: {predictions.shape}")
    print(f"Colonne predictions: {predictions.columns}")
    print(f"Numero di indici in predictions: {len(predictions.index)}")
    print(f"Numero di indici in test_df_with_index: {len(test_df_with_index.index)}")

    # trovo solo gli indici comuni
    common_indices = predictions.index.intersection(test_df_with_index.index)
    print(f"Indici comuni: {len(common_indices)}")

    # prendo solo gli indici comuni
    y_true = test_df_with_index.loc[common_indices, "target"]
    y_pred = predictions.loc[common_indices, 'mean'].to_numpy()

    mae = calcola_mae(y_true, y_pred)
    rmse = calcola_rmse(y_true, y_pred)
    
    results.append({'split': fold+1, 'MAE_chronos': mae, 'RMSE_chronos': rmse})
    print(f"Split {fold+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
# risultati finali
results_df_chronos = pd.DataFrame(results)
results_df_chronos.set_index('split',inplace=True)
print("\n=== Backtest Results ===")
#salvo results_df_chronos
results_df_chronos.to_pickle('pickles/CHRONOS_CV_ElecDemand_multistep.pkl')
results_df_chronos

In [ ]:
#carico results_df_chronos
results_df_chronos = pd.read_pickle('pickles/CHRONOS_CV_ElecDemand_multistep.pkl')

In [ ]:
results_df_chronos

In [ ]:
# Estraggo le date
fold_data = []
train_df_chronos.reset_index(inplace=True)
n_splits = 5
for i in range(n_splits):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h
    
    if 'timestamp' in train_df_chronos.columns:
        start_date = train_df_chronos['timestamp'].iloc[test_start]
        end_date = train_df_chronos['timestamp'].iloc[test_end - 1]
    else:
        start_date = train_df_chronos.index[test_start]
        end_date = train_df_chronos.index[test_end - 1]
    
    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]
    
    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# creo dataframe finale
fold_intervals_df = pd.DataFrame(fold_data)
fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])

print("Fold Intervals DataFrame (before conversion):")
fold_intervals_df

In [ ]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_chronos['timestamp'], train_df_chronos['target'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot degli errori
results_df_chronos.reset_index(inplace=True)
ax = results_df_chronos.plot(x='split', y=['MAE_chronos','RMSE_chronos'], marker='o', title='TimeSeriesCV Errors')
ax.set_xticks(range(1, 6)) # Imposta le tacche sull'asse x da 1 a 5
plt.show()

Let's training on the whole dataset

In [ ]:
# Preparo dataset per training
predictor = TimeSeriesPredictor(
    prediction_length=len(test_df_chronos), 
    eval_metric="MASE",
    target="target",
    known_covariates_names=futr_exog_list,
    freq="30min"
)
test_known_covariates = test_df_chronos[futr_exog_list]

In [ ]:
# Fit del modello
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {
                "model_path": "bolt_small",
                "ag_args": {"name_suffix": "ZeroShot"}
            },
            {
                "model_path": "bolt_mini",
                "covariate_regressor": "NN_TORCH",
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
        ],
    },
    time_limit=600,
    enable_ensemble=True,
)
# Valutazione del modello in fase di training
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [ ]:
# PREVISIONI MULTI STEP 
predictions = predictor.predict(
    train_df_chronos,
    known_covariates=test_known_covariates  # Solo le covariate realmente note in anticipo
)
predictions.reset_index(inplace=True)

predictions_0shot = predictor.predict(
    train_df_chronos,
    known_covariates=test_known_covariates,
    model="ChronosZeroShot[bolt_small]"
)
predictions_0shot.reset_index(inplace=True)

In [ ]:
# Rescaling dei dati
target_col_position = feature_cols.index('target')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
predictions['mean'] = predictions['mean'] * (max_val - min_val) + min_val
predictions['0.1'] = predictions['0.1'] * (max_val - min_val) + min_val
predictions['0.5'] = predictions['0.5'] * (max_val - min_val) + min_val
predictions['0.9'] = predictions['0.9'] * (max_val - min_val) + min_val
predictions_0shot['mean'] = predictions_0shot['mean'] * (max_val - min_val) + min_val
predictions_0shot['0.5'] = predictions_0shot['0.5'] * (max_val - min_val) + min_val
predictions_0shot['0.9'] = predictions_0shot['0.9'] * (max_val - min_val) + min_val

In [ ]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=train_df_chronos['timestamp'], y=train_df_original['target'], mode='lines', name='Training Data'))
# Dati di test

fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='With Regressors', line=dict(color='green')))
# PREVISIONI CHRONOS 0-shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='0-shot Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="CHRONOS forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_chronos['timestamp'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [ ]:
# metriche di valutazione
test_df_original.reset_index(inplace=True)
merged_df_chronos = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']],
        on=['timestamp', 'item_id']
)
metriche_chronos = calcola_metriche(merged_df_chronos['target'],merged_df_chronos['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos").round(5)
metriche_chronos

In [ ]:
# metriche di valutazione 0 shot
merged_df_chronos_0shot = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions_0shot[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']],
        on=['timestamp', 'item_id']
)
metriche_chronos_0shot = calcola_metriche(merged_df_chronos_0shot['target'],merged_df_chronos_0shot['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos_0shot[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos 0 shot").round(5)
metriche_chronos_0shot

#### final evaluations

In [ ]:
final_metrics = metriche_xgb.join(metriche_lstm).join(metriche_np).join(metriche_chronos).join(metriche_chronos_0shot).join(metriche_tft).round(4)
final_metrics

In [ ]:
# FINAL PREDICTIONS PLOT FOR OIL PRICE
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI XGBOOST
fig.add_trace(go.Scatter(
    x=forecast_xgb['ds'], 
    y=forecast_xgb['forecast'], 
    mode='lines', 
    name='XGBoost', 
    line=dict(color='purple')
))

# PREVISIONI LSTM
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=Y_hat_df_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='pink')
))

# PREVISIONI NEURAL PROPHET
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='aquamarine')
))


# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=forecast_test['ds'], 
                         #y=Y_hat_df_tft['TFT'], 
                         y=Y_hat_df_tft,
                         mode='lines', 
    name='TFT', line=dict(color='yellow')))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS', line=dict(color='green')))
    
# PREVISIONI CHRONOS 0 shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS 0-shot', line=dict(color='lightgreen')))

# Layout del grafico
fig.update_layout(
    title="Forecasts for Electricity demand", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [ ]:
#sistemo i vari dataset prima del merge
cv_metrics_df_lstm.drop(columns=('Model'),inplace=True)
results_df_np.rename(columns={'split': 'Fold'}, inplace=True)
results_df_chronos.rename(columns={'split': 'Fold','MAE':'MAE_chronos','RMSE':'RMSE_chronos'}, inplace=True)
cv_metrics_df_xgboost.rename(columns={'MAE': 'MAE_XGB','RMSE':'RMSE_XGB'}, inplace=True)

In [ ]:
# Final plot metrics TimeSeriesCV

# faccio il merge di tutte le tabelle dei vari CV
final_cv = cv_metrics_df_lstm.merge(results_df_chronos, 
                            on="Fold",how="inner").merge(cv_metrics_df_xgboost[['MAE_XGB','RMSE_XGB','Fold']],
                            on="Fold",how="inner").merge(results_df_np,
                            on="Fold", how="inner").merge(cv_metrics_df_tft,
                            on="Fold",how="inner")
# Rescaling CV metrics data
columns_to_modify = ['MAE_nprophet', 'RMSE_nprophet', 'MAE_TFT', 'RMSE_TFT', #'MAE_lstm', 'RMSE_lstm',
                        'MAE_chronos', 'RMSE_chronos', 'MAE_XGB', 'RMSE_XGB']
for col in columns_to_modify:
    final_cv[f'{col}'] = final_cv[f'{col}'] * (max_val - min_val) + min_val

model_colors = {
    'XGB': '#9467bd',    # Purple
    'lstm': '#e377c2',       # Pink
    'nprophet': '#7fffd4',         # Aquamarine
    'TFT': '#ffff00',        # Yellow
    'chronos': '#2ca02c',    # Green
}


# Create the figure
fig = go.Figure()

# Add traces for each model - MAE (darker) and RMSE (lighter)
for model, color in model_colors.items():
    # Add MAE line (darker)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'MAE_{model}'],
        mode='lines+markers',
        name=f'{model} MAE',
        line=dict(color=color, width=3),
        marker=dict(size=8)
    ))
    
    # Add RMSE line (lighter with same color but different dash pattern)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'RMSE_{model}'],
        mode='lines+markers',
        name=f'{model} RMSE',
        line=dict(color=color, width=2, dash='dash'),
        marker=dict(size=6)
    ))

# Update layout
fig.update_layout(
    title='Model Performance Comparison (MAE and RMSE)',
    template='plotly_dark',
    xaxis=dict(title='Fold',
        tickmode='linear',
        tick0=1, dtick=1
    ),    yaxis=dict(
        title='Error Value'
    ),    legend=dict(
        orientation="v"
    ),    hovermode="closest")
fig.show()

In [ ]:
# Confronto TFT/CHRONOS
tft_chronos = metriche_chronos.join(metriche_tft)
tft_chronos                                    

### **F1 RACE**

In [0]:
df = f1[f1['Driver']=='LEC'] # prendiamo i dati di un solo pilota

In [0]:
# funzione per creare feature
def create_time_series_features(df, target_col):
    df = df.copy()
    lags = [1, 8]
    # Lag features (1-4)
    for lag in lags:
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    
    # Differencing features
    df['diff_1'] = (df[target_col] - df[target_col].shift(1)).shift(1)
    df['diff_8'] = (df[target_col] - df[target_col].shift(8)).shift(1)  # circa 1 secondo prima
    #df['diff_700'] = df[target_col] - df[target_col].shift(700) #circa il giro prima 
    
    # Caratteristiche temporali dal timestamp
    df['millisecond'] = df['ds'].dt.microsecond / 1000  # millisecondi
    df['second'] = df['ds'].dt.second
    df['minute'] = df['ds'].dt.minute
    
    # Caratteristiche cicliche per il tempo
    df['second_sin'] = np.sin(2 * np.pi * df['second']/60)
    df['second_cos'] = np.cos(2 * np.pi * df['second']/60)
    df['minute_sin'] = np.sin(2 * np.pi * df['minute']/60)
    df['minute_cos'] = np.cos(2 * np.pi * df['minute']/60)
    
    # progressione proporzionale lungo il giro
    df['lap_position'] = (df['ds'] - df.groupby('LapNumber')['ds'].transform('min')) / \
                          (df.groupby('LapNumber')['ds'].transform('max') - df.groupby('LapNumber')['ds'].transform('min'))
    # Permette di confrontare facilmente punti simili della pista tra giri diversi;
    # aiuta a identificare pattern che dipendono dalla posizione nella pista
    
    # Rolling statistics (medie mobili)
    window_sizes = [4, 8]
    for window in window_sizes:
        df[f'rolling_mean_{window}'] = df[target_col].rolling(window=window, min_periods=1).mean().shift(1)
        df[f'rolling_std_{window}'] = df[target_col].rolling(window=window, min_periods=1).std().shift(1)
    return df

In [0]:
#preparazione dataset
df = df.reset_index().rename(columns={'Date': 'ds','Speed':'y'})
df['unique_id'] = 'serie_1'
df['ds'] = pd.to_datetime(df['ds'])
df.drop(columns=['Driver','Status'],inplace=True)

##### ARIMA + ETS

In [0]:
# divisione in train e test
df_arima = df[['y','ds','unique_id']].iloc[9:]

df_arima = TimeSeriesDataFrame(
    df_arima,
    id_column='unique_id',
    timestamp_column='ds'
)
df_arima = df_arima.convert_frequency("160ms").interpolate() #qui perdo qualche osservazione
df_arima.reset_index(inplace=True)
df_arima.rename(columns={'timestamp': 'ds', 'item_id':'unique_id'}, inplace=True)

train_size = int(len(df_arima) * 0.9)
train_df = df_arima.iloc[:train_size]
test_df = df_arima.iloc[train_size:]
# queste due ci serviranno dopo
season_length = 700 # 
horizon = len(test_df) # number of predictions
train_df = pd.DataFrame(train_df)
test_df = pd.DataFrame(test_df)
train_df.shape, test_df.shape

In [0]:
# train e test plot
fig = go.Figure()
#train dat
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Train'))
#test data 
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Test'))
#layout
fig.update_layout(template='plotly_dark', title='Train and Test Data Plot')
fig.show()

In [0]:
# definisco i modelli
models = [
    AutoARIMA(), # ci dovrebbe andare season_length=season_length, ma ci mette 30min+
    AutoETS() #season_length=season_length
]
# creo il forecaster
sf = StatsForecast(models=models, freq='131ms')

In [0]:
# fit
import time
start_time = time.time()
sf.fit(train_df)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
#summary of the model ARIMA
result=sf.fitted_[0,0].model_
print(result.keys())
print("Parametri ARMA:",result['arma'])

In [0]:
arima_string(sf.fitted_[0,0].model_)

In [0]:
#summary of the model ETS
result2 = sf.fitted_[0,1].model_
print(result2.keys())
print("Modello:",result2['method'])

Cross Validation

In [0]:
#CROSS VALIDATION
crossvalidation_df = sf.cross_validation(
    df=train_df,
    h=horizon, 
    step_size=len(test_df),
    n_windows=5
)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = crossvalidation_df['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = crossvalidation_df[crossvalidation_df['cutoff'] == fold]
    
    for model in ['AutoARIMA', 'AutoETS']:
        # Filtra i dati per il modello corrente
        model_data = fold_data[['y', f'{model}']]
        model_data = model_data.dropna()
        
        # Calcola MAE e RMSE
        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data[f'{model}'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data[f'{model}'])
        
        # Aggiungi i risultati
        cv_metrics.append({
            'Fold': i+1,
            'Model': model,
            'MAE': mae,
            'RMSE': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_arima_ets = pd.DataFrame(cv_metrics)

In [0]:
# PLOT CV
#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_arima_ets)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_arima_ets.groupby('Model').mean().reset_index()
print(mean_metrics[['Model', 'MAE', 'RMSE']])

# Ora creiamo un grafico che combina la serie temporale con le metriche di errore per visualizzare dove si verificano gli errori maggiori

# Prima dobbiamo identificare gli intervalli temporali per ogni fold
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = crossvalidation_df[crossvalidation_df['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df['unique_id'].unique():
    series_data = train_df[train_df['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

# Plot MAE sul primo asse
for model in cv_metrics_df_arima_ets['Model'].unique():
    model_data = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_arima_ets['Model'].unique():
    model_data = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()

In [0]:
fold_intervals #EXPANDING WINDOW CROSS VALIDATION

Let's forecast

In [0]:
# let's forecast
Y_hat_df = sf.forecast(df=train_df, h=len(test_df), fitted=True, level=[95])
#see fitted values vs true values
values=sf.forecast_fitted_values() #qui viene aggiunta anche la vera y
#salvo i valori fittati nel training in un pickle
# with open('pickles/ARIMAETS_fit_f1.pkl', 'wb') as file:
#     pickle.dump(values, file)
values.head(2)

In [0]:
# carico i valori fittati se ne avessi bisogno 
# with open('pickles/ARIMAETS_fit_f1.pkl', 'rb') as file:
#      values = pickle.load(file)

In [0]:
# # prediction on test set
test_pred = sf.forecast(df=train_df, h=len(test_df), level=[95])
Y_hat_df = test_df.merge(Y_hat_df, how='inner', on=['unique_id', 'ds'])
# with open('pickles/ARIMAETS_pred_f1.pkl', 'wb') as file:
#     pickle.dump(Y_hat_df, file)

CARICO LE PREVISIONI

In [0]:
Y_hat_df = pd.read_pickle('pickles/ARIMAETS_pred_f1.pkl')

In [0]:
# Calcola metriche
metriche_arima = calcola_metriche(Y_hat_df['y'],Y_hat_df['AutoARIMA'],train_df['y'], 
                                modelname="ARIMA").round(5)
metriche_ets = calcola_metriche(Y_hat_df['y'],Y_hat_df['AutoETS'],train_df['y'], 
                                modelname="ETS").round(5)
metriche_arima, metriche_ets

In [0]:
# Forecast plot
# Creiamo un dataframe completo che include training, test e previsioni future
full_df = pd.concat([train_df, test_df], axis=0)
# Aggiungiamo le previsioni al dataframe completo
forecast_df_etsA = full_df.merge(Y_hat_df, how='outer', on=['unique_id', 'ds'])

fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Training Data'))

# Dati di test
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(x=test_pred['ds'], y=test_pred['AutoARIMA'], mode='lines', 
    name='AutoARIMA Forecast', line=dict(color='green')))

# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(x=test_pred['ds'], y=test_pred['AutoETS'], mode='lines', 
    name='AutoETS Forecast', line=dict(color='orange', dash='dot')))

# Layout del grafico
fig.update_layout(title="Forecast Comparison: AutoARIMA vs AutoETS", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

##### XGBOOST

In [0]:
# df_xgb
df_xgb = create_time_series_features(df, target_col='y') 
df_xgb = df_xgb.dropna().reset_index(drop=True) #droppo i NaN creati dai lag
df_xgb = TimeSeriesDataFrame(
    df_xgb,
    id_column='unique_id',
    timestamp_column='ds'
)
df_xgb = df_xgb.convert_frequency("160ms").interpolate() #qui perdo qualche osservazione
df_xgb.reset_index(inplace=True)
df_xgb.rename(columns={'timestamp': 'ds', 'item_id':'unique_id'}, inplace=True)
df_xgb.drop(columns=['unique_id'],inplace=True)
print(df_xgb.columns)
# df_xgb.head()

In [0]:
# Split train/test
train_size = int(len(df_xgb) * 0.9)
train_df_xgb = df_xgb.iloc[:train_size]
test_df_xgb = df_xgb.iloc[train_size:]

# Separazione features e target
feature_cols = [col for col in df_xgb.columns if col not in ['y', 'ds']]
X_train_xgb, y_train_xgb = train_df_xgb[feature_cols], train_df_xgb['y']
X_test_xgb, y_test_xgb = test_df_xgb[feature_cols], test_df_xgb['y']
X_train_xgb.shape, y_train_xgb.shape, X_test_xgb.shape, y_test_xgb.shape

In [0]:
# Scaling dei dati

#features
scaler_x = MinMaxScaler()
X_train_scaled = scaler_x.fit_transform(X_train_xgb)
X_test_scaled = scaler_x.transform(X_test_xgb)

# target, non serve che scalo y_test
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train_xgb.values.reshape(-1, 1))

# Ricreo df con stessi indici e nomi
X_train_xgb = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train_xgb.index)
X_test_xgb = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test_xgb.index)
y_train_xgb = pd.Series(y_train_scaled.flatten(), name='y', index=y_train_xgb.index)

#aggiorno train_df_xgb
train_df_xgb[feature_cols] = X_train_xgb
train_df_xgb['y'] = y_train_xgb

#controllo le shape
X_train_xgb.shape, X_test_xgb.shape, y_train_xgb.shape, y_test_xgb.shape

Ottimizzazione Optuna (Bayesiana)

In [0]:
# Optuna function
cv_metrics_log = []  # List globale per salvare metriche fold per fold

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'verbosity': 0,
        'seed': 42
    }

    tscv = TimeSeriesSplit(n_splits=5,test_size=len(test_df_xgb))
    y = train_df_xgb['y']
    df_xgb_feature = train_df_xgb.drop(columns=['y','ds'])
    all_rmse = []

    for fold, (train_idx, test_idx) in enumerate(tscv.split(df_xgb_feature)):
        X_train, X_test = df_xgb_feature.iloc[train_idx], df_xgb_feature.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        dtest = xgb.DMatrix(X_test.values, label=y_test.values)

        model = xgb.train(params, dtrain, num_boost_round=500,
                          evals=[(dtest, 'eval')],
                          early_stopping_rounds=30, verbose_eval=False)

        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mae = calcola_mae(y_test, preds)  # tua funzione custom

        all_rmse.append(rmse)

        # Recupera le date (se esiste colonna 'ds')
        start_date = df_xgb.iloc[test_idx]['ds'].min() if 'ds' in df_xgb.columns else None
        end_date = df_xgb.iloc[test_idx]['ds'].max() if 'ds' in df_xgb.columns else None

        # Logga le metriche della fold
        cv_metrics_log.append({
            'Trial': trial.number,
            'Fold': fold + 1,
            'MAE': mae,
            'RMSE': rmse,
            'start_date': start_date,
            'end_date': end_date,
            'Model': 'XGBoost'  # Puoi modificarlo se usi più modelli
        })
    return np.mean(all_rmse)

In [0]:
# Optuna optimization
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=2))
study.optimize(objective, n_trials=50) #aumentare su databricks

In [0]:
# Best parameters
print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)
print("Best trial:",study.best_trial.number)
besttrial = study.best_trial.number

In [0]:
cv_metrics_log = pd.DataFrame(cv_metrics_log)
cv_metrics_log = cv_metrics_log[cv_metrics_log['Trial']==besttrial]
cv_metrics_log

Vediamo come ha performato nel miglior trial

In [0]:
#PLOT CV
cv_metrics_df_xgboost = cv_metrics_log.copy()

# Stampa le metriche per modello
print("Metriche per modello:")
print(cv_metrics_df_xgboost)

# Calcola e stampa le metriche medie per modello
mean_metrics = cv_metrics_df_xgboost.mean(numeric_only=True).reset_index()

# Crea il DataFrame con gli intervalli delle fold
fold_intervals_df = cv_metrics_df_xgboost[['Fold', 'start_date', 'end_date']].drop_duplicates()

# Ora creiamo il grafico combinato
plt.figure(figsize=(16, 12))

# === PLOT 1: Serie temporale originale ===
plt.subplot(2, 1, 1)
#for unique_id in train_df_xgb['unique_id'].unique():
series_data = train_df_xgb.copy()
plt.plot(series_data['ds'], series_data['y'], label='Serie unica')

# Evidenzia le fold con colori
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)],
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10,
             bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# === PLOT 2: Metriche MAE e RMSE ===
plt.subplot(2, 1, 2)
ax1 = plt.gca()
ax2 = ax1.twinx()

# MAE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# RMSE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Legenda combinata
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


Training coi migliori parametri trovati da Optuna

In [0]:
# #salvo i best params
# best_params = study.best_params
# with open ('pickles/XGB_bestparams_f1.pkl', 'wb') as file:
#     pickle.dump(best_params, file)

In [0]:
#carico i best_params
best_params = pd.read_pickle('pickles/XGB_bestparams_f1.pkl')
best_params.update({'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 22})

dtrain_xgb = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
start_time = time.time()
#fit 
model = xgb.train(best_params, dtrain_xgb, num_boost_round=500)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Plot feature importance
ax = xgb.plot_importance(model, importance_type='gain', max_num_features=10)
for text in ax.texts:
    text.set_visible(False)
plt.title('Feature Importance (Gain)')
plt.tight_layout()
plt.show()

MULTI-STEP FORECAST

In [0]:
def direct_multistep_forecast(train_data, feature_cols, target_col, horizon=None):
    """
    Addestra modelli separati per ogni orizzonte temporale futuro
    """
    forecasts = []
    models = []
    
    # Crea dataframe per date future
    last_date = train_data['ds'].max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta('131ms'), periods=horizon, freq='131ms') #dopo modificare
    
    # Per ogni step futuro, addestra un modello dedicato
    for h in range(1, horizon+1):
        print(f"Training model for horizon {h}")
        
        # Prepara target con shift inverso per prevedere h passi avanti
        df_horizon = train_data.copy()
        df_horizon[f'y_horizon_{h}'] = df_horizon[target_col].shift(-h)
        df_horizon = df_horizon.dropna()
        
        # Prendi features e target per questo orizzonte
        X = df_horizon[feature_cols]
        y = df_horizon[f'y_horizon_{h}']
        
        # Split train/validation
        train_size = int(len(X) * 0.95)
        X_train, X_val = X.iloc[:train_size], X.iloc[train_size:]
        y_train, y_val = y.iloc[:train_size], y.iloc[train_size:]
        
        # Addestra modello
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        params = best_params
        model = xgb.train(
            params,
            dtrain,
            evals=[(dtrain, 'train'), (dval, 'val')],
            num_boost_round=30,
            early_stopping_rounds=4,
            verbose_eval=False
        )
        models.append(model)
        
    # Crea input per la previsione (l'ultimo punto noto)
    last_point = xgb.DMatrix(train_data.iloc[[-1]][feature_cols])
    
    # Prevedi ciascun orizzonte con il modello dedicato
    for i, model in enumerate(models):
        pred = model.predict(last_point)[0]
        forecasts.append(pred)
    return pd.DataFrame({'ds': future_dates, 'forecast': forecasts})

In [0]:
# #Forecasting, took 20/30 minutes
# forecast_xgb = direct_multistep_forecast(train_df_xgb, feature_cols, target_col='y',horizon=len(test_df_xgb)+700)
# with open('pickles/XGB_pred_f1.pkl', 'wb') as file:
#     pickle.dump(forecast_xgb, file)

In [0]:
# # carico le previsioni
# with open('pickles/XGB_pred_f1.pkl', 'rb') as file:
#     forecast_xgb = pickle.load(file)

RECURSIVE FORECAST

In [0]:
dtest_xgb = xgb.DMatrix(X_test_xgb)
forecast_xgb = pd.DataFrame(model.predict(dtest_xgb),columns=['forecast'])
forecast_xgb['ds'] = test_df_xgb['ds'].values

Rescaling

In [0]:
# RE-Scaling dei dati
forecast_xgb['forecast'] = scaler_y.inverse_transform(forecast_xgb[['forecast']])
train_df_xgb['y'] = scaler_y.inverse_transform(train_df_xgb[['y']])

In [0]:
# Plot forecast
fig = go.Figure()
# Training data
fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=train_df_xgb['y'], mode='lines', name='Training'))
# Test data
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=test_df_xgb['y'], mode='lines', name='Test'))
# Forecast
fig.add_trace(go.Scatter(x=forecast_xgb['ds'], y=forecast_xgb['forecast'], mode='lines', 
                         name='Forecast', line=dict(color='deepskyblue')))
# Forecast dirette
fig.update_layout(title='Forecast vs Real Data', template='plotly_dark')
fig.show()

Calcolo metriche

In [0]:
# Calcola metriche
merged_df = pd.merge(test_df_xgb[['ds', 'y']], forecast_xgb, on='ds', how='left')
metriche_xgb = calcola_metriche(merged_df['y'],merged_df['forecast'],
                                train_df_xgb['y'], modelname="XGBoost")
metriche_xgb

##### LSTM

In [0]:
# divisione in train e test
df_lstm = create_time_series_features(df, target_col='y') 
df_lstm = df_lstm.dropna().reset_index(drop=True)
df_lstm = TimeSeriesDataFrame(
    df_lstm,
    id_column='unique_id',
    timestamp_column='ds'
)
df_lstm = df_lstm.convert_frequency("160ms").interpolate() #qui perdo qualche osservazione
df_lstm.reset_index(inplace=True)
df_lstm.rename(columns={'timestamp': 'ds', 'item_id':'unique_id'}, inplace=True)

train_size = int(len(df_lstm) * 0.9)
train_df_lstm = df_lstm.iloc[:train_size]
test_df_lstm = df_lstm.iloc[train_size:]
print(train_df_lstm.columns)
train_df_lstm.head()

In [0]:
train_df_lstm.shape, test_df_lstm.shape

CV da lanciare solo su Databricks, qui si blocca tutto

In [0]:
# Optuna function
futr_exog_list = train_df_lstm.drop(columns=['unique_id', 'ds', 'y']).columns.tolist()
folds = pd.DataFrame()
h = len(test_df_lstm)
cv_metrics_df_lstm = pd.DataFrame()
def objective(trial):
    cv_metrics = []
    # Hyperparametri da ottimizzare
    encoder_n_layers = trial.suggest_int("encoder_n_layers", 1, 4)
    encoder_hidden_size = trial.suggest_categorical("encoder_hidden_size", [32, 256])
    decoder_hidden_size = trial.suggest_categorical("decoder_hidden_size", [32, 256])
    decoder_layers = trial.suggest_int("decoder_layers", 1, 4)
    learning_rate = trial.suggest_float("learning_rate", 1e-3, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [128,1024])

    # Fisso il seed per riproducibilità
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

    # Definizione modello LSTM
    lstm = LSTM(
        h=h,  # nel tuning
        input_size=h,
        loss=DistributionLoss(distribution="Normal", level=[95]),
        scaler_type='robust',
        encoder_n_layers=encoder_n_layers,
        encoder_hidden_size=encoder_hidden_size,
        decoder_hidden_size=decoder_hidden_size,
        decoder_layers=decoder_layers,
        futr_exog_list=futr_exog_list,
        max_steps=20, 
        batch_size=batch_size,
        learning_rate=learning_rate,
        recurrent=False
    )

    nf = NeuralForecast(models=[lstm], freq='160ms')

    # cross-validation per tuning
    df_cv = nf.cross_validation(
        df=train_df_lstm,
        step_size=h,
        n_windows=5
    )

    # Calcola MAE per ogni fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'LSTM']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'LSTM',
            'MAE_lstm': mae,
            'RMSE_lstm': rmse
        })

    # Media MAE su tutte le fold
    cv_metrics_df_lstm = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_lstm['MAE_lstm'].mean()

    return mean_mae 

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=10) 

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
# # salvo i best params trovati da Optuna
with open('pickles/LSTM_bestPar_F1.pkl', 'wb') as file:
    pickle.dump(best_params, file)

Carico i best params

In [0]:
with open('pickles/LSTM_bestPar_F1.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [0]:
# final model
h = len(test_df_lstm)
best_lstm = LSTM(
    h=h,
    input_size=h,
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=50,
    #early_stop_patience_steps=20,
    batch_size=2048,
    learning_rate=best_params['learning_rate'],
    recurrent=False
)
nf = NeuralForecast(models=[best_lstm], freq='160ms')

In [0]:
# final CV
# df_cv = nf.cross_validation(
#     df=train_df_lstm[10000:], #la CPU non regge tutte le unità
#     step_size=len(test_df_lstm),
#     n_windows=5
# )
# with open('pickles/LSTM_CV_F1.pkl', 'wb') as file:
#      pickle.dump(df_cv, file)

with open('pickles/LSTM_CV_F1.pkl', 'rb') as file:
    df_cv = pickle.load(file)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'LSTM']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'LSTM',
        'MAE_lstm': mae,
        'RMSE_lstm': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_lstm = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_lstm)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_lstm.groupby('Model').mean().reset_index()
print(mean_metrics)

In [0]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_lstm['unique_id'].unique():
    series_data = train_df_lstm[train_df_lstm['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_lstm['Model'] = 'lstm'
# Plot MAE sul primo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_lstm'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_lstm'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()

In [0]:
# Final training
h = len(test_df_lstm)
# Fisso il seed per riproducibilità
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

final_model = LSTM(
    h=h,
    input_size=h,
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=best_params['encoder_n_layers'],
    encoder_hidden_size=best_params['encoder_hidden_size'],
    decoder_hidden_size=best_params['decoder_hidden_size'],
    decoder_layers=best_params['decoder_layers'],
    futr_exog_list=futr_exog_list,
    max_steps=100,
    #early_stop_patience_steps=20,
    batch_size=256,
    learning_rate=0.01,
    recurrent=False
)
nf = NeuralForecast(models=[final_model], freq='160ms')

start_time = time.time()
#fit 
nf.fit(
    df=train_df_lstm[12000:], #la cpu non regge tutto il dataset
    val_size=len(test_df_lstm)
    )
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

Previsioni

In [0]:
#predictions
Y_hat_df_lstm = nf.predict(futr_df=test_df_lstm)
#salvo in pickle
with open('pickles/LSTM_F1.pkl', 'wb') as file:
    pickle.dump(Y_hat_df_lstm, file)

In [0]:
#carico le previsioni:
with open('pickles/LSTM_f1.pkl', 'rb') as file:
    Y_hat_df_lstm = pickle.load(file)
#queste previsioni sono state generate con 'max_steps' = 50, ma ci mette 15 minuti

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_lstm['ds'], y=train_df_lstm['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_lstm['ds'], y=test_df_lstm['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM'], mode='lines', 
    name='LSTM Forecast', line=dict(color='green')))

# PREVISIONI MEDIANA LSTM
# fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM-median'], mode='lines', 
#     name='MEDIAN Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="LSTM forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_lstm['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")

fig.show()

In [0]:
# Calcola metriche
merged_df_lstm = pd.merge(test_df_lstm[['ds', 'y']], Y_hat_df_lstm, on='ds', how='left')
#merged_df_lstm.dropna()
metriche_lstm = calcola_metriche(merged_df_lstm['y'],merged_df_lstm['LSTM'],train_df_lstm['y'],modelname="LSTM")
metriche_lstm

##### NEURAL PROPHET

In [0]:
df_np = create_time_series_features(df, target_col='y') 
df_np = df_np.dropna().reset_index(drop=True)
df_np = TimeSeriesDataFrame(
    df_np,
    id_column='unique_id',
    timestamp_column='ds'
)
df_np = df_np.convert_frequency("160ms").interpolate() #qui perdo qualche osservazione
df_np.reset_index(inplace=True)
df_np.rename(columns={'timestamp': 'ds', 'item_id':'unique_id'}, inplace=True)
#metto come booleana la variabile brake
df_np['Brake'] = df_np['Brake'].astype(int)
df_np.drop(columns="unique_id",inplace=True)
df_np.columns

In [0]:
train_size = int(len(df_np) * 0.9)
train_df_np = df_np.iloc[:train_size]
test_df_np = df_np.iloc[train_size:]

In [0]:
# Scaling dei dati
train_df_original = train_df_np.copy()
test_df_original = test_df_np.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_np['ds']
test_meta = test_df_np['ds']

feature_cols = [col for col in train_df_np.columns if col not in ['ds']]
train_idx = train_df_np.index
test_idx = test_df_np.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_np[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_np[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_np = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_np = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_np.shape, test_df_np.shape

TimeSeriesCV

In [0]:
# preparazione dataset
np_df_train = train_df_np[['ds', 'y']].copy()
np_df_test = test_df_np[['ds', 'y']].copy()

# Aggiungi le colonne dei regressori
regressors = ['DistanceToDriverAhead', 'RPM', 'nGear', 'Throttle', 'Brake',
       'DRS', 'LapNumber', 'lag_1', 'lag_8', 'diff_1', 'diff_8', 'millisecond',
       'second', 'minute', 'second_sin', 'second_cos', 'minute_sin',
       'minute_cos', 'lap_position', 'rolling_mean_4', 'rolling_std_4',
       'rolling_mean_8', 'rolling_std_8']

for reg in regressors:
    np_df_train[reg] = train_df_np[reg]
    np_df_test[reg] = test_df_np[reg]
# Definisci il modello
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.001, 
    batch_size=16,
    daily_seasonality=True,
    weekly_seasonality=True,
    loss_func='Huber'
)

# Aggiungi i regressori
for reg in regressors:
    neuralprophet.add_future_regressor(reg)

In [0]:
# TIME SERIES CROSS VALIDATION
initial_train_size = len(train_df_np)-5*len(test_df_np)
h = len(test_df_np)
n_windows = 5

# Verifica
required_length = initial_train_size + h * n_windows
if len(np_df_train) < required_length:
    raise ValueError("Dataset troppo corto per questo schema di cross-validation.")

# Reset risultati
results = []

for i in range(n_windows):
    print(f"\n--- Fold {i+1}/{n_windows} ---")

    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    train_window = np_df_train.iloc[:train_end]
    test_window = np_df_train.iloc[test_start:test_end]

    print(f"Train: {train_window.shape}, Test: {test_window.shape}")

    # Modello
    model = NeuralProphet(
        quantiles=[0.025, 0.975],
        learning_rate=0.001,
        batch_size=64,
        daily_seasonality=True,
        weekly_seasonality=True,
        loss_func='Huber',
        global_normalization=True,
        unknown_data_normalization=True
    )

    for reg in regressors:
        model.add_future_regressor(reg)

    # Fit
    _ = model.fit(train_window, freq="15min", epochs=100)

    # Previsione
    forecast = model.predict(test_window)

    # Metriche
    y_true = test_window['y'].values
    y_pred = forecast['yhat1'].values

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    results.append({'split': i+1, 'MAE_np': mae, 'RMSE_np': rmse})
    print(f"Split {i+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# Risultati
results_df_np = pd.DataFrame(results)
print("\n=== Backtest Results ===")
print(results_df_np)

# Plot
results_df_np.plot(x='split', y=['MAE_np', 'RMSE_np'], marker='o', title='Backtesting Errors')


In [0]:
# Salvo results df CV
# with open('pickles/NP_CV_F1.pkl', 'wb') as file:
#     pickle.dump(results_df_np, file)

In [0]:
# Carico i risultati della CV
with open('pickles/NP_CV_F1.pkl', 'rb') as file:
    results_df_np = pickle.load(file)

In [0]:
# Mi ricavo le date
initial_train_size = len(train_df_np)-5*len(test_df_np)
h = len(test_df_np)
n_windows = 5

# Crea lista per i dati delle fold
fold_data = []

for i in range(n_windows):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    # Ricava date da colonna 'ds' o dall'indice
    if 'ds' in np_df_train.columns:
        start_date = np_df_train['ds'].iloc[test_start]
        end_date = np_df_train['ds'].iloc[test_end - 1]
    else:
        start_date = np_df_train.index[test_start]
        end_date = np_df_train.index[test_end - 1]

    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]

    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)

# Converto le date in datetime se necessario
if all(isinstance(date, str) for date in fold_intervals_df['start_date']):
    fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
    fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])

print("Fold Intervals DataFrame:")
print(fold_intervals_df)

In [0]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_np['ds'], train_df_np['y'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

results_df_np.plot(x='split', y=['MAE_np', 'RMSE_np'], marker='o', title='Backtesting Errors')

Now let's training on the whole dataset

In [0]:
# preparazione dataset
np_df_train = train_df_np[['ds', 'y']].copy()
np_df_test = test_df_np[['ds', 'y']].copy()

# Aggiungi le colonne dei regressori
regressors = ['DistanceToDriverAhead', 'RPM', 'nGear', 'Throttle', 'Brake',
       'DRS', 'LapNumber', 'lag_1', 'lag_8', 'diff_1', 'diff_8', 'millisecond',
       'second', 'minute', 'second_sin', 'second_cos', 'minute_sin',
       'minute_cos', 'lap_position', 'rolling_mean_4', 'rolling_std_4',
       'rolling_mean_8', 'rolling_std_8']

for reg in regressors:
    np_df_train[reg] = train_df_np[reg]
    np_df_test[reg] = test_df_np[reg]
    

# Fisso il seed per riproducibilità
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


# Definisci il modello
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],
    learning_rate=0.01,
    batch_size=16,
    loss_func='Huber'
)

# Aggiungi i regressori
for reg in regressors:
    neuralprophet.add_future_regressor(reg)

In [0]:
# FIT
start_time = time.time()
metrics_df = neuralprophet.fit(np_df_train, freq="160ms", epochs=100)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
forecast_test = neuralprophet.predict(np_df_test)
# #salvo le prediction sul test in locale
with open('pickles/NP_f1.pkl', 'wb') as file:
    pickle.dump(forecast_test, file)

CARICO LE PREVISIONI DEL TEST SET

In [0]:
#carico le previsioni del test
with open('pickles/NP_f1.pkl', 'rb') as file:
    forecast_test = pickle.load(file)

In [0]:
#Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
forecast_test['yhat1'] = forecast_test['yhat1'] * (max_val - min_val) + min_val
forecast_test['yhat1 2.5%'] = forecast_test['yhat1 2.5%'] * (max_val - min_val) + min_val
forecast_test['yhat1 97.5%'] = forecast_test['yhat1 97.5%'] * (max_val - min_val) + min_val

In [0]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))

# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)
fig.show()

In [0]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast_test['ds'], 'forecast': forecast_test['yhat1']})
merged_df = pd.merge(test_df_original[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
mae_val = mean_absolute_error(merged_df['y'], merged_df['forecast'])
metriche_np = calcola_metriche(merged_df['y'], merged_df['forecast'],train_df_original['y'],
                               modelname="NeuralProphet").round(10)
metriche_np

#### TFT

In [0]:
# Preparing dataset
df_tft = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_tft = df_tft.dropna().reset_index(drop=True) 
df_tft['item_id'] = df_tft['unique_id'] 
df_tft.drop(columns="unique_id", inplace=True) 
df_tft.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_tft = TimeSeriesDataFrame(
    df_tft,
    id_column='item_id',
    timestamp_column='timestamp'
)
df_tft = df_tft.convert_frequency("160ms").interpolate()
df_tft.reset_index(inplace=True)
df_tft.rename(columns={'timestamp': 'ds','item_id':'unique_id','target':'y'}, inplace=True)
train_size = int(len(df_tft) * 0.9)
train_df_tft = df_tft.iloc[:train_size]
test_df_tft = df_tft.iloc[train_size:]
train_df_tft.shape, test_df_tft.shape

In [0]:
# Scaling dei dati
#Salvo per dopo
train_df_original = train_df_tft.copy() 
test_df_original = test_df_tft.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_tft[['ds', 'unique_id']]
test_meta = test_df_tft[['ds', 'unique_id']]

feature_cols = [col for col in train_df_tft.columns if col not in ['ds', 'unique_id']]
train_idx = train_df_tft.index
test_idx = test_df_tft.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_tft[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_tft[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_tft = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_tft = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_tft.shape, test_df_tft.shape

In [0]:
# carico la cv
cv_metrics_df_tft = pd.read_pickle("pickles/cv_metrics_TFT_F1.pkl")
cv_metrics_df_tft.plot(x='Fold', y=['MAE_tft', 'RMSE_tft'], marker='o', title='Chronos Backtesting Errors')

##### MAIN Training (Colab)

In [0]:
# Training per GPU

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

h = len(test_df_tft)
futr_exog_list = train_df_tft.drop(columns=['unique_id', 'ds', 'y']).columns.tolist()

# Configurazione conservativa per la memoria
best_tft = TFT(
    h=h,  
    input_size=800, #should be 800
    hidden_size=8, 
    n_rnn_layers=1,
    rnn_type='lstm',
    grn_activation='ReLU',
    loss=DistributionLoss(distribution="StudentT", level=[90]),
    learning_rate=0.01,
    futr_exog_list=futr_exog_list,
    max_steps=200,
    batch_size=4,  
    early_stop_patience_steps=-1,
    scaler_type="standard",
    enable_progress_bar=True,
    accelerator="auto",
    one_rnn_initial_state=False,
    windows_batch_size=4,  
    start_padding_enabled=True
)
nf = NeuralForecast(models=[best_tft], freq="160ms")
# Pulire la memoria GPU prima di iniziare IL FIT
torch.cuda.empty_cache()
gc.collect()
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
start_time = time.time()

#fit 
nf.fit(df=train_df_tft[15000:], # nel training scartato 15k unità per memoria GPU
       val_size=len(test_df_tft)) 

end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

  | Name                    | Type                     | Params | Mode \
-----------------------------------------------------------------------------\
0 | loss                    | DistributionLoss         | 3      | train\
1 | padder_train            | ConstantPad1d            | 0      | train\
2 | scaler                  | TemporalNorm             | 0      | train\
3 | embedding               | TFTEmbedding             | 384    | train\
4 | temporal_encoder        | TemporalCovariateEncoder | 28.7 K | train\
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 1.2 K  | train\
6 | output_adapter          | Linear                   | 27     | train\
-----------------------------------------------------------------------------\
30.3 K    Trainable params\
3         Non-trainable params\
30.3 K    Total params\
0.121     Total estimated model params size (MB)\
440       Modules in train mode\

In [0]:
# Predict
Y_hat_df_tft = nf.predict(futr_df=test_df_tft)

Carico le previsioni fatte da Colab

In [0]:
Y_hat_df_tft = pd.read_csv('pickles/TFT_F1_best.csv')
Y_hat_df_tft['ds'] = pd.to_datetime(Y_hat_df_tft['ds'])

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
Y_hat_df_tft['TFT'] = Y_hat_df_tft['TFT'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-median'] = Y_hat_df_tft['TFT-median'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-lo-90'] = Y_hat_df_tft['TFT-lo-90'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-hi-90'] = Y_hat_df_tft['TFT-hi-90'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_tft['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_tft['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='lightgreen')))

# QUANTILI TFT
# Fascia tra P10 e P90
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_tft['ds']) + 
    list(Y_hat_df_tft['ds'][::-1]),
    y=list(Y_hat_df_tft['TFT-hi-90']) + list(Y_hat_df_tft['TFT-lo-90'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="TFT forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_tft['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# Calcola metriche
merged_df_tft = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_tft, on='ds', how='left')
merged_df_tft.rename(columns={'TFT-lo-90': '0.1','TFT-median':'0.5', 'TFT-hi-90':'0.9'},inplace=True)
#merged_df_tft.dropna(inplace=True)
metriche_tft = calcola_metriche(merged_df_tft['y'],merged_df_tft['TFT'],train_df_original['y'],
                                y_pred_quantiles=merged_df_tft[['0.1','0.5','0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="TFT").round(10)
metriche_tft

##### TFT Explainability

In [0]:
best_tft = torch.load('pickles/best_tft_F1_model.pth', map_location=torch.device('cpu'))

In [0]:
attention = best_tft.attention_weights()
def plot_attention(
    self, plot: str = "time", output: str = "plot", width: int = 800, height: int = 400
):
    """
    Plot the attention weights.

    Args:
        plot (str, optional): The type of plot to generate. Can be one of the following:
            - 'time': Display the mean attention weights over time.
            - 'all': Display the attention weights for each horizon.
            - 'heatmap': Display the attention weights as a heatmap.
            - An integer in the range [1, model.h) to display the attention weights for a specific horizon.
        output (str, optional): The type of output to generate. Can be one of the following:
            - 'plot': Display the plot directly.
            - 'figure': Return the plot as a figure object.
        width (int, optional): Width of the plot in pixels. Default is 800.
        height (int, optional): Height of the plot in pixels. Default is 400.

    Returns:
        matplotlib.figure.Figure: If `output` is 'figure', the function returns the plot as a figure object.
    """

    attention = (
        self.mean_on_batch(self.interpretability_params["attn_wts"])
        .mean(dim=0)
        .cpu()
        .numpy()
    )

    fig, ax = plt.subplots(figsize=(width / 100, height / 100))

    if plot == "time":
        attention = attention[self.input_size :, :].mean(axis=0)
        ax.plot(np.arange(-self.input_size, self.h), attention)
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Mean Attention")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "all":
        for i in range(self.input_size, attention.shape[0]):
            ax.plot(
                np.arange(-self.input_size, self.h),
                attention[i, :],
                label=f"horizon {i-self.input_size+1}",
            )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Attention per horizon")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "heatmap":
        cax = ax.imshow(
            attention,
            aspect="auto",
            cmap="viridis",
            extent=[-self.input_size, self.h, -self.input_size, self.h],
        )
        fig.colorbar(cax)
        ax.set_title("Attention Heatmap")
        ax.set_xlabel("Attention (current time step)")
        ax.set_ylabel("Attention (previous time step)")

    elif isinstance(plot, int) and (plot in np.arange(1, self.h + 1)):
        i = self.input_size + plot - 1
        ax.plot(
            np.arange(-self.input_size, self.h),
            attention[i, :],
            label=f"horizon {plot}",
        )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title(f"Attention weight for horizon {plot}")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    else:
        raise ValueError(
            'plot has to be in ["time","all","heatmap"] or integer in range(1,model.h)'
        )

    plt.tight_layout()

    if output == "plot":
        plt.show()
    elif output == "figure":
        return fig
    else:
        raise ValueError(f"Invalid output: {output}. Expected 'plot' or 'figure'.")

In [0]:
# MEAN ATTENTION
plot_attention(best_tft, plot="time")
# pesi di attenzione medi nel tempo su tutti gli orizzonti di previsione.
# La curva blu rappresenta quanto il modello ha "guardato" (attention) 
# quel punto temporale in input durante la previsione.
# Valori più alti significano che il modello ritiene quel punto temporale
# particolarmente rilevante per fare le previsioni

In [0]:
# ATTENTION OF A SPECIFIC TIME STEPS
plot_attention(best_tft, plot=44)
# pesi di attenzione per ogni orizzonte di previsione separatamente.

In [0]:
plot_attention(best_tft, plot="heatmap")

In [0]:
# PAST VARIABLE IMPORTANCES
feature_importances = best_tft.feature_importances()
print(feature_importances.keys())
print("\n ==========PAST FEATURE IMPORTANCES==========")
feature_importances["Past variable importance over time"].mean().sort_values().plot(kind="barh")

Questo grafico mostra quanto il modello ha fatto affidamento sulle feature passate \
(quelle che il modello ha potuto osservare prima del punto di previsione). Serve a costruire il contesto dell'input

In [0]:
# FUTURE VARIABLE IMPORTANCES
print("\n ==========FUTURE FEATURE IMPORTANCES==========")
feature_importances["Future variable importance over time"].mean().sort_values().plot(kind="barh")

Mostra quanto il modello ha sfruttato le feature che sono servite a guidare le previsioni passo-passo nel decoder

DIFFERENZA TRA PAST E FUTURE: \
La diversa importanza riflette quanto ciascuna variabile aiuta nell’analisi del contesto passato (encoder -->> PAST) \
o nella generazione delle previsioni (decoder -->> FUTURE).

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME
df_importances = feature_importances["Past variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances.index))

for col in df_importances.columns:
    p = ax.bar(np.arange(-len(df_importances), 0), df_importances[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances[col]
ax.set_title("Past variable importance over time")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)

plt.show()

Questo grafico mostra QUANDO (e QUANTO) nel passato ogni feature è stata più utile per costruire il contesto predittivo.

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME
df_importances2 = feature_importances["Future variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances2.index))
for col in df_importances2.columns:
    p = ax.bar(np.arange(-len(df_importances2), 0), df_importances2[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances2[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.grid(True)
ax.legend()
plt.show()

Per ogni feature viene mostrato quanto è stata importante per il modello a ciascun orizzonte futuro

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances3 = feature_importances["Past variable importance over time"]
mean_attention = (
    best_tft
    .attention_weights()[best_tft.input_size :, :]
    .mean(axis=0)[: best_tft.input_size]
)
df_importances3 = df_importances3.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances3.index))

for col in df_importances3.columns:
    p = ax.bar(np.arange(-len(df_importances3), 0), df_importances3[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances3[col]
ax.set_title("Past variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances3), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances4 = feature_importances["Future variable importance over time"]
mean_attention = (
    best_tft
    .attention_weights()[best_tft.input_size :, :]
    .mean(axis=0)[: best_tft.input_size]
)
df_importances4 = df_importances4.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances4.index))

for col in df_importances4.columns:
    p = ax.bar(np.arange(-len(df_importances4), 0), df_importances4[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances4[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances4), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
best_tft.feature_importance_correlations()

#### CHRONOS

In [0]:
# Preparing dataset
df_chronos = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos = df_chronos.dropna().reset_index(drop=True) 
df_chronos['item_id'] = df_chronos['unique_id'] 
df_chronos.drop(columns="unique_id", inplace=True) 
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)
df_chronos = df_chronos.convert_frequency("160ms").interpolate()
df_chronos.columns

In [0]:
# Split train e test
known_covariates_names = [
    'DistanceToDriverAhead', 'RPM', 'nGear', 'Throttle', 'Brake',
       'DRS', 'LapNumber', 'lag_1', 'lag_8', 'diff_1', 'diff_8', 'millisecond',
       'second', 'minute', 'second_sin', 'second_cos', 'minute_sin',
       'minute_cos', 'lap_position', 'rolling_mean_4', 'rolling_std_4',
       'rolling_mean_8', 'rolling_std_8']

# Split train e test
train_size = int(len(df_chronos) * 0.9)
train_df_chronos = df_chronos.iloc[:train_size]
test_df_chronos = df_chronos.iloc[train_size:]

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [0]:
# Scaling dei dati
train_df_original = train_df_chronos.copy() 
test_df_original = test_df_chronos.copy()

train_idx = train_df_chronos.index
test_idx = test_df_chronos.index

feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

#scaler
scaler_x = MinMaxScaler()
train_df_chronos = pd.DataFrame(scaler_x.fit_transform(train_df_chronos[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_chronos = pd.DataFrame(scaler_x.transform(test_df_chronos[feature_cols]),
                            columns=feature_cols, index=test_idx)
#controllo le shape
train_df_chronos.shape, test_df_chronos.shape

TimeSeriesCV

In [0]:
#TimeSeriesCV
h = len(test_df_chronos)  # prediction length
n_splits = 5
initial_train_size = len(train_df_chronos) - 5*h
results = []

for fold in range(n_splits):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")
    
    train_end = initial_train_size + fold * h
    test_start = train_end
    test_end = test_start + h
    train_df_fold = train_df_chronos.iloc[:train_end]
    test_df_fold = train_df_chronos.iloc[test_start:test_end]
    
    train_df_fold.reset_index(inplace=True)
    test_df_fold.reset_index(inplace=True)
    
    train_df_fold["timestamp"] = pd.to_datetime(train_df_fold["timestamp"])
    test_df_fold["timestamp"] = pd.to_datetime(test_df_fold["timestamp"])
    
    # Future timestamps 
    future_index = pd.date_range(
        start=train_df_fold["timestamp"].max() + pd.Timedelta("160ms"),
        periods=h,
        freq="160ms")
    
    # Preparo test set con known_covariates
    test_df_for_prediction = test_df_fold[test_df_fold["timestamp"].isin(future_index)].copy()
    test_df_for_prediction = test_df_for_prediction.set_index("timestamp")
    test_df_for_prediction = test_df_for_prediction.reindex(future_index)  # forza continuità
    test_df_for_prediction["item_id"] = train_df_fold["item_id"].iloc[0]  # supponiamo 1 sola serie
    test_df_for_prediction = test_df_for_prediction.reset_index().rename(columns={"index": "timestamp"})
    test_df_for_prediction = test_df_for_prediction.set_index(["item_id", "timestamp"])
    
    # Preparo il train con multindex
    train_df_fold = train_df_fold.set_index(["item_id", "timestamp"])
    print(f"Train: {train_df_fold.shape}, Test: {test_df_fold.shape}")
    
    # inizializzo il predictor
    predictor = TimeSeriesPredictor(
        prediction_length=h,
        target="target",
        known_covariates_names=known_covariates_names,
        eval_metric="MASE",
        freq="160ms")
    
    # fit del modello
    predictor.fit(train_df_fold,
        hyperparameters={
            "Chronos": [
                {"model_path": "bolt_small", "ag_args": {"name_suffix": "ZeroShot"}},
                {
                    "model_path": "bolt_small",
                    "covariate_regressor": "CAT",
                    "target_scaler": "standard",
                    "ag_args": {"name_suffix": "WithRegressor"},
                }
            ]
        },
        time_limit=600,
        enable_ensemble=True,)
    
    predictions = predictor.predict(train_df_fold, known_covariates=test_df_for_prediction)

    test_df_with_index = test_df_fold.set_index(["item_id", "timestamp"])

    # Debug
    print(f"Forma predictions: {predictions.shape}")
    print(f"Colonne predictions: {predictions.columns}")
    print(f"Numero di indici in predictions: {len(predictions.index)}")
    print(f"Numero di indici in test_df_with_index: {len(test_df_with_index.index)}")

    # trovo solo gli indici comuni
    common_indices = predictions.index.intersection(test_df_with_index.index)
    print(f"Indici comuni: {len(common_indices)}")

    # prendo solo gli indici comuni
    y_true = test_df_with_index.loc[common_indices, "target"]
    y_pred = predictions.loc[common_indices, 'mean'].to_numpy()

    mae = calcola_mae(y_true, y_pred)
    rmse = calcola_rmse(y_true, y_pred)
    
    results.append({'split': fold+1, 'MAE_chronos': mae, 'RMSE_chronos': rmse})
    print(f"Split {fold+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
# risultati finali
results_df_chronos = pd.DataFrame(results)
results_df_chronos.set_index('split',inplace=True)
print("\n=== Backtest Results ===")
results_df_chronos

In [0]:
# Estraggo le date
fold_data = []
train_df_chronos.reset_index(inplace=True)
for i in range(n_splits):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h
    
    # Ricava date da colonna 'ds' o dall'indice
    if 'timestamp' in train_df_chronos.columns:
        start_date = train_df_chronos['timestamp'].iloc[test_start]
        end_date = train_df_chronos['timestamp'].iloc[test_end - 1]
    else:
        start_date = train_df_chronos.index[test_start]
        end_date = train_df_chronos.index[test_end - 1]
    
    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]
    
    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)
fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])
# Stampa il dataframe prima della conversione per verificare i valori
print("Fold Intervals DataFrame")
fold_intervals_df

In [0]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_chronos['timestamp'], train_df_chronos['target'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot degli errori
results_df_chronos.reset_index(inplace=True)
ax = results_df_chronos.plot(x='split', y=['MAE_chronos','RMSE_chronos'], marker='o', title='TimeSeriesCV Errors')
ax.set_xticks(range(1, 6)) # Imposta le tacche sull'asse x da 1 a 5
plt.show()

In [0]:
results_df_chronos

Let's training on the whole dataset

In [0]:
# Fit del modello

# SOLO PER DATABRICKS
# sys.modules['sklearn.metrics._regression'].mean_absolute_error = sklearn.metrics.mean_absolute_error

predictor = TimeSeriesPredictor(
    prediction_length=len(test_df_chronos), 
    eval_metric="MASE",
    target="target",
    known_covariates_names=known_covariates_names,
    freq="160ms"
)

train_df_chronos = TimeSeriesDataFrame(
    train_df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Aggiunta di più modelli nella configurazione per migliorare le performance
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {
                "model_path": "bolt_base",
                "ag_args": {"name_suffix": "ZeroShot"}},
            {
                "model_path": "bolt_small",
                "covariate_regressor": "CAT", 
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
            {
                "model_path": "bolt_mini",
                "covariate_regressor": "NN_TORCH", 
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            }
        ],
    },
    time_limit=600,
    enable_ensemble=True, 
)

# Valutazione del modello in fase di training
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [0]:
# PREVISIONI 1 CLASSICHE
# train_df_chronos.set_index(['item_id','timestamp'],inplace=True)
# test_df_chronos.set_index(['item_id','timestamp'],inplace=True)
predictions = predictor.predict(
    train_df_chronos, 
    known_covariates=test_df_chronos
)
predictions.reset_index(inplace=True)
#prediction 0 shot
predictions_0shot = predictor.predict(
    train_df_chronos, 
    known_covariates=test_df_chronos,
    model="ChronosZeroShot[bolt_base]"
)
predictions_0shot.reset_index(inplace=True)

In [0]:
# Rescaling dei dati
target_col_position = feature_cols.index('target')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
predictions['mean'] = predictions['mean'] * (max_val - min_val) + min_val
predictions['0.1'] = predictions['0.1'] * (max_val - min_val) + min_val
predictions['0.5'] = predictions['0.5'] * (max_val - min_val) + min_val
predictions['0.9'] = predictions['0.9'] * (max_val - min_val) + min_val
predictions_0shot['mean'] = predictions_0shot['mean'] * (max_val - min_val) + min_val
predictions_0shot['0.5'] = predictions_0shot['0.5'] * (max_val - min_val) + min_val
predictions_0shot['0.9'] = predictions_0shot['0.9'] * (max_val - min_val) + min_val

In [0]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=train_df_chronos['timestamp'], y=train_df_original['target'], mode='lines', name='Training Data'))
# Dati di test

fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='With Regressors', line=dict(color='green')))
# PREVISIONI CHRONOS 0-shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='0-shot Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="CHRONOS forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_chronos['timestamp'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [0]:
# metriche di valutazione
test_df_original.reset_index(inplace=True)
merged_df_chronos = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)
metriche_chronos = calcola_metriche(merged_df_chronos['target'],merged_df_chronos['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos").round(5)
metriche_chronos

In [0]:
# metriche di valutazione 0 shot
merged_df_chronos_0shot = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions_0shot[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']],
        on=['timestamp', 'item_id']
)
metriche_chronos_0shot = calcola_metriche(merged_df_chronos_0shot['target'],merged_df_chronos_0shot['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos_0shot[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos 0 shot").round(5)
metriche_chronos_0shot

#### final evaluations

In [0]:
final_metrics = metriche_arima.join(metriche_ets).join(metriche_xgb).join(metriche_lstm).join(metriche_np).join(metriche_chronos).join(metriche_chronos_0shot).join(metriche_tft)
final_metrics

In [0]:
# FINAL PREDICTIONS PLOT FOR SOLAR ENERGY
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['target'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI AUTOARIMA
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoARIMA'], 
    mode='lines', 
    name='ARIMA', 
    line=dict(color='grey')
))

# PREVISIONI AUTOETS
fig.add_trace(go.Scatter(
    x=forecast_df_etsA['ds'], 
    y=forecast_df_etsA['AutoETS'], 
    mode='lines', 
    name='ETS', 
    line=dict(color='orange')
))

# PREVISIONI XGBOOST
fig.add_trace(go.Scatter(
    x=forecast_xgb['ds'], 
    y=forecast_xgb['forecast'], 
    mode='lines', 
    name='XGBoost', 
    line=dict(color='purple')
))

# PREVISIONI LSTM
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=Y_hat_df_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='pink')
))

# PREVISIONI NEURAL PROPHET
# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='aquamarine')
))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='yellow')))

# PREVISIONI CHRONOS
test_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS', line=dict(color='green')))
    
 # PREVISIONI CHRONOS 0 shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='CHRONOS 0-SHOT', line=dict(color='lightgreen')))
   
# Layout del grafico
fig.update_layout(
    title="Forecasts for White Noise", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [0]:
#sistemo i vari dataset prima del merge
cv_metrics_df_lstm.drop(columns=('Model'),inplace=True)
df_arima = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == 'AutoARIMA'][['Fold',
                                                                    'MAE','RMSE']]
df_ets = cv_metrics_df_arima_ets[cv_metrics_df_arima_ets['Model'] == 'AutoETS'][['Fold',
                                                                    'MAE','RMSE']]
results_df_np.rename(columns={'split': 'Fold'}, inplace=True)
# df_ets.rename
# df_arima.rename
results_df_chronos.rename(columns={'split': 'Fold','MAE':'MAE_chronos','RMSE':'RMSE_chronos'}, inplace=True)
cv_metrics_df_xgboost.rename(columns={'MAE': 'MAE_XGB','RMSE':'RMSE_XGB'}, inplace=True)
df_arima.rename(columns={'MAE':'MAE_AutoARIMA','RMSE':'RMSE_AutoARIMA'}, inplace=True)
df_ets.rename(columns={'MAE':'MAE_AutoETS','RMSE':'RMSE_AutoETS'}, inplace=True)

In [0]:
# Final plot metrics TimeSeriesCV

# faccio il merge di tutte le tabelle dei vari CV
final_cv = df_ets.merge(cv_metrics_df_lstm, 
                            on="Fold",how="inner").merge(results_df_chronos, 
                            on="Fold",how="inner").merge(cv_metrics_df_xgboost[['MAE_XGB','RMSE_XGB','Fold']],
                            on="Fold",how="inner").merge(df_arima,
                            on="Fold",how="inner").merge(results_df_np,
                            on="Fold", how="inner").merge(cv_metrics_df_tft,
                            on="Fold",how="inner")
# Rescaling CV metrics data
columns_to_modify = ['MAE_np', 'RMSE_np', 'MAE_tft', 'RMSE_tft',
                        'MAE_chronos', 'RMSE_chronos', 'MAE_XGB', 'RMSE_XGB']
for col in columns_to_modify:
    final_cv[f'{col}'] = final_cv[f'{col}'] * (max_val - min_val) + min_val

model_colors = {
    'AutoARIMA': '#808080',      # Grey
    'AutoETS': '#ff7f0e',        # Orange
    'XGB': '#9467bd',    # Purple
    'lstm': '#e377c2',       # Pink
    'np': '#7fffd4',         # Aquamarine
    'tft': '#ffff00',        # Yellow
    'chronos': '#2ca02c',    # Green
}


# Create the figure
fig = go.Figure()

# Add traces for each model - MAE (darker) and RMSE (lighter)
for model, color in model_colors.items():
    # Add MAE line (darker)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'MAE_{model}'],
        mode='lines+markers',
        name=f'{model} MAE',
        line=dict(color=color, width=3),
        marker=dict(size=8)
    ))
    
    # Add RMSE line (lighter with same color but different dash pattern)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'RMSE_{model}'],
        mode='lines+markers',
        name=f'{model} RMSE',
        line=dict(color=color, width=2, dash='dash'),
        marker=dict(size=6)
    ))

# Update layout
fig.update_layout(
    title='Model Performance Comparison (MAE and RMSE)',
    template='plotly_dark',
    xaxis=dict(title='Fold',
        tickmode='linear',
        tick0=1, dtick=1
    ),    yaxis=dict(
        title='Error Value'
    ),    legend=dict(
        orientation="v"
    ),    hovermode="closest")
fig.show()

In [0]:
# Confronto TFT/CHRONOS
tft_chronos = metriche_chronos.join(metriche_tft)
tft_chronos                                    

### **F1 RACE (MULTISTEP)**

In [ ]:
futr_exog_list = ['millisecond', 'second', 'minute', 'second_sin', 
                  'second_cos','minute_sin','minute_cos','lap_position']
future_features = ['ds', 'y'] + futr_exog_list

In [ ]:
df = f1[f1['Driver']=='LEC'] # prendiamo i dati di un solo pilota

In [ ]:
# funzione per creare feature
def create_time_series_features(df, target_col):
    df = df.copy()
    lags = [1, 8]
    # Lag features (1-4)
    for lag in lags:
        df[f'lag_{lag}'] = df[target_col].shift(lag)
    
    # Differencing features
    df['diff_1'] = (df[target_col] - df[target_col].shift(1)).shift(1)
    df['diff_8'] = (df[target_col] - df[target_col].shift(8)).shift(1)  # circa 1 secondo prima
    #df['diff_700'] = df[target_col] - df[target_col].shift(700) #circa il giro prima 
    
    # Caratteristiche temporali dal timestamp
    df['millisecond'] = df['ds'].dt.microsecond / 1000  # millisecondi
    df['second'] = df['ds'].dt.second
    df['minute'] = df['ds'].dt.minute
    
    # Caratteristiche cicliche per il tempo
    df['second_sin'] = np.sin(2 * np.pi * df['second']/60)
    df['second_cos'] = np.cos(2 * np.pi * df['second']/60)
    df['minute_sin'] = np.sin(2 * np.pi * df['minute']/60)
    df['minute_cos'] = np.cos(2 * np.pi * df['minute']/60)
    
    # progressione proporzionale lungo il giro
    df['lap_position'] = (df['ds'] - df.groupby('LapNumber')['ds'].transform('min')) / \
                          (df.groupby('LapNumber')['ds'].transform('max') - df.groupby('LapNumber')['ds'].transform('min'))
    # Permette di confrontare facilmente punti simili della pista tra giri diversi;
    # aiuta a identificare pattern che dipendono dalla posizione nella pista
    
    # Rolling statistics (medie mobili)
    window_sizes = [4, 8]
    for window in window_sizes:
        df[f'rolling_mean_{window}'] = df[target_col].rolling(window=window, min_periods=1).mean().shift(1)
        df[f'rolling_std_{window}'] = df[target_col].rolling(window=window, min_periods=1).std().shift(1)
    return df

In [ ]:
#preparazione dataset
df = df.reset_index().rename(columns={'Date': 'ds','Speed':'y'})
df['unique_id'] = 'serie_1'
df['ds'] = pd.to_datetime(df['ds'])
df.drop(columns=['Driver','Status'],inplace=True)
h = 1500 #(i due giri successivi circa)

##### XGBOOST

In [ ]:
# df_xgb
df_xgb = create_time_series_features(df, target_col='y') 
df_xgb = df_xgb.dropna().reset_index(drop=True) #droppo i NaN creati dai lag
df_xgb = TimeSeriesDataFrame(
    df_xgb,
    id_column='unique_id',
    timestamp_column='ds'
)
df_xgb = df_xgb.convert_frequency("160ms").interpolate() #qui perdo qualche osservazione
df_xgb.reset_index(inplace=True)
df_xgb.rename(columns={'timestamp': 'ds', 'item_id':'unique_id'}, inplace=True)

hist_exog_features = df_xgb.drop(columns=['unique_id', 'ds', 'y'] + futr_exog_list).columns.tolist() #dovrebbe funzionare

df_xgb.drop(columns=['unique_id'],inplace=True)
df_xgb = df_xgb[future_features]
print(df_xgb.columns)
# df_xgb.head()

In [ ]:
# Split train/test
train_df_xgb = df_xgb.iloc[:-h]
test_df_xgb = df_xgb.iloc[-h:]

# Separazione features e target
feature_cols = [col for col in df_xgb.columns if col not in ['y', 'ds']]
X_train_xgb, y_train_xgb = train_df_xgb[feature_cols], train_df_xgb['y']
X_test_xgb, y_test_xgb = test_df_xgb[feature_cols], test_df_xgb['y']
X_train_xgb.shape, y_train_xgb.shape, X_test_xgb.shape, y_test_xgb.shape

In [ ]:
# train e test plot
fig = go.Figure()
#train dat
fig.add_trace(go.Scatter(x=df_xgb['ds'], y=y_train_xgb, mode='lines', name='Train'))
#test data 
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=y_test_xgb, mode='lines', name='Test'))
#layout
fig.update_layout(template='plotly_dark', title='Train and Test Data Plot')
fig.show()

In [ ]:
# Scaling dei dati

#features
scaler_x = MinMaxScaler()
X_train_scaled = scaler_x.fit_transform(X_train_xgb)
X_test_scaled = scaler_x.transform(X_test_xgb)

# target, non serve che scalo y_test
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train_xgb.values.reshape(-1, 1))

# Ricreo df con stessi indici e nomi
X_train_xgb = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train_xgb.index)
X_test_xgb = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test_xgb.index)
y_train_xgb = pd.Series(y_train_scaled.flatten(), name='y', index=y_train_xgb.index)

#aggiorno train_df_xgb
train_df_xgb[feature_cols] = X_train_xgb
train_df_xgb['y'] = y_train_xgb

#controllo le shape
X_train_xgb.shape, X_test_xgb.shape, y_train_xgb.shape, y_test_xgb.shape

Ottimizzazione Optuna (Bayesiana)

In [ ]:
# Optuna function
cv_metrics_log = []  # List globale per salvare metriche fold per fold

def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'verbosity': 0,
        'seed': 42
    }

    tscv = TimeSeriesSplit(n_splits=5,test_size=len(test_df_xgb))
    y = train_df_xgb['y']
    df_xgb_feature = train_df_xgb.drop(columns=['y','ds'])
    all_rmse = []

    for fold, (train_idx, test_idx) in enumerate(tscv.split(df_xgb_feature)):
        X_train, X_test = df_xgb_feature.iloc[train_idx], df_xgb_feature.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        dtest = xgb.DMatrix(X_test.values, label=y_test.values)

        model = xgb.train(params, dtrain, num_boost_round=500,
                          evals=[(dtest, 'eval')],
                          early_stopping_rounds=30, verbose_eval=False)

        preds = model.predict(dtest)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mae = calcola_mae(y_test, preds)  # tua funzione custom

        all_rmse.append(rmse)

        # Recupera le date (se esiste colonna 'ds')
        start_date = df_xgb.iloc[test_idx]['ds'].min() if 'ds' in df_xgb.columns else None
        end_date = df_xgb.iloc[test_idx]['ds'].max() if 'ds' in df_xgb.columns else None

        # Logga le metriche della fold
        cv_metrics_log.append({
            'Trial': trial.number,
            'Fold': fold + 1,
            'MAE': mae,
            'RMSE': rmse,
            'start_date': start_date,
            'end_date': end_date,
            'Model': 'XGBoost'  # Puoi modificarlo se usi più modelli
        })
    return np.mean(all_rmse)

In [ ]:
# Optuna optimization
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=2))
study.optimize(objective, n_trials=50) #aumentare su databricks

In [ ]:
best_params = pd.read_pickle('pickles/XGB_bestparams_f1.pkl')

In [ ]:
# Best parameters
print("Best params:", study.best_params)
print("Best RMSE:", study.best_value)
print("Best trial:",study.best_trial.number)
besttrial = study.best_trial.number

In [ ]:
cv_metrics_log = pd.DataFrame(cv_metrics_log)
cv_metrics_log = cv_metrics_log[cv_metrics_log['Trial']==besttrial]
cv_metrics_log

Vediamo come ha performato nel miglior trial

In [ ]:
#PLOT CV
cv_metrics_df_xgboost = cv_metrics_log.copy()

# Stampa le metriche per modello
print("Metriche per modello:")
print(cv_metrics_df_xgboost)

# Calcola e stampa le metriche medie per modello
mean_metrics = cv_metrics_df_xgboost.mean(numeric_only=True).reset_index()

# Crea il DataFrame con gli intervalli delle fold
fold_intervals_df = cv_metrics_df_xgboost[['Fold', 'start_date', 'end_date']].drop_duplicates()

# Ora creiamo il grafico combinato
plt.figure(figsize=(16, 12))

# === PLOT 1: Serie temporale originale ===
plt.subplot(2, 1, 1)
#for unique_id in train_df_xgb['unique_id'].unique():
series_data = train_df_xgb.copy()
plt.plot(series_data['ds'], series_data['y'], label='Serie unica')

# Evidenzia le fold con colori
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)],
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'{row["Fold"]}', ha='center', fontsize=10,
             bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# === PLOT 2: Metriche MAE e RMSE ===
plt.subplot(2, 1, 2)
ax1 = plt.gca()
ax2 = ax1.twinx()

# MAE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# RMSE
for model in cv_metrics_df_xgboost['Model'].unique():
    model_data = cv_metrics_df_xgboost[cv_metrics_df_xgboost['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Legenda combinata
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()


In [ ]:
# carico le cv metrics
cv_metrics_df_xgboost = pd.read_pickle('pickles/cv_metrics_df_xgboost_f1.pkl')
print(cv_metrics_df_xgboost)
cv_metrics_df_xgboost.plot(x='fold', y=['MAE', 'RMSE'], marker='o', title='XGBoost CV Metrics')

Training coi migliori parametri trovati da Optuna

In [0]:
# #salvo i best params
# best_params = study.best_params
# with open ('pickles/XGB_bestparams_f1.pkl', 'wb') as file:
#     pickle.dump(best_params, file)

In [ ]:
# train con best params
best_params.update({'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 42})

dtrain_xgb = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
import time
start_time = time.time()
#fit 
model = xgb.train(best_params, dtrain_xgb, num_boost_round=500)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [ ]:
# Plot feature importance
ax = xgb.plot_importance(model, importance_type='gain', max_num_features=10)
for text in ax.texts:
    text.set_visible(False)
plt.title('Feature Importance (Gain)')
plt.tight_layout()
plt.show()

In [ ]:
dtest_xgb = xgb.DMatrix(X_test_xgb)
forecast_xgb = pd.DataFrame(model.predict(dtest_xgb),columns=['forecast'])
forecast_xgb['ds'] = test_df_xgb['ds'].values

In [ ]:
# RE-Scaling dei dati
forecast_xgb['forecast'] = scaler_y.inverse_transform(forecast_xgb[['forecast']])
train_df_xgb['y'] = scaler_y.inverse_transform(train_df_xgb[['y']])

In [ ]:
# Plot forecast
fig = go.Figure()
# Training data
fig.add_trace(go.Scatter(x=train_df_xgb['ds'], y=train_df_xgb['y'], mode='lines', name='Training'))
# Test data
fig.add_trace(go.Scatter(x=test_df_xgb['ds'], y=test_df_xgb['y'], mode='lines', name='Test'))
# Forecast
fig.add_trace(go.Scatter(x=forecast_xgb['ds'], y=forecast_xgb['forecast'], mode='lines', 
                         name='Forecast', line=dict(color='deepskyblue')))
# Forecast dirette
fig.update_layout(title='Forecast vs Real Data', template='plotly_dark')
fig.show()

Calcolo metriche

In [ ]:
# Calcola metriche
merged_df = pd.merge(test_df_xgb[['ds', 'y']], forecast_xgb, on='ds', how='left')
metriche_xgb = calcola_metriche(merged_df['y'],merged_df['forecast'],
                                train_df_xgb['y'], modelname="XGBoost")
metriche_xgb

##### LSTM

In [ ]:
# divisione in train e test
df_lstm = create_time_series_features(df, target_col='y') 
df_lstm = df_lstm.dropna().reset_index(drop=True)
df_lstm = TimeSeriesDataFrame(
    df_lstm,
    id_column='unique_id',
    timestamp_column='ds'
)
df_lstm = df_lstm.convert_frequency("160ms").interpolate() #qui perdo qualche osservazione
df_lstm.reset_index(inplace=True)
df_lstm.rename(columns={'timestamp': 'ds', 'item_id':'unique_id'}, inplace=True)

train_df_lstm = df_lstm.iloc[:-h]
test_df_lstm = df_lstm.iloc[-h:]
print(train_df_lstm.columns)
train_df_lstm.head()

In [ ]:
# Optuna function
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_lstm = pd.DataFrame()
def objective(trial):
    # Hyperparametri da ottimizzare
    encoder_n_layers = trial.suggest_int("encoder_n_layers", 1, 2,4)
    encoder_hidden_size = trial.suggest_categorical("encoder_hidden_size", [32, 256])
    decoder_hidden_size = trial.suggest_categorical("decoder_hidden_size", [32, 256])
    decoder_layers = trial.suggest_int("decoder_layers", 1, 2,4)
    input_size = trial.suggest_categorical("input_size", [int(h), int(h*2), int(h/2)])
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [64,128])

    # Definizione modello LSTM
    lstm = LSTM(
        h=len(test_df_lstm), 
        input_size=input_size,
        loss=DistributionLoss(distribution="Normal", level=[95]),
        scaler_type='robust',
        encoder_n_layers=encoder_n_layers,
        encoder_hidden_size=encoder_hidden_size,
        decoder_hidden_size=decoder_hidden_size,
        decoder_layers=decoder_layers,
        futr_exog_list=futr_exog_list,
        max_steps=50,
        batch_size=batch_size,
        learning_rate=learning_rate,
        recurrent=False
    )

    nf = NeuralForecast(models=[lstm], freq='160ms')

    # cross-validation per tuning
    df_cv = nf.cross_validation(
        df=train_df_lstm,
        step_size=len(test_df_lstm),
        n_windows=5
    )

    # Calcola MAE per ogni fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'LSTM']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'LSTM',
            'MAE_lstm': mae,
            'RMSE_lstm': rmse
        })

    # Media MAE su tutte le fold
    cv_metrics_df_lstm = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_lstm['MAE_lstm'].mean()

    return mean_mae 

In [ ]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=10) 

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
# # salvo i best params trovati da Optuna
with open('pickles/LSTM_bestPar_F1_multistep.pkl', 'wb') as file:
    pickle.dump(best_params, file)

Carico i best params

In [ ]:
# carico i best pars
with open('pickles/LSTM_bestPar_F1_multistep.pkl', 'rb') as file:
    best_params = pickle.load(file)

In [ ]:
# final model
h = len(test_df_lstm)
best_lstm = LSTM(
    h=h,
    input_size=h*2,
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=2,
    encoder_hidden_size=64,
    decoder_hidden_size=32,
    decoder_layers=1,
    futr_exog_list=futr_exog_list,
    max_steps=50,
    #early_stop_patience_steps=20,
    batch_size=512,
    learning_rate=0.01,
    recurrent=False
)
nf = NeuralForecast(models=[best_lstm], freq='160ms')

In [ ]:
# final CV
# df_cv = nf.cross_validation(
#     df=train_df_lstm[10000:], #la CPU non regge tutte le unità
#     step_size=len(test_df_lstm),
#     n_windows=5
# )
# with open('pickles/LSTM_CV_F1_multistep.pkl', 'wb') as file:
#      pickle.dump(df_cv, file)

with open('pickles/LSTM_CV_F1_multistep.pkl', 'rb') as file:
    df_cv = pickle.load(file)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'LSTM']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['LSTM'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['LSTM'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'LSTM',
        'MAE_lstm': mae,
        'RMSE_lstm': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_lstm = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_lstm)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_lstm.groupby('Model').mean().reset_index()
print(mean_metrics)

In [ ]:
# PLOT CV
# Prima dobbiamo identificare gli intervalli temporali per ogni fold
folds = df_cv['cutoff'].unique()
fold_intervals = []
for i, cutoff in enumerate(folds):
    # Ottieni le date della fold corrente
    fold_data = df_cv[df_cv['cutoff'] == cutoff]
    start_date = fold_data['ds'].min()
    end_date = fold_data['ds'].max()
    fold_intervals.append({
        'Fold': i+1,
        'start_date': start_date,
        'end_date': end_date
    })

fold_intervals_df = pd.DataFrame(fold_intervals)

# Ora creiamo un grafico combinato: serie temporale + metriche di errore
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
for unique_id in train_df_lstm['unique_id'].unique():
    series_data = train_df_lstm[train_df_lstm['unique_id'] == unique_id]
    plt.plot(series_data['ds'], series_data['y'], label=f'Serie {unique_id}')

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot inferiore: metriche MAE e RMSE per ogni fold
plt.subplot(2, 1, 2)

# Crea un grafico con due assi y
ax1 = plt.gca()
ax2 = ax1.twinx()

cv_metrics_df_lstm['Model'] = 'lstm'
# Plot MAE sul primo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax1.plot(model_data['Fold'], model_data['MAE_lstm'], 'o-', label=f'MAE - {model}')

ax1.set_xlabel('Fold')
ax1.set_ylabel('MAE', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot RMSE sul secondo asse
for model in cv_metrics_df_lstm['Model'].unique():
    model_data = cv_metrics_df_lstm[cv_metrics_df_lstm['Model'] == model]
    ax2.plot(model_data['Fold'], model_data['RMSE_lstm'], 's--', label=f'RMSE - {model}')

ax2.set_ylabel('RMSE', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Combina le legende
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Metriche di Errore per Fold')
plt.tight_layout()
plt.show()

In [ ]:
# Final training
import time
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

best_lstm = LSTM(
    h=h,
    input_size=int(h*2),
    loss=DistributionLoss(distribution="Normal", level=[95]),
    scaler_type='robust',
    encoder_n_layers=2,
    encoder_hidden_size=64,
    decoder_hidden_size=32,
    decoder_layers=2,
    futr_exog_list=futr_exog_list,
    max_steps=200,
    #early_stop_patience_steps=20,
    batch_size=64,
    learning_rate=0.025,
    recurrent=False
)
nf = NeuralForecast(models=[best_lstm], freq='160ms')

start_time = time.time()
#fit 
nf.fit(
    df=train_df_lstm[12500:], #la cpu non regge tutto il dataset
    val_size=len(test_df_lstm)
    )
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

Previsioni

In [ ]:
#predictions
Y_hat_df_lstm = nf.predict(futr_df=test_df_lstm[['unique_id', 'ds'] + futr_exog_list])
#salvo in pickle
with open('pickles/LSTM_F1_multistep.pkl', 'wb') as file:
    pickle.dump(Y_hat_df_lstm, file)

In [ ]:
#carico le previsioni:
with open('pickles/LSTM_f1_multistep.pkl', 'rb') as file:
    Y_hat_df_lstm = pickle.load(file)

In [ ]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_lstm['ds'], y=train_df_lstm['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_lstm['ds'], y=test_df_lstm['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIA LSTM
fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM'], mode='lines', 
    name='LSTM Forecast', line=dict(color='green')))

# PREVISIONI MEDIANA LSTM
# fig.add_trace(go.Scatter(x=Y_hat_df_lstm['ds'], y=Y_hat_df_lstm['LSTM-median'], mode='lines', 
#     name='MEDIAN Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="LSTM forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_lstm['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")

fig.show()

In [ ]:
# Calcola metriche
merged_df_lstm = pd.merge(test_df_lstm[['ds', 'y']], Y_hat_df_lstm, on='ds', how='left')
#merged_df_lstm.dropna()
metriche_lstm = calcola_metriche(merged_df_lstm['y'],merged_df_lstm['LSTM'],train_df_lstm['y'],modelname="LSTM")
metriche_lstm

##### NEURAL PROPHET

In [ ]:
df_np = create_time_series_features(df, target_col='y') 
df_np = df_np.dropna().reset_index(drop=True)
df_np = TimeSeriesDataFrame(
    df_np,
    id_column='unique_id',
    timestamp_column='ds'
)
df_np = df_np.convert_frequency("160ms").interpolate() #qui perdo qualche osservazione
df_np.reset_index(inplace=True)
df_np.rename(columns={'timestamp': 'ds', 'item_id':'unique_id'}, inplace=True)
#metto come booleana la variabile brake
df_np['Brake'] = df_np['Brake'].astype(int)
df_np.drop(columns="unique_id",inplace=True)
df_np = df_np[future_features]
train_df_np = df_np.iloc[:-h]
test_df_np = df_np.iloc[-h:]
df_np.columns

In [ ]:
# Scaling dei dati
train_df_original = train_df_np.copy()
test_df_original = test_df_np.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_np['ds']
test_meta = test_df_np['ds']

feature_cols = [col for col in train_df_np.columns if col not in ['ds']]
train_idx = train_df_np.index
test_idx = test_df_np.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_np[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_np[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_np = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_np = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_np.shape, test_df_np.shape

TimeSeriesCV

In [ ]:
# CV
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

initial_train_size = len(train_df_np) - 5 * len(test_df_np)
h = len(test_df_np)
n_windows = 5

results = []

for i in range(n_windows):
    print(f"\n--- Fold {i+1}/{n_windows} ---")

    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    train_window = train_df_np.iloc[:train_end].copy()
    test_window = train_df_np.iloc[test_start:test_end].copy()

    print(f"Train: {train_window.shape}, Test: {test_window.shape}")

    # Modello NeuralProphet
    model = NeuralProphet(
        quantiles=[0.025, 0.975],
        learning_rate=0.01,
        batch_size=64,
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
        loss_func='Huber',
    )

    # Aggiungi regressori
    for reg in futr_exog_list:
        model.add_future_regressor(reg)

    # Fit
    _ = model.fit(train_window, freq="160ms", epochs=50)

    # Predizione
    future_df = test_window.copy()  # Deve contenere anche i regressori futuri
    forecast = model.predict(future_df)

    # Metriche
    y_true = test_window['y'].values
    y_pred = forecast['yhat1'].values

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    results.append({'Fold': i+1, 'MAE_nprophet': mae, 'RMSE_nprophet': rmse})
    print(f"Split {i+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# Risultati CV
results_df_np = pd.DataFrame(results)
print("\n=== Backtest Results ===")
print(results_df_np)

# Plot Errori
results_df_np.plot(
    x='Fold',
    y=['MAE_nprophet', 'RMSE_nprophet'],
    marker='o',
    title='TimeSeriesCV Errors',
    figsize=(10, 5)
)
plt.grid(True)
plt.show()

In [ ]:
# Salvo results df CV
with open('pickles/NP_CV_F1_multistep.pkl', 'wb') as file:
    pickle.dump(results_df_np, file)

In [ ]:
# Carico i risultati della CV
with open('pickles/NP_CV_F1_multistep.pkl', 'rb') as file:
    results_df_np = pickle.load(file)

In [ ]:
# Mi ricavo le date
initial_train_size = len(train_df_np)-5*len(test_df_np)
h = len(test_df_np)
n_windows = 5

# Crea lista per i dati delle fold
fold_data = []

for i in range(n_windows):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h

    # Ricava date da colonna 'ds' o dall'indice
    if 'ds' in train_df_np.columns:
        start_date = train_df_np['ds'].iloc[test_start]
        end_date = train_df_np['ds'].iloc[test_end - 1]
    else:
        start_date = train_df_np.index[test_start]
        end_date = train_df_np.index[test_end - 1]

    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]

    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)

# Converto le date in datetime se necessario
if all(isinstance(date, str) for date in fold_intervals_df['start_date']):
    fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
    fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])

print("Fold Intervals DataFrame:")
print(fold_intervals_df)

In [ ]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_np['ds'], train_df_np['y'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

results_df_np.plot(x='Fold', y=['MAE_nprophet', 'RMSE_nprophet'], marker='o', title='Backtesting Errors')

Now let's training on the whole dataset

In [ ]:
# FIT
# Fissa i seed per riproducibilità
import random
import time
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

np_df_train = train_df_np[future_features].copy()

# Modello senza regressori esterni
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.2, 
    batch_size=512,
    daily_seasonality=True,    # Impara la stagionalità dai dati
    weekly_seasonality=True,   # Impara pattern settimanali
    yearly_seasonality=True,   # Impara pattern annuali
    loss_func='Huber'
)
# Aggiungi solo i regressori deterministici
for reg in futr_exog_list:
    neuralprophet.add_future_regressor(reg)
start_time = time.time()
metrics_df = neuralprophet.fit(np_df_train, freq="160ms",epochs=200)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [ ]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
forecast_test = neuralprophet.predict(test_df_np[future_features])
# #salvo le prediction sul test in locale
with open('pickles/NP_f1_multistep.pkl', 'wb') as file:
    pickle.dump(forecast_test, file)

CARICO LE PREVISIONI DEL TEST SET

In [ ]:
#carico le previsioni del test
with open('pickles/NP_f1_multistep.pkl', 'rb') as file:
    forecast_test = pickle.load(file)

In [ ]:
#Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
forecast_test['yhat1'] = forecast_test['yhat1'] * (max_val - min_val) + min_val
forecast_test['yhat1 2.5%'] = forecast_test['yhat1 2.5%'] * (max_val - min_val) + min_val
forecast_test['yhat1 97.5%'] = forecast_test['yhat1 97.5%'] * (max_val - min_val) + min_val

In [ ]:
# Forecast plot
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='green')
))

# Layout del grafico
fig.update_layout(
    title="Neural Prophet Forecast", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)
fig.show()

In [ ]:
# Calcola metriche
# df con previsioni e valori reali
forecast_df = pd.DataFrame({'ds': forecast_test['ds'], 'forecast': forecast_test['yhat1']})
merged_df = pd.merge(test_df_original[['ds', 'y']], forecast_df, on='ds', how='left')
# Calcola metriche
mae_val = mean_absolute_error(merged_df['y'], merged_df['forecast'])
metriche_np = calcola_metriche(merged_df['y'], merged_df['forecast'],train_df_original['y'],
                               modelname="NeuralProphet").round(10)
metriche_np

#### TFT

In [ ]:
# Preparing dataset
df_tft = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_tft = df_tft.dropna().reset_index(drop=True) 
df_tft['item_id'] = df_tft['unique_id'] 
df_tft.drop(columns="unique_id", inplace=True) 
df_tft.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_tft = TimeSeriesDataFrame(
    df_tft,
    id_column='item_id',
    timestamp_column='timestamp'
)
df_tft = df_tft.convert_frequency("160ms").interpolate()
df_tft.reset_index(inplace=True)
df_tft.rename(columns={'timestamp': 'ds','item_id':'unique_id','target':'y'}, inplace=True)
hist_exog_features = df_tft.drop(columns=['unique_id', 'ds', 'y'] + futr_exog_list).columns.tolist() #should work
train_df_tft = df_tft.iloc[:-h]
test_df_tft = df_tft.iloc[-h:]
train_df_tft.shape, test_df_tft.shape

In [ ]:
# Scaling dei dati
#Salvo per dopo
train_df_original = train_df_tft.copy() 
test_df_original = test_df_tft.copy()

# Salvo le colonne da riattaccare
train_meta = train_df_tft[['ds', 'unique_id']]
test_meta = test_df_tft[['ds', 'unique_id']]

feature_cols = [col for col in train_df_tft.columns if col not in ['ds', 'unique_id']]
train_idx = train_df_tft.index
test_idx = test_df_tft.index

#scaler
scaler_x = MinMaxScaler()
train_df_scaled = pd.DataFrame(scaler_x.fit_transform(train_df_tft[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_scaled = pd.DataFrame(scaler_x.transform(test_df_tft[feature_cols]),
                            columns=feature_cols, index=test_idx)

#riconcateno con ds e unique_id
train_df_tft = pd.concat([train_df_scaled, train_meta], axis=1)
test_df_tft = pd.concat([test_df_scaled, test_meta], axis=1)

#controllo le shape
train_df_tft.shape, test_df_tft.shape

##### TimeSeriesCV - fine tuning Optuna (LENTO)

In [0]:
# Optuna function
folds = pd.DataFrame()
cv_metrics = []
cv_metrics_df_tft = pd.DataFrame()
h = len(test_df_tft)

def objective(trial):
    # Hyperparameters to optimize for TFT
    input_size = trial.suggest_categorical("input_size", [h, int(h/2)])
    hidden_size = trial.suggest_categorical("hidden_size", [32, 64, 128, 256])
    n_rnn_layers = trial.suggest_int("n_rnn_layers", 1, 4)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    grn_activation = trial.suggest_categorical("grn_activation", ["ReLU", "ELU", "Sigmoid"])
    
    # Define TFT model
    tft = TFT(
        h=h,  # forecast horizon
        input_size=input_size,
        hidden_size=hidden_size,
        n_rnn_layers=n_rnn_layers,
        rnn_type='lstm',
        grn_activation=grn_activation,
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=learning_rate,
        futr_exog_list=futr_exog_list,
        hist_exog_list=hist_exog_features,
        max_steps=50, #aumentare a 100
        val_check_steps=10,
        batch_size=batch_size,
        early_stop_patience_steps=-1,
        scaler_type="standard",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=64
)

    nf = NeuralForecast(models=[tft], freq='160ms')

    # Cross-validation for tuning
    df_cv = nf.cross_validation(
        df=train_df_tft[20000:],
        step_size=int(h/2),
        n_windows=5
    )

    # Calculate MAE for each fold
    folds = df_cv['cutoff'].unique()

    for i, fold in enumerate(folds):
        fold_data = df_cv[df_cv['cutoff'] == fold]
        model_data = fold_data[['y', 'TFT']].dropna()

        mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
        rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])

        cv_metrics.append({
            'Fold': i+1,
            'Model': 'TFT',
            'MAE_tft': mae,
            'RMSE_tft': rmse
        })

    # Average MAE across all folds
    cv_metrics_df_tft = pd.DataFrame(cv_metrics)
    mean_mae = cv_metrics_df_tft['MAE_tft'].mean()

    return mean_mae

In [0]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=10) #NON GIRA TROPPO PESANTE, OUT OF MEMORY ERROR DI CUDA  

In [0]:
print("Best hyperparameters:", study.best_trial.params)
print("Best MAE:", study.best_value)
best_params=study.best_trial.params

In [0]:
# # Path nella repo Git
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_BestPars_F1_multistep.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(best_params, f)

In [0]:
# # #carico i best params
# with open("pickles/TFT_BestPars_F1_multistep.pkl", 'rb') as file:
#     best_params = pickle.load(file)

In [0]:
# final CV

# final model
best_tft = TFT(
        h=h,  
        input_size=int(h/2), 
        hidden_size=32,
        n_rnn_layers=1,
        rnn_type='lstm',
        grn_activation="ELU",
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=0.05,
        futr_exog_list=futr_exog_list,
        hist_exog_list = hist_exog_features,
        max_steps=200, #aumentare a 100
        val_check_steps=10,
        batch_size=8,
        early_stop_patience_steps=-1,
        scaler_type="standard",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=8
)
nf = NeuralForecast(models=[best_tft],freq="15min")
# final CV
df_cv = nf.cross_validation(
    df=train_df_tft[20000:],
    step_size=len(test_df_tft),
    n_windows=5
)
# Estrai i dati di cross-validation per calcolare le metriche
cv_metrics = []

# Identifica le fold basate sui timestamp unici
folds = df_cv['cutoff'].unique()

for i, fold in enumerate(folds):
    fold_data = df_cv[df_cv['cutoff'] == fold]
    
    # Filtra i dati per il modello corrente
    model_data = fold_data[['y', 'TFT']]
    model_data = model_data.dropna()
        
    # Calcola MAE e RMSE
    mae = calcola_mae(y_true=model_data['y'], y_pred=model_data['TFT'])
    rmse = calcola_rmse(y_true=model_data['y'], y_pred=model_data['TFT'])
        
    # Aggiungi i risultati
    cv_metrics.append({
        'Fold': i+1,
        'Model': 'TFT',
        'MAE_TFT': mae,
        'RMSE_TFT': rmse
        })

# Converti i risultati in DataFrame
cv_metrics_df_tft = pd.DataFrame(cv_metrics)

#Stampa le metriche nelle varie folds
print("Metriche per modello:")
print(cv_metrics_df_tft)
print("Metriche medie per modello:")
mean_metrics = cv_metrics_df_tft.groupby('Model').mean().reset_index()
print(mean_metrics)

In [ ]:
# carico la CV fatta su Colab
with open('pickles/TFT_CV_F1_multistep.pkl', 'rb') as file:
    cv_metrics_df_tft = pickle.load(file)

In [ ]:
cv_metrics_df_tft.plot(x='Fold', y=['MAE_TFT', 'RMSE_TFT'], marker='o', title='Backtesting Errors')

In [ ]:
cv_metrics_df_tft

##### Final training

In [0]:
# Final training
import time
# final model
best_tft = TFT(
        h=h,  
        input_size=int(h*2),
        hidden_size=16,
        n_rnn_layers=2,
        rnn_type='lstm',
        grn_activation="ELU",
        loss=DistributionLoss(distribution="StudentT", level=[90]),
        learning_rate=0.005,
        futr_exog_list=futr_exog_list,
        hist_exog_list=hist_exog_features,
        max_steps=500, #aumentare a 100
        val_check_steps=25,
        batch_size=8,
        early_stop_patience_steps=-1,
        scaler_type="standard",
        enable_progress_bar=True,
        accelerator="auto",
        one_rnn_initial_state=False,
        windows_batch_size=8,

)
nf = NeuralForecast(models=[best_tft],freq="160ms")
# Pulire la memoria GPU prima di iniziare IL FIT
torch.cuda.empty_cache()
gc.collect()
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
start_time = time.time()

#fit 
nf.fit(df=train_df_tft[15000:], # nel training scartato 15k unità per memoria GPU
       val_size=len(test_df_tft)) 

end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# Predict
Y_hat_df_tft = nf.predict(futr_df=test_df_tft[['unique_id','ds'] + futr_exog_list])

In [0]:
# #salvo le pred in pickle
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_pred_F1_multistep.pkl"
# with open(save_path, 'wb') as file:
#     pickle.dump(Y_hat_df_tft, file)

In [ ]:
# Rescaling dei dati
target_col_position = feature_cols.index('y')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
Y_hat_df_tft['TFT'] = Y_hat_df_tft['TFT'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-median'] = Y_hat_df_tft['TFT-median'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-lo-90'] = Y_hat_df_tft['TFT-lo-90'] * (max_val - min_val) + min_val
Y_hat_df_tft['TFT-hi-90'] = Y_hat_df_tft['TFT-hi-90'] * (max_val - min_val) + min_val

In [ ]:
# #carico le pred
with open("pickles/TFT_pred_F1_multistep.pkl", 'rb') as file:
    Y_hat_df_tft = pickle.load(file)

In [ ]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
fig.add_trace(go.Scatter(x=train_df_tft['ds'], y=train_df_original['y'], mode='lines', name='Training Data'))
# Dati di test
fig.add_trace(go.Scatter(x=test_df_tft['ds'], y=test_df_original['y'], mode='lines', name='Testing Data'))

# PREVISIONI MEDIANA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT-median'], mode='lines', 
    name='MEDIAN Forecast', line=dict(color='green')))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='lightgreen')))

# QUANTILI TFT
# Fascia tra P10 e P90
fig.add_trace(go.Scatter(
    x=list(Y_hat_df_tft['ds']) + 
    list(Y_hat_df_tft['ds'][::-1]),
    y=list(Y_hat_df_tft['TFT-hi-90']) + list(Y_hat_df_tft['TFT-lo-90'][::-1]),
    fill='toself',
    fillcolor='rgba(144, 238, 144, 0.3)',  # lightgreen sfumato
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='P10-P90'
))

# Layout del grafico
fig.update_layout(title="TFT forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_tft['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [ ]:
# Calcola metriche
merged_df_tft = pd.merge(test_df_original[['ds', 'y']], Y_hat_df_tft, on='ds', how='left')
merged_df_tft.rename(columns={'TFT-lo-90': '0.1','TFT-median':'0.5', 'TFT-hi-90':'0.9'},inplace=True)
#merged_df_tft.dropna(inplace=True)
metriche_tft = calcola_metriche(merged_df_tft['y'],merged_df_tft['TFT'],train_df_original['y'],
                                y_pred_quantiles=merged_df_tft[['0.1','0.5','0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="TFT").round(10)
metriche_tft

##### TFT Explainability

In [0]:
best_tft = nf.models[0]

In [0]:
# # NON MI FA SALVARE IL MODELLO PERCHE' TROPPO PESANTE
# save_path = "/Workspace/Users/p.pierdomenico@iconsulting.biz/timeseries_ai/TFT_model_F1_multistep.pkl"
# with open(save_path, "wb") as f:
#     pickle.dump(best_tft, f)

In [0]:
# # # carico il modello tft
# with open('pickles/TFT_model_F1_multistep.pkl', 'rb') as file:
#     best_tft = pickle.load(file)

In [0]:
attention = best_tft.attention_weights()
def plot_attention(
    self, plot: str = "time", output: str = "plot", width: int = 800, height: int = 400
):
    """
    Plot the attention weights.

    Args:
        plot (str, optional): The type of plot to generate. Can be one of the following:
            - 'time': Display the mean attention weights over time.
            - 'all': Display the attention weights for each horizon.
            - 'heatmap': Display the attention weights as a heatmap.
            - An integer in the range [1, model.h) to display the attention weights for a specific horizon.
        output (str, optional): The type of output to generate. Can be one of the following:
            - 'plot': Display the plot directly.
            - 'figure': Return the plot as a figure object.
        width (int, optional): Width of the plot in pixels. Default is 800.
        height (int, optional): Height of the plot in pixels. Default is 400.

    Returns:
        matplotlib.figure.Figure: If `output` is 'figure', the function returns the plot as a figure object.
    """

    attention = (
        self.mean_on_batch(self.interpretability_params["attn_wts"])
        .mean(dim=0)
        .cpu()
        .numpy()
    )

    fig, ax = plt.subplots(figsize=(width / 100, height / 100))

    if plot == "time":
        attention = attention[self.input_size :, :].mean(axis=0)
        ax.plot(np.arange(-self.input_size, self.h), attention)
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Mean Attention")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "all":
        for i in range(self.input_size, attention.shape[0]):
            ax.plot(
                np.arange(-self.input_size, self.h),
                attention[i, :],
                label=f"horizon {i-self.input_size+1}",
            )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Attention per horizon")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "heatmap":
        cax = ax.imshow(
            attention,
            aspect="auto",
            cmap="viridis",
            extent=[-self.input_size, self.h, -self.input_size, self.h],
        )
        fig.colorbar(cax)
        ax.set_title("Attention Heatmap")
        ax.set_xlabel("Attention (current time step)")
        ax.set_ylabel("Attention (previous time step)")

    elif isinstance(plot, int) and (plot in np.arange(1, self.h + 1)):
        i = self.input_size + plot - 1
        ax.plot(
            np.arange(-self.input_size, self.h),
            attention[i, :],
            label=f"horizon {plot}",
        )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title(f"Attention weight for horizon {plot}")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    else:
        raise ValueError(
            'plot has to be in ["time","all","heatmap"] or integer in range(1,model.h)'
        )

    plt.tight_layout()

    if output == "plot":
        plt.show()
    elif output == "figure":
        return fig
    else:
        raise ValueError(f"Invalid output: {output}. Expected 'plot' or 'figure'.")

In [0]:
# MEAN ATTENTION
plot_attention(best_tft, plot="time")
# pesi di attenzione medi nel tempo su tutti gli orizzonti di previsione.
# La curva blu rappresenta quanto il modello ha "guardato" (attention) 
# quel punto temporale in input durante la previsione.
# Valori più alti significano che il modello ritiene quel punto temporale
# particolarmente rilevante per fare le previsioni

In [0]:
# ATTENTION OF A SPECIFIC TIME STEPS
plot_attention(best_tft, plot=44)
# pesi di attenzione per ogni orizzonte di previsione separatamente.

In [0]:
plot_attention(best_tft, plot="heatmap")

In [0]:
# PAST VARIABLE IMPORTANCES
feature_importances = best_tft.feature_importances()
print(feature_importances.keys())
print("\n ==========PAST FEATURE IMPORTANCES==========")
feature_importances["Past variable importance over time"].mean().sort_values().plot(kind="barh")

Questo grafico mostra quanto il modello ha fatto affidamento sulle feature passate \
(quelle che il modello ha potuto osservare prima del punto di previsione). Serve a costruire il contesto dell'input

In [0]:
# FUTURE VARIABLE IMPORTANCES
print("\n ==========FUTURE FEATURE IMPORTANCES==========")
feature_importances["Future variable importance over time"].mean().sort_values().plot(kind="barh")

Mostra quanto il modello ha sfruttato le feature che sono servite a guidare le previsioni passo-passo nel decoder

DIFFERENZA TRA PAST E FUTURE: \
La diversa importanza riflette quanto ciascuna variabile aiuta nell’analisi del contesto passato (encoder -->> PAST) \
o nella generazione delle previsioni (decoder -->> FUTURE).

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME
df_importances = feature_importances["Past variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances.index))

for col in df_importances.columns:
    p = ax.bar(np.arange(-len(df_importances), 0), df_importances[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances[col]
ax.set_title("Past variable importance over time")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)

plt.show()

Questo grafico mostra QUANDO (e QUANTO) nel passato ogni feature è stata più utile per costruire il contesto predittivo.

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME
df_importances2 = feature_importances["Future variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances2.index))
for col in df_importances2.columns:
    p = ax.bar(np.arange(-len(df_importances2), 0), df_importances2[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances2[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.grid(True)
ax.legend()
plt.show()

Per ogni feature viene mostrato quanto è stata importante per il modello a ciascun orizzonte futuro

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances3 = feature_importances["Past variable importance over time"]
mean_attention = (
    best_tft
    .attention_weights()[best_tft.input_size :, :]
    .mean(axis=0)[: best_tft.input_size]
)
df_importances3 = df_importances3.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances3.index))

for col in df_importances3.columns:
    p = ax.bar(np.arange(-len(df_importances3), 0), df_importances3[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances3[col]
ax.set_title("Past variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances3), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances4 = feature_importances["Future variable importance over time"]
mean_attention = (
    best_tft
    .attention_weights()[best_tft.input_size :, :]
    .mean(axis=0)[: best_tft.input_size]
)
df_importances4 = df_importances4.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances4.index))

for col in df_importances4.columns:
    p = ax.bar(np.arange(-len(df_importances4), 0), df_importances4[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances4[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances4), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
best_tft.feature_importance_correlations()

#### CHRONOS

In [ ]:
# Preparing dataset
df_chronos = create_time_series_features(df, target_col='y') 

# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos = df_chronos.dropna().reset_index(drop=True) 
df_chronos['item_id'] = df_chronos['unique_id'] 
df_chronos.drop(columns="unique_id", inplace=True) 
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)
df_chronos = df_chronos.convert_frequency("160ms").interpolate() #qui perdo qualche osservazione
# Definizione delle covariate note
known_covariates_names=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id','target']]
feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

# Split train e test
train_df_chronos = df_chronos.iloc[:-h]
test_df_chronos = df_chronos.iloc[-h:]

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [ ]:
# Scaling dei dati
train_df_original = train_df_chronos.copy() 
test_df_original = test_df_chronos.copy()

train_idx = train_df_chronos.index
test_idx = test_df_chronos.index

feature_cols=[col for col in df_chronos.columns if col not in ['timestamp', 'item_id']] #mi serve dopo nello scaling

#scaler
scaler_x = MinMaxScaler()
train_df_chronos = pd.DataFrame(scaler_x.fit_transform(train_df_chronos[feature_cols]),
                             columns=feature_cols, index=train_idx)
test_df_chronos = pd.DataFrame(scaler_x.transform(test_df_chronos[feature_cols]),
                            columns=feature_cols, index=test_idx)
#controllo le shape
train_df_chronos.shape, test_df_chronos.shape

TimeSeriesCV

In [ ]:
#TimeSeriesCV
h = len(test_df_chronos)  # prediction length
n_splits = 5
initial_train_size = len(train_df_chronos) - 5*h
results = []

for fold in range(n_splits):
    print(f"\n--- Fold {fold+1}/{n_splits} ---")
    
    train_end = initial_train_size + fold * h
    test_start = train_end
    test_end = test_start + h
    train_df_fold = train_df_chronos.iloc[:train_end]
    test_df_fold = train_df_chronos.iloc[test_start:test_end]
    
    train_df_fold.reset_index(inplace=True)
    test_df_fold.reset_index(inplace=True)
    
    train_df_fold["timestamp"] = pd.to_datetime(train_df_fold["timestamp"])
    test_df_fold["timestamp"] = pd.to_datetime(test_df_fold["timestamp"])
    
    # Future timestamps 
    future_index = pd.date_range(
        start=train_df_fold["timestamp"].max() + pd.Timedelta("160ms"),
        periods=h,
        freq="160ms")
    
    # Preparo test set con known_covariates
    test_df_for_prediction = test_df_fold[test_df_fold["timestamp"].isin(future_index)].copy()
    test_df_for_prediction = test_df_for_prediction.set_index("timestamp")
    test_df_for_prediction = test_df_for_prediction.reindex(future_index)  # forza continuità
    test_df_for_prediction["item_id"] = train_df_fold["item_id"].iloc[0]  # supponiamo 1 sola serie
    test_df_for_prediction = test_df_for_prediction.reset_index().rename(columns={"index": "timestamp"})
    test_df_for_prediction = test_df_for_prediction.set_index(["item_id", "timestamp"])
    
    # Preparo il train con multindex
    train_df_fold = train_df_fold.set_index(["item_id", "timestamp"])
    print(f"Train: {train_df_fold.shape}, Test: {test_df_fold.shape}")
    
    # inizializzo il predictor
    predictor = TimeSeriesPredictor(
        prediction_length=h,
        target="target",
        known_covariates_names=known_covariates_names,
        eval_metric="MASE",
        freq="160ms")
    
    # fit del modello
    predictor.fit(train_df_fold,
        hyperparameters={
            "Chronos": [
                {"model_path": "bolt_small", "ag_args": {"name_suffix": "ZeroShot"}},
                {
                    "model_path": "bolt_small",
                    "covariate_regressor": "CAT",
                    "target_scaler": "standard",
                    "ag_args": {"name_suffix": "WithRegressor"},
                }
            ]
        },
        time_limit=600,
        enable_ensemble=True,)
    
    predictions = predictor.predict(train_df_fold, known_covariates=test_df_for_prediction)

    test_df_with_index = test_df_fold.set_index(["item_id", "timestamp"])

    # Debug
    print(f"Forma predictions: {predictions.shape}")
    print(f"Colonne predictions: {predictions.columns}")
    print(f"Numero di indici in predictions: {len(predictions.index)}")
    print(f"Numero di indici in test_df_with_index: {len(test_df_with_index.index)}")

    # trovo solo gli indici comuni
    common_indices = predictions.index.intersection(test_df_with_index.index)
    print(f"Indici comuni: {len(common_indices)}")

    # prendo solo gli indici comuni
    y_true = test_df_with_index.loc[common_indices, "target"]
    y_pred = predictions.loc[common_indices, 'mean'].to_numpy()

    mae = calcola_mae(y_true, y_pred)
    rmse = calcola_rmse(y_true, y_pred)
    
    results.append({'split': fold+1, 'MAE_chronos': mae, 'RMSE_chronos': rmse})
    print(f"Split {fold+1} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
# risultati finali
results_df_chronos = pd.DataFrame(results)
results_df_chronos.set_index('split',inplace=True)
print("\n=== Backtest Results ===")
results_df_chronos

In [ ]:
# Estraggo le date
fold_data = []
train_df_chronos.reset_index(inplace=True)
for i in range(n_splits):
    train_end = initial_train_size + i * h
    test_start = train_end
    test_end = test_start + h
    
    # Ricava date da colonna 'ds' o dall'indice
    if 'timestamp' in train_df_chronos.columns:
        start_date = train_df_chronos['timestamp'].iloc[test_start]
        end_date = train_df_chronos['timestamp'].iloc[test_end - 1]
    else:
        start_date = train_df_chronos.index[test_start]
        end_date = train_df_chronos.index[test_end - 1]
    
    # Format
    if hasattr(start_date, 'strftime'):
        start_date_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
        end_date_str = end_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        start_date_str = str(start_date).split('.')[0]
        end_date_str = str(end_date).split('.')[0]
    
    fold_data.append({
        'Fold': i + 1,
        'start_date': start_date_str,
        'end_date': end_date_str
    })

# Crea DataFrame finale
fold_intervals_df = pd.DataFrame(fold_data)
fold_intervals_df['start_date'] = pd.to_datetime(fold_intervals_df['start_date'])
fold_intervals_df['end_date'] = pd.to_datetime(fold_intervals_df['end_date'])
# Stampa il dataframe prima della conversione per verificare i valori
print("Fold Intervals DataFrame")
fold_intervals_df

In [ ]:
# PLOT CV
plt.figure(figsize=(16, 12))

# Plot principale: serie temporale originale
plt.subplot(2, 1, 1)
plt.plot(train_df_chronos['timestamp'], train_df_chronos['target'])

# Evidenzia le fold con colori di sfondo diversi
colors = ['lightblue', 'lightgreen', 'lightgray', 'lightpink', 'red']
for i, row in fold_intervals_df.iterrows():
    plt.axvspan(row['start_date'], row['end_date'], alpha=0.3, color=colors[i % len(colors)], 
                label=f'Fold {row["Fold"]}' if i == 0 else "")
    # Aggiungi etichetta della fold al centro dell'intervallo
    mid_point = row['start_date'] + (row['end_date'] - row['start_date']) / 2
    y_pos = plt.ylim()[1] * 0.9
    plt.text(mid_point, y_pos, f'Fold {row["Fold"]}', ha='center', fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

plt.title('Serie Temporale con Folds della Cross Validation')
plt.xlabel('Data')
plt.ylabel('Valore')
plt.legend(loc='upper left')

# Plot degli errori
results_df_chronos.reset_index(inplace=True)
ax = results_df_chronos.plot(x='split', y=['MAE_chronos','RMSE_chronos'], marker='o', title='TimeSeriesCV Errors')
ax.set_xticks(range(1, 6)) # Imposta le tacche sull'asse x da 1 a 5
plt.show()

Let's training on the whole dataset

In [ ]:
# Preparo dataset per training
predictor = TimeSeriesPredictor(
    prediction_length=len(test_df_chronos), 
    eval_metric="MASE",
    target="target",
    known_covariates_names=futr_exog_list,
    freq="160ms"
)
test_known_covariates = test_df_chronos[futr_exog_list]

In [ ]:
# Fit del modello
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {
                "model_path": "bolt_small",
                "ag_args": {"name_suffix": "ZeroShot"}
            },
            {
                "model_path": "bolt_mini",
                "covariate_regressor": "NN_TORCH",
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            },
        ],
    },
    time_limit=600,
    enable_ensemble=True,
)
# Valutazione del modello in fase di training
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [ ]:
# PREVISIONI MULTI STEP 
predictions = predictor.predict(
    train_df_chronos,
    known_covariates=test_known_covariates,  # Solo le covariate realmente note in anticipo
)
predictions.reset_index(inplace=True)

predictions_0shot = predictor.predict(
    train_df_chronos,
    known_covariates=test_known_covariates,
    model="ChronosZeroShot[bolt_small]"
)
predictions_0shot.reset_index(inplace=True)

In [ ]:
# Rescaling dei dati
target_col_position = feature_cols.index('target')
min_val = scaler_x.data_min_[target_col_position]  # min value della feature target
max_val = scaler_x.data_max_[target_col_position]  # max value della feature target
# Trasformazione inversa manuale
predictions['mean'] = predictions['mean'] * (max_val - min_val) + min_val
predictions['0.1'] = predictions['0.1'] * (max_val - min_val) + min_val
predictions['0.5'] = predictions['0.5'] * (max_val - min_val) + min_val
predictions['0.9'] = predictions['0.9'] * (max_val - min_val) + min_val
predictions_0shot['mean'] = predictions_0shot['mean'] * (max_val - min_val) + min_val
predictions_0shot['0.5'] = predictions_0shot['0.5'] * (max_val - min_val) + min_val
predictions_0shot['0.9'] = predictions_0shot['0.9'] * (max_val - min_val) + min_val

In [ ]:
# Plot forecast
# Creare la figura
fig = go.Figure()
# Dati di training
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=train_df_chronos['timestamp'], y=train_df_original['target'], mode='lines', name='Training Data'))
# Dati di test

fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI CHRONOS
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='With Regressors', line=dict(color='green')))
# PREVISIONI CHRONOS 0-shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='0-shot Forecast', line=dict(color='yellow')))

# Layout del grafico
fig.update_layout(title="CHRONOS forecast", 
    xaxis_title="Timestamp [t]", yaxis_title="Value", 
    template='plotly_dark', legend=dict(font=dict(size=15))
)

# linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_chronos['timestamp'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(x=ultimo_test, y=0.9, yref="paper", text="Fine Test Set", 
                  showarrow=True, arrowhead=2, arrowcolor="white")
fig.show()

In [ ]:
# metriche di valutazione
test_df_original.reset_index(inplace=True)
merged_df_chronos = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']], 
        on=['timestamp', 'item_id']
)
metriche_chronos = calcola_metriche(merged_df_chronos['target'],merged_df_chronos['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos").round(5)
metriche_chronos

In [ ]:
# metriche di valutazione 0 shot
merged_df_chronos_0shot = pd.merge(
        test_df_original[['timestamp', 'target', 'item_id']], 
        predictions_0shot[['timestamp', 'mean', 'item_id','0.1','0.5','0.9']],
        on=['timestamp', 'item_id']
)
metriche_chronos_0shot = calcola_metriche(merged_df_chronos_0shot['target'],merged_df_chronos_0shot['mean'],
                                train_df_original['target'], y_pred_quantiles=merged_df_chronos_0shot[['0.1', '0.5', '0.9']],
                                quantiles=[0.1, 0.5, 0.9], modelname="Chronos 0 shot").round(5)
metriche_chronos_0shot

#### final evaluations

In [ ]:
final_metrics = metriche_xgb.join(metriche_lstm).join(metriche_np).join(metriche_chronos).join(metriche_chronos_0shot).join(metriche_tft)
final_metrics

In [ ]:
# FINAL PREDICTIONS PLOT FOR SOLAR ENERGY
fig = go.Figure()

# Dati di training e test
fig.add_trace(go.Scatter(x=train_df_np['ds'], y=train_df_original['target'], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test_df_np['ds'], y=test_df_original['target'], mode='lines', name='Testing Data'))

# PREVISIONI XGBOOST
fig.add_trace(go.Scatter(
    x=forecast_xgb['ds'], 
    y=forecast_xgb['forecast'], 
    mode='lines', 
    name='XGBoost', 
    line=dict(color='purple')
))

# PREVISIONI LSTM
fig.add_trace(go.Scatter(
    x=Y_hat_df_lstm['ds'], 
    y=Y_hat_df_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='pink')
))

# PREVISIONI NEURAL PROPHET
# In-sample forecast (test period)
fig.add_trace(go.Scatter(
    x=forecast_test['ds'], 
    y=forecast_test['yhat1'], 
    mode='lines', 
    name='NeuralProphet', 
    line=dict(color='aquamarine')
))

# PREVISIONI MEDIA TFT
fig.add_trace(go.Scatter(x=Y_hat_df_tft['ds'], y=Y_hat_df_tft['TFT'], mode='lines', 
    name='TFT Forecast', line=dict(color='yellow')))

# PREVISIONI CHRONOS
test_df_chronos.reset_index(inplace=True)
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions['mean'], mode='lines', 
    name='CHRONOS', line=dict(color='green')))
    
 # PREVISIONI CHRONOS 0 shot
fig.add_trace(go.Scatter(x=test_df_chronos['timestamp'], y=predictions_0shot['mean'], mode='lines', 
    name='CHRONOS 0-SHOT', line=dict(color='lightgreen')))
   
# Layout del grafico
fig.update_layout(
    title="Forecasts for White Noise", 
    xaxis_title="Timestamp [t]", 
    yaxis_title="Value", 
    template='plotly_dark', 
    legend=dict(font=dict(size=15))
)

# Linea verticale per separare i dati di test dai dati futuri
ultimo_test = test_df_np['ds'].max()
fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
fig.add_annotation(
    x=ultimo_test, 
    y=0.9, 
    yref="paper", 
    text="Fine Test Set", 
    showarrow=True, 
    arrowhead=2, 
    arrowcolor="white"
)
fig.show()

In [ ]:
#sistemo i vari dataset prima del merge
cv_metrics_df_lstm.drop(columns=('Model'),inplace=True)
results_df_np.rename(columns={'split': 'Fold'}, inplace=True)
results_df_chronos.rename(columns={'split': 'Fold','MAE':'MAE_chronos','RMSE':'RMSE_chronos'}, inplace=True)
cv_metrics_df_xgboost.rename(columns={'MAE': 'MAE_XGB','RMSE':'RMSE_XGB','fold':'Fold'}, inplace=True)

In [ ]:
# Final plot metrics TimeSeriesCV

# faccio il merge di tutte le tabelle dei vari CV
final_cv = cv_metrics_df_lstm.merge(results_df_chronos, 
                            on="Fold",how="inner").merge(cv_metrics_df_xgboost[['MAE_XGB','RMSE_XGB','Fold']],
                            on="Fold",how="inner").merge(results_df_np,
                            on="Fold", how="inner").merge(cv_metrics_df_tft,
                            on="Fold",how="inner")
# Rescaling CV metrics data
columns_to_modify = ['MAE_nprophet', 'RMSE_nprophet', 'MAE_TFT', 'RMSE_TFT', #'MAE_lstm', 'RMSE_lstm',
                        'MAE_chronos', 'RMSE_chronos', 'MAE_XGB', 'RMSE_XGB']
for col in columns_to_modify:
    final_cv[f'{col}'] = final_cv[f'{col}'] * (max_val - min_val) + min_val

model_colors = {
    'XGB': '#9467bd',    # Purple
    'lstm': '#e377c2',       # Pink
    'nprophet': '#7fffd4',         # Aquamarine
    'TFT': '#ffff00',        # Yellow
    'chronos': '#2ca02c',    # Green
}


# Create the figure
fig = go.Figure()

# Add traces for each model - MAE (darker) and RMSE (lighter)
for model, color in model_colors.items():
    # Add MAE line (darker)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'MAE_{model}'],
        mode='lines+markers',
        name=f'{model} MAE',
        line=dict(color=color, width=3),
        marker=dict(size=8)
    ))
    
    # Add RMSE line (lighter with same color but different dash pattern)
    fig.add_trace(go.Scatter(
        x=final_cv['Fold'],
        y=final_cv[f'RMSE_{model}'],
        mode='lines+markers',
        name=f'{model} RMSE',
        line=dict(color=color, width=2, dash='dash'),
        marker=dict(size=6)
    ))

# Update layout
fig.update_layout(
    title='Model Performance Comparison (MAE and RMSE)',
    template='plotly_dark',
    xaxis=dict(title='Fold',
        tickmode='linear',
        tick0=1, dtick=1
    ),    yaxis=dict(
        title='Error Value'
    ),    legend=dict(
        orientation="v"
    ),    hovermode="closest")
fig.show()

In [ ]:
# Confronto TFT/CHRONOS
tft_chronos = metriche_chronos.join(metriche_tft)
tft_chronos                                    

### **FAVORITA SALES**, panel

In [ ]:
#preparazione dataset
df = favorita_train.reset_index().rename(columns={'DATE_TIME': 'ds','sales':'y','Store':'unique_id'})
df['ds'] = pd.to_datetime(df['ds'])
df.drop(columns=['index'],inplace=True)
df = df[df['unique_id'] != 52] #dava problemi nei vari fit (tutti valori costanti, no variabilità)

In [ ]:
# funzione create features per store
def create_time_series_features(df, target_col):
    df = df.copy()
    
    # Ordina per sicurezza
    df = df.sort_values(['unique_id', 'ds'])

    # Applicazione delle feature per ogni store
    def fe_group(group):
        # Lag features (1-7)
        for lag in range(1, 8):
            group[f'lag_{lag}'] = group[target_col].shift(lag)

        # Differencing features
        group['diff_1'] = (group[target_col] - group[target_col].shift(1)).shift(1)
        group['diff_7'] = (group[target_col] - group[target_col].shift(7)).shift(1)

        # Rolling means & std
        group['rolling_mean_3'] = group[target_col].rolling(window=3).mean().shift(1)
        group['rolling_mean_7'] = group[target_col].rolling(window=7).mean().shift(1)
        group['rolling_std_3'] = group[target_col].rolling(window=3).std().shift(1)
        group['rolling_std_7'] = group[target_col].rolling(window=7).std().shift(1)

        return group

    # Applica la funzione per ogni store
    df = df.groupby('unique_id', group_keys=False).apply(fe_group)

    # Feature temporali (non dipendono da target)
    df['dayofweek'] = df['ds'].dt.dayofweek
    df['month'] = df['ds'].dt.month
    df['quarter'] = df['ds'].dt.quarter

    # Feature cicliche
    df['day_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    return df

##### LSTM

In [ ]:
df_lstm = create_time_series_features(df, target_col='y') 
df_lstm = df_lstm.dropna().reset_index(drop=True)

In [ ]:
# 2. Suddivisione in train/test
#divido train/test ogni serie storica
def split_train_test_by_id(df, test_frac=0.1):
    train_list = []
    test_list = []
    for uid, group in df.groupby("unique_id"):
        group = group.sort_values("ds")  # Ordina temporalmente
        split_idx = int(len(group) * (1 - test_frac))
        train_list.append(group.iloc[:split_idx])
        test_list.append(group.iloc[split_idx:])
    return pd.concat(train_list), pd.concat(test_list)
train_df_lstm, test_df_lstm = split_train_test_by_id(df_lstm, test_frac=0.1)
h = len(test_df_lstm) // 53


In [0]:
train_df_lstm.shape, test_df_lstm.shape

In [0]:
# creating static df
# Step 1: One-hot encoding normale
ohe_df = pd.get_dummies(train_df_lstm['unique_id'].drop_duplicates(), prefix='store')

# Step 2: Inverti i valori (0 -> 1 e 1 -> 0)
ohe_df = 1 - ohe_df

# Step 3: Aggiungi la colonna unique_id
ohe_df['unique_id'] = train_df_lstm['unique_id'].drop_duplicates().values

# Step 4: Reordina colonne
static_df = ohe_df[['unique_id'] + [col for col in ohe_df.columns if col != 'unique_id']]

# Cast a int
static_df = static_df.astype(int)

In [0]:
# TimeSeriesCV
h = len(test_df_lstm) // train_df_lstm['unique_id'].nunique()  # horizon
# === MODELLI ===
models = [LSTM(h=h, 
               input_size=2*h,
               loss=DistributionLoss(distribution="Normal", level=[95]),
               scaler_type='robust',
               encoder_n_layers=1,
               encoder_hidden_size=64,
               decoder_hidden_size=64,
               decoder_layers=1,
               max_steps=50, # su databricks va aumentato
               futr_exog_list=train_df_lstm.drop(columns=['unique_id', 'ds', 'y']).columns.tolist(),
               batch_size=32,
               learning_rate=0.01,
               recurrent=False,
               start_padding_enabled=True
               )
          ]

# === INIT ===
nf = NeuralForecast(models=models, freq='B')
# === CROSS-VALIDATION ===
cv_df = nf.cross_validation(df=train_df_lstm, 
                            static_df=static_df,
                            val_size=h, 
                            n_windows=5)
for uid in df['unique_id'].unique():
    y_true = cv_df[cv_df['unique_id'] == uid]['y']
    y_pred = cv_df[cv_df['unique_id'] == uid]['LSTM']  
    print(f'{uid} MAE:', calcola_mae(y_true, y_pred),
          f'{uid} RMSE:', calcola_rmse(y_true, y_pred))

In [0]:
# === FIT FINALE DEL MODELLO ===
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

models = [LSTM(h=h, 
               input_size=int(3*h),
               loss=DistributionLoss(distribution="Normal", level=[95]),
               scaler_type='robust',
               encoder_n_layers=2, 
               encoder_hidden_size=128,
               decoder_hidden_size=64,
               decoder_layers=2, 
               max_steps=100,
               futr_exog_list=train_df_lstm.drop(columns=['unique_id', 'ds', 'y']).columns.tolist(),
               batch_size=32, 
               learning_rate=0.005, 
               recurrent=False,
               start_padding_enabled=True
               )
          ]
nf = NeuralForecast(models=models, freq='B') 
print("=== STARTING THE MODEL FIT ===")

start_time = time.time()
#fit 
nf.fit(train_df_lstm, val_size=4*h)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
#predictions
Y_hat_df_lstm = nf.predict(futr_df=test_df_lstm)
#salvo in pickle
with open('pickles/LSTM_favorita.pkl', 'wb') as file:
    pickle.dump(Y_hat_df_lstm, file)

CARICO LE PREVISIONI

In [0]:
# #carico le previsioni:
with open('pickles/LSTM_favorita.pkl', 'rb') as file:
    Y_hat_df_lstm = pickle.load(file)

In [0]:
# forecast plot per store
store_ids = [1,3,5,20,35,54] #5 store a caso

for store in store_ids:
    train_store = train_df_lstm[train_df_lstm['unique_id'] == store]
    test_store = test_df_lstm[test_df_lstm['unique_id'] == store]
    forecast_store = Y_hat_df_lstm[Y_hat_df_lstm['unique_id'] == store]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=train_store['ds'], y=train_store['y'], mode='lines', name='Training Data'))
    fig.add_trace(go.Scatter(x=test_store['ds'], y=test_store['y'], mode='lines', name='Testing Data'))
    fig.add_trace(go.Scatter(
        x=forecast_store['ds'], 
        y=forecast_store['LSTM'], 
        mode='lines', 
        name='LSTM Forecast', 
        line=dict(color='green')
    ))

    ultimo_test = test_store['ds'].max()
    fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
    fig.add_annotation(
        x=ultimo_test, 
        y=0.9, 
        yref="paper", 
        text="Fine Test Set", 
        showarrow=True, 
        arrowhead=2, 
        arrowcolor="white"
    )

    fig.update_layout(
        title=f"Forecast - Store {store}",
        xaxis_title="Timestamp [t]",
        yaxis_title="Sales",
        template='plotly_dark',
        legend=dict(font=dict(size=15))
    )

    fig.show()

In [0]:
# Calcola metriche per ogni store
forecast_df = Y_hat_df_lstm[['ds', 'LSTM', 'unique_id']]
merged_df = pd.merge(test_df_lstm[['ds', 'y', 'unique_id']], Y_hat_df_lstm, on=['ds', 'unique_id'], how='left')
store_metrics_lstm = []
for store in df_lstm['unique_id'].unique():  
    store_df = merged_df[merged_df['unique_id'] == store]
    metrics = calcola_metriche(store_df['y'], store_df['LSTM'],
                               y_train=train_df_lstm['y'], modelname="LSTM")
    metrics['Store_ID'] = store
    store_metrics_lstm.append(metrics)

In [0]:
store_metrics_lstm[13] #posso selezionare qualunque store io voglia valutare

In [0]:
# MEDIE E MEDIANE delle metriche
mae_values = []
mape_values = []
smape_values = []
rmse_values = []
mase_values_lstm = []

# Collect metrics from each store
for i in range(len(store_metrics_lstm)):
    mae_values.append(store_metrics_lstm[i].iloc[0, 0])  
    mape_values.append(store_metrics_lstm[i].iloc[2, 0])  
    smape_values.append(store_metrics_lstm[i].iloc[3, 0]) 
    rmse_values.append(store_metrics_lstm[i].iloc[1, 0]) 
    mase_values_lstm.append(store_metrics_lstm[i].iloc[4, 0]) 

# Calculate means
mae_mean = np.mean(mae_values)
mape_mean = np.mean(mape_values)
smape_mean = np.mean(smape_values)
rmse_mean = np.mean(rmse_values)
mase_mean = np.mean(mase_values_lstm)

# Calculate medians
mae_median = np.median(mae_values)
mape_median = np.median(mape_values)
smape_median = np.median(smape_values)
rmse_median = np.median(rmse_values)
mase_median = np.median(mase_values_lstm)

# Create a DataFrame to store the aggregated metrics
mean_metrics_lstm = pd.DataFrame({
    'Metric': ['MAE', 'MAPE', 'SMAPE', 'RMSE', 'MASE'],
    'Mean_LSTM': [mae_mean, mape_mean, smape_mean, rmse_mean, mase_mean],
    'Median_LSTM':[mae_median, mape_median, smape_median, rmse_median, mase_median]
})
mean_metrics_lstm

In [0]:
plt.plot(mase_values_lstm)
plt.axhline(y=1, color='red', linestyle='--')
print("MASE >= 1:", np.sum(np.array(mase_values_lstm) >= 1), "Percentuale:",
      (np.sum(np.array(mase_values_lstm) > 1) / len(mase_values_lstm) * 100).round(2), "% degli store")
print("MASE < 1:", np.sum(np.array(mase_values_lstm) < 1), "Percentuale:",
      (np.sum(np.array(mase_values_lstm) < 1) / len(mase_values_lstm) * 100).round(2), "% degli store")

##### NEURAL PROPHET

In [ ]:
# 1. Prepara il dataset
df_np = create_time_series_features(df, target_col='y').dropna()
df_np['ID'] = df_np['unique_id'].astype(str)
df_np.drop(columns=['unique_id'], inplace=True)
df_np['settimana_del_anno'] = df_np['settimana_del_anno'].astype(int)

In [ ]:
# 2. Suddivisione in train/test
#divido train/test ogni serie storica
def split_train_test_by_id(df, test_frac=0.1):
    train_list = []
    test_list = []
    for uid, group in df.groupby("ID"):
        group = group.sort_values("ds")  # Ordina temporalmente
        split_idx = int(len(group) * (1 - test_frac))
        train_list.append(group.iloc[:split_idx])
        test_list.append(group.iloc[split_idx:])
    return pd.concat(train_list), pd.concat(test_list)
train_df_np, test_df_np = split_train_test_by_id(df_np, test_frac=0.1)


In [ ]:
train_df_np.shape, test_df_np.shape

In [ ]:
# 3. Selezione colonne
cols_base = ['ds', 'y', 'ID']
regressors = [
        'onpromotion', 'promo', 'oil price', 'giorno_settimana',
       'settimana_del_mese', 'settimana_del_anno', 'giorno_del_mese',
       'mese_del_anno', 'rolling_mean_30', 'sales_lag_1', 'sales_lag_2',
       'sales_lag_7', 'sales_lag_14', 'sales_lag_28', 'lag_1', 'lag_2',
       'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'diff_1', 'diff_7',
       'dayofweek', 'month', 'quarter', 'day_sin', 'day_cos', 'month_sin',
       'month_cos', 'rolling_mean_3', 'rolling_mean_7', 'rolling_std_3',
       'rolling_std_7'
]

train_df_np = train_df_np[cols_base + regressors]
test_df_np = test_df_np[cols_base + regressors]

train_df_np = train_df_np.astype({col: 'float' for col in train_df_np.columns if col not in ['ds', 'ID']})
test_df_np = test_df_np.astype({col: 'float' for col in test_df_np.columns if col not in ['ds', 'ID']})

In [ ]:
# 4. Definisco il modello
model = NeuralProphet(
    # n_forecasts=1,
    # n_lags=0,
    quantiles=[0.025, 0.975],
    learning_rate=0.001,
    batch_size=64,
    daily_seasonality=True,
    weekly_seasonality=True,
    loss_func='Huber',
    #seasonality_mode='multiplicative',
    global_normalization=True,
    unknown_data_normalization=True
)

# 5. Aggiungo i regressori
for reg in regressors:
    model.add_future_regressor(reg)

In [ ]:
# 6. Fit del modello
import time
start_time = time.time()
metrics_df = model.fit(
    train_df_np,
    freq="B", 
    epochs = 100
)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
# 7. Previsione
forecast_test = model.predict(test_df_np)

In [0]:
# salvo le prediction sul test in locale
with open('pickles/NP_favorita.pkl', 'wb') as file:
    pickle.dump(forecast_test, file)

CARICO LE PREVISIONI

In [0]:
# carico le previsioni del test
with open('pickles/NP_favorita.pkl', 'rb') as file:
    forecast_test = pickle.load(file)

In [0]:
# forecast plot per store
store_ids = ['1','3','5','20','35','54'] # degli store a caso

for store in store_ids:
    train_store = train_df_np[train_df_np['ID'] == store]
    test_store = test_df_np[test_df_np['ID'] == store]
    forecast_store = forecast_test[forecast_test['ID'] == store]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=train_store['ds'], y=train_store['y'], mode='lines', name='Training Data'))
    fig.add_trace(go.Scatter(x=test_store['ds'], y=test_store['y'], mode='lines', name='Testing Data'))
    fig.add_trace(go.Scatter(
        x=forecast_store['ds'], 
        y=forecast_store['yhat1'], 
        mode='lines', 
        name='NeuralProphet Forecast', 
        line=dict(color='green')
    ))

    ultimo_test = test_store['ds'].max()
    fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
    fig.add_annotation(
        x=ultimo_test, 
        y=0.9, 
        yref="paper", 
        text="Fine Test Set", 
        showarrow=True, 
        arrowhead=2, 
        arrowcolor="white"
    )

    fig.update_layout(
        title=f"Forecast - Store {store}",
        xaxis_title="Timestamp [t]",
        yaxis_title="Sales",
        template='plotly_dark',
        legend=dict(font=dict(size=15))
    )

    fig.show()

In [0]:
# Calcola metriche per ogni store
forecast_df = forecast_test[['ds', 'yhat1', 'ID']]
merged_df = pd.merge(test_df_np[['ds', 'y', 'ID']], forecast_df, on=['ds', 'ID'], how='left')
store_metrics_np = []
for store in df_np['ID'].unique():  # primi 5 store
    store_df = merged_df[merged_df['ID'] == store]
    metrics = calcola_metriche(store_df['y'], store_df['yhat1'],
                               y_train=train_df_np['y'], modelname="NeuralProphet")
    metrics['Store_ID'] = store
    store_metrics_np.append(metrics)

In [0]:
store_metrics_np[12] #posso selezionare qualunque store voglio vedere

In [0]:
# MEDIE E MEDIANE delle metriche
mae_values = []
mape_values = []
smape_values = []
rmse_values = []
mase_values_np = []

# Collect metrics from each store
for i in range(len(store_metrics_np)):
    mae_values.append(store_metrics_np[i].iloc[0, 0])  
    mape_values.append(store_metrics_np[i].iloc[2, 0])  
    smape_values.append(store_metrics_np[i].iloc[3, 0]) 
    rmse_values.append(store_metrics_np[i].iloc[1, 0]) 
    mase_values_np.append(store_metrics_np[i].iloc[4, 0]) 

# Calculate means
mae_mean = np.mean(mae_values)
mape_mean = np.mean(mape_values)
smape_mean = np.mean(smape_values)
rmse_mean = np.mean(rmse_values)
mase_mean = np.mean(mase_values_np)

# Calculate medians
mae_median = np.median(mae_values)
mape_median = np.median(mape_values)
smape_median = np.median(smape_values)
rmse_median = np.median(rmse_values)
mase_median = np.median(mase_values_np)

# Create a DataFrame to store the aggregated metrics
mean_metrics_np = pd.DataFrame({
    'Metric': ['MAE', 'MAPE', 'SMAPE', 'RMSE', 'MASE'],
    'Mean_NP': [mae_mean, mape_mean, smape_mean, rmse_mean, mase_mean],
    'Median_NP':[mae_median, mape_median, smape_median, rmse_median, mase_median]
})
mean_metrics_np

In [0]:
plt.plot(mase_values_np)
plt.axhline(y=1, color='red', linestyle='--')
print("MASE >= 1:", np.sum(np.array(mase_values_np) > 1), "Percentuale:",
      (np.sum(np.array(mase_values_np) >= 1) / len(mase_values_np) * 100).round(2), "% degli store")
print("MASE < 1:", np.sum(np.array(mase_values_np) < 1), "Percentuale:",
      (np.sum(np.array(mase_values_np) < 1) / len(mase_values_np) * 100).round(2), "% degli store")

#### TFT

In [0]:
# Preparing dataset
df_tft = create_time_series_features(df, target_col='y') 
df_tft = df_tft.dropna().reset_index(drop=True)
# 2. Suddivisione in train/test
#divido train/test ogni serie storica
def split_train_test_by_id(df, test_frac=0.1):
    train_list = []
    test_list = []
    for uid, group in df.groupby("unique_id"):
        group = group.sort_values("ds")  # Ordina temporalmente
        split_idx = int(len(group) * (1 - test_frac))
        train_list.append(group.iloc[:split_idx])
        test_list.append(group.iloc[split_idx:])
    return pd.concat(train_list), pd.concat(test_list)
train_df_tft, test_df_tft = split_train_test_by_id(df_tft, test_frac=0.1)
train_df_tft.shape, test_df_tft.shape


In [0]:
# creating static df
# Step 1: One-hot encoding normale
ohe_df = pd.get_dummies(train_df_tft['unique_id'].drop_duplicates(), prefix='store')

# Step 2: Inverti i valori (0 -> 1 e 1 -> 0)
ohe_df = 1 - ohe_df

# Step 3: Aggiungi la colonna unique_id
ohe_df['unique_id'] = train_df_tft['unique_id'].drop_duplicates().values

# Step 4: Reordina colonne
static_df = ohe_df[['unique_id'] + [col for col in ohe_df.columns if col != 'unique_id']]

# Cast a int
static_df = static_df.astype(int)

In [0]:
# model fit
h = len(test_df_tft) // train_df_tft['unique_id'].nunique()
nf = NeuralForecast(
    models=[
        TFT(
            h=h,
            input_size=31,
            hidden_size=32, # deve essere divisibile per 4
            grn_activation="ELU",
            rnn_type="lstm",
            n_rnn_layers=2, # prima 1
            one_rnn_initial_state=False,
            loss=DistributionLoss(distribution="StudentT", level=[95]),
            learning_rate=0.005, # prima 0.01
            futr_exog_list=train_df_tft.drop(columns=['unique_id', 'ds', 'y']).columns.tolist(),
            max_steps=100,
            val_check_steps=25,
            early_stop_patience_steps=10,
            scaler_type="standard",
            windows_batch_size=128,
            enable_progress_bar=True,
        ),
    ],
    freq="B", 
)
start_time = time.time()
#fit 
nf.fit(df=train_df_tft, static_df=static_df, val_size=h)
end_time = time.time()
tft_model = nf.models[0]
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
Y_hat_df_tft = nf.predict(futr_df=test_df_tft)
#salvo in pickle
# with open('pickles/TFT_favorita.pkl', 'wb') as file:
#     pickle.dump(Y_hat_df_tft, file)

Carico le previsioni TFT

In [0]:
# #carico le previsioni:
with open('pickles/TFT_favorita.pkl', 'rb') as file:
    Y_hat_df_tft = pickle.load(file)

In [0]:
# forecast plot per store
store_ids = [1,3,5,20,35,54] #5 store a caso

for store in store_ids:
    train_store = train_df_tft[train_df_tft['unique_id'] == store]
    test_store = test_df_tft[test_df_tft['unique_id'] == store]
    forecast_store = Y_hat_df_tft[Y_hat_df_tft['unique_id'] == store]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=train_store['ds'], y=train_store['y'], mode='lines', name='Training Data'))
    fig.add_trace(go.Scatter(x=test_store['ds'], y=test_store['y'], mode='lines', name='Testing Data'))
    fig.add_trace(go.Scatter(
        x=forecast_store['ds'], 
        y=forecast_store['TFT'], 
        mode='lines', 
        name='TFT Forecast', 
        line=dict(color='green')
    ))

    ultimo_test = test_store['ds'].max()
    fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
    fig.add_annotation(
        x=ultimo_test, 
        y=0.9, 
        yref="paper", 
        text="Fine Test Set", 
        showarrow=True, 
        arrowhead=2, 
        arrowcolor="white"
    )

    fig.update_layout(
        title=f"Forecast - Store {store}",
        xaxis_title="Timestamp [t]",
        yaxis_title="Sales",
        template='plotly_dark',
        legend=dict(font=dict(size=15))
    )

    fig.show()

In [0]:
# Calcola metriche per ogni store
forecast_df = Y_hat_df_tft[['ds', 'TFT', 'unique_id']]
merged_df = pd.merge(test_df_tft[['ds', 'y', 'unique_id']], Y_hat_df_tft, on=['ds', 'unique_id'], how='left')
store_metrics_tft = []
for store in df_tft['unique_id'].unique():  # 5 store
    store_df = merged_df[merged_df['unique_id'] == store]
    metrics = calcola_metriche(store_df['y'], store_df['TFT'],
                               y_train=train_df_tft['y'], modelname="TFT")
    metrics['Store_ID'] = store
    store_metrics_tft.append(metrics)

In [0]:
# MEDIE E MEDIANE delle metriche
mae_values = []
mape_values = []
smape_values = []
rmse_values = []
mase_values_tft = []

# Collect metrics from each store
for i in range(len(store_metrics_tft)):
    mae_values.append(store_metrics_tft[i].iloc[0, 0])  
    mape_values.append(store_metrics_tft[i].iloc[2, 0])  
    smape_values.append(store_metrics_tft[i].iloc[3, 0]) 
    rmse_values.append(store_metrics_tft[i].iloc[1, 0]) 
    mase_values_tft.append(store_metrics_tft[i].iloc[4, 0]) 

# Calculate means
mae_mean = np.mean(mae_values)
mape_mean = np.mean(mape_values)
smape_mean = np.mean(smape_values)
mae_std_mean = np.mean(rmse_values)
mase_mean = np.mean(mase_values_tft)

# Calculate medians
mae_median = np.median(mae_values)
mape_median = np.median(mape_values)
smape_median = np.median(smape_values)
mae_std_median = np.median(rmse_values)
mase_median = np.median(mase_values_tft)

# Create a DataFrame to store the aggregated metrics
mean_metrics_tft = pd.DataFrame({
    'Metric': ['MAE', 'MAPE', 'SMAPE', 'RMSE', 'MASE'],
    'Mean_TFT': [mae_mean, mape_mean, smape_mean, mae_std_mean, mase_mean],
    'Median_TFT':[mae_median, mape_median, smape_median, mae_std_median, mase_median]
})
mean_metrics_tft

In [0]:
plt.plot(mase_values_tft)
plt.axhline(y=1, color='red', linestyle='--')
print("MASE >= 1:", np.sum(np.array(mase_values_tft) >= 1), "Percentuale:",
      (np.sum(np.array(mase_values_tft) > 1) / len(mase_values_tft) * 100).round(2), "% degli store")
print("MASE < 1:", np.sum(np.array(mase_values_tft) < 1), "Percentuale:",
      (np.sum(np.array(mase_values_tft) < 1) / len(mase_values_tft) * 100).round(2), "% degli store")

##### TFT Explainability

In [0]:
# # salvo il modello tft
# with open('pickles/TFT_model_favorita.pkl', 'wb') as file:
#     pickle.dump(tft_model, file)

In [0]:
# # carico il modello tft
with open('pickles/TFT_model_favorita.pkl', 'rb') as file:
    tft_model = pickle.load(file)

In [0]:
attention = tft_model.attention_weights()
def plot_attention(
    self, plot: str = "time", output: str = "plot", width: int = 800, height: int = 400
):
    """
    Plot the attention weights.

    Args:
        plot (str, optional): The type of plot to generate. Can be one of the following:
            - 'time': Display the mean attention weights over time.
            - 'all': Display the attention weights for each horizon.
            - 'heatmap': Display the attention weights as a heatmap.
            - An integer in the range [1, model.h) to display the attention weights for a specific horizon.
        output (str, optional): The type of output to generate. Can be one of the following:
            - 'plot': Display the plot directly.
            - 'figure': Return the plot as a figure object.
        width (int, optional): Width of the plot in pixels. Default is 800.
        height (int, optional): Height of the plot in pixels. Default is 400.

    Returns:
        matplotlib.figure.Figure: If `output` is 'figure', the function returns the plot as a figure object.
    """

    attention = (
        self.mean_on_batch(self.interpretability_params["attn_wts"])
        .mean(dim=0)
        .cpu()
        .numpy()
    )

    fig, ax = plt.subplots(figsize=(width / 100, height / 100))

    if plot == "time":
        attention = attention[self.input_size :, :].mean(axis=0)
        ax.plot(np.arange(-self.input_size, self.h), attention)
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Mean Attention")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "all":
        for i in range(self.input_size, attention.shape[0]):
            ax.plot(
                np.arange(-self.input_size, self.h),
                attention[i, :],
                label=f"horizon {i-self.input_size+1}",
            )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Attention per horizon")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "heatmap":
        cax = ax.imshow(
            attention,
            aspect="auto",
            cmap="viridis",
            extent=[-self.input_size, self.h, -self.input_size, self.h],
        )
        fig.colorbar(cax)
        ax.set_title("Attention Heatmap")
        ax.set_xlabel("Attention (current time step)")
        ax.set_ylabel("Attention (previous time step)")

    elif isinstance(plot, int) and (plot in np.arange(1, self.h + 1)):
        i = self.input_size + plot - 1
        ax.plot(
            np.arange(-self.input_size, self.h),
            attention[i, :],
            label=f"horizon {plot}",
        )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title(f"Attention weight for horizon {plot}")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    else:
        raise ValueError(
            'plot has to be in ["time","all","heatmap"] or integer in range(1,model.h)'
        )

    plt.tight_layout()

    if output == "plot":
        plt.show()
    elif output == "figure":
        return fig
    else:
        raise ValueError(f"Invalid output: {output}. Expected 'plot' or 'figure'.")

In [0]:
# MEAN ATTENTION
plot_attention(tft_model, plot="time")
# pesi di attenzione medi nel tempo su tutti gli orizzonti di previsione.
# La curva blu rappresenta quanto il modello ha "guardato" (attention) 
# quel punto temporale in input durante la previsione.
# Valori più alti significano che il modello ritiene quel punto temporale
# particolarmente rilevante per fare le previsioni

In [0]:
# ATTENTION OF A SPECIFIC TIME STEPS
plot_attention(tft_model, plot=4)
# pesi di attenzione per ogni orizzonte di previsione separatamente.

In [0]:
plot_attention(tft_model, plot="heatmap")

In [0]:
# PAST VARIABLE IMPORTANCES
feature_importances = tft_model.feature_importances()
print(feature_importances.keys())
print("\n ==========PAST FEATURE IMPORTANCES==========")
feature_importances["Past variable importance over time"].mean().sort_values().plot(kind="barh")

Questo grafico mostra quanto il modello ha fatto affidamento sulle feature passate \
(quelle che il modello ha potuto osservare prima del punto di previsione). Serve a costruire il contesto dell'input

In [0]:
# FUTURE VARIABLE IMPORTANCES
print("\n ==========FUTURE FEATURE IMPORTANCES==========")
feature_importances["Future variable importance over time"].mean().sort_values().plot(kind="barh")

Mostra quanto il modello ha sfruttato le feature che sono servite a guidare le previsioni passo-passo nel decoder

DIFFERENZA TRA PAST E FUTURE: \
La diversa importanza riflette quanto ciascuna variabile aiuta nell’analisi del contesto passato (encoder -->> PAST) \
o nella generazione delle previsioni (decoder -->> FUTURE).

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME
df_importances = feature_importances["Past variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances.index))

for col in df_importances.columns:
    p = ax.bar(np.arange(-len(df_importances), 0), df_importances[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances[col]
ax.set_title("Past variable importance over time")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)

plt.show()

Questo grafico mostra QUANDO (e QUANTO) nel passato ogni feature è stata più utile per costruire il contesto predittivo.

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME
df_importances2 = feature_importances["Future variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances2.index))
for col in df_importances2.columns:
    p = ax.bar(np.arange(-len(df_importances2), 0), df_importances2[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances2[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.grid(True)
ax.legend()
plt.show()

Per ogni feature viene mostrato quanto è stata importante per il modello a ciascun orizzonte futuro

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances3 = feature_importances["Past variable importance over time"]
mean_attention = (
    tft_model
    .attention_weights()[tft_model.input_size :, :]
    .mean(axis=0)[: tft_model.input_size]
)
df_importances3 = df_importances3.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances3.index))

for col in df_importances3.columns:
    p = ax.bar(np.arange(-len(df_importances3), 0), df_importances3[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances3[col]
ax.set_title("Past variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances3), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION

# Estrai l'importanza delle future variables
df_importances4 = feature_importances["Future variable importance over time"]

# Estrai tutte le attention weights mediate nel tempo
full_attention = tft_model.attention_weights()[tft_model.input_size:, :].mean(axis=0)

# Estrai solo la parte relativa ai 12 time steps futuri
n_future_steps = df_importances4.shape[0]
mean_attention_future = full_attention[-n_future_steps:]

# Applica la ponderazione riga per riga (sul tempo)
df_importances4 = df_importances4.multiply(mean_attention_future, axis=0)

# Plot
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances4.index))

for col in df_importances4.columns:
    ax.bar(
        np.arange(1, len(df_importances4) + 1), 
        df_importances4[col].values, 
        width=0.6, 
        label=col, 
        bottom=bottom
    )
    bottom += df_importances4[col].values

# Titoli e assi
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Future time steps")
ax.grid(True)

# Linea media di attenzione
ax.plot(
    np.arange(1, len(df_importances4) + 1),
    mean_attention_future,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention"
)
ax.legend()
plt.show()

In [0]:
tft_model.feature_importance_correlations()

#### CHRONOS

In [0]:
# Preparing dataset
df_chronos = create_time_series_features(df, target_col='y').dropna()
# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos['item_id'] = df_chronos['unique_id'] 
df_chronos.drop(columns="unique_id", inplace=True) 
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Definizione delle covariate note
known_covariates_names = ['onpromotion', 'promo', 'oil price', 'giorno_settimana',
       'settimana_del_mese', 'settimana_del_anno', 'giorno_del_mese',
       'mese_del_anno', 'rolling_mean_30', 'sales_lag_1', 'sales_lag_2',
       'sales_lag_7', 'sales_lag_14', 'sales_lag_28', 'lag_1', 'lag_2',
       'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'diff_1', 'diff_7',
       'dayofweek', 'month', 'quarter', 'day_sin', 'day_cos', 'month_sin',
       'month_cos', 'rolling_mean_3', 'rolling_mean_7', 'rolling_std_3',
       'rolling_std_7']

# Split train e test
#divido train/test ogni serie storica
def split_train_test_by_id(df, test_frac=0.1):
    train_list = []
    test_list = []
    for uid, group in df.groupby("item_id"):
        group = group.sort_values("timestamp")  # Ordina temporalmente
        split_idx = int(len(group) * (1 - test_frac))
        train_list.append(group.iloc[:split_idx])
        test_list.append(group.iloc[split_idx:])
    return pd.concat(train_list), pd.concat(test_list)
train_df_chronos, test_df_chronos = split_train_test_by_id(df_chronos, test_frac=0.1)

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [0]:
# Fit del modello
predictor = TimeSeriesPredictor(
    prediction_length=h,
    eval_metric="MASE",
    target="target",
    known_covariates_names=known_covariates_names,
    freq="B" 
)

# Aggiunta di più modelli nella configurazione per migliorare le performance
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {"model_path": "bolt_small", "ag_args": {"name_suffix": "ZeroShot"}},
            {
                "model_path": "bolt_mini",
                "covariate_regressor": "NN_TORCH", 
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "WithRegressor"},
            }
        ],
    },
    time_limit=600,
    enable_ensemble=True, 
)

# Valutazione del modello
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [0]:
# Previsioni
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)
train_df_chronos.set_index(["item_id","timestamp"],inplace=True)
test_df_chronos.set_index(["item_id","timestamp"],inplace=True)

predictions = predictor.predict(
    data=train_df_chronos, 
    known_covariates=test_df_chronos
)
predictions.reset_index(inplace=True)
# Previsioni
predictions_0shot = predictor.predict(
    data=train_df_chronos, 
    known_covariates=test_df_chronos,
    model = "ChronosZeroShot[bolt_small]"
)
predictions_0shot.reset_index(inplace=True)

In [0]:
# forecast plot per store
store_ids = [1,3,5,20,35,54] #5 store a caso
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)
for store in store_ids:
    train_store = train_df_chronos[train_df_chronos['item_id'] == store]
    test_store = test_df_chronos[test_df_chronos['item_id'] == store]
    forecast_store = predictions[predictions['item_id'] == store]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=train_store['timestamp'], y=train_store['target'], mode='lines', name='Training Data'))
    fig.add_trace(go.Scatter(x=test_store['timestamp'], y=test_store['target'], mode='lines', name='Testing Data'))
    fig.add_trace(go.Scatter(
        x=forecast_store['timestamp'], 
        y=forecast_store['mean'], 
        mode='lines', 
        name='CHRONOS Forecast', 
        line=dict(color='green')
    ))

    ultimo_test = test_store['timestamp'].max()
    fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
    fig.add_annotation(
        x=ultimo_test, 
        y=0.9, 
        yref="paper", 
        text="Fine Test Set", 
        showarrow=True, 
        arrowhead=2, 
        arrowcolor="white"
    )

    fig.update_layout(
        title=f"Forecast - Store {store}",
        xaxis_title="Timestamp [t]",
        yaxis_title="Sales",
        template='plotly_dark',
        legend=dict(font=dict(size=15))
    )

    fig.show()

In [0]:
# Calcola metriche per ogni store
test_df_chronos.reset_index(inplace=True)
forecast_df = predictions[['timestamp', 'mean', 'item_id']]
merged_df = pd.merge(test_df_chronos[['timestamp', 'target', 'item_id']], predictions, 
                     on=['timestamp', 'item_id'], how='right')
store_metrics_chronos = []
df_chronos.reset_index(inplace=True)
for store in df_chronos['item_id'].unique():  # 5 store
    store_df = merged_df[merged_df['item_id'] == store]
    metrics = calcola_metriche(y_true=store_df['target'], y_pred=store_df['mean'],
                               y_train=train_df_chronos['target'],
                               modelname="Chronos")
    metrics['Store_ID'] = store
    store_metrics_chronos.append(metrics)

In [0]:
store_metrics_chronos[13] #posso selezionare qualunque store io voglia valutare

In [0]:
# MEDIE E MEDIANE delle metriche
mae_values = []
mape_values = []
smape_values = []
rmse_values = []
mase_values_chronos = []

# Collect metrics from each store
for i in range(len(store_metrics_chronos)):
    mae_values.append(store_metrics_chronos[i].iloc[0, 0])  
    mape_values.append(store_metrics_chronos[i].iloc[2, 0])  
    smape_values.append(store_metrics_chronos[i].iloc[3, 0]) 
    rmse_values.append(store_metrics_chronos[i].iloc[1, 0]) 
    mase_values_chronos.append(store_metrics_chronos[i].iloc[4, 0]) 

# Calculate means
mae_mean = np.mean(mae_values)
mape_mean = np.mean(mape_values)
smape_mean = np.mean(smape_values)
rmse_mean = np.mean(rmse_values)
mase_mean = np.mean(mase_values_chronos)

# Calculate medians
mae_median = np.median(mae_values)
mape_median = np.median(mape_values)
smape_median = np.median(smape_values)
rmse_median = np.median(rmse_values)
mase_median = np.median(mase_values_chronos)

# Create a DataFrame to store the aggregated metrics
mean_metrics_chronos = pd.DataFrame({
    'Metric': ['MAE', 'MAPE', 'SMAPE', 'RMSE', 'MASE'],
    'Mean_Chronos': [mae_mean, mape_mean, smape_mean, rmse_mean, mase_mean],
    'Median_Chronos':[mae_median, mape_median, smape_median, rmse_median, mase_median]
})
mean_metrics_chronos

In [0]:
plt.plot(mase_values_chronos)
plt.axhline(y=1, color='red', linestyle='--')
print("MASE >= 1:", np.sum(np.array(mase_values_chronos) >= 1), "Percentuale:",
      (np.sum(np.array(mase_values_chronos) > 1) / len(mase_values_chronos) * 100).round(2), "% degli store")
print("MASE < 1:", np.sum(np.array(mase_values_chronos) < 1), "Percentuale:",
      (np.sum(np.array(mase_values_chronos) < 1) / len(mase_values_chronos) * 100).round(2), "% degli store")

#### valutazioni finali

In [0]:
## PLOT UNICO con tutte le previsioni per quegli store
# forecast plot per store
store_ids = ['1','3','5','20','35','54'] # degli store a caso

for store in store_ids:
    train_store = train_df_np[train_df_np['ID'] == store]
    test_store = test_df_np[test_df_np['ID'] == store]
    forecast_store_np = forecast_test[forecast_test['ID'] == store]
    
    predictions.rename(columns={'item_id':'ID'}, inplace=True)
    predictions['ID'] = predictions['ID'].astype(str)
    forecast_store_chronos = predictions[predictions['ID'] == store]
    
    predictions_0shot.rename(columns={'item_id':'ID'}, inplace=True)
    predictions_0shot['ID'] = predictions_0shot['ID'].astype(str)
    forecast_0shot_store_chronos = predictions_0shot[predictions_0shot['ID'] == store]
    
    
    Y_hat_df_lstm.rename(columns={'unique_id':'ID'}, inplace=True)
    Y_hat_df_lstm['ID'] = Y_hat_df_lstm['ID'].astype(str)
    forecast_store_lstm = Y_hat_df_lstm[Y_hat_df_lstm['ID'] == store]
    
    # Y_hat_df_tft.rename(columns={'unique_id':'ID'}, inplace=True)
    # Y_hat_df_tft['ID'] = Y_hat_df_tft['ID'].astype(str)
    # forecast_store_tft = Y_hat_df_tft[Y_hat_df_tft['ID'] == store]
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=train_store['ds'], y=train_store['y'], mode='lines', name='Training Data'))
    fig.add_trace(go.Scatter(x=test_store['ds'], y=test_store['y'], mode='lines', name='Testing Data'))
    fig.add_trace(go.Scatter(
        x=forecast_store_np['ds'], 
        y=forecast_store_np['yhat1'], 
        mode='lines', 
        name='NeuralProphet', 
        line=dict(color='pink')
    ))
    fig.add_trace(go.Scatter(
    x=forecast_store_lstm['ds'], 
    y=forecast_store_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='yellow')
    ))
    # fig.add_trace(go.Scatter(
    # x=forecast_store_tft['ds'], 
    # y=forecast_store_tft['TFT'], 
    # mode='lines', 
    # name='TFT', 
    # line=dict(color='lightblue')
    # ))
    fig.add_trace(go.Scatter(
    x=forecast_store_chronos['timestamp'], 
    y=forecast_store_chronos['mean'], 
    mode='lines', 
    name='Chronos', 
    line=dict(color='lightgreen')
    ))
    # fig.add_trace(go.Scatter(
    # x=forecast_0shot_store_chronos['timestamp'], 
    # y=forecast_0shot_store_chronos['mean'], 
    # mode='lines', 
    # name='Chronos 0 shot', 
    # line=dict(color='green')
    # ))
    
    fig.update_layout(
        title=f"Forecast - Store {store}",
        xaxis_title="Timestamp [t]",
        yaxis_title="Sales",
        template='plotly_dark',
        legend=dict(font=dict(size=15))
    )
    fig.show()

In [0]:
final_metrics = pd.merge(mean_metrics_lstm, mean_metrics_np, on='Metric', how='outer')
final_metrics = pd.merge(final_metrics, mean_metrics_chronos, on='Metric', how='outer')
final_metrics = pd.merge(final_metrics, mean_metrics_tft, on='Metric', how='outer')
final_metrics

In [0]:
## PLOT DEI MASE PER STORE
# Imposta stile
sns.set(style='whitegrid')
plt.figure(figsize=(12, 6))

# Linea di riferimento a MASE = 1
plt.axhline(y=1, color='red', linestyle='--', label='MASE = 1')

# Plot dei MASE
plt.plot(mase_values_lstm, label='LSTM', marker='o')
plt.plot(mase_values_np, label='NeuralProphet', marker='s')
plt.plot(mase_values_chronos, label='Chronos', marker='^')
plt.plot(mase_values_tft, label='TFT', marker='d',color="aquamarine") 

# Etichette e titolo
plt.title('MASE per Store', fontsize=16)
plt.xlabel('Store ID (indice)', fontsize=12)
plt.ylabel('MASE', fontsize=12)
store_ids = df['unique_id'].unique()
plt.xticks(ticks=range(len(store_ids)), labels=store_ids, rotation=90)

# Legenda
plt.legend(title='Modelli')
plt.tight_layout()
plt.show()

### **FAVORITA SALES**, panel (multistep)

H=90 gg

In [0]:
#preparazione dataset
df = favorita_train.reset_index().rename(columns={'DATE_TIME': 'ds','sales':'y','Store':'unique_id'})
df['ds'] = pd.to_datetime(df['ds'])
df.drop(columns=['index'],inplace=True)
df = df[df['unique_id'] != 52] #dava problemi nei vari fit (tutti valori costanti, no variabilità)

In [0]:
# funzione create features per store
def create_time_series_features(df, target_col):
    df = df.copy()
    
    # Ordina per sicurezza
    df = df.sort_values(['unique_id', 'ds'])

    # Applicazione delle feature per ogni store
    def fe_group(group):
        # Lag features (1-7)
        for lag in range(1, 8):
            group[f'lag_{lag}'] = group[target_col].shift(lag)

        # Differencing features
        group['diff_1'] = (group[target_col] - group[target_col].shift(1)).shift(1)
        group['diff_7'] = (group[target_col] - group[target_col].shift(7)).shift(1)

        # Rolling means & std
        group['rolling_mean_3'] = group[target_col].rolling(window=3).mean().shift(1)
        group['rolling_mean_7'] = group[target_col].rolling(window=7).mean().shift(1)
        group['rolling_std_3'] = group[target_col].rolling(window=3).std().shift(1)
        group['rolling_std_7'] = group[target_col].rolling(window=7).std().shift(1)

        return group

    # Applica la funzione per ogni store
    df = df.groupby('unique_id', group_keys=False).apply(fe_group)
    df['quarter'] = df['ds'].dt.quarter

    # Feature cicliche
    df['day_sin'] = np.sin(2 * np.pi * df['giorno_settimana'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['giorno_settimana'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['mese_del_anno'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['mese_del_anno'] / 12)

    return df

In [0]:
h = 90
futr_exog_list = ['giorno_settimana','settimana_del_mese', 
                  'settimana_del_anno','giorno_del_mese', 'mese_del_anno',
                  'day_sin',  'day_cos', 'month_sin', 'month_cos', 'quarter']
future_features = ['ds', 'y'] + futr_exog_list

##### LSTM

In [0]:
df_lstm = create_time_series_features(df, target_col='y') 
df_lstm = df_lstm.dropna().reset_index(drop=True)

In [0]:
# 2. Suddivisione in train/test
#divido train/test ogni serie storica
def split_train_test_by_id(df, test_frac=0.1):
    train_list = []
    test_list = []
    for uid, group in df.groupby("unique_id"):
        group = group.sort_values("ds")  # Ordina temporalmente
        split_idx = int(len(group) * (1 - test_frac))
        train_list.append(group.iloc[:split_idx])
        test_list.append(group.iloc[split_idx:])
    return pd.concat(train_list), pd.concat(test_list)
train_df_lstm, test_df_lstm = split_train_test_by_id(df_lstm, test_frac=0.076)
train_df_lstm.shape, test_df_lstm.shape

In [0]:
# creating static df
# Step 1: One-hot encoding normale
ohe_df = pd.get_dummies(train_df_lstm['unique_id'].drop_duplicates(), prefix='store')

# Step 2: Inverti i valori (0 -> 1 e 1 -> 0)
ohe_df = 1 - ohe_df

# Step 3: Aggiungi la colonna unique_id
ohe_df['unique_id'] = train_df_lstm['unique_id'].drop_duplicates().values

# Step 4: Reordina colonne
static_df = ohe_df[['unique_id'] + [col for col in ohe_df.columns if col != 'unique_id']]

# Cast a int
static_df = static_df.astype(int)

In [0]:
# === FIT FINALE DEL MODELLO ===
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

models = [LSTM(h=h, 
               input_size=int(4*h),
               loss=DistributionLoss(distribution="StudentT", level=[95]),
               scaler_type='standard',
               encoder_n_layers=3, 
               encoder_hidden_size=256,
               decoder_hidden_size=128,
               decoder_layers=2, 
               max_steps=200,
               futr_exog_list=futr_exog_list,
               batch_size=128, 
               learning_rate=0.005, 
               recurrent=True,
               start_padding_enabled=True
               )
          ]
nf = NeuralForecast(models=models, freq='B') 
print("=== STARTING THE MODEL FIT ===")

start_time = time.time()
#fit 
nf.fit(train_df_lstm, val_size=4*h)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
#predictions
Y_hat_df_lstm = nf.predict(futr_df=test_df_lstm[['unique_id', 'ds'] + futr_exog_list])
#salvo in pickle
with open('pickles/LSTM_favorita_multistep.pkl', 'wb') as file:
    pickle.dump(Y_hat_df_lstm, file)

CARICO LE PREVISIONI

In [0]:
# # #carico le previsioni:
with open('pickles/LSTM_favorita_multistep.pkl', 'rb') as file:
    Y_hat_df_lstm = pickle.load(file)

In [0]:
# forecast plot per store
store_ids = [1,3,5,20,35,54] #5 store a caso

for store in store_ids:
    train_store = train_df_lstm[train_df_lstm['unique_id'] == store]
    test_store = test_df_lstm[test_df_lstm['unique_id'] == store]
    forecast_store = Y_hat_df_lstm[Y_hat_df_lstm['unique_id'] == store]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=train_store['ds'], y=train_store['y'], mode='lines', name='Training Data'))
    fig.add_trace(go.Scatter(x=test_store['ds'], 
                             y=test_store['y'],
                             mode='lines', 
                             name='Testing Data',
                             #line=dict(color='red')
                             ))
    fig.add_trace(go.Scatter(
        x=forecast_store['ds'], 
        y=forecast_store['LSTM'], 
        mode='lines', 
        name='LSTM Forecast', 
        line=dict(color='green')
    ))

    ultimo_test = test_store['ds'].max()
    fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
    fig.add_annotation(
        x=ultimo_test, 
        y=0.9, 
        yref="paper", 
        text="Fine Test Set", 
        showarrow=True, 
        arrowhead=2, 
        arrowcolor="white"
    )

    fig.update_layout(
        title=f"Forecast - Store {store}",
        xaxis_title="Timestamp [t]",
        yaxis_title="Sales",
        template='plotly_dark',
        legend=dict(font=dict(size=15))
    )

    fig.show()

In [0]:
# Calcola metriche per ogni store
forecast_df = Y_hat_df_lstm[['ds', 'LSTM', 'unique_id']]
merged_df = pd.merge(test_df_lstm[['ds', 'y', 'unique_id']], Y_hat_df_lstm, on=['ds', 'unique_id'], how='left')
store_metrics_lstm = []
for store in df_lstm['unique_id'].unique():  
    store_df = merged_df[merged_df['unique_id'] == store]
    metrics = calcola_metriche(store_df['y'], store_df['LSTM'],
                               y_train=train_df_lstm['y'], modelname="LSTM")
    metrics['Store_ID'] = store
    store_metrics_lstm.append(metrics)

In [0]:
# MEDIE E MEDIANE delle metriche
mae_values = []
mape_values = []
smape_values = []
rmse_values = []
mase_values_lstm = []

# Collect metrics from each store
for i in range(len(store_metrics_lstm)):
    mae_values.append(store_metrics_lstm[i].iloc[0, 0])  
    mape_values.append(store_metrics_lstm[i].iloc[2, 0])  
    smape_values.append(store_metrics_lstm[i].iloc[3, 0]) 
    rmse_values.append(store_metrics_lstm[i].iloc[1, 0]) 
    mase_values_lstm.append(store_metrics_lstm[i].iloc[4, 0]) 

# Calculate means
mae_mean = np.mean(mae_values)
mape_mean = np.mean(mape_values)
smape_mean = np.mean(smape_values)
rmse_mean = np.mean(rmse_values)
mase_mean = np.mean(mase_values_lstm)

# Calculate medians
mae_median = np.median(mae_values)
mape_median = np.median(mape_values)
smape_median = np.median(smape_values)
rmse_median = np.median(rmse_values)
mase_median = np.median(mase_values_lstm)

# Create a DataFrame to store the aggregated metrics
mean_metrics_lstm = pd.DataFrame({
    'Metric': ['MAE', 'MAPE', 'SMAPE', 'RMSE', 'MASE'],
    'Mean_LSTM': [mae_mean, mape_mean, smape_mean, rmse_mean, mase_mean],
    'Median_LSTM':[mae_median, mape_median, smape_median, rmse_median, mase_median]
})
mean_metrics_lstm

In [0]:
plt.plot(mase_values_lstm)
plt.axhline(y=1, color='red', linestyle='--')
print("MASE >= 1:", np.sum(np.array(mase_values_lstm) >= 1), "Percentuale:",
      (np.sum(np.array(mase_values_lstm) > 1) / len(mase_values_lstm) * 100).round(2), "% degli store")
print("MASE < 1:", np.sum(np.array(mase_values_lstm) < 1), "Percentuale:",
      (np.sum(np.array(mase_values_lstm) < 1) / len(mase_values_lstm) * 100).round(2), "% degli store")

##### NEURAL PROPHET

In [0]:
# 1. Prepara il dataset
df_np = create_time_series_features(df, target_col='y').dropna()
df_np['ID'] = df_np['unique_id'].astype(str)
df_np.drop(columns=['unique_id'], inplace=True)
df_np['settimana_del_anno'] = df_np['settimana_del_anno'].astype(int)

In [0]:
# 2. Suddivisione in train/test
#divido train/test ogni serie storica
def split_train_test_by_id(df, test_frac=0.1):
    train_list = []
    test_list = []
    for uid, group in df.groupby("ID"):
        group = group.sort_values("ds")  # Ordina temporalmente
        split_idx = int(len(group) * (1 - test_frac))
        train_list.append(group.iloc[:split_idx])
        test_list.append(group.iloc[split_idx:])
    return pd.concat(train_list), pd.concat(test_list)
train_df_np, test_df_np = split_train_test_by_id(df_np, test_frac=0.076)
train_df_np.shape, test_df_np.shape

In [0]:
# FIT
# Fissa i seed per riproducibilità
import random
import time
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

np_df_train = train_df_np[future_features + ['ID']].copy()

# Modello senza regressori esterni
neuralprophet = NeuralProphet(
    quantiles=[0.025, 0.975],    
    learning_rate=0.01, 
    batch_size=64,
    daily_seasonality=True,    # Impara la stagionalità dai dati
    weekly_seasonality=True,   # Impara pattern settimanali
    yearly_seasonality=True,   # Impara pattern annuali
    loss_func='Huber'
)
# Aggiungi solo i regressori deterministici
for reg in futr_exog_list:
    neuralprophet.add_future_regressor(reg)
start_time = time.time()
metrics_df = neuralprophet.fit(np_df_train, freq="B",epochs=100)
end_time = time.time()
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
# plot delle metriche in training
plt.figure(figsize=(14, 7))
plt.plot(metrics_df['epoch'], metrics_df['Loss'], label='Train Loss', color='blue')
plt.plot(metrics_df['epoch'], metrics_df['MAE'], label='Train MAE', color='green')
plt.plot(metrics_df['epoch'], metrics_df['RMSE'], label='Train RMSE', color='red')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training & Validation Metrics by Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [0]:
# 7. Previsione
forecast_test = neuralprophet.predict(test_df_np[future_features+['ID']])

In [0]:
# salvo le prediction sul test in locale
with open('pickles/NP_favorita_multistep.pkl', 'wb') as file:
    pickle.dump(forecast_test, file)

CARICO LE PREVISIONI

In [0]:
# carico le previsioni del test
with open('pickles/NP_favorita_multistep.pkl', 'rb') as file:
    forecast_test = pickle.load(file)

In [0]:
# forecast plot per store
store_ids = ['1','3','5','20','35','54'] # degli store a caso

for store in store_ids:
    train_store = train_df_np[train_df_np['ID'] == store]
    test_store = test_df_np[test_df_np['ID'] == store]
    forecast_store = forecast_test[forecast_test['ID'] == store]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=train_store['ds'], y=train_store['y'], mode='lines', name='Training Data'))
    fig.add_trace(go.Scatter(x=test_store['ds'], y=test_store['y'], mode='lines', name='Testing Data'))
    fig.add_trace(go.Scatter(
        x=forecast_store['ds'], 
        y=forecast_store['yhat1'], 
        mode='lines', 
        name='NeuralProphet Forecast', 
        line=dict(color='green')
    ))

    ultimo_test = test_store['ds'].max()
    fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
    fig.add_annotation(
        x=ultimo_test, 
        y=0.9, 
        yref="paper", 
        text="Fine Test Set", 
        showarrow=True, 
        arrowhead=2, 
        arrowcolor="white"
    )

    fig.update_layout(
        title=f"Forecast - Store {store}",
        xaxis_title="Timestamp [t]",
        yaxis_title="Sales",
        template='plotly_dark',
        legend=dict(font=dict(size=15))
    )

    fig.show()

In [0]:
# Calcola metriche per ogni store
forecast_df = forecast_test[['ds', 'yhat1', 'ID']]
merged_df = pd.merge(test_df_np[['ds', 'y', 'ID']], forecast_df, on=['ds', 'ID'], how='left')
store_metrics_np = []
for store in df_np['ID'].unique():  # primi 5 store
    store_df = merged_df[merged_df['ID'] == store]
    metrics = calcola_metriche(store_df['y'], store_df['yhat1'],
                               y_train=train_df_np['y'], modelname="NeuralProphet")
    metrics['Store_ID'] = store
    store_metrics_np.append(metrics)

In [0]:
# MEDIE E MEDIANE delle metriche
mae_values = []
mape_values = []
smape_values = []
rmse_values = []
mase_values_np = []

# Collect metrics from each store
for i in range(len(store_metrics_np)):
    mae_values.append(store_metrics_np[i].iloc[0, 0])  
    mape_values.append(store_metrics_np[i].iloc[2, 0])  
    smape_values.append(store_metrics_np[i].iloc[3, 0]) 
    rmse_values.append(store_metrics_np[i].iloc[1, 0]) 
    mase_values_np.append(store_metrics_np[i].iloc[4, 0]) 

# Calculate means
mae_mean = np.mean(mae_values)
mape_mean = np.mean(mape_values)
smape_mean = np.mean(smape_values)
rmse_mean = np.mean(rmse_values)
mase_mean = np.mean(mase_values_np)

# Calculate medians
mae_median = np.median(mae_values)
mape_median = np.median(mape_values)
smape_median = np.median(smape_values)
rmse_median = np.median(rmse_values)
mase_median = np.median(mase_values_np)

# Create a DataFrame to store the aggregated metrics
mean_metrics_np = pd.DataFrame({
    'Metric': ['MAE', 'MAPE', 'SMAPE', 'RMSE', 'MASE'],
    'Mean_NP': [mae_mean, mape_mean, smape_mean, rmse_mean, mase_mean],
    'Median_NP':[mae_median, mape_median, smape_median, rmse_median, mase_median]
})
mean_metrics_np

In [0]:
plt.plot(mase_values_np)
plt.axhline(y=1, color='red', linestyle='--')
print("MASE >= 1:", np.sum(np.array(mase_values_np) > 1), "Percentuale:",
      (np.sum(np.array(mase_values_np) >= 1) / len(mase_values_np) * 100).round(2), "% degli store")
print("MASE < 1:", np.sum(np.array(mase_values_np) < 1), "Percentuale:",
      (np.sum(np.array(mase_values_np) < 1) / len(mase_values_np) * 100).round(2), "% degli store")

#### TFT

In [0]:
# Preparing dataset
df_tft = create_time_series_features(df, target_col='y') 
df_tft = df_tft.dropna().reset_index(drop=True)
# 2. Suddivisione in train/test
#divido train/test ogni serie storica
def split_train_test_by_id(df, test_frac=0.1):
    train_list = []
    test_list = []
    for uid, group in df.groupby("unique_id"):
        group = group.sort_values("ds")  # Ordina temporalmente
        split_idx = int(len(group) * (1 - test_frac))
        train_list.append(group.iloc[:split_idx])
        test_list.append(group.iloc[split_idx:])
    return pd.concat(train_list), pd.concat(test_list)
train_df_tft, test_df_tft = split_train_test_by_id(df_tft, test_frac=0.076)
train_df_tft.shape, test_df_tft.shape


In [0]:
# creating static df
# Step 1: One-hot encoding normale
ohe_df = pd.get_dummies(train_df_tft['unique_id'].drop_duplicates(), prefix='store')

# Step 2: Inverti i valori (0 -> 1 e 1 -> 0)
ohe_df = 1 - ohe_df

# Step 3: Aggiungi la colonna unique_id
ohe_df['unique_id'] = train_df_tft['unique_id'].drop_duplicates().values

# Step 4: Reordina colonne
static_df = ohe_df[['unique_id'] + [col for col in ohe_df.columns if col != 'unique_id']]

# Cast a int
static_df = static_df.astype(int)

In [0]:
# model fit
h = len(test_df_tft) // train_df_tft['unique_id'].nunique()
nf = NeuralForecast(
    models=[
        TFT(
            h=h,
            input_size=h*3,
            hidden_size=120, # deve essere divisibile per 4
            grn_activation="ReLU",
            rnn_type="lstm",
            n_rnn_layers=4, 
            one_rnn_initial_state=False,
            loss=DistributionLoss(distribution="StudentT", level=[95]),
            learning_rate=0.0075, # prima 0.01
            futr_exog_list=futr_exog_list,
            hist_exog_list=hist_exog_features,
            max_steps=100,
            val_check_steps=25,
            early_stop_patience_steps=10,
            scaler_type="standard",
            windows_batch_size=128, #prima 128
            enable_progress_bar=True,
        ),
    ],
    freq="B", 
)
start_time = time.time()
#fit 
nf.fit(df=train_df_tft, static_df=static_df, val_size=h)
end_time = time.time()
tft_model = nf.models[0]
training_time = end_time - start_time
print(f"Tempo di training: {training_time:.4f} secondi")

In [0]:
Y_hat_df_tft = nf.predict(futr_df=test_df_tft)
#salvo in pickle
with open('pickles/TFT_favorita_multistep.pkl', 'wb') as file:
    pickle.dump(Y_hat_df_tft, file)

Carico le previsioni TFT

In [0]:
# #carico le previsioni:
with open('pickles/TFT_favorita_multistep.pkl', 'rb') as file:
    Y_hat_df_tft = pickle.load(file)

In [0]:
# forecast plot per store
store_ids = [1,3,5,20,35,54] #5 store a caso

for store in store_ids:
    train_store = train_df_tft[train_df_tft['unique_id'] == store]
    test_store = test_df_tft[test_df_tft['unique_id'] == store]
    forecast_store = Y_hat_df_tft[Y_hat_df_tft['unique_id'] == store]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=train_store['ds'], y=train_store['y'], mode='lines', name='Training Data'))
    fig.add_trace(go.Scatter(x=test_store['ds'], y=test_store['y'], mode='lines', name='Testing Data'))
    fig.add_trace(go.Scatter(
        x=forecast_store['ds'], 
        y=forecast_store['TFT'], 
        mode='lines', 
        name='TFT Forecast', 
        line=dict(color='green')
    ))

    ultimo_test = test_store['ds'].max()
    fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
    fig.add_annotation(
        x=ultimo_test, 
        y=0.9, 
        yref="paper", 
        text="Fine Test Set", 
        showarrow=True, 
        arrowhead=2, 
        arrowcolor="white"
    )

    fig.update_layout(
        title=f"Forecast - Store {store}",
        xaxis_title="Timestamp [t]",
        yaxis_title="Sales",
        template='plotly_dark',
        legend=dict(font=dict(size=15))
    )

    fig.show()

In [0]:
# Calcola metriche per ogni store
forecast_df = Y_hat_df_tft[['ds', 'TFT', 'unique_id']]
merged_df = pd.merge(test_df_tft[['ds', 'y', 'unique_id']], Y_hat_df_tft, on=['ds', 'unique_id'], how='left')
store_metrics_tft = []
for store in df_tft['unique_id'].unique():  # 5 store
    store_df = merged_df[merged_df['unique_id'] == store]
    metrics = calcola_metriche(store_df['y'], store_df['TFT'],
                               y_train=train_df_tft['y'], modelname="TFT")
    metrics['Store_ID'] = store
    store_metrics_tft.append(metrics)

In [0]:
# MEDIE E MEDIANE delle metriche
mae_values = []
mape_values = []
smape_values = []
rmse_values = []
mase_values_tft = []

# Collect metrics from each store
for i in range(len(store_metrics_tft)):
    mae_values.append(store_metrics_tft[i].iloc[0, 0])  
    mape_values.append(store_metrics_tft[i].iloc[2, 0])  
    smape_values.append(store_metrics_tft[i].iloc[3, 0]) 
    rmse_values.append(store_metrics_tft[i].iloc[1, 0]) 
    mase_values_tft.append(store_metrics_tft[i].iloc[4, 0]) 

# Calculate means
mae_mean = np.mean(mae_values)
mape_mean = np.mean(mape_values)
smape_mean = np.mean(smape_values)
mae_std_mean = np.mean(rmse_values)
mase_mean = np.mean(mase_values_tft)

# Calculate medians
mae_median = np.median(mae_values)
mape_median = np.median(mape_values)
smape_median = np.median(smape_values)
mae_std_median = np.median(rmse_values)
mase_median = np.median(mase_values_tft)

# Create a DataFrame to store the aggregated metrics
mean_metrics_tft = pd.DataFrame({
    'Metric': ['MAE', 'MAPE', 'SMAPE', 'RMSE', 'MASE'],
    'Mean_TFT': [mae_mean, mape_mean, smape_mean, mae_std_mean, mase_mean],
    'Median_TFT':[mae_median, mape_median, smape_median, mae_std_median, mase_median]
})
mean_metrics_tft

In [0]:
plt.plot(mase_values_tft)
plt.axhline(y=1, color='red', linestyle='--')
print("MASE >= 1:", np.sum(np.array(mase_values_tft) >= 1), "Percentuale:",
      (np.sum(np.array(mase_values_tft) > 1) / len(mase_values_tft) * 100).round(2), "% degli store")
print("MASE < 1:", np.sum(np.array(mase_values_tft) < 1), "Percentuale:",
      (np.sum(np.array(mase_values_tft) < 1) / len(mase_values_tft) * 100).round(2), "% degli store")

##### TFT EXPLAINABILITY

In [0]:
# salvo il modello tft
# with open('pickles/TFT_model_Favorita_multistep.pkl', 'wb') as file:
#     pickle.dump(tft_model, file)

In [0]:
# # carico il modello tft
with open('pickles/TFT_model_Favorita_multistep.pkl', 'rb') as file:
    tft_model = pickle.load(file)

In [0]:
attention = tft_model.attention_weights()
def plot_attention(
    self, plot: str = "time", output: str = "plot", width: int = 800, height: int = 400
):
    """
    Plot the attention weights.

    Args:
        plot (str, optional): The type of plot to generate. Can be one of the following:
            - 'time': Display the mean attention weights over time.
            - 'all': Display the attention weights for each horizon.
            - 'heatmap': Display the attention weights as a heatmap.
            - An integer in the range [1, model.h) to display the attention weights for a specific horizon.
        output (str, optional): The type of output to generate. Can be one of the following:
            - 'plot': Display the plot directly.
            - 'figure': Return the plot as a figure object.
        width (int, optional): Width of the plot in pixels. Default is 800.
        height (int, optional): Height of the plot in pixels. Default is 400.

    Returns:
        matplotlib.figure.Figure: If `output` is 'figure', the function returns the plot as a figure object.
    """

    attention = (
        self.mean_on_batch(self.interpretability_params["attn_wts"])
        .mean(dim=0)
        .cpu()
        .numpy()
    )

    fig, ax = plt.subplots(figsize=(width / 100, height / 100))

    if plot == "time":
        attention = attention[self.input_size :, :].mean(axis=0)
        ax.plot(np.arange(-self.input_size, self.h), attention)
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Mean Attention")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "all":
        for i in range(self.input_size, attention.shape[0]):
            ax.plot(
                np.arange(-self.input_size, self.h),
                attention[i, :],
                label=f"horizon {i-self.input_size+1}",
            )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title("Attention per horizon")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    elif plot == "heatmap":
        cax = ax.imshow(
            attention,
            aspect="auto",
            cmap="viridis",
            extent=[-self.input_size, self.h, -self.input_size, self.h],
        )
        fig.colorbar(cax)
        ax.set_title("Attention Heatmap")
        ax.set_xlabel("Attention (current time step)")
        ax.set_ylabel("Attention (previous time step)")

    elif isinstance(plot, int) and (plot in np.arange(1, self.h + 1)):
        i = self.input_size + plot - 1
        ax.plot(
            np.arange(-self.input_size, self.h),
            attention[i, :],
            label=f"horizon {plot}",
        )
        ax.axvline(
            x=0, color="black", linewidth=3, linestyle="--", label="prediction start"
        )
        ax.set_title(f"Attention weight for horizon {plot}")
        ax.set_xlabel("time")
        ax.set_ylabel("Attention")
        ax.legend()

    else:
        raise ValueError(
            'plot has to be in ["time","all","heatmap"] or integer in range(1,model.h)'
        )

    plt.tight_layout()

    if output == "plot":
        plt.show()
    elif output == "figure":
        return fig
    else:
        raise ValueError(f"Invalid output: {output}. Expected 'plot' or 'figure'.")

In [0]:
# MEAN ATTENTION
plot_attention(tft_model, plot="time")
# pesi di attenzione medi nel tempo su tutti gli orizzonti di previsione.
# La curva blu rappresenta quanto il modello ha "guardato" (attention) 
# quel punto temporale in input durante la previsione.
# Valori più alti significano che il modello ritiene quel punto temporale
# particolarmente rilevante per fare le previsioni

In [0]:
# ATTENTION OF A SPECIFIC TIME STEPS
plot_attention(tft_model, plot=4)
# pesi di attenzione per ogni orizzonte di previsione separatamente.

In [0]:
plot_attention(tft_model, plot="heatmap")

In [0]:
# PAST VARIABLE IMPORTANCES
feature_importances = tft_model.feature_importances()
print(feature_importances.keys())
print("\n ==========PAST FEATURE IMPORTANCES==========")
feature_importances["Past variable importance over time"].mean().sort_values().plot(kind="barh")

Questo grafico mostra quanto il modello ha fatto affidamento sulle feature passate \
(quelle che il modello ha potuto osservare prima del punto di previsione). Serve a costruire il contesto dell'input

In [0]:
# FUTURE VARIABLE IMPORTANCES
print("\n ==========FUTURE FEATURE IMPORTANCES==========")
feature_importances["Future variable importance over time"].mean().sort_values().plot(kind="barh")

Mostra quanto il modello ha sfruttato le feature che sono servite a guidare le previsioni passo-passo nel decoder

DIFFERENZA TRA PAST E FUTURE: \
La diversa importanza riflette quanto ciascuna variabile aiuta nell’analisi del contesto passato (encoder -->> PAST) \
o nella generazione delle previsioni (decoder -->> FUTURE).

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME
df_importances = feature_importances["Past variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances.index))

for col in df_importances.columns:
    p = ax.bar(np.arange(-len(df_importances), 0), df_importances[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances[col]
ax.set_title("Past variable importance over time")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)

plt.show()

Questo grafico mostra QUANDO (e QUANTO) nel passato ogni feature è stata più utile per costruire il contesto predittivo.

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME
df_importances2 = feature_importances["Future variable importance over time"]
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances2.index))
for col in df_importances2.columns:
    p = ax.bar(np.arange(-len(df_importances2), 0), df_importances2[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances2[col]
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.grid(True)
ax.legend()
plt.show()

Per ogni feature viene mostrato quanto è stata importante per il modello a ciascun orizzonte futuro

In [0]:
# PAST VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION
df_importances3 = feature_importances["Past variable importance over time"]
mean_attention = (
    tft_model
    .attention_weights()[tft_model.input_size :, :]
    .mean(axis=0)[: tft_model.input_size]
)
df_importances3 = df_importances3.multiply(mean_attention, axis=0)

fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances3.index))

for col in df_importances3.columns:
    p = ax.bar(np.arange(-len(df_importances3), 0), df_importances3[col].values, 0.6, label=col, bottom=bottom)
    bottom += df_importances3[col]
ax.set_title("Past variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Time")
ax.legend()
ax.grid(True)
plt.plot(
    np.arange(-len(df_importances3), 0),
    mean_attention,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention",
)
plt.legend()
plt.show()

In [0]:
# FUTURE VARIABLE IMPORTANCE OVER TIME PONDERATED BY ATTENTION

# Estrai l'importanza delle future variables
df_importances4 = feature_importances["Future variable importance over time"]

# Estrai tutte le attention weights mediate nel tempo
full_attention = tft_model.attention_weights()[tft_model.input_size:, :].mean(axis=0)

# Estrai solo la parte relativa ai 12 time steps futuri
n_future_steps = df_importances4.shape[0]
mean_attention_future = full_attention[-n_future_steps:]

# Applica la ponderazione riga per riga (sul tempo)
df_importances4 = df_importances4.multiply(mean_attention_future, axis=0)

# Plot
fig, ax = plt.subplots(figsize=(20, 10))
bottom = np.zeros(len(df_importances4.index))

for col in df_importances4.columns:
    ax.bar(
        np.arange(1, len(df_importances4) + 1), 
        df_importances4[col].values, 
        width=0.6, 
        label=col, 
        bottom=bottom
    )
    bottom += df_importances4[col].values

# Titoli e assi
ax.set_title("Future variable importance over time ponderated by attention")
ax.set_ylabel("Importance")
ax.set_xlabel("Future time steps")
ax.grid(True)

# Linea media di attenzione
ax.plot(
    np.arange(1, len(df_importances4) + 1),
    mean_attention_future,
    color="black",
    marker="o",
    linestyle="-",
    linewidth=2,
    label="mean_attention"
)
ax.legend()
plt.show()

In [0]:
tft_model.feature_importance_correlations()

#### CHRONOS

In [0]:
# Preparing dataset
df_chronos = create_time_series_features(df, target_col='y').dropna()
# Creazione dell'oggetto TimeSeriesDataFrame
df_chronos['item_id'] = df_chronos['unique_id'] 
df_chronos.drop(columns="unique_id", inplace=True) 
df_chronos.rename(columns={'ds': 'timestamp', 'y': 'target'}, inplace=True)

# Converti in TimeSeriesDataFrame
df_chronos = TimeSeriesDataFrame(
    df_chronos,
    id_column='item_id',
    timestamp_column='timestamp'
)
# Split train e test
#divido train/test ogni serie storica
def split_train_test_by_id(df, test_frac=0.1):
    train_list = []
    test_list = []
    for uid, group in df.groupby("item_id"):
        group = group.sort_values("timestamp")  # Ordina temporalmente
        split_idx = int(len(group) * (1 - test_frac))
        train_list.append(group.iloc[:split_idx])
        test_list.append(group.iloc[split_idx:])
    return pd.concat(train_list), pd.concat(test_list)
train_df_chronos, test_df_chronos = split_train_test_by_id(df_chronos, test_frac=0.076)

# Verifica dimensioni
print(f"Train shape: {train_df_chronos.shape}, Test shape: {test_df_chronos.shape}")

In [0]:
# Fit del modello
predictor = TimeSeriesPredictor(
    prediction_length=h,
    eval_metric="MASE",
    target="target",
    known_covariates_names=futr_exog_list,
    freq="B" 
)

# Aggiunta di più modelli nella configurazione per migliorare le performance
predictor.fit(
    train_df_chronos,
    hyperparameters={
        "Chronos": [
            {"model_path": "bolt_small", "ag_args": {"name_suffix": "ZeroShot"}},
            {
                "model_path": "bolt_small",
                "covariate_regressor": "CAT", 
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "smallCAT"},
            },
            {
                "model_path": "bolt_mini",
                "covariate_regressor": "CAT", 
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "miniCAT"},
            },
            {
                "model_path": "bolt_small",
                "covariate_regressor": "NN_TORCH", 
                "target_scaler": "standard",
                "ag_args": {"name_suffix": "small_NNtorch"},
            }
        ],
    },
    time_limit=600,
    enable_ensemble=True, 
)

# Valutazione del modello
leaderboard = predictor.leaderboard(train_df_chronos)
print(leaderboard)

In [0]:
# Previsioni
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)
train_df_chronos.set_index(["item_id","timestamp"],inplace=True)
test_df_chronos.set_index(["item_id","timestamp"],inplace=True)

test_known_covariates = test_df_chronos[futr_exog_list]
predictions = predictor.predict(
    data=train_df_chronos, 
    known_covariates=test_known_covariates
)
predictions.reset_index(inplace=True)
# Previsioni
predictions_0shot = predictor.predict(
    data=train_df_chronos, 
    known_covariates=test_known_covariates,
    model = "ChronosZeroShot[bolt_small]"
)
predictions_0shot.reset_index(inplace=True)

In [0]:
# forecast plot per store
store_ids = [1,3,5,20,35,54] #5 store a caso
train_df_chronos.reset_index(inplace=True)
test_df_chronos.reset_index(inplace=True)
for store in store_ids:
    train_store = train_df_chronos[train_df_chronos['item_id'] == store]
    test_store = test_df_chronos[test_df_chronos['item_id'] == store]
    forecast_store = predictions[predictions['item_id'] == store]

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=train_store['timestamp'], y=train_store['target'], mode='lines', name='Training Data'))
    fig.add_trace(go.Scatter(x=test_store['timestamp'], y=test_store['target'], mode='lines', name='Testing Data'))
    fig.add_trace(go.Scatter(
        x=forecast_store['timestamp'], 
        y=forecast_store['mean'], 
        mode='lines', 
        name='CHRONOS Forecast', 
        line=dict(color='green')
    ))

    ultimo_test = test_store['timestamp'].max()
    fig.add_vline(x=ultimo_test, line_dash="dash", line_color="white", opacity=0.5)
    fig.add_annotation(
        x=ultimo_test, 
        y=0.9, 
        yref="paper", 
        text="Fine Test Set", 
        showarrow=True, 
        arrowhead=2, 
        arrowcolor="white"
    )

    fig.update_layout(
        title=f"Forecast - Store {store}",
        xaxis_title="Timestamp [t]",
        yaxis_title="Sales",
        template='plotly_dark',
        legend=dict(font=dict(size=15))
    )

    fig.show()

In [0]:
# Calcola metriche per ogni store
#test_df_chronos.reset_index(inplace=True)
forecast_df = predictions[['timestamp', 'mean', 'item_id']]
merged_df = pd.merge(test_df_chronos[['timestamp', 'target', 'item_id']], predictions, 
                     on=['timestamp', 'item_id'], how='right')
store_metrics_chronos = []
df_chronos.reset_index(inplace=True)
for store in df_chronos['item_id'].unique():  # 5 store
    store_df = merged_df[merged_df['item_id'] == store]
    metrics = calcola_metriche(y_true=store_df['target'], y_pred=store_df['mean'],
                               y_train=train_df_chronos['target'],
                               modelname="Chronos")
    metrics['Store_ID'] = store
    store_metrics_chronos.append(metrics)

In [0]:
# Calcola metriche per ogni store 0 shot
#test_df_chronos.reset_index(inplace=True)
forecast_df_0shot = predictions_0shot[['timestamp', 'mean', 'item_id']]
merged_df_0shot = pd.merge(test_df_chronos[['timestamp', 'target', 'item_id']], predictions_0shot, 
                     on=['timestamp', 'item_id'], how='right')
store_metrics_chronos_0shot = []
#df_chronos.reset_index(inplace=True)
for store in df_chronos['item_id'].unique():  # 5 store
    store_df = merged_df_0shot[merged_df_0shot['item_id'] == store]
    metrics = calcola_metriche(y_true=store_df['target'], y_pred=store_df['mean'],
                               y_train=train_df_chronos['target'],
                               modelname="Chronos_0shot")
    metrics['Store_ID'] = store
    store_metrics_chronos_0shot.append(metrics)

In [0]:
# MEDIE E MEDIANE delle metriche
mae_values = []
mape_values = []
smape_values = []
rmse_values = []
mase_values_chronos = []

# Collect metrics from each store
for i in range(len(store_metrics_chronos)):
    mae_values.append(store_metrics_chronos[i].iloc[0, 0])  
    mape_values.append(store_metrics_chronos[i].iloc[2, 0])  
    smape_values.append(store_metrics_chronos[i].iloc[3, 0]) 
    rmse_values.append(store_metrics_chronos[i].iloc[1, 0]) 
    mase_values_chronos.append(store_metrics_chronos[i].iloc[4, 0]) 

# Calculate means
mae_mean = np.mean(mae_values)
mape_mean = np.mean(mape_values)
smape_mean = np.mean(smape_values)
rmse_mean = np.mean(rmse_values)
mase_mean = np.mean(mase_values_chronos)

# Calculate medians
mae_median = np.median(mae_values)
mape_median = np.median(mape_values)
smape_median = np.median(smape_values)
rmse_median = np.median(rmse_values)
mase_median = np.median(mase_values_chronos)

# Create a DataFrame to store the aggregated metrics
mean_metrics_chronos = pd.DataFrame({
    'Metric': ['MAE', 'MAPE', 'SMAPE', 'RMSE', 'MASE'],
    'Mean_Chronos': [mae_mean, mape_mean, smape_mean, rmse_mean, mase_mean],
    'Median_Chronos':[mae_median, mape_median, smape_median, rmse_median, mase_median]
})
mean_metrics_chronos

In [0]:
plt.plot(mase_values_chronos)
plt.axhline(y=1, color='red', linestyle='--')
print("MASE >= 1:", np.sum(np.array(mase_values_chronos) >= 1), "Percentuale:",
      (np.sum(np.array(mase_values_chronos) > 1) / len(mase_values_chronos) * 100).round(2), "% degli store")
print("MASE < 1:", np.sum(np.array(mase_values_chronos) < 1), "Percentuale:",
      (np.sum(np.array(mase_values_chronos) < 1) / len(mase_values_chronos) * 100).round(2), "% degli store")

#### valutazioni finali

In [0]:
## PLOT UNICO con tutte le previsioni per quegli store
# forecast plot per store
store_ids = ['1','3','5','20','35','54'] # degli store a caso

for store in store_ids:
    train_store = train_df_np[train_df_np['ID'] == store]
    test_store = test_df_np[test_df_np['ID'] == store]
    forecast_store_np = forecast_test[forecast_test['ID'] == store]
    
    predictions.rename(columns={'item_id':'ID'}, inplace=True)
    predictions['ID'] = predictions['ID'].astype(str)
    forecast_store_chronos = predictions[predictions['ID'] == store]
    
    predictions_0shot.rename(columns={'item_id':'ID'}, inplace=True)
    predictions_0shot['ID'] = predictions_0shot['ID'].astype(str)
    forecast_0shot_store_chronos = predictions_0shot[predictions_0shot['ID'] == store]
    
    
    Y_hat_df_lstm.rename(columns={'unique_id':'ID'}, inplace=True)
    Y_hat_df_lstm['ID'] = Y_hat_df_lstm['ID'].astype(str)
    forecast_store_lstm = Y_hat_df_lstm[Y_hat_df_lstm['ID'] == store]
    
    # Y_hat_df_tft.rename(columns={'unique_id':'ID'}, inplace=True)
    # Y_hat_df_tft['ID'] = Y_hat_df_tft['ID'].astype(str)
    # forecast_store_tft = Y_hat_df_tft[Y_hat_df_tft['ID'] == store]
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=train_store['ds'], y=train_store['y'], mode='lines', name='Training Data'))
    fig.add_trace(go.Scatter(x=test_store['ds'], y=test_store['y'], mode='lines', name='Testing Data'))
    fig.add_trace(go.Scatter(
        x=forecast_store_np['ds'], 
        y=forecast_store_np['yhat1'], 
        mode='lines', 
        name='NeuralProphet', 
        line=dict(color='pink')
    ))
    fig.add_trace(go.Scatter(
    x=forecast_store_lstm['ds'], 
    y=forecast_store_lstm['LSTM'], 
    mode='lines', 
    name='LSTM', 
    line=dict(color='yellow')
    ))
    # fig.add_trace(go.Scatter(
    # x=forecast_store_tft['ds'], 
    # y=forecast_store_tft['TFT'], 
    # mode='lines', 
    # name='TFT', 
    # line=dict(color='lightblue')
    # ))
    fig.add_trace(go.Scatter(
    x=forecast_store_chronos['timestamp'], 
    y=forecast_store_chronos['mean'], 
    mode='lines', 
    name='Chronos', 
    line=dict(color='lightgreen')
    ))
    # fig.add_trace(go.Scatter(
    # x=forecast_0shot_store_chronos['timestamp'], 
    # y=forecast_0shot_store_chronos['mean'], 
    # mode='lines', 
    # name='Chronos 0 shot', 
    # line=dict(color='green')
    # ))
    
    fig.update_layout(
        title=f"Forecast - Store {store}",
        xaxis_title="Timestamp [t]",
        yaxis_title="Sales",
        template='plotly_dark',
        legend=dict(font=dict(size=15))
    )
    fig.show()

In [0]:
final_metrics = pd.merge(mean_metrics_lstm, mean_metrics_np, on='Metric', how='outer')
final_metrics = pd.merge(final_metrics, mean_metrics_chronos, on='Metric', how='outer')
final_metrics = pd.merge(final_metrics, mean_metrics_tft, on='Metric', how='outer')
final_metrics

In [0]:
## PLOT DEI MASE PER STORE
# Imposta stile
sns.set(style='whitegrid')
plt.figure(figsize=(12, 6))

# Linea di riferimento a MASE = 1
plt.axhline(y=1, color='red', linestyle='--', label='MASE = 1')

# Plot dei MASE
plt.plot(mase_values_lstm, label='LSTM', marker='o')
plt.plot(mase_values_np, label='NeuralProphet', marker='s')
plt.plot(mase_values_chronos, label='Chronos', marker='^')
plt.plot(mase_values_tft, label='TFT', marker='d',color="aquamarine") 

# Etichette e titolo
plt.title('MASE per Store', fontsize=16)
plt.xlabel('Store ID (indice)', fontsize=12)
plt.ylabel('MASE', fontsize=12)
store_ids = df['unique_id'].unique()
plt.xticks(ticks=range(len(store_ids)), labels=store_ids, rotation=90)

# Legenda
plt.legend(title='Modelli')
plt.tight_layout()
plt.show()